# 📊 PyFlow: Deep PYUSD Analysis with Google Cloud's Premium RPC Methods

**Hackathon Context:** This notebook is developed for the **PayPal x Google Cloud Web3 Bounty**, demonstrating how **Google Cloud Platform's Blockchain Node Engine** unlocks powerful, cost-effective analysis of the **PayPal USD (PYUSD)** stablecoin on Ethereum.

---

## The Challenge: Unlocking Deep Blockchain Insights

Understanding the intricate movements, internal logic, and real-world interactions of stablecoins like **PYUSD** often requires deep, computationally intensive blockchain analysis. Standard block explorers and basic RPC calls provide only surface-level data, while accessing advanced tracing and state inspection methods on most platforms is prohibitively expensive or heavily rate-limited due to their high computational demands.

## The Solution: PyFlow leveraging GCP's Advantage

This notebook, **PyFlow**, provides a comprehensive toolkit for advanced PYUSD analysis by specifically utilizing **Google Cloud Platform's premium RPC debug and trace methods**.

> **🚀 GCP's Unique Offering: Cost-Effective Access to High-Multiplier Methods**
>
> Many advanced RPC methods carry significant **request multipliers** due to their computational intensity. For example, a method with a `50x` multiplier consumes the equivalent quota/cost of 50 basic calls (like `eth_call`). Methods like `trace_replayTransaction` have an even higher `100x` multiplier.
>
> **GCP's Blockchain Node Engine stands out by offering generous free quotas even for these high-multiplier methods**, effectively democratizing access to capabilities previously reserved for specialized infrastructure or high budgets.

This allows PyFlow to perform analysis typically infeasible elsewhere, such as:

*   **Forensic Accounting:** Tracing PYUSD flow through complex multi-contract DeFi interactions using methods like `debug_traceTransaction` (`50x`).
*   **Gas Optimization Analysis:** Pinpointing exact gas costs within internal PYUSD functions or integrations.
*   **Security Investigations:** Replaying failed transactions (`trace_replayTransaction`, `100x`) or examining state changes (`stateDiff` via replay).
*   **Smart Contract Auditing:** Verifying internal logic, storage layout (`debug_storageRangeAt`, `50x`), and event emission (`eth_getLogs`, `50x`).
*   **Network Health Insights:** Analyzing pending transaction queues (`txpool_status`, `50x`) and estimating confirmation times.

## 🛠️ Methods Explored (with GCP Request Multipliers):

This notebook provides practical implementations and analysis using the following GCP-powered methods for PYUSD on Ethereum. Multipliers indicate the relative request cost compared to a standard call:

*   **Detailed Tracing:**
    *   `debug_traceTransaction` (`50x`): In-depth EVM execution trace (using `callTracer` & `structLog`).
    *   `trace_transaction` (`50x`): Alternative transaction tracing method.
*   **Block-Level Analysis:**
    *   `trace_block` (`50x`, Mainnet only): Trace all transactions within a specified block.
    *   `debug_traceBlockByNumber` / `debug_traceBlockByHash` (`50x`): Alternative block tracing.
*   **State Replay & Simulation:**
    *   `trace_replayTransaction` (`100x`, Mainnet only): Re-execute a past transaction with tracers.
    *   `trace_replayBlockTransactions` (`100x`, Mainnet only): Re-execute all transactions in a block with tracers.
    *   `trace_call` (`50x`, Mainnet only): Simulate transaction calls without sending to the network.
*   **State & Data Retrieval:**
    *   `eth_getLogs` (`50x`): Efficiently fetch specific PYUSD events (e.g., Transfers, Approvals).
    *   `eth_getCode` (`10x`): Retrieve deployed contract bytecode.
    *   `debug_storageRangeAt` (`50x`): Inspect raw contract storage slots.
    *   `eth_getProof` (`50x`): Fetch Merkle proofs for state verification.
*   **Network Monitoring:**
    *   `txpool_status` (`50x`): Analyze pending/queued transaction counts.

*(Note: Multipliers are based on GCP documentation and highlight the computational intensity absorbed by the service.)*

---

**💡 Goal:** By the end of this notebook, you will understand how to leverage GCP's unique RPC capabilities, including high-multiplier methods offered with generous quotas, to perform advanced, cost-effective blockchain intelligence specifically tailored for the PYUSD stablecoin.


## 🛠️ Environment Setup: Installing Dependencies for PyFlow
---

This cell installs the necessary Python packages to run the PyFlow analysis notebook. It sets up a complete environment for interacting with the Ethereum blockchain (via GCP), analyzing PYUSD data, generating visualizations, and connecting to Google Cloud services.

### 📊 Key Dependencies & Purpose:

| Category                 | Packages                                                       | Purpose                                                             |
| :----------------------- | :------------------------------------------------------------- | :------------------------------------------------------------------ |
| **Core Blockchain/Data** | `web3`, `pandas`, `numpy`                                    | Ethereum RPC interaction, data manipulation                         |
| **Visualization**        | `matplotlib`, `plotly`, `seaborn`, `networkx`, `graphviz`, `pygraphviz`    | Charts, transaction graphs, visual analysis                         |
| **Google Cloud**         |  `gspread`, `oauth2client` | Accessing Google Sheets export, Authentication |
| **Ethereum Utilities**   | `eth-utils`, `rlp`, `tqdm`                                   | Cryptographic functions, RLP encoding, progress bars                |
| **Notebook Enhancement** | `ipywidgets`, `rich`                                         | Interactive controls, improved console output                     |

### ⚙️ Runtime Notes:

*   **Environment:** Designed primarily for Google Colab.
*   **Resources:** A standard Colab runtime is usually sufficient, but a High-RAM runtime is recommended for analyzing very large blocks or complex transaction traces. GPU is generally not required.
*   **Colab Features:** The setup automatically installs system-level `graphviz` and enables interactive data tables within Colab.

> **⏳ Installation Time:** The process uses `pip` and typically completes in **1-2 minutes**. Please ensure this cell executes successfully before proceeding.

In [ ]:
# =============================================================================================
# 🛠️ Environment Setup and Package Installation
# =============================================================================================
# This cell installs and configures all necessary packages for blockchain data analysis.
# The setup process may take 1-2 minutes to complete.

import sys
import subprocess
import time
from IPython.display import clear_output
from rich.console import Console
from rich.theme import Theme
from rich.progress import Progress, SpinnerColumn, TextColumn

# Auto-adapting color theme that works well in both light and dark terminals
custom_theme = Theme({
    "info": "cyan3",          # Informational messages
    "success": "spring_green3", # Success indicators
    "warning": "gold3",       # Warning messages
    "error": "red3",          # Error messages
    "highlight": "royal_blue1"  # Highlighted information
})

# Create console with auto color system detection for better visual feedback
console = Console(theme=custom_theme)

console.print("\n✨ Environment Setup and Package Installation ✨", style="bold cyan3")
console.print("─────────────────────────────────────────────────", style="cyan3")

console.print("🔄 Starting package installation process...", style="info")

# Function to install packages and handle errors with better formatting
# This provides visual feedback during the installation process
def install_packages(packages, description):
    """Install specified packages with progress indicator and error handling"""
    with Progress(
        SpinnerColumn(),
        TextColumn(f"[info]Installing {description}..."),
        transient=True,
    ) as progress:
        task = progress.add_task("", total=None)
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + packages.split())
            console.print(f"✓ {description} installed", style="success")
            return True
        except subprocess.CalledProcessError:
            console.print(f"❌ Error installing {description}", style="error")
            return False

# =============================================================================================
# Core Libraries Installation
# =============================================================================================

# Install core data processing libraries
# - web3: For blockchain interaction and smart contract calls
# - pandas: For data manipulation and analysis
# - numpy: For numerical operations
# - matplotlib: For basic visualization
success = install_packages("web3==6.11.1 pandas numpy matplotlib",
                         "Core data libraries (web3, pandas, numpy, matplotlib)")

# Install advanced visualization and analysis libraries
# - plotly: For interactive charts
# - seaborn: For statistical visualizations
# - networkx: For blockchain transaction network analysis
if success:
    success = install_packages("plotly seaborn networkx",
                             "Visualization and analysis libraries (plotly, seaborn, networkx)")

# Install visualization export libraries
# - kaleido: For high-quality Plotly chart exports to PNG/PDF/SVG
if success:
    success = install_packages("kaleido",
                             "Visualization export library (required for exporting charts to images)")

# Install Google API libraries for data access and storage
# - gspread: For Google Sheets integration
# - oauth2client: For authentication with Google services
if success:
    success = install_packages("gspread oauth2client",
                             "Google API libraries (gspread, oauth2client)")

# Install interactive widgets for Jupyter/Colab notebooks
# - ipywidgets: For creating interactive controls and dashboards
if success:
    success = install_packages("ipywidgets",
                             "Interactive widgets for Jupyter notebooks")

# Install Ethereum proof verification and analysis libraries
# - eth-utils: For cryptographic functions and general Ethereum utilities
# - rlp: For Recursive Length Prefix encoding used in Ethereum
# - tqdm: For progress visualization in notebook environments
# - graphviz: For visualizing Merkle proofs and contract structures
if success:
    success = install_packages("eth-utils rlp tqdm graphviz",
                             "Ethereum proof verification libraries")

# For Colab environments, install system-level graphviz for visualization

try:
    # Check if running in Google Colab
    import google.colab
    console.print("\n\n🔄 Installing system dependencies for visualization...", style="info")
    # Install graphviz system package (used for rendering graphs)
    subprocess.check_call(['apt-get', '-qq', 'install', 'graphviz'])
    console.print("✓ System-level graphviz installed for advanced visualizations", style="success")

    # Install libgraphviz-dev system package
    console.print("\n\n🔄 Installing libgraphviz Python package...")
    subprocess.check_call(['apt-get', '-qq', 'install', 'libgraphviz-dev'])
    console.print("✓ System-level graphviz installed for advanced visualizations", style="success")

    # Install libgraphviz-dev system package
    console.print("\n\n🔄 Installing pygraphviz Python package...")
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'pygraphviz'])
    console.print("✓ pygraphviz Python connector installed.", style="success")

    # Enable enhanced data visualization and export capabilities
    console.print("\n\n🔄 Enabling enhanced data visualization and export...", style="info")

    # Enable interactive data tables from Google Colab
    try:
        from google.colab import data_table
        data_table.enable_dataframe_formatter()
        console.print("✓ Interactive data tables enabled", style="success")
    except ImportError:
        console.print("⚠️ Could not enable interactive data tables", style="warning")

    # Import additional components for file exports and Google Sheets integration
    try:
        # For direct CSV/JSON downloads
        import base64
        import io

        # For Google Sheets export
        from google.colab import output
        from googleapiclient.discovery import build
        from googleapiclient.http import MediaInMemoryUpload

        console.print("✓ Export functionality enabled", style="success")
    except ImportError:
        console.print("⚠️ Some export functions may be limited", style="warning")

    # Verify data table display
    try:
        from IPython.display import display, HTML
        console.print("✓ Data table display verified", style="success")
    except:
        console.print("⚠️ Data table display verification failed", style="warning")

except ImportError:
    console.print("ℹ️ Not running in Colab, skipping system-level installations", style="info")
except Exception as e:
    console.print(f"⚠️ Note: Could not install graphviz system package: {e}. Some visualizations may be limited.", style="warning")

# Final status message with extra spacing
if success:
    console.print("\n\n📦 ✓ All required packages installed successfully!", style="success")
else:
    console.print("\n\n⚠️ [bold]Some packages failed to install. Please check the errors above.[/bold]", style="warning")

# =============================================================================================
# Environment Verification
# =============================================================================================
# Verify all packages imported correctly and set up the analytics environment

try:
    # Core data and utility libraries
    import os
    import json
    import time
    import warnings
    import hashlib

    # Data analysis stack
    import numpy as np
    import pandas as pd

    # Network and graph analysis
    import networkx as nx

    # Visualization libraries
    import matplotlib.pyplot as plt
    import seaborn as sns
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots

    # Date handling
    from datetime import datetime, timedelta

    # Collections and data structures
    from collections import defaultdict, Counter

    # Blockchain interaction libraries
    from web3 import Web3
    from web3.exceptions import TransactionNotFound
    from web3.middleware import geth_poa_middleware
    from hexbytes import HexBytes

    # Google Cloud and authentication (for gspread)
    from google.colab import auth # Kept for potential gspread auth if needed in Colab
    from google.oauth2 import service_account
    import gspread
    from oauth2client.client import GoogleCredentials
    from oauth2client.service_account import ServiceAccountCredentials

    # Import ipywidgets components for interactive dashboards
    import ipywidgets as widgets
    from ipywidgets import interact, interactive, fixed, interact_manual

    # Rich components for improved CLI-style output
    from rich.panel import Panel
    from rich.syntax import Syntax
    from rich.table import Table

    # Ethereum specific utilities for cryptography and data structures
    from eth_utils import keccak, to_bytes, to_hex
    import rlp
    from tqdm.notebook import tqdm
    from graphviz import Digraph

    # Suppress warnings for cleaner output
    warnings.filterwarnings('ignore')

    # Configure pandas display settings for better readability
    pd.set_option('display.max_columns', None)  # Show all columns
    pd.set_option('display.max_rows', 100)      # Reasonable number of rows
    pd.set_option('display.float_format', '{:.6f}'.format)  # Format for amounts

    # Setup Plotly for better Jupyter/Colab integration
    import plotly.io as pio
    pio.templates.default = "plotly_white"

    console.print("\n\n🚀 ✓ Setup complete! Analytics platform initialized.", style="success")
except ImportError as e:
    console.print(f"\n\n❌ Error importing libraries: {e}", style="error")
    console.print("⚠️ Some required packages may not have been installed correctly.", style="warning")

## 🔑 Configuration & Authentication: Connecting to GCP and Ethereum RPC
---

This crucial cell configures PyFlow to connect to Google Cloud Platform services (for authentication and Google Sheets) and the necessary Ethereum networks via GCP's Blockchain Node Engine. **You MUST edit this cell with your specific credentials before running it.**

### 📋 Step 1: Provide Your GCP Credentials

To use GCP's Blockchain RPC and potentially Google Sheets, you need:

1.  **Your GCP Project ID:**
    *   **Purpose:** Identifies your Google Cloud project, primarily used here to scope authentication requests for Google Drive/Sheets access.
    *   **How to Obtain:** If you don't have one, create it at [GCP Console](https://console.cloud.google.com/projectcreate).
    *   **➡️ ACTION REQUIRED:** Find the `GCP_PROJECT_ID` variable in the code below and replace `"YOUR_PROJECT_ID"` with your actual Project ID string.
2.  **Your GCP Blockchain RPC Endpoints (with API Key):**
    *   **Purpose:** Secure URLs to connect to Ethereum Mainnet and testnets via GCP. This is key to interacting with the blockchain using `web3.py`.
    *   **How to Obtain:**
        1.  Go to the [GCP Blockchain Node Engine Console](https://console.cloud.google.com/blockchain/node-engine) in your project.
        2.  Enable the API if you haven't already.
        3.  Copy the **full HTTPS RPC endpoint URL** (including `?key=...`) for **Ethereum Mainnet**. Optionally, copy URLs for testnets (Holesky, Sepolia) if needed.
    *   **➡️ ACTION REQUIRED:** Find the `BLOCKCHAIN_RPC` dictionary in the code below. Replace the placeholder URLs (e.g., `"https://blockchain.googleapis.com/..."`) with your *complete* copied endpoint URLs for `'mainnet'`, `'holesky'`, and `'sepolia'`.

    *Example Format (Use your actual URLs):*
    ```python
    BLOCKCHAIN_RPC = {
        'ethereum': {
            'mainnet': 'https://YOUR_MAINNET_ENDPOINT_URL?key=YOUR_API_KEY',
            # Optional testnets:
            'holesky': 'https://YOUR_HOLESKY_ENDPOINT_URL?key=YOUR_API_KEY',
            'sepolia': 'https://YOUR_SEPOLIA_ENDPOINT_URL?key=YOUR_API_KEY'
        }
    }
    ```

### 🌐 Step 2: Enable Required GCP APIs

Ensure the following APIs are **enabled** in your GCP Project *before* running this cell for authentication and Google Sheets integration to work correctly:

*   **Blockchain Node Engine API:** ([Enable Link](Coming Soon))
*   **Google Drive API:** Required for Google Sheets export/import functionality via `gspread`. ([Enable Link](https://console.cloud.google.com/apis/library/drive.googleapis.com))
*   **Google Sheets API:** Required for Google Sheets export/import functionality via `gspread`. ([Enable Link](https://console.cloud.google.com/apis/library/sheets.googleapis.com))

> #### **💡Tip: Follow README for Video Instructions.**

### 🔐 Step 3: Run the Cell & Authenticate

When you execute the code cell below:

1.  It will define constants (like PYUSD contract addresses) and configurations (trace settings).
2.  It will attempt to **authenticate** your Google account (via a pop-up in Colab) to grant access to the enabled GCP services needed for Google Sheets (Drive, Sheets). Follow the prompts.
3.  It will initialize `web3.py` clients using your provided RPC endpoints.
4.  It will initialize a client for Google Sheets (`gspread`).
5.  It will perform **connection tests** (check RPC node block height) and display a status summary.

> **⚠️ IMPORTANT:** Double-check that you have replaced the placeholder `GCP_PROJECT_ID` and the **full** `BLOCKCHAIN_RPC` URLs before running. The notebook relies heavily on a successful connection to the **Ethereum Mainnet** endpoint via GCP for most subsequent analysis.

In [ ]:
# =============================================================================================
# 📋 Configuration and Authentication for Blockchain Analytics
# =============================================================================================
# This cell configures & Authenticates for blockchain data analysis.

# Import necessary libraries
import google.auth

try:
    from google.colab import auth
except ImportError:
    auth = None
    print("Note: Not running in Google Colab, standard gcloud auth will be used if available.")

import gspread
from web3 import Web3
from web3.middleware import geth_poa_middleware
from rich.console import Console
from rich.theme import Theme
from rich.progress import Progress, SpinnerColumn, TextColumn, TimeElapsedColumn
import time
import os

# Auto-adapting color theme that works well in both light and dark terminals
custom_theme = Theme({
    "info": "cyan3",
    "success": "spring_green3",
    "warning": "gold3",
    "error": "red3",
    "highlight": "royal_blue1"
})

# Ensure console is created with the theme
console = Console(theme=custom_theme)

# =============================================================================================
# Contract Configuration: PYUSD Stablecoin Addresses
# =============================================================================================

# Main PYUSD Contract addresses (used for querying transactions and events)
PYUSD_PROXY = Web3.to_checksum_address('0x6c3ea9036406852006290770bedfcaba0e23a0e8')
PYUSD_IMPLEMENTATION = Web3.to_checksum_address('0x8EcaE0B0402E29694B3Af35d5943D4631Ee568dC')
SUPPLY_CONTROL_PROXY = Web3.to_checksum_address('0x31d9bDEa6F104606C954f8FE6ba614F1BD347Ec3')
SUPPLY_CONTROL_IMPLEMENTATION = Web3.to_checksum_address('0xFaB5891ED867a1195303251912013b92c4fc3a1D')

# PYUSD Contract Registry with implementation contracts
PYUSD_CONTRACTS = {
    PYUSD_PROXY.lower(): "PYUSD Token",
    PYUSD_IMPLEMENTATION.lower(): "PYUSD Implementation",
    SUPPLY_CONTROL_PROXY.lower(): "Supply Control",
    SUPPLY_CONTROL_IMPLEMENTATION.lower(): "Supply Control Impl"
}

# Define event topics first
TRANSFER_EVENT_TOPIC = Web3.keccak(text="Transfer(address,address,uint256)").hex()
APPROVAL_EVENT_TOPIC = Web3.keccak(text="Approval(address,address,uint256)").hex()
PAUSED_EVENT_TOPIC = Web3.keccak(text="Paused(address)").hex()
UNPAUSED_EVENT_TOPIC = Web3.keccak(text="Unpaused(address)").hex()

# Comprehensive PYUSD configuration
PYUSD_CONFIG = {
    'ethereum': {
        'address': PYUSD_PROXY,
        'implementation': PYUSD_IMPLEMENTATION,
        'decimals': 6,
        'symbol': 'PYUSD',
        'deployment_block': 15921958,
        'transfer_event_topic': TRANSFER_EVENT_TOPIC,
        'approval_event_topic': APPROVAL_EVENT_TOPIC,
        'pause_event_topic': PAUSED_EVENT_TOPIC,
        'unpause_event_topic': UNPAUSED_EVENT_TOPIC
    }
}

PYUSD_ADDRESS_LOWER_ETH = PYUSD_CONFIG['ethereum']['address'].lower()

# PYUSD Function Signature Registry
PYUSD_SIGNATURES = {
    '0xa9059cbb': {"name": "transfer(address,uint256)", "type": "function", "category": "token_movement"},
    '0x095ea7b3': {"name": "approve(address,uint256)", "type": "function", "category": "allowance"},
    '0x23b872dd': {"name": "transferFrom(address,address,uint256)", "type": "function", "category": "token_movement"},
    '0x40c10f19': {"name": "mint(address,uint256)", "type": "function", "category": "supply_change"},
    '0x42966c68': {"name": "burn(uint256)", "type": "function", "category": "supply_change"},
    '0x18160ddd': {"name": "totalSupply()", "type": "function", "category": "view"},
    '0x70a08231': {"name": "balanceOf(address)", "type": "function", "category": "view"},
    '0xdd62ed3e': {"name": "allowance(address,address)", "type": "function", "category": "view"},
    '0x313ce567': {"name": "decimals()", "type": "function", "category": "view"},
    '0x06fdde03': {"name": "name()", "type": "function", "category": "view"},
    '0x95d89b41': {"name": "symbol()", "type": "function", "category": "view"},
    '0x8456cb59': {"name": "pause()", "type": "function", "category": "control"},
    '0x3f4ba83a': {"name": "unpause()", "type": "function", "category": "control"},
    '0x5c975abb': {"name": "paused()", "type": "function", "category": "view"},
    '0xf2fde38b': {"name": "transferOwnership(address)", "type": "function", "category": "admin"},
    '0x8da5cb5b': {"name": "owner()", "type": "function", "category": "view"},
    '0x715018a6': {"name": "renounceOwnership()", "type": "function", "category": "admin"}
}

# PYUSD Event Signature Registry with decoders - using the defined event topics
PYUSD_EVENTS = {
    TRANSFER_EVENT_TOPIC: {
        "name": "Transfer(address,address,uint256)",
        "decoder": lambda topics, data: {
            "from": Web3.to_checksum_address('0x' + topics[1][-40:]),
            "to": Web3.to_checksum_address('0x' + topics[2][-40:]),
            "value": int(data, 16)
        }
    },
    APPROVAL_EVENT_TOPIC: {
        "name": "Approval(address,address,uint256)",
        "decoder": lambda topics, data: {
            "owner": Web3.to_checksum_address('0x' + topics[1][-40:]),
            "spender": Web3.to_checksum_address('0x' + topics[2][-40:]),
            "value": int(data, 16)
        }
    },
    PAUSED_EVENT_TOPIC: {
        "name": "Paused(address)",
        "decoder": lambda topics, data: {
            "account": Web3.to_checksum_address('0x' + topics[1][-40:]) if len(topics) > 1 else None
        }
    },
    UNPAUSED_EVENT_TOPIC: {
        "name": "Unpaused(address)",
        "decoder": lambda topics, data: {
            "account": Web3.to_checksum_address('0x' + topics[1][-40:]) if len(topics) > 1 else None
        }
    }
}

# Tracing configurations
TRACE_CONFIGS = {
    "callTracer": {
        "withLog": True,
        "enableReturnData": True,
        "enableMemory": True,
        "enableStack": True
    },
    "structLog": {
        "disableStorage": False,
        "disableMemory": False,
        "disableStack": False,
        "fullStorage": True
    }
}

# Gas analysis categories
GAS_CATEGORIES = {
    "token_movement": ["transfer", "transferFrom"],
    "supply_change": ["mint", "burn"],
    "allowance": ["approve", "increaseAllowance", "decreaseAllowance"],
    "control": ["pause", "unpause"],
    "admin": ["transferOwnership", "renounceOwnership", "addMinter", "removeMinter"],
    "view": ["balanceOf", "allowance", "totalSupply", "decimals", "name", "symbol", "paused", "owner"],
    "other": []
}


# =============================================================================================
# Data Source Configuration: Google Cloud & Blockchain RPC
# =============================================================================================

# Replace "YOUR_PROJECT_ID" with your actual GCP Project ID (needed for GSheets auth scope)
GCP_PROJECT_ID = "YOUR_PROJECT_ID"
if "YOUR_PROJECT_ID" in GCP_PROJECT_ID or not GCP_PROJECT_ID:
    console.print("[error]🚨 CRITICAL: Please replace placeholder or provide your actual GCP Project ID in GCP_PROJECT_ID.", style="bold red")

# Replace "YOUR_MAINNET_BLOCKCHAIN_RPC_URL", "YOUR_HOLESKY_BLOCKCHAIN_RPC_URL", "YOUR_SEPOLIA_BLOCKCHAIN_RPC_URL" with your FULL RPC endpoint URLs including your API key.
# e.g., 'mainnet': 'https://blockchain.googleapis.com/v1/projects/...........'
BLOCKCHAIN_RPC = {
    'ethereum': {
        'holesky': 'YOUR_HOLESKY_BLOCKCHAIN_RPC_URL',
        'mainnet': 'YOUR_MAINNET_BLOCKCHAIN_RPC_URL',
        'sepolia': 'YOUR_SEPOLIA_BLOCKCHAIN_RPC_URL'
    }
}

# Basic Validation checks for placeholders in URLs
rpc_urls_valid = True
if 'ethereum' not in BLOCKCHAIN_RPC or not BLOCKCHAIN_RPC['ethereum']: # Check if ethereum key exists and has entries
    console.print("[error]🚨 CRITICAL: BLOCKCHAIN_RPC['ethereum'] is missing or empty.", style="bold red")
    rpc_urls_valid = False
else:
    for network, url in BLOCKCHAIN_RPC.get('ethereum', {}).items():
        # More robust check for placeholders or incomplete URLs
        if not url or "xxx" in url.lower() or "/v1/" not in url or "key=" not in url or url.endswith("key="):
            console.print(f"[error]🚨 CRITICAL: RPC URL for '{network}' seems invalid or uses placeholder ('{url}'). Use full URL with API key.", style="bold red")
            rpc_urls_valid = False

# Transaction tracing configuration (for detailed transaction analysis)
# Increased timeout for complex traces
DEFAULT_TRACE_CONFIG = {
    'tracer': 'callTracer',
    'timeout': '120s',
    'tracerConfig': {
        'onlyTopCall': False,
        'withLog': True,
    }
}

STRUCTLOG_TRACE_CONFIG = {
    'tracer': 'structLog',
    # 'timeout': '120s',
}

# =============================================================================================
# Global Client Variables
# =============================================================================================
gc_sheets = None
w3_clients = {}

# =============================================================================================
# Utility Functions: Testing & System Verification
# =============================================================================================

def authenticate_gcp(progress, task_id):
    """Authenticate to GCP, initialize Google Sheets client (updates progress descriptively)."""
    global gc_sheets
    progress.update(task_id, description="[info]Initiating GCP Authentication...")
    if not GCP_PROJECT_ID or "project_id" in GCP_PROJECT_ID:
         progress.update(task_id, description="[error]GCP Auth Failed (No Project ID)")
         return False
    auth_success = False
    gs_init_success = False
    effective_project_id = GCP_PROJECT_ID

    try:
        # Step 1: Authentication (Colab vs. Default)
        creds = None
        if auth: # If google.colab.auth was imported successfully
            progress.update(task_id, description="[info]Waiting for Colab user authentication...")
            auth.authenticate_user(project_id=effective_project_id) # Use project ID here if needed by Colab auth context
            auth_success = True
            progress.update(task_id, description="[info]Colab user authenticated. Getting credentials...")
            creds, _ = google.auth.default() # Get default credentials after Colab auth
        else:
            # Attempt standard ADC (Application Default Credentials) - works in VM, Cloud Shell, local gcloud auth login
            progress.update(task_id, description="[info]Attempting default GCP authentication...")
            try:
                creds, inferred_project_id = google.auth.default()
                if not creds:
                     raise Exception("Could not get default credentials.")
                auth_success = True
                progress.update(task_id, description="[info]Default GCP credentials obtained.")
            except Exception as adc_error:
                progress.update(task_id, description="[error]Default GCP Authentication Failed!")
                console.print(f"❌ Default GCP Auth error: {adc_error}", style="error")
                return False # Hard stop if auth fails

        # Ensure we have credentials before proceeding
        if not creds:
             progress.update(task_id, description="[error]Credentials not obtained after auth attempt.")
             return False

        # Step 2: Initialize Google Sheets Client
        progress.update(task_id, description="[info]Initializing Google Sheets client...")
        gc_sheets = gspread.authorize(creds)
        # Perform a minimal check (e.g., list spreadsheets) if needed, but authorize usually suffices
        # gc_sheets.list_spreadsheet_files(max_results=1)
        gs_init_success = True

        progress.update(task_id, description="[success]GCP Authentication & GSheets Client Initialized")
        return True # Overall success

    except Exception as e:
        error_stage = "GCP Setup Error!"
        if not auth_success:
            error_stage = "GCP Authentication Failed!"
        elif not gs_init_success:
            error_stage = "Google Sheets Client Init Failed!"
        progress.update(task_id, description=f"[error]{error_stage}")
        console.print(f"❌ {error_stage}: {str(e)}", style="error")
        if not gs_init_success: gc_sheets = None
        return False


def initialize_all_web3_clients(progress, task_id):
    """Initializes Web3 clients for all networks (updates progress descriptively)."""
    global w3_clients
    progress.update(task_id, description="[info]Initializing Web3 Clients...")
    w3_clients = {}
    success_count = 0
    total_networks = 0
    if 'ethereum' not in BLOCKCHAIN_RPC or not BLOCKCHAIN_RPC['ethereum']:
        progress.update(task_id, description="[error]No valid RPC Config found!")
        return False
    total_networks = len(BLOCKCHAIN_RPC['ethereum'])
    network_statuses = []

    for network, rpc_url in BLOCKCHAIN_RPC['ethereum'].items():
        progress.update(task_id, description=f"[info]Connecting to {network.capitalize()}...")
        time.sleep(0.1) # Small delay for visual update
        # Find the original url value before the loop modified it
        original_url = BLOCKCHAIN_RPC.get('ethereum', {}).get(network, "")
        is_invalid_url = not original_url or "xxx" in original_url.lower() or "/v1/" not in original_url or "key=" not in original_url or original_url.endswith("key=")
        if is_invalid_url:
            w3_clients[network] = None
            network_statuses.append(f"{network.capitalize()}:[error]Skipped (Invalid URL)[/error]")
            progress.update(task_id, description=f"[warning]Skipping {network.capitalize()} (Invalid URL)...")
            continue
        try:
            # Increased timeout slightly for potentially slower network conditions
            provider = Web3.HTTPProvider(rpc_url, request_kwargs={'timeout': 120})
            w3_client = Web3(provider)
            # Inject middleware only if needed (e.g., for PoA testnets like Goerli, Rinkeby - less relevant for Mainnet/Sepolia/Holesky now)
            # Check chain ID if necessary to decide on middleware, but generally safe to add
            w3_client.middleware_onion.inject(geth_poa_middleware, layer=0)

            # Test connection with get_block_number
            block_num = w3_client.eth.get_block_number()
            w3_clients[network] = w3_client
            success_count += 1
            network_statuses.append(f"{network.capitalize()}:[success]OK (Block: {block_num:,})[/success]")
            progress.update(task_id, description=f"[info]Connecting... ({success_count}/{total_networks} OK)")
        except Exception as e:
            error_short = type(e).__name__
            # Add more detail for common errors
            if "Max retries exceeded" in str(e): error_short = "ConnectionTimeout"
            elif "Failed to establish a new connection" in str(e): error_short = "ConnectionRefused"
            network_statuses.append(f"{network.capitalize()}:[error]{error_short}[/error]")
            w3_clients[network] = None
            progress.update(task_id, description=f"[warning]Failed {network.capitalize()} ({error_short})...")
            console.print(f"[warning]Web3 connection error for {network.capitalize()}: {e}", style="warning")


    final_web3_status = f"[success]Web3 Clients Initialized ({success_count}/{total_networks} OK)"
    if success_count == 0:
        final_web3_status = "[error]Web3 Client Init Failed (All Networks)"
    elif success_count < total_networks:
         final_web3_status = f"[warning]Web3 Clients Initialized ({success_count}/{total_networks} OK)"
    progress.update(task_id, description=final_web3_status)

    # Return True if mainnet client initialized successfully
    return w3_clients.get('mainnet') is not None

# =============================================================================================
# Main Execution: System Initialization and Status Check (with Progress)
# =============================================================================================

# --- Title Display ---
console.print("\n✨ Configuration and Authentication ✨", style="bold cyan3")
console.print("───────────────────────────────────────", style="cyan3")

# Initialize status variables
gcp_auth_success = False
any_web3_success = False
mainnet_ready = False

# Use Rich Progress for initialization steps
# Set transient=True to make the progress bar disappear on completion
with Progress(
    SpinnerColumn(),
    TextColumn("[progress.description]{task.description}"),
    TimeElapsedColumn(),
    console=console, # Ensure progress uses the themed console
    transient=True # Make spinner disappear when done
) as progress:
    # Add tasks for each step (total=1 means they complete in one update)
    auth_task = progress.add_task("GCP Authentication...", total=1)
    web3_task = progress.add_task("Initializing Web3 Clients...", total=1)

    # --- Run Initialization Steps (functions update progress description) ---
    gcp_auth_success = authenticate_gcp(progress, auth_task)
    progress.update(auth_task, completed=1) # Mark as done

    if rpc_urls_valid:
        any_web3_success = initialize_all_web3_clients(progress, web3_task)
    else:
        progress.update(web3_task, description="[error]Skipped Web3 Init (Invalid URLs)")
    progress.update(web3_task, completed=1)


# --- Get Final Status ---
mainnet_ready = any_web3_success # mainnet_ready directly reflects if mainnet client succeeded
overall_success = gcp_auth_success and mainnet_ready

# --- Display Intermediate Success Messages (Now that progress is done) ---
if gcp_auth_success:
    console.print(f"✓ User authentication successful!", style="success")
    if gc_sheets: console.print(f"✓ Google Sheets client initialized", style="success")
else:
    console.print(f"❌ GCP authentication failed.", style="error")

# Updated Web3 success message based on w3_clients dictionary
if w3_clients:
    connected_nets = [net.capitalize() for net, client in w3_clients.items() if client]
    if connected_nets:
        console.print(f"✓ Web3 clients connected: {', '.join(connected_nets)}", style="success")
    elif rpc_urls_valid: # Only show warning if init was attempted but failed all
         console.print("[warning]Web3 clients initialized, but none connected successfully.", style="warning")
else:
     # This case occurs if rpc_urls_valid was False
     console.print("[error]Web3 client initialization skipped due to invalid RPC URLs.", style="error")


# --- Display Final System Status Summary ---
console.print("\n\n📊 System Status Summary", style="highlight")
# RPC Status
if w3_clients:
    for network, client in w3_clients.items():
        connected = client is not None
        status_msg = "[red]Failed/Skipped[/red]" # Default if not connected
        if connected:
             try:
                 block_num = client.eth.block_number # Get block number again for final status
                 block_num_str = f"(Block #{block_num:,})"
                 status_msg = f"[green]Connected[/green] {block_num_str}"
             except Exception as e:
                  # If client exists but fails here, mark as unresponsive
                  status_msg = f"[orange3]Unresponsive ({type(e).__name__})[/orange3]"
                  w3_clients[network] = None # Clear bad client for consistency
                  console.print(f"[warning]RPC ({network.capitalize()}) became unresponsive: {e}", style="warning")
        else:
            # Add reason if skipped due to invalid URL during init
            original_url = BLOCKCHAIN_RPC.get('ethereum', {}).get(network, "")
            if not original_url or "xxx" in original_url.lower():
                 status_msg = "[red]Skipped (Invalid URL)[/red]"

        console.print(f"  • Ethereum RPC ({network.capitalize()}): {status_msg}")
elif not rpc_urls_valid:
    console.print("  • Ethereum RPC: [red]Skipped (Invalid URLs)[/red]")
else:
    console.print("  • Ethereum RPC: [red]Initialization Failed[/red]")

# GCP Status
status_auth = "[green]Successful[/green]" if gcp_auth_success else "[red]Failed[/red]"
console.print(f"  • GCP Authentication: {status_auth}")

# Google Sheets Status
status_gs = "[green]Initialized[/green]" if gc_sheets else "[red]Not Initialized[/red]"
if not gcp_auth_success: # Also mark skipped if auth failed
    status_gs = "[yellow]Skipped[/yellow]"
console.print(f"  • Google Sheets Client: {status_gs}")

# --- Final Ready/Failure Message with Checkmark ---
if overall_success:
    console.print("\n\n[bold green]✓ Configuration Complete:[/bold green] System Ready for Ethereum Blockchain Analytics (via RPC) and Google Sheets")
    # Optional warnings for testnets can still be useful
    if 'holesky' in w3_clients and w3_clients.get('holesky') is None: console.print("  [warning](Note: Holesky testnet connection failed/skipped/unresponsive)", style="warning")
    if 'sepolia' in w3_clients and w3_clients.get('sepolia') is None: console.print("  [warning](Note: Sepolia testnet connection failed/skipped/unresponsive)", style="warning")
else:
     failure_reasons = []
     if not gcp_auth_success: failure_reasons.append("GCP Auth Failed")
     if not w3_clients.get('mainnet'): failure_reasons.append("Mainnet RPC Connection Failed/Skipped")
     if gcp_auth_success and not gc_sheets: failure_reasons.append("Google Sheets Client Failed") # Check if GSheets failed despite auth success
     if not rpc_urls_valid: failure_reasons.append("Invalid RPC URL Placeholders")


     reason_str = ', '.join(failure_reasons) if failure_reasons else "Unknown Issues"
     console.print(f"\n\n[bold red]❌ Configuration Failed:[/bold red] System setup encountered issues ({reason_str}). Review status messages above.")

## 1.1 🎯 Analysis Targets & Utility Functions
---

This cell handles target validation and prepares the essential utilities needed for transaction/block analysis:

1.  **🌐 Network Verification:**
    *   Verifies connectivity to Ethereum Mainnet and available testnets
    *   Displays current block heights and chain IDs
    *   Attempts reconnection if mainnet client is unavailable

2.  **🎯 Define Analysis Targets:**
    *   Set the specific Ethereum Mainnet **transaction hash** (`TARGET_TX_HASH`) or **block identifier** (`TARGET_BLOCK_IDENTIFIER`) you wish to analyze
    *   **ACTION:** Modify these values to analyze different transactions or blocks relevant to PYUSD
    *   Targets are automatically validated for proper format and existence on-chain

3.  **🔧 Helper Function Library:**
    *   Initializes essential functions used throughout the notebook for:
        *   Making raw RPC requests (`make_rpc_request`) to Ethereum nodes
        *   Decoding PYUSD-specific function calls and events
        *   Formatting blockchain values (ETH/PYUSD amounts, gas)
        *   Handling addresses and transaction data
        *   Creating visualizations for transaction analysis

> **Note:** If a target validation fails, specific diagnostic information will be displayed to help troubleshoot the issue.

In [ ]:
# =============================================================================================
# 🎯 Target Selection & Helper Functions for Tracing
# =============================================================================================
# This cell validates targets, initializes tracing configurations, and sets up helper functions.

# Ensure web3 clients dictionary is loaded from the previous cell
if 'w3_clients' not in locals() or not isinstance(w3_clients, dict):
     raise NameError("Web3 clients dictionary 'w3_clients' not initialized. Please run '🔑 Configuration & Authentication: Connecting to GCP and Ethereum RPC' Cell")

# --- Network Status Verification ---
with Progress(
    SpinnerColumn(),
    TextColumn("[progress.description]{task.description}"),
    console=console,
    transient=True
) as progress:
    verification_task = progress.add_task("[info]Verifying network connections...", total=1)

    w3_mainnet = w3_clients.get('mainnet')
    if not w3_mainnet or not w3_mainnet.is_connected():
         # Attempt re-initialization with visual feedback
         progress.update(verification_task, description="[warning]⚠️ Mainnet client not found. Attempting re-initialization...")
         web3_task = progress.add_task("[info]Reinitializing Web3 clients...", total=1)
         initialize_all_web3_clients(progress, web3_task)
         progress.update(web3_task, completed=1)

         # Check if reconnection succeeded
         w3_mainnet = w3_clients.get('mainnet')
         if not w3_mainnet or not w3_mainnet.is_connected():
              raise ConnectionError("Cannot proceed without a connected Mainnet Web3 client. Check your RPC endpoints.")

    # Get testnet client if available (for testing function implementations)
    progress.update(verification_task, description="[info]Checking for testnet availability...")
    w3_testnet = w3_clients.get('sepolia') or w3_clients.get('holesky') # Prefer Sepolia if available
    if w3_testnet and w3_testnet.is_connected():
        # Find the network name ('holesky' or 'sepolia')
        testnet_name = next((name for name in ['sepolia', 'holesky'] if w3_clients.get(name) == w3_testnet), None)
    else:
        testnet_name = None # No connections with testnet

    progress.update(verification_task, completed=1)

# --- Network Status Table ---
console.print("[bold cyan3 size=20]🌐 Network Connections[/]", justify="left")
console.print("────────────────────────", style="cyan3")

network_table = Table(show_header=True, header_style="bold cyan3")
network_table.add_column("Network", style="dim")
network_table.add_column("Status", justify="center")
network_table.add_column("Block Height", justify="right")
network_table.add_column("Chain ID", justify="right")

# Add Mainnet info
latest_block_mainnet = w3_mainnet.eth.block_number if w3_mainnet else "N/A"
chain_id_mainnet = w3_mainnet.eth.chain_id if w3_mainnet else "N/A"
network_table.add_row(
    "Ethereum Mainnet",
    "[spring_green3]Connected[/spring_green3]" if w3_mainnet and w3_mainnet.is_connected() else "[red3]Disconnected[/red3]",
    f"{latest_block_mainnet:,}" if isinstance(latest_block_mainnet, int) else str(latest_block_mainnet),
    str(chain_id_mainnet)
)

# Add Testnet info if available
if testnet_name:
    latest_block_testnet = w3_testnet.eth.block_number if w3_testnet else "N/A"
    chain_id_testnet = w3_testnet.eth.chain_id if w3_testnet else "N/A"
    network_table.add_row(
        f"{testnet_name.capitalize()} Testnet",
        "[spring_green3]Connected[/spring_green3]" if w3_testnet and w3_testnet.is_connected() else "[red3]Disconnected[/red3]",
        f"{latest_block_testnet:,}" if isinstance(latest_block_testnet, int) else str(latest_block_testnet),
        str(chain_id_testnet)
    )
else:
    network_table.add_row("Testnet", "[gold3]Not Available[/gold3]", "N/A", "N/A")

# Display the network connection table
console.print(network_table)
console.print("\n\n")  # Empty lines for better spacing

############################################################
# 🎯 DEFINE YOUR ETHEREUM ANALYSIS TARGET
# Set BOTH values below for optimal analysis capabilities.
############################################################

# Transaction Hash for analysis
# Required by certain analysis functions.
TARGET_TX_HASH = "YOUR_TARGET_TX_HASH"

# Block Number/Identifier for analysis
# Required by other analysis functions (often related to block context).
# Use the block containing the target transaction, or the specific block you want to analyze.
TARGET_BLOCK_IDENTIFIER = YOUR_TARGET_BLOCK_NUMBER # Or block hash, "latest", etc.

# Important Notes:
# - Full analysis experience requires both TARGET_TX_HASH and TARGET_BLOCK_IDENTIFIER.
# - Some functions specifically need the block context, others the transaction details.
# - If both values are set, TARGET_TX_HASH takes precedence in situations where
#   only one identifier can be used as the primary target.
############################################################

# Import necessary packages for helper functions
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
from graphviz import Digraph
from IPython.display import display, Javascript
from hexbytes import HexBytes
from rich.panel import Panel
from rich.syntax import Syntax
from rich.table import Table
import json
from datetime import datetime

# Plotly configuration for Colab
try:
    display(Javascript('''
        require.config({
            paths: {
                plotly: 'https://cdn.plot.ly/plotly-latest.min.js'
            }
        });
    '''))
except Exception as e:
    console.print(f"[warning]Could not re-configure Plotly: {e}", style="warning")

# --- Target Validation Functions ---
def validate_tx_hash(tx_hash):
    """Validates a transaction hash with detailed diagnostics"""
    if not tx_hash:
        return {
            'valid': False,
            'status': "Missing",
            'status_color': "red3",
            'details': "Transaction hash not provided"
        }

    if not isinstance(tx_hash, str):
        return {
            'valid': False,
            'status': "Invalid Type",
            'status_color': "red3",
            'details': f"Expected string, got {type(tx_hash).__name__}"
        }

    if not tx_hash.startswith('0x'):
        return {
            'valid': False,
            'status': "Invalid Format",
            'status_color': "red3",
            'details': "Transaction hash must start with '0x'"
        }

    if len(tx_hash) < 66:
        return {
            'valid': False,
            'status': "Too Short",
            'status_color': "red3",
            'details': f"Length is {len(tx_hash)}, should be 66 characters"
        }

    if len(tx_hash) > 66:
        return {
            'valid': False,
            'status': "Too Long",
            'status_color': "red3",
            'details': f"Length is {len(tx_hash)}, should be 66 characters (including '0x')"
        }

    # Check if characters are valid hex
    try:
        int(tx_hash[2:], 16)
    except ValueError:
        return {
            'valid': False,
            'status': "Invalid Hex",
            'status_color': "red3",
            'details': "Contains non-hexadecimal characters"
        }

    # At this point, the format is valid, try to retrieve it
    try:
        tx_receipt = w3_mainnet.eth.get_transaction_receipt(tx_hash[:66])  # Truncate to valid length for retrieval
        tx_details = w3_mainnet.eth.get_transaction(tx_hash[:66])

        # Format gas info
        gas_used = tx_receipt.get('gasUsed', 0)
        gas_limit = tx_details.get('gas', 0)
        gas_percentage = (gas_used / gas_limit * 100) if gas_limit else 0

        # Check if transaction was successful
        is_success = tx_receipt.get('status') == 1

        return {
            'valid': True,
            'status': "Found" if is_success else "Failed Transaction",
            'status_color': "spring_green3" if is_success else "gold3",
            'details': f"Block: {tx_receipt.get('blockNumber')}, Gas: {gas_used:,}/{gas_limit:,} ({gas_percentage:.1f}%)",
            'data': {
                'receipt': tx_receipt,
                'transaction': tx_details,
                'success': is_success
            }
        }
    except Exception as e:
        return {
            'valid': True,  # Format is valid but transaction not found
            'status': "Valid Format, Not Found",
            'status_color': "gold3",
            'details': f"Error: {str(e)}"  # No truncation
        }

def validate_block_identifier(block_id):
    """Validates a block identifier with detailed diagnostics"""
    if block_id is None:
        return {
            'valid': False,
            'status': "Missing",
            'status_color': "red3",
            'details': "Block identifier not provided"
        }

    # For integer block numbers
    if isinstance(block_id, int):
        if block_id < 0:
            return {
                'valid': False,
                'status': "Negative Number",
                'status_color': "red3",
                'details': "Block number cannot be negative"
            }

        # Check if block is within realistic range
        latest_block = w3_mainnet.eth.block_number if w3_mainnet else None
        if latest_block and block_id > latest_block:
            return {
                'valid': False,
                'status': "Future Block",
                'status_color': "red3",
                'details': f"Block {block_id:,} exceeds current mainnet height ({latest_block:,})"
            }

        # Block is valid by format, try to retrieve it
        try:
            block_details = w3_mainnet.eth.get_block(block_id)

            # Format timestamp from Unix timestamp
            block_time = datetime.fromtimestamp(block_details.timestamp)

            return {
                'valid': True,
                'status': "Found",
                'status_color': "spring_green3",
                'details': f"Time: {block_time.strftime('%Y-%m-%d %H:%M:%S')}, TX Count: {len(block_details['transactions'])}",
                'data': block_details
            }
        except Exception as e:
            return {
                'valid': True,  # Format is valid but block not found
                'status': "Valid Format, Not Found",
                'status_color': "gold3",
                'details': f"Error: {str(e)}"  # No truncation
            }

    # For hex string or hash
    if isinstance(block_id, str):
        if not block_id.startswith('0x'):
            return {
                'valid': False,
                'status': "Invalid Format",
                'status_color': "red3",
                'details': "Block hash must start with '0x'"
            }

        # Check if characters are valid hex
        try:
            int(block_id[2:], 16)
        except ValueError:
            return {
                'valid': False,
                'status': "Invalid Hex",
                'status_color': "red3",
                'details': "Contains non-hexadecimal characters"
            }

        # Block hash is valid by format, try to retrieve it
        try:
            block_details = w3_mainnet.eth.get_block(block_id)

            # Format timestamp from Unix timestamp
            block_time = datetime.fromtimestamp(block_details.timestamp)

            return {
                'valid': True,
                'status': "Found",
                'status_color': "spring_green3",
                'details': f"Time: {block_time.strftime('%Y-%m-%d %H:%M:%S')}, TX Count: {len(block_details['transactions'])}",
                'data': block_details
            }
        except Exception as e:
            return {
                'valid': True,  # Format is valid but block not found
                'status': "Valid Format, Not Found",
                'status_color': "gold3",
                'details': f"Error: {str(e)}"  # No truncation
            }

    # If not int or str
    return {
        'valid': False,
        'status': "Invalid Type",
        'status_color': "red3",
        'details': f"Expected int or string, got {type(block_id).__name__}"
    }

# --- Target Verification & Information Retrieval with progress indicator ---
with Progress(
    SpinnerColumn(),
    TextColumn("[progress.description]{task.description}"),
    console=console,
    transient=True
) as progress:
    validate_task = progress.add_task("[info]Validating analysis targets...", total=2)

    # Validate transaction hash
    progress.update(validate_task, description="[info]Analyzing transaction hash...")
    if 'TARGET_TX_HASH' in locals() and TARGET_TX_HASH:
        tx_validation = validate_tx_hash(TARGET_TX_HASH)
        tx_valid = tx_validation['valid']
    else:
        tx_valid = False
        tx_validation = {
            'status': "Skipped",
            'status_color': "gold3",
            'details': "No transaction hash provided"
        }

    progress.update(validate_task, advance=1)

    # Validate block identifier
    progress.update(validate_task, description="[info]Analyzing block identifier...")
    if 'TARGET_BLOCK_IDENTIFIER' in locals() and TARGET_BLOCK_IDENTIFIER is not None:
        block_validation = validate_block_identifier(TARGET_BLOCK_IDENTIFIER)
        block_valid = block_validation['valid']
    else:
        block_valid = False
        block_validation = {
            'status': "Skipped",
            'status_color': "gold3",
            'details': "No block identifier provided"
        }

    progress.update(validate_task, completed=1)

# --- Target Status Display (Key-Value Format) ---
console.print("[bold royal_blue1 size=20]🎯 Analysis Targets[/]", justify="left")
console.print("────────────────────", style="royal_blue1")
console.print("\n")  # Extra spacing

# Transaction Target
console.print("[bold]Target Type - Transaction Hash[/bold]")
if 'TARGET_TX_HASH' in locals() and TARGET_TX_HASH:
    console.print(f"[dim]Value:[/dim] [cyan3]{TARGET_TX_HASH}[/cyan3]")
else:
    console.print("[dim]Value:[/dim] Nil")

console.print(f"[dim]Status:[/dim] [{tx_validation['status_color']}]{tx_validation.get('status', 'Unknown')}[/{tx_validation['status_color']}]")
console.print(f"[dim]Diagnostics:[/dim] {tx_validation.get('details', 'No information available')}")

# Separator
console.print("\n" + "─" * 50 + "\n")

# Block Target
console.print("[bold]Target Type - Block[/bold]")
if 'TARGET_BLOCK_IDENTIFIER' in locals() and TARGET_BLOCK_IDENTIFIER is not None:
    console.print(f"[dim]Value:[/dim] [cyan3]{TARGET_BLOCK_IDENTIFIER}[/cyan3]")
else:
    console.print("[dim]Value:[/dim] Nil")

console.print(f"[dim]Status:[/dim] [{block_validation['status_color']}]{block_validation.get('status', 'Unknown')}[/{block_validation['status_color']}]")
console.print(f"[dim]Diagnostics:[/dim] {block_validation.get('details', 'No information available')}")

console.print("\n")  # Extra spacing

# Show target selection status message
if not tx_valid and not block_valid:
    console.print("[error]⚠️ No valid targets found. Please set either TARGET_TX_HASH or TARGET_BLOCK_IDENTIFIER.[/error]", style="bold red3")
elif tx_valid and block_valid:
    # Check if both targets exist but the transaction validation found the transaction and the block validation found the block
    if tx_validation.get('status') == "Found" and block_validation.get('status') == "Found":
        console.print("[info]ℹ️ Both transaction and block targets are found. Transaction analysis will take priority.[/info]", style="bold royal_blue1")
    else:
        console.print("[info]ℹ️ Both transaction and block targets are provided but at least one wasn't found. Valid target will be used.[/info]", style="bold royal_blue1")
elif tx_valid:
    if tx_validation.get('status') == "Found":
        console.print("[info]✓ Transaction target is valid and found. Ready for analysis.[/info]", style="bold spring_green3")
    else:
        console.print("[warning]⚠️ Transaction target format is valid but transaction wasn't found. Check the hash.[/warning]", style="bold gold3")
elif block_valid:
    if block_validation.get('status') == "Found":
        console.print("[info]✓ Block target is valid and found. Ready for analysis.[/info]", style="bold spring_green3")
    else:
        console.print("[warning]⚠️ Block target format is valid but block wasn't found. Check the number/hash.[/warning]", style="bold gold3")

console.print("\n")  # Empty line for better spacing

# --- Helper Functions ---
def make_rpc_request(method, params, network='mainnet'):
    """Helper function to make raw RPC requests via the specified network's provider."""
    w3_client = w3_clients.get(network)
    if not w3_client or not w3_client.is_connected():
        console.print(f"[error]Web3 client for '{network}' not available or not connected.", style="error")
        return None
    try:
        response = w3_client.provider.make_request(method, params)
        if 'error' in response:
            console.print(f"[error]RPC Error ({method} on {network}): {response['error']['message']} (Code: {response['error']['code']})", style="error")
            return None
        return response.get('result')
    except Exception as e:
        console.print(f"[error]Exception during RPC call ({method} on {network}): {str(e)}", style="error")
        return None

# --- PYUSD Contract Helpers ---
def is_pyusd_contract(address):
    """Checks if an address is a known PYUSD contract."""
    if not address:
        return False
    address_lower = address.lower() if isinstance(address, str) else str(address).lower()
    return address_lower in PYUSD_CONTRACTS

def get_contract_name(address):
    """Gets the friendly name for a contract address."""
    if not address:
        return "Unknown"
    address_lower = address.lower() if isinstance(address, str) else str(address).lower()
    return PYUSD_CONTRACTS.get(address_lower, "Other Contract")

def decode_pyusd_function(input_data):
    """Decodes PYUSD function calls using the signature registry."""
    if not input_data or input_data == '0x':
        return "Empty call data"

    method_sig = input_data[:10]
    if method_sig in PYUSD_SIGNATURES:
        return PYUSD_SIGNATURES[method_sig]["name"]
    return f"Unknown function: {method_sig}"

def get_function_category(input_data):
    """Gets the category of a function from its input data."""
    if not input_data or input_data == '0x':
        return "other"

    method_sig = input_data[:10]
    if method_sig in PYUSD_SIGNATURES:
        return PYUSD_SIGNATURES[method_sig]["category"]
    return "other"

def decode_pyusd_event(topic0, topics, data):
    """Decodes PYUSD events using the event registry."""
    if not topic0 or topic0 not in PYUSD_EVENTS:
        return {"name": "Unknown event", "details": "Cannot decode"}

    event_info = PYUSD_EVENTS[topic0]

    try:
        decoded = event_info["decoder"](topics, data)
        return {
            "name": event_info["name"],
            "decoded": decoded
        }
    except Exception as e:
        return {
            "name": event_info["name"],
            "error": str(e)
        }

# --- Formatting Helpers ---
def format_value_pyusd(value_raw):
    """Formats raw PYUSD value (int) to decimal string."""
    if value_raw is None: return "0 PYUSD"
    try:
        decimals = PYUSD_CONFIG['ethereum']['decimals']
        value_float = int(value_raw) / (10**decimals)
        return f"{value_float:,.{decimals}f} PYUSD"
    except (ValueError, TypeError):
        return "Invalid PYUSD Value"

def format_value_eth(value_wei):
    """Formats Wei value to Ether string."""
    if value_wei is None: return "0 ETH"
    try:
        # Ensure value_wei is int or can be converted (handle hex strings from traces)
        if isinstance(value_wei, str):
            value_int = int(value_wei, 16)
        else:
             value_int = int(value_wei)
        # Use mainnet client for formatting ETH value regardless of target network
        if w3_mainnet: # Check if mainnet client exists
            return f"{w3_mainnet.from_wei(value_int, 'ether'):.6f} ETH"
        else:
            return f"{value_int / 1e18:.6f} ETH (approx)" # Fallback if mainnet client missing
    except (ValueError, TypeError, AttributeError):
        return "Invalid ETH Value"

def format_gas(gas):
    """Formats gas value (int or hex string)."""
    if gas is None: return "N/A"
    try:
       gas_int = int(gas, 16) if isinstance(gas, str) and gas.startswith('0x') else int(gas)
       return f"{gas_int:,}"
    except (ValueError, TypeError):
        return "Invalid Gas Value"

def shorten_address(address):
    """Shortens an Ethereum address for display."""
    if not isinstance(address, str) or not address.startswith('0x') or len(address) != 42:
        return str(address) # Return original if not a valid address string
    return f"{address[:6]}...{address[-4:]}"

def display_json(data, title="JSON Output"):
    """Pretty prints JSON data using Rich."""
    if data is None:
        console.print(f"[warning]{title}: No data to display.", style="warning")
        return
    try:
        # Use default=str to handle potential non-serializable types like HexBytes
        json_str = json.dumps(data, indent=2, default=str)
        console.print(Panel(Syntax(json_str, "json", theme="default", line_numbers=False),
                      title=title, border_style="cyan3", expand=False))
    except Exception as e:
        console.print(f"[error]Could not display JSON for {title}: {e}", style="error")
        # Fallback to printing raw data (limited length)
        try:
            raw_str = str(data)
            console.print(raw_str[:1000] + ("..." if len(raw_str) > 1000 else ""))
        except Exception:
            console.print("[error]Could not print raw data fallback.", style="error")

# --- Visualization Helpers ---
def create_gas_usage_chart(gas_data, title="Gas Usage Distribution"):
    """Creates a pie chart showing gas usage distribution."""
    try:
        fig = px.pie(gas_data, values='gas_used', names='category',
                      title=title)
        fig.update_layout(template="plotly_white")
        return fig
    except Exception as e:
        console.print(f"[warning]Could not create gas usage chart: {e}", style="warning")
        return None

def create_call_sequence_chart(calls_df, highlight_pyusd=True):
    """Creates a bar chart showing the sequence of calls with gas usage."""
    try:
        if highlight_pyusd:
            fig = px.bar(calls_df, x='id', y='gasUsed', color='is_pyusd',
                         title='Gas Usage by Call Sequence',
                         hover_data=['type', 'depth', 'from', 'to'])
        else:
            fig = px.bar(calls_df, x='id', y='gasUsed',
                         title='Gas Usage by Call Sequence',
                         hover_data=['type', 'depth', 'from', 'to'])

        fig.update_layout(template="plotly_white")
        return fig
    except Exception as e:
        console.print(f"[warning]Could not create call sequence chart: {e}", style="warning")
        return None

# --- Final Tracing Configuration Setup ---
# Set up trace configuration based on targets
if tx_valid and tx_validation.get('status') == "Found":
    # If valid transaction hash found, prepare for transaction tracing
    ACTIVE_TRACE_CONFIG = DEFAULT_TRACE_CONFIG.copy()
    console.print(f"[info]Trace configuration ready for transaction analysis", style="info")
elif block_valid and block_validation.get('status') == "Found":
    # If valid block found, prepare for block tracing
    ACTIVE_TRACE_CONFIG = STRUCTLOG_TRACE_CONFIG.copy()
    console.print(f"[info]Trace configuration ready for block analysis", style="info")
else:
    # If no valid targets, set a default trace config
    ACTIVE_TRACE_CONFIG = DEFAULT_TRACE_CONFIG.copy()
    console.print(f"[warning]Using default trace configuration (no valid target found)", style="warning")

# Final status message
status_icon = "✓" if (tx_valid and tx_validation.get('status') == "Found") or (block_valid and block_validation.get('status') == "Found") else "⚠️"
status_style = "bold spring_green3" if status_icon == "✓" else "bold gold3"
console.print(f"[{status_style}]{status_icon} Target selection and helper functions initialized[/{status_style}]")

## 1.2 🔍 `debug_traceTransaction` - Deep Dive into Transaction Execution
---
This section utilizes the powerful `debug_traceTransaction` RPC method to dissect the internal workings of a specific PYUSD transaction defined by `TARGET_TX_HASH`. This goes far beyond standard block explorers by revealing the step-by-step execution flow within the EVM. We will explore two main tracers: `callTracer` and `structLog`.

### 1.2.1 Using `callTracer`: Mapping Internal Calls, Gas & Events (Recommended)

The `callTracer` provides a structured, hierarchical view of the transaction's execution flow. It's generally the most useful tracer for understanding high-level interactions, gas consumption patterns, and event emissions.

> **🚀 Leveraging GCP's Premium RPC Capabilities**
>
> *   **Method:** `debug_traceTransaction` with `tracer: "callTracer"`
> *   **Multiplier:** `50x` (Consumes 50x the quota/cost of a basic call)
> *   **GCP Advantage:** Performing this detailed trace is computationally intensive. GCP's generous free quotas for this high-multiplier method make such in-depth analysis accessible and cost-effective.
> *   **PYUSD Insight:** `callTracer` allows us to:
>     *   Visualize the **exact interaction path** when PYUSD interacts with other DeFi protocols (e.g., DEXs, lending platforms).
>     *   Identify specific **internal PYUSD function calls** (`transfer`, `approve`, `mint`, `burn`) and their parameters within the overall transaction.
>     *   Pinpoint **gas consumption** within specific PYUSD operations vs. external contract interactions.
>     *   Verify the **emission and content** of crucial PYUSD events like `Transfer` and `Approval`.

**Analysis Workflow:**

1.  **Fetch Trace:** The code calls `debug_traceTransaction` using the `TARGET_TX_HASH` and the `callTracer` configuration.
2.  **Parse Data:** The complex JSON response is processed by `parse_call_trace` to extract structured information about calls, logs, gas, and state changes.
3.  **Visualize & Summarize:** The results are presented through:
    *   **Trace Overview & Metrics:** Key stats like call count, depth, gas, and status.
    *   **Interaction Graphs:** High-level contract interactions and detailed call sequences (Interactive Plotly). PYUSD calls are highlighted.
    *   **Token Flow:** A specific graph visualizing PYUSD movements (transfers, mints, burns).
    *   **State & Gas Analysis:** Tables and charts showing PYUSD state changes and gas usage breakdown.
    *   **Event Log Analysis:** Decoded PYUSD events emitted.
    *   **Data Tables:** DataFrames showing key calls and high gas usage operations.
    *   **Recommendations:** Automated observations based on trace patterns.
    *   **Export Options:** Download parsed data (CSV/JSON) or export a report to Google Sheets (Colab).

**💡 What to Look For:**
*   **Call Graph:** Observe the sequence and depth of calls. Identify the green/blue highlighted nodes representing PYUSD/Supply Controller interactions. Note the gas usage (`Gas: ...`) on each node.
*   **Token Flow Graph:** Track how PYUSD moved between addresses.
*   **Gas Usage Pie Chart:** See which *types* of operations (transfer, approval, supply change) consumed the most gas.
*   **Event Table:** Correlate events like `Transfer` with the calls shown in the graph.
*   **Recommendations:** Check for automated insights about gas or complexity.

In [ ]:
# =============================================================================================
# 🔬 Trace Transaction using debug_traceTransaction (callTracer)
# =============================================================================================
# This cell validates transaction targets, initializes blockchain tracing configurations, and sets up helper functions.
# It prepares the environment for detailed transaction analysis by:
# - Validating the transaction hash format and existence
# - Configuring tracer parameters for debug_traceTransaction
# - Setting up utility functions for address formatting, value conversion, and data processing
# - Initializing transaction-specific constants and analysis options

import base64
import json
import os
from datetime import datetime
from IPython.display import HTML, display
import ipywidgets as widgets
from IPython.display import clear_output
from IPython.display import Javascript
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import networkx as nx
from rich.console import Console
from rich.table import Table
from rich.panel import Panel

def download_csv_direct(df, filename=None):
    """Creates a direct download for CSV without intermediate display."""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"pyusd_data_{timestamp}.csv"

    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()

    # Create direct download HTML
    html = f'''
    <script>
    function download(filename, data) {{
        const a = document.createElement('a');
        a.href = data;
        a.download = filename;
        document.body.appendChild(a);
        a.click();
        document.body.removeChild(a);
    }}

    download("{filename}", "data:text/csv;base64,{payload}");
    </script>
    <div>Downloading {filename}...</div>
    '''
    return HTML(html)

def download_json_direct(data, filename=None):
    """Creates a direct download for JSON without intermediate display."""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"pyusd_data_{timestamp}.json"

    # Convert to JSON string (handling non-serializable objects)
    json_str = json.dumps(data, default=str, indent=2)
    b64 = base64.b64encode(json_str.encode()).decode()

    # Create direct download HTML
    html = f'''
    <script>
    function download(filename, data) {{
        const a = document.createElement('a');
        a.href = data;
        a.download = filename;
        document.body.appendChild(a);
        a.click();
        document.body.removeChild(a);
    }}

    download("{filename}", "data:application/json;base64,{b64}");
    </script>
    <div>Downloading {filename}...</div>
    '''
    return HTML(html)

def export_to_google_sheets(df, data_dict, tx_hash):
    """Export analysis data to Google Sheets with rich formatting and visualization references."""
    # Show loading message
    console.print("[cyan3]Exporting to Google Sheets...", style="info")

    try:
        # Create a new Google Sheet with meaningful title
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        sheet_title = f"PYUSD TX Analysis {tx_hash[:10]} {timestamp}"

        # Use the global gc_sheets client that's already authenticated
        spreadsheet = gc_sheets.create(sheet_title)

        # Get the default worksheet and rename it
        worksheet = spreadsheet.get_worksheet(0)
        worksheet.update_title("Transaction Analysis")

        # Set up a header with transaction info
        header_values = [
            ["PYUSD Transaction Analysis"],
            [f"Transaction: {tx_hash}"],
            [f"Analysis Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"],
            [""],  # Empty row for spacing
        ]
        worksheet.update("A1", header_values)

        # Format the header with bold text and colored background
        worksheet.format("A1:A1", {
            "textFormat": {"bold": True, "fontSize": 14},
            "backgroundColor": {"red": 0.9, "green": 0.9, "blue": 1.0}
        })

        worksheet.format("A2:A3", {
            "textFormat": {"bold": True, "fontSize": 12}
        })

        current_row = 5  # Start after header

        # 1. Add transaction stats summary with improved formatting
        if "transaction_stats" in data_dict:
            stats = data_dict["transaction_stats"]

            # Add section title
            worksheet.update(f"A{current_row}", [["Transaction Metrics"]])
            worksheet.format(f"A{current_row}:A{current_row}", {
                "textFormat": {"bold": True, "fontSize": 12},
                "backgroundColor": {"red": 0.8, "green": 0.9, "blue": 1.0}
            })
            current_row += 1

            # Add stats data
            stats_rows = []
            stats_rows.append(["Metric", "Value"])  # Header row
            for key, value in stats.items():
                # Format keys and values appropriately
                formatted_key = key.replace("_", " ").title()
                if key == 'total_gas':
                    formatted_value = f"{value:,} gas units"
                else:
                    formatted_value = str(value)
                stats_rows.append([formatted_key, formatted_value])

            # Add stats table
            stats_start_row = current_row
            worksheet.update(f"A{stats_start_row}", stats_rows)

            # Format stats table header
            worksheet.format(f"A{stats_start_row}:B{stats_start_row}", {
                "textFormat": {"bold": True},
                "backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}
            })

            current_row += len(stats_rows) + 1  # Add space after table

        # 2. Add Contract Interaction Graph reference
        worksheet.update(f"A{current_row}", [["Contract Interaction Overview"]])
        worksheet.format(f"A{current_row}:A{current_row}", {
            "textFormat": {"bold": True, "fontSize": 12},
            "backgroundColor": {"red": 0.8, "green": 0.8, "blue": 1.0}
        })
        current_row += 1

        worksheet.update(f"A{current_row}", [["📊 Contract interaction visualization is available in the notebook"]])
        current_row += 2

        # 3. Add Call Graph Visualization reference
        worksheet.update(f"A{current_row}", [["Detailed Call Graph Visualization"]])
        worksheet.format(f"A{current_row}:A{current_row}", {
            "textFormat": {"bold": True, "fontSize": 12},
            "backgroundColor": {"red": 0.7, "green": 0.8, "blue": 1.0}
        })
        current_row += 1

        worksheet.update(f"A{current_row}", [["📊 Detailed call graph visualization is available in the notebook"]])
        current_row += 2

        # 4. Add gas usage section
        if "gas_distribution" in data_dict and data_dict["gas_distribution"]:
            gas_data = data_dict["gas_distribution"]

            # Add section title
            worksheet.update(f"A{current_row}", [["Gas Usage Analysis"]])
            worksheet.format(f"A{current_row}:A{current_row}", {
                "textFormat": {"bold": True, "fontSize": 12},
                "backgroundColor": {"red": 1.0, "green": 0.9, "blue": 0.7}
            })
            current_row += 1

            # Create gas usage table
            gas_header = ["Category", "Gas Used", "Percentage"]
            gas_rows = [gas_header]

            total_gas = sum(item["gas_used"] for item in gas_data)
            for item in gas_data:
                category = item["category"].replace("_", " ").title()
                gas_used = item["gas_used"]
                percentage = (gas_used / total_gas * 100) if total_gas > 0 else 0
                gas_rows.append([category, f"{gas_used:,}", f"{percentage:.1f}%"])

            gas_table_row = current_row
            worksheet.update(f"A{gas_table_row}", gas_rows)

            # Format gas table headers
            worksheet.format(f"A{gas_table_row}:C{gas_table_row}", {
                "textFormat": {"bold": True},
                "backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}
            })

            current_row += len(gas_rows) + 1

            # Add pie chart reference
            worksheet.update(f"A{current_row}", [["📊 Gas usage pie chart visualization is available in the notebook"]])
            current_row += 2

        # 5. Add PYUSD Token Flow section
        if "pyusd_transfers" in data_dict and data_dict["pyusd_transfers"]:
            transfers = data_dict["pyusd_transfers"]

            # Add section title
            worksheet.update(f"A{current_row}", [["PYUSD Token Flow Analysis"]])
            worksheet.format(f"A{current_row}:A{current_row}", {
                "textFormat": {"bold": True, "fontSize": 12},
                "backgroundColor": {"red": 0.7, "green": 1.0, "blue": 0.8}
            })
            current_row += 1

            # Add description
            worksheet.update(f"A{current_row}", [["This shows the movement of PYUSD tokens in this transaction."]])
            current_row += 1

            # Add transfer data as a table
            transfer_header = ["From", "To", "Amount", "Gas Used"]
            transfer_rows = [transfer_header]

            for transfer in transfers:
                from_addr = shorten_address(transfer["from"])
                to_addr = shorten_address(transfer["to"])
                amount = format_value_pyusd(transfer["amount"])
                gas = f"{transfer.get('gas_used', 0):,}"
                transfer_rows.append([from_addr, to_addr, amount, gas])

            transfer_table_row = current_row
            worksheet.update(f"A{transfer_table_row}", transfer_rows)

            # Format transfer table headers
            worksheet.format(f"A{transfer_table_row}:D{transfer_table_row}", {
                "textFormat": {"bold": True},
                "backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}
            })

            current_row += len(transfer_rows) + 1

            # Add flow graph reference
            worksheet.update(f"A{current_row}", [["🔄 Token flow visualization is available in the notebook"]])
            current_row += 2

        # 6. Add PYUSD State Changes
        if "state_changes" in data_dict and data_dict["state_changes"]:
            state_changes = data_dict["state_changes"]

            # Add section title
            worksheet.update(f"A{current_row}", [["PYUSD State Changes"]])
            worksheet.format(f"A{current_row}:A{current_row}", {
                "textFormat": {"bold": True, "fontSize": 12},
                "backgroundColor": {"red": 1.0, "green": 0.8, "blue": 1.0}
            })
            current_row += 1

            # Add description
            worksheet.update(f"A{current_row}", [["The following state changes occurred in PYUSD contracts:"]])
            current_row += 1

            # Create state changes table
            if isinstance(state_changes, list) and state_changes:
                # Define headers
                headers = ["contract", "function", "type", "amount", "gas_used"]
                display_headers = ["Contract", "Function", "Type", "Amount", "Gas Used"]

                # Create the table data
                state_rows = [display_headers]  # Header row
                for change in state_changes:
                    row_data = []
                    for key in headers:
                        if key == "amount" and "amount" in change:
                            # Format PYUSD amounts nicely
                            value = format_value_pyusd(change["amount"])
                        elif key == "gas_used":
                            value = f"{change.get(key, 0):,}"
                        else:
                            value = str(change.get(key, ""))
                        row_data.append(value)
                    state_rows.append(row_data)

                # Add to sheet
                state_start_row = current_row
                worksheet.update(f"A{state_start_row}", state_rows)

                # Format headers
                header_range = f"A{state_start_row}:E{state_start_row}"
                worksheet.format(header_range, {
                    "textFormat": {"bold": True},
                    "backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}
                })

                current_row += len(state_rows) + 1

        # 7. Add Event Analysis summary
        if "logs_df" in data_dict and isinstance(data_dict["logs_df"], pd.DataFrame) and not data_dict["logs_df"].empty:
            logs_df = data_dict["logs_df"]
            pyusd_logs = logs_df[logs_df['is_pyusd']] if 'is_pyusd' in logs_df.columns else pd.DataFrame()

            if not pyusd_logs.empty:
                # Add section title
                worksheet.update(f"A{current_row}", [["PYUSD Events Analysis"]])
                worksheet.format(f"A{current_row}:A{current_row}", {
                    "textFormat": {"bold": True, "fontSize": 12},
                    "backgroundColor": {"red": 0.7, "green": 1.0, "blue": 0.8}
                })
                current_row += 1

                # Add summary count
                worksheet.update(f"A{current_row}", [[f"Found {len(pyusd_logs)} PYUSD events in this transaction."]])
                current_row += 1

                # Add event counts if available
                if 'event_name' in pyusd_logs.columns:
                    event_counts = pyusd_logs['event_name'].value_counts()

                    # Create event counts table
                    event_table = [["Event Type", "Count"]]  # Header row
                    for event, count in event_counts.items():
                        event_table.append([event, str(count)])

                    # Add to sheet
                    event_table_row = current_row
                    worksheet.update(f"A{event_table_row}", event_table)

                    # Format headers
                    worksheet.format(f"A{event_table_row}:B{event_table_row}", {
                        "textFormat": {"bold": True},
                        "backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}
                    })

                    current_row += len(event_table) + 1

                # Add event details
                worksheet.update(f"A{current_row}", [["PYUSD Event Details:"]])
                worksheet.format(f"A{current_row}:A{current_row}", {"textFormat": {"bold": True}})
                current_row += 1

                # Display key event details
                event_cols = ['contract', 'event_name', 'details']
                event_cols = [col for col in event_cols if col in pyusd_logs.columns]

                if event_cols:
                    # Convert to list for sheet
                    event_data = [event_cols]  # Header row
                    for _, row in pyusd_logs[event_cols].iterrows():
                        # Format each value appropriately
                        row_values = []
                        for col in event_cols:
                            if pd.isnull(row[col]):
                                value = ""
                            else:
                                value = str(row[col])
                            row_values.append(value)
                        event_data.append(row_values)

                    # Add to sheet
                    event_start_row = current_row
                    worksheet.update(f"A{event_start_row}", event_data)

                    # Format headers
                    header_range = f"A{event_start_row}:{chr(65+len(event_cols)-1)}{event_start_row}"
                    worksheet.format(header_range, {
                        "textFormat": {"bold": True},
                        "backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}
                    })

                    current_row += len(event_data) + 1

                # Add transfer value summary if available
                if 'amount' in pyusd_logs.columns and 'is_transfer' in pyusd_logs.columns:
                    transfer_logs = pyusd_logs[pyusd_logs['is_transfer']]
                    if not transfer_logs.empty:
                        total_transferred = transfer_logs['amount'].sum()
                        worksheet.update(f"A{current_row}", [[f"Total PYUSD transferred: {format_value_pyusd(total_transferred)}"]])
                        current_row += 2

        # 8. Add Recommendations
        if "recommendations" in data_dict and data_dict["recommendations"]:
            recommendations = data_dict["recommendations"]

            # Add section title
            worksheet.update(f"A{current_row}", [["Analysis Observations & Recommendations"]])
            worksheet.format(f"A{current_row}:A{current_row}", {
                "textFormat": {"bold": True, "fontSize": 12},
                "backgroundColor": {"red": 0.7, "green": 0.9, "blue": 1.0}
            })
            current_row += 1

            # Add each recommendation
            for rec in recommendations:
                worksheet.update(f"A{current_row}", [[rec]])
                current_row += 1

            current_row += 1  # Extra space

        # 9. Add main DataFrame data (selected columns only for better readability)
        if not df.empty:
            # Add a section title
            worksheet.update(f"A{current_row}", [["Key Contract Calls"]])
            worksheet.format(f"A{current_row}:A{current_row}", {
                "textFormat": {"bold": True, "fontSize": 12},
                "backgroundColor": {"red": 0.8, "green": 0.8, "blue": 1.0}
            })
            current_row += 1

            # Select important columns to display
            display_cols = ['id', 'type', 'depth', 'contract', 'function_category', 'gasUsed', 'is_pyusd']
            display_cols = [col for col in display_cols if col in df.columns]

            # First show PYUSD calls
            pyusd_calls = df[df['is_pyusd']] if 'is_pyusd' in df.columns else pd.DataFrame()

            if not pyusd_calls.empty:
                worksheet.update(f"A{current_row}", [[f"Found {len(pyusd_calls)} PYUSD-related calls:"]])
                current_row += 1

                # Convert DataFrame to list of lists for the worksheet
                pyusd_df_values = [display_cols] + pyusd_calls[display_cols].values.tolist()

                # Format values for display
                for i in range(1, len(pyusd_df_values)):
                    for j, col in enumerate(display_cols):
                        val = pyusd_df_values[i][j]
                        if col == 'gasUsed' and pd.notnull(val):
                            pyusd_df_values[i][j] = f"{val:,}"
                        elif pd.isnull(val):
                            pyusd_df_values[i][j] = "NULL"
                        else:
                            pyusd_df_values[i][j] = str(val)

                worksheet.update(f"A{current_row}", pyusd_df_values)

                # Format the DataFrame header
                worksheet.format(f"A{current_row}:{chr(65+len(display_cols)-1)}{current_row}", {
                    "textFormat": {"bold": True},
                    "backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}
                })

                # Add alternating row colors for readability
                data_rows = len(pyusd_df_values)
                for i in range(2, data_rows + 1, 2):
                    row_num = current_row + i - 1
                    worksheet.format(f"A{row_num}:{chr(65+len(display_cols)-1)}{row_num}", {
                        "backgroundColor": {"red": 0.97, "green": 0.97, "blue": 1.0}
                    })

                current_row += len(pyusd_df_values) + 2

            # Then show highest gas usage calls
            worksheet.update(f"A{current_row}", [["Highest Gas Usage Calls:"]])
            worksheet.format(f"A{current_row}:A{current_row}", {"textFormat": {"bold": True}})
            current_row += 1

            # Get top 5 by gas usage
            high_gas_cols = ['id', 'type', 'contract', 'function_category', 'gasUsed']
            high_gas_cols = [col for col in high_gas_cols if col in df.columns]

            high_gas_calls = df.nlargest(5, 'gasUsed') if 'gasUsed' in df.columns else pd.DataFrame()

            if not high_gas_calls.empty:
                # Convert to list for sheet
                high_gas_data = [high_gas_cols]  # Header row
                for _, row in high_gas_calls[high_gas_cols].iterrows():
                    # Format each value appropriately
                    row_values = []
                    for col in high_gas_cols:
                        if col == 'gasUsed':
                            value = f"{row[col]:,}" if pd.notnull(row[col]) else "0"
                        elif pd.isnull(row[col]):
                            value = "NULL"
                        else:
                            value = str(row[col])
                        row_values.append(value)
                    high_gas_data.append(row_values)

                # Add to sheet
                worksheet.update(f"A{current_row}", high_gas_data)

                # Format headers
                header_range = f"A{current_row}:{chr(65+len(high_gas_cols)-1)}{current_row}"
                worksheet.format(header_range, {
                    "textFormat": {"bold": True},
                    "backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}
                })

                # Add alternating row colors
                for i in range(2, len(high_gas_data) + 1, 2):
                    row_num = current_row + i - 1
                    worksheet.format(f"A{row_num}:{chr(65+len(high_gas_cols)-1)}{row_num}", {
                        "backgroundColor": {"red": 0.97, "green": 0.97, "blue": 1.0}
                    })

        # Try to auto-resize columns for better readability
        try:
            worksheet.columns_auto_resize(0, 10)  # Attempt to resize first 10 columns
        except:
            pass  # Ignore if not supported

        # Return spreadsheet URL and title for opening
        spreadsheet_url = f"https://docs.google.com/spreadsheets/d/{spreadsheet.id}"
        return (spreadsheet_url, sheet_title)

    except Exception as e:
        console.print(f"[error]Error creating Google Sheet: {str(e)}", style="error")
        raise Exception(f"Error creating Google Sheet: {str(e)}")

# Function to display loading indicator while rendering visualization
def show_loading_indicator():
    """Display a loading indicator while rendering graphs"""
    loading_html = """
    <div style="display: flex; justify-content: center; align-items: center; height: 50px;">
        <div style="text-align: center;">
            <div class="spinner-border" role="status">
                <span class="sr-only">Loading...</span>
            </div>
            <p style="margin-top: 10px; color: #555;">Generating visualization...</p>
        </div>
    </div>
    <style>
    .spinner-border {
        display: inline-block;
        width: 2rem;
        height: 2rem;
        vertical-align: text-bottom;
        border: 0.25em solid currentColor;
        border-right-color: transparent;
        border-radius: 50%;
        animation: spinner-border .75s linear infinite;
    }
    @keyframes spinner-border {
        to { transform: rotate(360deg); }
    }
    </style>
    """
    display(HTML(loading_html))

# Function to get human-readable function descriptions
def get_function_description(input_data, is_pyusd, contract_name):
    """Get a human-readable function description."""
    if not input_data or input_data == '0x':
        return "Contract Creation" if not input_data else "ETH Transfer"

    # Extract method signature (first 10 characters including 0x)
    method_sig = input_data[:10] if len(input_data) >= 10 else input_data

    # Check PYUSD signatures first
    if method_sig in PYUSD_SIGNATURES:
        function_info = PYUSD_SIGNATURES[method_sig]
        return function_info["name"]

    # Check common ERC20/generic function signatures
    common_sigs = {
        '0xa9059cbb': "transfer(address,uint256)",
        '0x095ea7b3': "approve(address,uint256)",
        '0x23b872dd': "transferFrom(address,address,uint256)",
        '0x18160ddd': "totalSupply()",
        '0x70a08231': "balanceOf(address)",
        '0xdd62ed3e': "allowance(address,address)",
        '0x06fdde03': "name()",
        '0x95d89b41': "symbol()",
        '0x313ce567': "decimals()",
        '0x8da5cb5b': "owner()",
        '0x715018a6': "renounceOwnership()",
        '0xf2fde38b': "transferOwnership(address)",
        '0x01ffc9a7': "supportsInterface(bytes4)",
        '0x3644e515': "DOMAIN_SEPARATOR()",
        '0x7ecebe00': "nonces(address)",
        '0xd505accf': "permit(address,address,uint256,uint256,uint8,bytes32,bytes32)"
    }

    return common_sigs.get(method_sig, f"Function {method_sig}")

# function for creating interactive Plotly Contract Interaction Graph
def create_plotly_contract_interaction_graph(contract_interactions):
    """Creates an interactive Plotly Network graph for contract interactions with directional arrows"""
    import math

    if not contract_interactions:
        return None

    # Create a networkx graph from the interaction data
    G = nx.DiGraph()

    # Add nodes for all contracts in interactions
    contracts_seen = set()
    for src, dst in contract_interactions:
        if src not in contracts_seen:
            src_name = PYUSD_CONTRACTS.get(src, "External Contract")
            G.add_node(src, name=src_name, is_pyusd=(src in PYUSD_CONTRACTS))
            contracts_seen.add(src)

        if dst not in contracts_seen:
            dst_name = PYUSD_CONTRACTS.get(dst, "External Contract")
            G.add_node(dst, name=dst_name, is_pyusd=(dst in PYUSD_CONTRACTS))
            contracts_seen.add(dst)

        # Add edge
        G.add_edge(src, dst)

    # Calculate layout with more spacing for readability
    pos = nx.spring_layout(G, seed=42, k=1.5)

    # Create edge traces with arrows
    edge_traces = []

    for edge in G.edges():
        src, dst = edge
        src_name = G.nodes[src]['name']
        dst_name = G.nodes[dst]['name']

        x0, y0 = pos[src]
        x1, y1 = pos[dst]

        # Calculate direction vector for arrow
        dx = x1 - x0
        dy = y1 - y0

        # Normalize the vector
        length = math.sqrt(dx**2 + dy**2)
        if length > 0:
            udx = dx / length
            udy = dy / length
        else:
            udx, udy = 0, 0

        # Position arrow slightly before the destination node (80% along the edge)
        arrow_ratio = 0.8
        arrow_x = x0 + arrow_ratio * dx
        arrow_y = y0 + arrow_ratio * dy

        # Angle for the arrow in degrees
        angle = math.degrees(math.atan2(dy, dx))

        # Main edge line
        edge_trace = go.Scatter(
            x=[x0, x1],
            y=[y0, y1],
            line=dict(width=1.5, color='rgba(50, 50, 50, 0.8)'),
            hoverinfo='text',
            text=f"From: {src_name}<br>To: {dst_name}<br>From address: {src}<br>To address: {dst}",
            mode='lines',
            showlegend=False
        )

        # Arrow marker
        arrow_trace = go.Scatter(
            x=[arrow_x],
            y=[arrow_y],
            mode='markers',
            marker=dict(
                symbol='triangle-right',
                size=12,
                color='rgba(50, 50, 50, 0.8)',
                angle=angle  # Apply the calculated angle
            ),
            hoverinfo='none',
            showlegend=False
        )

        edge_traces.append(edge_trace)
        edge_traces.append(arrow_trace)

    # Create node trace
    node_x = []
    node_y = []
    node_colors = []
    node_sizes = []
    hover_texts = []
    node_addresses = []

    for node in G.nodes():
        node_data = G.nodes[node]
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

        # Node color based on contract type
        if node_data['is_pyusd']:
            if "PYUSD Token" in node_data['name']:
                node_colors.append('rgba(144, 238, 144, 0.9)')  # palegreen
            elif "Supply Control" in node_data['name']:
                node_colors.append('rgba(135, 206, 250, 0.9)')  # lightskyblue
            else:
                node_colors.append('rgba(224, 255, 255, 0.9)')  # lightcyan
        else:
            node_colors.append('rgba(211, 211, 211, 0.9)')  # lightgray

        # Node size: bigger for PYUSD contracts
        node_sizes.append(25 if node_data['is_pyusd'] else 18)

        # Full address for node label
        node_addresses.append(node)

        # Hover text with full contract information
        hover_texts.append(f"<b>{node_data['name']}</b><br>Address: {node}")

    # Create node trace with text labels showing full addresses
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        hoverinfo='text',
        text=node_addresses,
        textposition="bottom center",
        hovertext=hover_texts,
        marker=dict(
            showscale=False,
            color=node_colors,
            size=node_sizes,
            line=dict(width=1, color='#000')
        ),
        textfont=dict(
            family="monospace",
            size=10,
            color="black"
        )
    )

    # Add legend traces for different node types
    legend_traces = [
        go.Scatter(
            x=[None], y=[None],
            mode='markers',
            marker=dict(size=15, color='rgba(144, 238, 144, 0.9)'),
            name='PYUSD Token',
            showlegend=True
        ),
        go.Scatter(
            x=[None], y=[None],
            mode='markers',
            marker=dict(size=15, color='rgba(135, 206, 250, 0.9)'),
            name='Supply Control',
            showlegend=True
        ),
        go.Scatter(
            x=[None], y=[None],
            mode='markers',
            marker=dict(size=15, color='rgba(211, 211, 211, 0.9)'),
            name='External Contract',
            showlegend=True
        )
    ]

    # Create figure with styled layout
    fig = go.Figure()

    # Add all traces
    for trace in edge_traces:
        fig.add_trace(trace)

    fig.add_trace(node_trace)

    for trace in legend_traces:
        fig.add_trace(trace)

    # Style the figure
    fig.update_layout(
        title='<b>Contract Interaction Overview</b>',
        titlefont=dict(size=16),
        showlegend=True,
        legend=dict(
            title="Contract Types",
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1,
            bgcolor="rgba(255, 255, 255, 0.8)"
        ),
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=60),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        template="plotly_white",
        height=600,
        paper_bgcolor='rgba(255,255,255,0.8)',
        plot_bgcolor='rgba(255,255,255,0.8)'
    )

    return fig

# function for creating interactive Plotly Call Graph
def create_plotly_call_graph(call_data_list):
    """Creates an interactive Plotly Network graph for the call trace hierarchy"""
    if not call_data_list:
        return None

    # Process relationships and parent-child connections
    # We need to reconstruct parent/child relationships based on depth
    for i, call in enumerate(call_data_list):
        # Initialize parent_id field if it doesn't exist
        if 'parent_id' not in call:
            call['parent_id'] = None

        # If not root node, find parent
        if call['depth'] > 0 and i > 0:
            # Look backward for potential parents at the previous depth level
            for j in range(i-1, -1, -1):
                potential_parent = call_data_list[j]
                if potential_parent['depth'] == call['depth'] - 1:
                    call['parent_id'] = potential_parent['id']
                    break

    # Create a networkx graph
    G = nx.DiGraph()

    # Add all nodes with their attributes
    for call in call_data_list:
        G.add_node(call['id'], **call)

    # Add edges based on parent-child relationships
    for call in call_data_list:
        if call['parent_id']:
            G.add_edge(call['parent_id'], call['id'])

    # Try using graphviz layout if available
    try:
        import pygraphviz as pgv
        pos = nx.nx_agraph.graphviz_layout(G, prog='dot')  # Hierarchical layout
    except:
        # Fallback to custom hierarchical layout
        pos = {}
        depth_to_nodes = {}

        for node, data in G.nodes(data=True):
            depth = data['depth']
            if depth not in depth_to_nodes:
                depth_to_nodes[depth] = []
            depth_to_nodes[depth].append(node)

        # Calculate positions based on depth
        max_depth = max(depth_to_nodes.keys()) if depth_to_nodes else 0

        for depth, nodes in depth_to_nodes.items():
            nodes.sort()  # For consistent layout
            node_count = len(nodes)
            for i, node in enumerate(nodes):
                # Horizontal spacing based on node count, vertical based on depth
                x_pos = i / (node_count + 1) if node_count > 1 else 0.5
                y_pos = 1.0 - (depth / (max_depth + 1)) if max_depth > 0 else 0.5
                pos[node] = (x_pos, y_pos)

    # Create edge traces with different colors by call type
    edge_traces_by_type = {}
    edge_types = set()

    for edge in G.edges():
        source, target = edge
        source_data = G.nodes[source]
        target_data = G.nodes[target]

        call_type = target_data['type']
        edge_types.add(call_type)

        if call_type not in edge_traces_by_type:
            edge_traces_by_type[call_type] = {
                'x': [], 'y': [], 'text': [], 'color': '', 'style': '', 'width': 1
            }

            # Set color and style based on call type
            if call_type == 'DELEGATECALL':
                edge_traces_by_type[call_type]['color'] = 'rgba(0, 0, 255, 0.7)'
                edge_traces_by_type[call_type]['style'] = 'dash'
                edge_traces_by_type[call_type]['width'] = 2
            elif call_type == 'STATICCALL':
                edge_traces_by_type[call_type]['color'] = 'rgba(0, 128, 0, 0.7)'
                edge_traces_by_type[call_type]['style'] = 'dot'
                edge_traces_by_type[call_type]['width'] = 1.5
            else:
                edge_traces_by_type[call_type]['color'] = 'rgba(128, 128, 128, 0.7)'
                edge_traces_by_type[call_type]['style'] = 'solid'
                edge_traces_by_type[call_type]['width'] = 1

        x0, y0 = pos[source]
        x1, y1 = pos[target]

        edge_traces_by_type[call_type]['x'].extend([x0, x1, None])
        edge_traces_by_type[call_type]['y'].extend([y0, y1, None])

        # Create full hover info
        hover_text = (
            f"<b>From:</b> {source_data['from']}<br>"
            f"<b>To:</b> {target_data['to']}<br>"
            f"<b>Type:</b> {target_data['type']}<br>"
            f"<b>Depth:</b> {target_data['depth']}<br>"
        )

        if target_data['value_eth'] > 0:
            hover_text += f"<b>Value:</b> {target_data['value_eth']} ETH<br>"

        hover_text += f"<b>Gas:</b> {target_data['gasUsed']:,}<br>"

        if target_data['is_pyusd']:
            hover_text += f"<b>Contract:</b> {target_data['contract']}<br>"

        if target_data.get('error'):
            hover_text += f"<b style='color:red'>Error:</b> {target_data['error']}<br>"

        edge_traces_by_type[call_type]['text'].append(hover_text)

    # Create scatter traces for each call type
    edge_traces = []
    for call_type, trace_data in edge_traces_by_type.items():
        edge_traces.append(
            go.Scatter(
                x=trace_data['x'],
                y=trace_data['y'],
                line=dict(
                    width=trace_data['width'],
                    color=trace_data['color'],
                    dash=trace_data['style']
                ),
                hoverinfo='text',
                text=trace_data['text'],
                mode='lines',
                name=call_type
            )
        )

    # Create node trace with improved hover information
    node_x = []
    node_y = []
    node_colors = []
    node_sizes = []
    node_text = []  # Function name displayed on node
    hover_texts = []

    for node in G.nodes():
        node_data = G.nodes[node]
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

        # Create more detailed text for nodes
        function_name = get_function_description(
            node_data.get('input_preview', '0x'),
            node_data['is_pyusd'],
            node_data.get('contract', 'Other')
        )

        # Short text for display (truncated for space)
        short_name = function_name.split('(')[0] if '(' in function_name else function_name
        if len(short_name) > 12:
            short_name = short_name[:10] + '...'
        node_text.append(short_name)

        # Create detailed hover text
        hover_text = f"<b style='font-size:12px'>{node_data['type']} Call</b><br>"
        hover_text += f"<b>From:</b> {node_data['from']}<br>"
        hover_text += f"<b>To:</b> {node_data['to']}<br>"
        hover_text += f"<b>Depth:</b> {node_data['depth']}<br>"

        if node_data['value_eth'] > 0:
            hover_text += f"<b>Value:</b> {node_data['value_eth']} ETH<br>"

        hover_text += f"<b>Gas Used:</b> {node_data['gasUsed']:,}<br>"

        if node_data['is_pyusd']:
            hover_text += f"<b>PYUSD Contract:</b> {node_data['contract']}<br>"
            hover_text += f"<b>Function Category:</b> {node_data['function_category']}<br>"

        hover_text += f"<b>Function:</b> {function_name}<br>"

        if node_data.get('error'):
            hover_text += f"<b style='color:red'>Error:</b> {node_data['error']}<br>"

        hover_texts.append(hover_text)

        # Node color based on contract type and error status
        if node_data.get('error'):
            node_colors.append('rgba(255, 99, 71, 0.9)')  # tomato for errors
        elif node_data['is_pyusd']:
            if "PYUSD Token" in node_data.get('contract', ''):
                node_colors.append('rgba(144, 238, 144, 0.9)')  # palegreen
            elif "Supply Control" in node_data.get('contract', ''):
                node_colors.append('rgba(135, 206, 250, 0.9)')  # lightskyblue
            else:
                node_colors.append('rgba(224, 255, 255, 0.9)')  # lightcyan
        else:
            # Gradient based on depth for non-PYUSD calls
            intensity = min(95, max(70, 95 - node_data['depth'] * 5))
            rgb_val = intensity / 100.0
            node_colors.append(f'rgba({int(rgb_val*255)}, {int(rgb_val*255)}, {int(rgb_val*255)}, 0.9)')

        # Size based on gas used
        gas_used = node_data['gasUsed']
        # Scale node size based on gas used (within reasonable bounds)
        size = max(15, min(40, 15 + (gas_used / 50000)))
        node_sizes.append(size)

    # Create node trace
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        hoverinfo='text',
        text=node_text,
        textposition="top center",
        hovertext=hover_texts,
        marker=dict(
            showscale=False,
            color=node_colors,
            size=node_sizes,
            line=dict(width=1, color='#000')
        ),
        textfont=dict(
            family="Arial",
            size=9,
            color="#333"
        )
    )

    # Create legend traces for node colors
    node_color_legend = [
        go.Scatter(
            x=[None], y=[None],
            mode='markers',
            marker=dict(size=15, color='rgba(144, 238, 144, 0.9)'),
            name='PYUSD Token',
            showlegend=True
        ),
        go.Scatter(
            x=[None], y=[None],
            mode='markers',
            marker=dict(size=15, color='rgba(135, 206, 250, 0.9)'),
            name='Supply Control',
            showlegend=True
        ),
        go.Scatter(
            x=[None], y=[None],
            mode='markers',
            marker=dict(size=15, color='rgba(211, 211, 211, 0.9)'),
            name='Other Contract',
            showlegend=True
        ),
        go.Scatter(
            x=[None], y=[None],
            mode='markers',
            marker=dict(size=15, color='rgba(255, 99, 71, 0.9)'),
            name='Error',
            showlegend=True
        )
    ]

    # Create figure with all traces
    fig = go.Figure()

    # Add traces
    for edge_trace in edge_traces:
        fig.add_trace(edge_trace)

    fig.add_trace(node_trace)

    # Add legend traces
    for legend_trace in node_color_legend:
        fig.add_trace(legend_trace)

    # Style the figure
    fig.update_layout(
        title='<b>Detailed Call Graph Visualization</b>',
        titlefont=dict(size=16),
        showlegend=True,
        legend=dict(
            title="Call Types",
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1,
            bgcolor="rgba(255, 255, 255, 0.8)"
        ),
        hovermode='closest',
        margin=dict(b=40, l=5, r=5, t=60),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        template="plotly_white",
        height=700,
        paper_bgcolor='rgba(255,255,255,0.8)',
        plot_bgcolor='rgba(255,255,255,0.8)'
    )

    return fig

# New function for creating interactive Plotly PYUSD Flow Graph
def create_plotly_flow_graph(transfers):
    """Creates an interactive Plotly Network graph for PYUSD token flows"""
    if not transfers:
        return None

    # Create a directed graph for token flows
    G = nx.DiGraph()

    # Track total transfer amount per edge for aggregation
    transfer_totals = {}

    # Calculate aggregate transfers
    for transfer in transfers:
        from_addr = transfer['from']
        to_addr = transfer['to']
        amount = transfer['amount']

        edge_key = (from_addr, to_addr)
        if edge_key in transfer_totals:
            transfer_totals[edge_key] += amount
        else:
            transfer_totals[edge_key] = amount

    # Add nodes and edges to networkx graph
    for (from_addr, to_addr), total_amount in transfer_totals.items():
        # Add nodes if they don't exist
        if from_addr not in G:
            G.add_node(from_addr, address=from_addr, label=shorten_address(from_addr))

        if to_addr not in G:
            G.add_node(to_addr, address=to_addr, label=shorten_address(to_addr))

        # Add edge with amount
        G.add_edge(from_addr, to_addr, amount=total_amount, label=format_value_pyusd(total_amount))

    # Calculate layout
    pos = nx.spring_layout(G, k=1.0, seed=42)

    # Create edge trace
    edge_x = []
    edge_y = []
    edge_text = []
    edge_amount_texts = []

    for edge in G.edges(data=True):
        source, target, data = edge

        x0, y0 = pos[source]
        x1, y1 = pos[target]

        # Calculate midpoint for displaying amount
        mid_x = (x0 + x1) / 2
        mid_y = (y0 + y1) / 2

        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

        amount_str = format_value_pyusd(data['amount'])
        edge_text.append(f"Transfer: {amount_str}<br>From: {shorten_address(source)}<br>To: {shorten_address(target)}")
        edge_amount_texts.append(amount_str)

    # Create edge trace with arrows
    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=2, color='rgba(50, 150, 50, 0.8)'),
        hoverinfo='text',
        text=edge_text,
        mode='lines',
        name='Transfer'
    )

    # Create a separate trace for each edge label (amount)
    edge_label_traces = []
    edge_idx = 0
    for edge in G.edges(data=True):
        source, target, data = edge
        x0, y0 = pos[source]
        x1, y1 = pos[target]

        # Calculate midpoint for the label
        mid_x = (x0 + x1) / 2
        mid_y = (y0 + y1) / 2

        # Add label trace
        edge_label_traces.append(
            go.Scatter(
                x=[mid_x],
                y=[mid_y],
                text=[format_value_pyusd(data['amount'])],
                mode='text',
                hoverinfo='none',
                showlegend=False,
                textfont=dict(
                    size=10,
                    color='darkgreen'
                )
            )
        )

        # Add arrow trace (small marker at the target end)
        # Calculate the position for the arrow (slightly before the target)
        arrow_ratio = 0.8  # How far along the edge to place the arrow (0.8 = 80% of the way to target)
        arrow_x = x0 + (x1 - x0) * arrow_ratio
        arrow_y = y0 + (y1 - y0) * arrow_ratio

        edge_label_traces.append(
            go.Scatter(
                x=[arrow_x],
                y=[arrow_y],
                mode='markers',
                marker=dict(
                    symbol='triangle-right',
                    size=12,
                    color='rgba(50, 150, 50, 0.8)',
                    angle=45
                ),
                hoverinfo='none',
                showlegend=False
            )
        )

        edge_idx += 1

    # Create node trace with full addresses
    node_x = []
    node_y = []
    node_text = []
    node_hover = []

    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

        # Use full address for display
        node_text.append(node)

        # Create hover text
        node_hover.append(f"<b>Address:</b> {node}")

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        hoverinfo='text',
        text=node_text,
        textposition="bottom center",
        hovertext=node_hover,
        marker=dict(
            color='rgba(144, 238, 144, 0.8)',  # palegreen
            size=25,
            line=dict(width=1, color='darkgreen'),
            symbol='circle'
        ),
        textfont=dict(
            family="monospace",
            size=9,
            color="black"
        ),
        name='Address'
    )

    # Create figure with styled layout
    fig = go.Figure()

    # Add all traces
    fig.add_trace(edge_trace)
    for label_trace in edge_label_traces:
        fig.add_trace(label_trace)
    fig.add_trace(node_trace)

    # Style the figure
    fig.update_layout(
        title='<b>PYUSD Token Flow Analysis</b>',
        titlefont=dict(size=16),
        showlegend=True,
        legend=dict(
            title="Elements",
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1,
            bgcolor="rgba(255, 255, 255, 0.8)"
        ),
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=60),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        template="plotly_white",
        height=600,
        paper_bgcolor='rgba(255,255,255,0.8)',
        plot_bgcolor='rgba(255,255,255,0.8)'
    )

    return fig

def parse_call_trace(trace_result, tx_hash):
    """Parses the output of callTracer and generates insights & visualizations."""
    if not trace_result:
        console.print("[warning]No trace result provided for parsing.", style="warning")
        return None, None, None, None, None, None, None, None

    # --- Basic Info Panel ---
    to_address = trace_result.get('to', 'N/A').lower()
    is_pyusd_tx = to_address in PYUSD_CONTRACTS
    pyusd_label = f"[bold green]({PYUSD_CONTRACTS[to_address]})[/bold green]" if is_pyusd_tx else ""

    # Extract gas metrics for analysis
    gas_used = int(trace_result.get('gasUsed', '0x0'), 16) if isinstance(trace_result.get('gasUsed', '0x0'), str) and trace_result.get('gasUsed', '0x0').startswith('0x') else int(trace_result.get('gasUsed', 0))

    overview_text = f"""
      [bold]Trace Summary for {shorten_address(tx_hash)}[/bold] {pyusd_label}
      Type: {trace_result.get('type', 'N/A')}
      From: {shorten_address(trace_result.get('from', 'N/A'))}
      To: {shorten_address(trace_result.get('to', 'N/A'))}
      Value: {format_value_eth(trace_result.get('value', '0x0'))}
      Gas Used: {format_gas(trace_result.get('gasUsed', '0x0'))} ({gas_used:,} units)
      Status: [bold red]Error: {trace_result['error']}[/bold red]""" if 'error' in trace_result else "[bold green]Success[/bold green]"
    console.print(Panel(overview_text, title="Trace Overview", border_style="cyan3", expand=False))

    node_counter = 0
    call_data_list = []
    state_changes = []

    # Track PYUSD transfer data for visualization
    pyusd_transfers = []

    # Track contract interactions for relationship mapping
    contract_interactions = set()

    # Track gas usage by category
    gas_by_category = {category: 0 for category in GAS_CATEGORIES.keys()}

    def add_nodes_edges(call, parent_node_id=None, depth=0, parent_addr=None):
        nonlocal node_counter
        current_node_id = f"node_{node_counter}"
        node_counter += 1

        call_type = call.get('type', 'N/A').upper()

        # Extract call data for label
        from_addr = call.get('from', 'N/A')
        from_addr_short = shorten_address(from_addr)
        to_addr = call.get('to', 'N/A')
        to_addr_lower = to_addr.lower() if to_addr else ''
        to_addr_short = shorten_address(to_addr)

        # Track contract interaction
        if parent_addr and to_addr_lower:
            contract_interactions.add((parent_addr.lower(), to_addr_lower))

        # Check for PYUSD contracts
        is_pyusd_call = to_addr_lower in PYUSD_CONTRACTS
        contract_name = PYUSD_CONTRACTS.get(to_addr_lower, None)

        # function signature detection for PYUSD calls
        input_data = call.get('input', '0x')
        function_category = "other"
        if input_data != '0x':
            method_sig = input_data[:10]

            if method_sig in PYUSD_SIGNATURES:
                function_info = PYUSD_SIGNATURES[method_sig]
                function_name = function_info["name"]
                function_category = function_info["category"]

                # Process specific functions for deeper analysis
                if is_pyusd_call and "PYUSD Token" in contract_name:
                    if method_sig == '0xa9059cbb':  # transfer
                        try:
                            # Extract params
                            to_offset = 10
                            to_param = "0x" + input_data[to_offset+24:to_offset+64]
                            amount_offset = 74
                            amount = int(input_data[amount_offset:amount_offset+64], 16)
                            pyusd_transfers.append({
                                'from': from_addr,
                                'to': to_param,
                                'amount': amount,
                                'gas_used': int(call.get('gasUsed', '0x0'), 16) if call.get('gasUsed', '0x0').startswith('0x') else int(call.get('gasUsed', 0))
                            })
                            # Track state change
                            state_changes.append({
                                'contract': contract_name,
                                'function': function_name,
                                'type': 'transfer',
                                'from': from_addr,
                                'to': to_param,
                                'amount': amount,
                                'gas_used': int(call.get('gasUsed', '0x0'), 16) if call.get('gasUsed', '0x0').startswith('0x') else int(call.get('gasUsed', 0))
                            })
                        except Exception:
                            pass
                    elif method_sig == '0x40c10f19':  # mint
                        try:
                            to_offset = 10
                            to_param = "0x" + input_data[to_offset+24:to_offset+64]
                            amount_offset = 74
                            amount = int(input_data[amount_offset:amount_offset+64], 16)
                            # Track state change
                            state_changes.append({
                                'contract': contract_name,
                                'function': function_name,
                                'type': 'mint',
                                'to': to_param,
                                'amount': amount,
                                'gas_used': int(call.get('gasUsed', '0x0'), 16) if call.get('gasUsed', '0x0').startswith('0x') else int(call.get('gasUsed', 0))
                            })
                        except Exception:
                            pass
                    elif method_sig == '0x9dc29fac':  # burn
                        try:
                            from_offset = 10
                            from_param = "0x" + input_data[from_offset+24:from_offset+64]
                            amount_offset = 74
                            amount = int(input_data[amount_offset:amount_offset+64], 16)
                            # Track state change
                            state_changes.append({
                                'contract': contract_name,
                                'function': function_name,
                                'type': 'burn',
                                'from': from_addr,
                                'amount': amount,
                                'gas_used': int(call.get('gasUsed', '0x0'), 16) if call.get('gasUsed', '0x0').startswith('0x') else int(call.get('gasUsed', 0))
                            })
                        except Exception:
                            pass

        # Update gas usage by category - AFTER function_category is defined
        call_gas = int(call.get('gasUsed', '0x0'), 16) if call.get('gasUsed', '0x0').startswith('0x') else int(call.get('gasUsed', 0))
        if is_pyusd_call:
            gas_by_category[function_category] += call_gas
        else:
            gas_by_category["other"] += call_gas

        # Extract output and error data
        output_data = call.get('output', '0x')
        error_msg = call.get('error')

        # Store call data for dataframe
        try:
            gas_used_val = int(call.get('gasUsed', '0x0'), 16) if call.get('gasUsed', '0x0').startswith('0x') else int(call.get('gasUsed', 0))
            value_raw_wei = int(call.get('value', '0x0'), 16) if call.get('value', '0x0').startswith('0x') else int(call.get('value', 0))
            value_eth_float = float(w3_mainnet.from_wei(value_raw_wei, 'ether')) if w3_mainnet else (value_raw_wei / 1e18)
        except (ValueError, TypeError, AttributeError):
            gas_used_val = 0
            value_eth_float = 0.0

        # Build call info for dataframe with data
        call_info = {
            'id': current_node_id,
            'parent_id': parent_node_id,  # Track parent-child relationship
            'type': call_type,
            'depth': depth,
            'from': from_addr,
            'to': to_addr,
            'value_eth': value_eth_float,
            'gasUsed': gas_used_val,
            'is_pyusd': is_pyusd_call,
            'contract': contract_name if is_pyusd_call else "Other",
            'function_category': function_category,
            'error': error_msg,
            'input_preview': input_data[:10] + "..." if len(input_data) > 10 else input_data,
            'output_preview': output_data[:10] + "..." if len(output_data) > 10 else output_data,
        }
        call_data_list.append(call_info)

        # Process sub-calls recursively
        if 'calls' in call and isinstance(call['calls'], list):
            for sub_call in call['calls']:
                add_nodes_edges(sub_call, current_node_id, depth + 1, to_addr)

    # --- Start processing the trace from the top-level call ---
    add_nodes_edges(trace_result, depth=0)

    # Create a dataframe from collected call data
    call_df = pd.DataFrame(call_data_list)

    # Create state changes dataframe
    state_changes_df = pd.DataFrame(state_changes) if state_changes else None

    # --- Extract Logs from trace with decoding ---
    logs_data = []
    log_counter_trace = 0

    def extract_logs_recursive(call):
        nonlocal log_counter_trace
        if 'logs' in call and isinstance(call['logs'], list):
            for log in call['logs']:
                # Ensure log is a dictionary
                if not isinstance(log, dict): continue

                log_details = {
                    "address": log.get("address", "N/A"),
                    "topics": log.get("topics", []),
                    "data": log.get("data", "0x"),
                    "log_index_trace": log_counter_trace
                }
                log_counter_trace += 1

                # Check if log is from PYUSD contract
                address_lower = log_details["address"].lower()
                is_pyusd_contract = address_lower in PYUSD_CONTRACTS

                # Basic data for all logs
                log_entry = {
                    "log_idx_trace": log_details["log_index_trace"],
                    "address": log_details["address"],
                    "contract": PYUSD_CONTRACTS.get(address_lower, "Other"),
                    "is_pyusd": is_pyusd_contract,
                    "topic0": log_details["topics"][0] if log_details["topics"] else "N/A",
                    "topic0_short": log_details["topics"][0][:10]+"..." if log_details["topics"] else "N/A",
                    "details": "Not Decoded",
                    "event_name": "Unknown"
                }

                # event decoding with the registry
                if is_pyusd_contract and isinstance(log_details["topics"], list) and log_details["topics"]:
                    event_topic = log_details["topics"][0]
                    if event_topic in PYUSD_EVENTS:
                        event_info = PYUSD_EVENTS[event_topic]
                        log_entry["event_name"] = event_info["name"]

                        try:
                            # Decode event data using registered decoder
                            decoded_data = event_info["decoder"](log_details["topics"], log_details["data"])

                            # Format details based on event type
                            if "Transfer" in event_info["name"]:
                                value_pyusd_str = format_value_pyusd(decoded_data["value"])
                                log_entry["details"] = f"PYUSD Transfer: {value_pyusd_str} from {shorten_address(decoded_data['from'])} to {shorten_address(decoded_data['to'])}"
                                log_entry["is_transfer"] = True
                                log_entry["amount"] = decoded_data["value"]
                                log_entry["from_addr"] = decoded_data["from"]
                                log_entry["to_addr"] = decoded_data["to"]
                            elif "Approval" in event_info["name"]:
                                value_pyusd_str = format_value_pyusd(decoded_data["value"])
                                log_entry["details"] = f"PYUSD Approval: {shorten_address(decoded_data['owner'])} approved {value_pyusd_str} for {shorten_address(decoded_data['spender'])}"
                                log_entry["is_approval"] = True
                                log_entry["amount"] = decoded_data["value"]
                                log_entry["owner"] = decoded_data["owner"]
                                log_entry["spender"] = decoded_data["spender"]
                            elif "Paused" in event_info["name"]:
                                account = decoded_data.get("account", "N/A")
                                log_entry["details"] = f"PYUSD Paused by {shorten_address(account) if account else 'N/A'}"
                                log_entry["is_pause"] = True
                            elif "Unpaused" in event_info["name"]:
                                account = decoded_data.get("account", "N/A")
                                log_entry["details"] = f"PYUSD Unpaused by {shorten_address(account) if account else 'N/A'}"
                                log_entry["is_unpause"] = True

                        except Exception as decode_err:
                            log_entry["details"] = f"PYUSD Event (Decode Error: {decode_err})"

                logs_data.append(log_entry)

        # Check sub-calls recursively
        if 'calls' in call and isinstance(call['calls'], list):
            for sub_call in call['calls']:
                extract_logs_recursive(sub_call)

    # Extract logs if present in trace config
    extract_logs_recursive(trace_result)
    logs_df = pd.DataFrame(logs_data) if logs_data else pd.DataFrame()

    # --- Create Contract Interaction Graph with Plotly ---
    contract_graph = None
    if contract_interactions:
        try:
            contract_graph = create_plotly_contract_interaction_graph(contract_interactions)
        except Exception as viz_err:
            console.print(f"[warning]Could not create contract interaction graph: {viz_err}", style="warning")

    # --- Create Detailed Call Graph with Plotly ---
    call_graph = None
    if call_data_list:
        try:
            call_graph = create_plotly_call_graph(call_data_list)
        except Exception as viz_err:
            console.print(f"[warning]Could not create detailed call graph: {viz_err}", style="warning")

    # Create PYUSD flow graph if transfers exist
    flow_graph = None
    if pyusd_transfers:
        try:
            flow_graph = create_plotly_flow_graph(pyusd_transfers)
        except Exception as flow_err:
            console.print(f"[warning]Could not create PYUSD flow graph: {flow_err}", style="warning")

    # Create Gas Usage by Category
    gas_category_df = pd.DataFrame(
        [{"category": k, "gas_used": v} for k, v in gas_by_category.items() if v > 0]
    )

    return call_graph, call_df, logs_df, flow_graph, contract_graph, gas_category_df, state_changes_df, pyusd_transfers

# =============================================================================================
# --- Execute callTracer Analysis ---
# =============================================================================================
if 'TARGET_TX_HASH' in locals() and validate_tx_hash: # Use the validation flag from setup cell
    console.print("\n\n[bold]🎯 Using callTracer on Mainnet[/bold]", style="cyan3")
    console.print("───────────────────────────────", style="cyan3")
    console.print(f"Target Transaction : {TARGET_TX_HASH}")

    # Use the trace config from the configuration
    trace_result_call = make_rpc_request("debug_traceTransaction",
                                         [TARGET_TX_HASH, {"tracer": "callTracer", "tracerConfig": TRACE_CONFIGS["callTracer"]}],
                                         network='mainnet')

    if trace_result_call:
        console.print("[success]Successfully received trace data.", style="success")

        # --- Parse and Visualize ---
        try:
            call_graph, call_df, logs_df, pyusd_flow, contract_graph, gas_category_df, state_changes_df, pyusd_transfers = parse_call_trace(trace_result_call, TARGET_TX_HASH)

            # Create output widgets for each visualization (add this right after parsing the trace)
            contract_graph_output = widgets.Output()
            call_graph_output = widgets.Output()
            flow_graph_output = widgets.Output()

            # 1. TRANSACTION OVERVIEW DASHBOARD
            console.print("\n\n[bold cyan3]🔍 PYUSD TRANSACTION ANALYSIS DASHBOARD[/bold cyan3]", justify="left")
            console.print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━", style="cyan3", justify="left")

            if call_df is not None and not call_df.empty:
                # Basic transaction metrics
                tx_stats = {
                    "total_calls": len(call_df),
                    "pyusd_calls": len(call_df[call_df['is_pyusd']]),
                    "max_call_depth": call_df['depth'].max(),
                    "total_gas": call_df['gasUsed'].sum(),
                    "errors": len(call_df[call_df['error'].notnull()])
                }

                stats_table = Table(title="Transaction Metrics", show_header=True, header_style="bold cyan")
                stats_table.add_column("Metric", style="dim")
                stats_table.add_column("Value")

                for k, v in tx_stats.items():
                    if k == 'total_gas':
                        stats_table.add_row(k.replace('_', ' ').title(), f"{v:,} gas units")
                    else:
                        stats_table.add_row(k.replace('_', ' ').title(), str(v))

                console.print(stats_table)

           # 2. Display Contract Interaction Graph
            if contract_graph:
                console.print("\n\n[bold cyan3]📊 Contract Interaction Overview[/bold cyan3]")
                console.print("─────────────────────────────────", style="cyan3")
                try:
                    # Simply display the visualization
                    display(contract_graph)
                    console.print("\n\n[info]This graph shows the high-level interactions between contracts in this transaction.", style="info")
                except Exception as viz_err:
                    console.print(f"[warning]Could not render contract interaction graph: {viz_err}", style="warning")

            # 3. Display Call Graph Visualization
            console.print("\n\n[bold cyan3]📊 Detailed Call Graph Visualization[/bold cyan3]")
            console.print("─────────────────────────────────────", style="cyan3")
            if call_graph:
                try:
                    # Simply display the visualization
                    display(call_graph)
                    console.print("\n\n[info]This visualization shows the detailed call hierarchy in this transaction.", style="info")
                except Exception as viz_err:
                    console.print(f"[warning]Could not render visualization: {viz_err}", style="warning")
            else:
                console.print("[warning]Call graph generation failed.", style="warning")

            # 4. Display PYUSD Flow Graph if available
            if pyusd_flow:
                console.print("\n\n[bold cyan3]🔄 PYUSD Token Flow Analysis[/bold cyan3]")
                console.print("─────────────────────────────", style="cyan3")
                try:
                    # Simply display the visualization
                    display(pyusd_flow)
                    console.print("\n\n[info]This graph shows the movement of PYUSD tokens in this transaction.", style="info")
                except Exception as flow_err:
                    console.print(f"[warning]Could not render PYUSD flow: {flow_err}", style="warning")

            # 5. State Changes Analysis
            if state_changes_df is not None and not state_changes_df.empty:
                console.print("\n\n[bold cyan3]🔄 PYUSD State Changes[/bold cyan3]")
                console.print("───────────────────────", style="cyan3")
                console.print("[info]The following state changes occurred in PYUSD contracts:", style="info")

                # Format amounts in dataframe
                state_changes_df['formatted_amount'] = state_changes_df['amount'].apply(format_value_pyusd)

                # Display state changes with appropriate columns
                display(state_changes_df[['contract', 'function', 'type', 'formatted_amount', 'gas_used']])

                # Summary of state impact
                if 'type' in state_changes_df.columns:
                    changes_by_type = state_changes_df['type'].value_counts()
                    console.print("\n[bold]State Change Summary:[/bold]")
                    for change_type, count in changes_by_type.items():
                        console.print(f"- {change_type.title()}: {count} operations")
            else:
                console.print("\n\n[info]No direct PYUSD state changes detected in this transaction.", style="info")

            # 6. Gas Usage Analysis
            if gas_category_df is not None and not gas_category_df.empty:
                console.print("\n\n[bold yellow3]⛽ Gas Usage Analysis[/bold yellow3]")
                console.print("──────────────────────", style="yellow3")

                # Create simple gas usage table
                gas_table = Table(title="Gas Usage by Operation Category", show_header=True, header_style="bold yellow3")
                gas_table.add_column("Category", style="dim")
                gas_table.add_column("Gas Used", justify="right")
                gas_table.add_column("Percentage", justify="right")

                total_gas = gas_category_df['gas_used'].sum()

                for _, row in gas_category_df.iterrows():
                    category = row['category'].replace('_', ' ').title()
                    gas_used = row['gas_used']
                    percentage = (gas_used / total_gas * 100) if total_gas > 0 else 0
                    gas_table.add_row(
                        category,
                        f"{gas_used:,}",
                        f"{percentage:.1f}%"
                    )

                console.print(gas_table)

                # Gas usage visualization
                try:
                    # Color by operation category for better visibility
                    fig_gas = px.pie(gas_category_df, values='gas_used', names='category',
                                     title=f'<b>Gas Usage Distribution ({shorten_address(TARGET_TX_HASH)})</b>')
                    fig_gas.update_layout(
                        template="plotly_white",
                        title={
                            'y': 0.95,
                            'x': 0.5,
                            'xanchor': 'center',
                            'yanchor': 'top',
                            'font': {'size': 16}
                        },
                        margin=dict(t=100, b=50, l=50, r=50)  # Increased top margin
                    )
                    fig_gas.show()
                except Exception as plot_err:
                    console.print(f"[warning]Could not generate gas usage plot: {plot_err}", style="warning")

            # 7. Filtered Call Data Table
            if call_df is not None and not call_df.empty:
                console.print("\n\n[bold cyan3]📋 Key Contract Calls[/bold cyan3]")
                console.print("─────────────────────", style="cyan3")

                # Focus on PYUSD calls for a cleaner view
                pyusd_calls = call_df[call_df['is_pyusd']]

                if not pyusd_calls.empty:
                    console.print(f"[success]Found {len(pyusd_calls)} PYUSD-related calls in this transaction.", style="success")

                    # Style the DataFrame for better visibility
                    def highlight_pyusd(val):
                        return 'background-color: palegreen' if val else ''

                    # Display with conditional formatting - only important columns
                    display_cols = ['id', 'type', 'depth', 'contract', 'function_category', 'gasUsed', 'error']
                    display(pyusd_calls[display_cols])
                else:
                    console.print("[info]No PYUSD-specific calls found in this transaction.", style="info")

                # Show high gas usage calls with function names
                console.print("\n[bold yellow3]Highest Gas Usage Calls:[/bold yellow3]")
                high_gas_calls = call_df.nlargest(5, 'gasUsed')

                # Add function description to high gas calls
                high_gas_calls['function_name'] = high_gas_calls.apply(
                    lambda row: get_function_description(
                        row['input_preview'],
                        row['is_pyusd'] if 'is_pyusd' in row else False,
                        row['contract'] if 'contract' in row else "Non-PYUSD Contract"
                    ),
                    axis=1
                )

                # Display with function name instead of category if available
                display(high_gas_calls[['id', 'type', 'contract', 'function_name', 'gasUsed']])

            # 8. PYUSD Event Analysis
            if logs_df is not None and not logs_df.empty:
                console.print("\n\n[bold green3]📝 PYUSD Events Analysis[/bold green3]")
                console.print("─────────────────────────", style="green3")

                # Highlight PYUSD logs
                pyusd_logs = logs_df[logs_df['is_pyusd']]
                if not pyusd_logs.empty:
                    console.print(f"[success]Found {len(pyusd_logs)} PYUSD events in this transaction.", style="success")

                    # Group by event type
                    if 'event_name' in pyusd_logs.columns:
                        event_counts = pyusd_logs['event_name'].value_counts()

                        event_table = Table(title="PYUSD Events", show_header=True, header_style="bold green3")
                        event_table.add_column("Event Type", style="dim")
                        event_table.add_column("Count", justify="right")

                        for event, count in event_counts.items():
                            event_table.add_row(event, str(count))

                        console.print(event_table)

                    # Display detailed event data
                    console.print("\n\n[bold green3]PYUSD Event Details:[/bold green3]")
                    display(pyusd_logs[['contract', 'event_name', 'details']])

                    # Transfer value analysis
                    if 'amount' in pyusd_logs.columns and 'is_transfer' in pyusd_logs.columns:
                        transfer_logs = pyusd_logs[pyusd_logs['is_transfer']]
                        if not transfer_logs.empty:
                            total_transferred = transfer_logs['amount'].sum()
                            console.print(f"\n\n[info][bold cyan3]Total PYUSD transferred:[bold cyan3] {format_value_pyusd(total_transferred)}", style="info")
                else:
                    console.print("[info]No PYUSD events found in this transaction.", style="info")
            else:
                console.print("[info]No logs extracted from trace.", style="info")

            # 9. Add Recommendations Section
            console.print("\n\n[bold cyan3]💡 Analysis Observations & Recommendations[/bold cyan3]")
            console.print("──────────────────────────────────────────", style="cyan3")

            recommendations = []

            # Check for high gas usage patterns
            if call_df is not None and not call_df.empty:
                high_gas_threshold = call_df['gasUsed'].sum() * 0.25  # 25% of total gas
                high_gas_ops = call_df[call_df['gasUsed'] > high_gas_threshold]
                if not high_gas_ops.empty:
                    recommendations.append(f"- High gas operations detected: {len(high_gas_ops)} calls used >25% of transaction gas")

            # Check for deep call stack
            if call_df is not None and 'depth' in call_df.columns:
                max_depth = call_df['depth'].max()
                if max_depth > 5:
                    recommendations.append(f"- Deep call stack detected (max depth: {max_depth})")

            # Check token flow patterns
            if logs_df is not None and 'is_transfer' in logs_df.columns:
                transfer_count = logs_df['is_transfer'].sum()
                if transfer_count > 3:
                    recommendations.append(f"- Complex token movement detected ({transfer_count} transfers)")

            # Add general PYUSD observations
            if call_df is not None and 'is_pyusd' in call_df.columns:
                pyusd_calls = call_df[call_df['is_pyusd']]
                if not pyusd_calls.empty:
                    recommendations.append(f"- Transaction involves {len(pyusd_calls)} PYUSD contract interactions")

            # Display recommendations
            if recommendations:
                for rec in recommendations:
                    console.print(rec)
            else:
                console.print("[info]No specific observations to highlight for this transaction.", style="info")

            # 10. Export Options
            console.print("\n\n[bold cyan3]📤 Export Options:[/bold cyan3]")
            console.print("──────────────────", style="cyan3")

            # Create export output area
            export_output = widgets.Output()

            # Create export buttons with proper styling
            export_buttons = widgets.HBox([
                widgets.Button(
                    description='Export to CSV',
                    button_style='primary',  # Green
                    layout=widgets.Layout(width='150px')
                ),
                widgets.Button(
                    description='Export as JSON',
                    button_style='warning',  # Orange
                    layout=widgets.Layout(width='150px')
                ),
                widgets.Button(
                    description='Export to Google Sheets',
                    button_style='info',     # Blue
                    layout=widgets.Layout(width='200px')
                )
            ])

            # Define export handlers with simplified loading indicators
            def export_csv(b):
                with export_output:
                    clear_output()
                    console.print("[cyan3]Exporting to CSV...", style="info")

                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                    filename = f"calltrace_{TARGET_TX_HASH[:10]}_{timestamp}.csv"

                    try:
                        # Export to CSV
                        result = download_csv_direct(call_df, filename)

                        # Clear loading message and show success
                        clear_output()
                        console.print("✓ Successfully exported to CSV", style="spring_green3")
                        display(result)
                    except Exception as e:
                        clear_output()
                        console.print(f"❌ Error exporting to CSV: {str(e)}", style="error")

            def export_json(b):
                with export_output:
                    clear_output()
                    console.print("[cyan3]Exporting to JSON...", style="info")

                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                    filename = f"calltrace_{TARGET_TX_HASH[:10]}_{timestamp}.json"

                    try:
                        # Prepare export data
                        export_data = {
                            "transaction_hash": TARGET_TX_HASH,
                            "analysis_type": "callTracer",
                            "transaction_stats": tx_stats if 'tx_stats' in locals() else {},
                            "state_changes": state_changes_df.to_dict('records') if state_changes_df is not None and not state_changes_df.empty else [],
                            "gas_by_category": gas_category_df.to_dict('records') if gas_category_df is not None and not gas_category_df.empty else [],
                            "pyusd_transfers": pyusd_transfers if pyusd_transfers is not None else []
                        }

                        # Export to JSON
                        result = download_json_direct(export_data, filename)

                        # Clear loading message and show success
                        clear_output()
                        console.print("✓ Successfully exported to JSON", style="spring_green3")
                        display(result)
                    except Exception as e:
                        clear_output()
                        console.print(f"❌ Error exporting to JSON: {str(e)}", style="error")

            def export_to_sheets(b):
                with export_output:
                    clear_output()
                    try:
                        # Collect all data for the sheet
                        export_data = {
                            "transaction_hash": TARGET_TX_HASH,
                            "transaction_stats": tx_stats if 'tx_stats' in locals() else {},
                            "gas_distribution": gas_category_df.to_dict('records') if gas_category_df is not None else [],
                            "pyusd_transfers": pyusd_transfers if pyusd_transfers is not None else [],
                            "state_changes": state_changes_df.to_dict('records') if state_changes_df is not None and not state_changes_df.empty else [],
                            "logs_df": logs_df if logs_df is not None and not logs_df.empty else None
                        }

                        # Call the export function
                        spreadsheet_url, sheet_title = export_to_google_sheets(call_df, export_data, TARGET_TX_HASH)

                        # Open the spreadsheet and display link
                        display(Javascript(f'window.open("{spreadsheet_url}", "_blank");'))

                        # 2. Display *only* the link and success message as static HTML output
                        clear_output(wait=True)
                        console.print("✓ Successfully exported to Google Sheets", style="spring_green3")
                        display(HTML(f'''
                        <div>Spreadsheet created and opened: <a href="{spreadsheet_url}" target="_blank">{sheet_title}</a></div>
                        '''))

                    except Exception as e:
                        clear_output()
                        console.print(f"❌ Error exporting to Google Sheets: {str(e)}", style="error")
                        html = f"<div style='color:red'>Error exporting to Google Sheets: {str(e)}</div>"
                        display(HTML(html))

                        # Fallback to CSV
                        console.print("[cyan3]Falling back to CSV download...", style="info")
                        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                        filename = f"calltrace_{TARGET_TX_HASH[:10]}_{timestamp}.csv"
                        result = download_csv_direct(call_df, filename)

                        clear_output()
                        console.print("✓ CSV fallback ready", style="spring_green3")
                        display(result)
                        display(HTML("<div>Falling back to CSV download due to Google Sheets error.</div>"))

            # Connect handlers to buttons
            export_buttons.children[0].on_click(export_csv)
            export_buttons.children[1].on_click(export_json)
            export_buttons.children[2].on_click(export_to_sheets)

            # Display button container and output area
            display(export_buttons)
            display(export_output)

        except Exception as parse_err:
            console.print(f"[error]Failed during parsing or visualization: {parse_err}", style="error")
            import traceback
            console.print(traceback.format_exc())
    else:
        console.print("[error]Failed to retrieve trace data from RPC node.", style="error")
else:
    console.print("[warning]No valid transaction hash found. Please set TARGET_TX_HASH to a valid transaction hash.", style="warning")

### 1.2.2 Using `structLog` Tracer: Opcode-Level Execution Analysis (Use Cautiously)

The `structLog` tracer dives deep into the Ethereum Virtual Machine (EVM), providing a step-by-step log of each opcode executed during the transaction. For each step, it details the program counter (PC), opcode, remaining gas, gas cost, stack contents, memory state, and storage changes (if enabled).

> **⚠️ Extreme Granularity & Resource Intensity**
>
> *   **Method:** `debug_traceTransaction` with `tracer: "structLog"`
> *   **Multiplier:** `50x` (Same base multiplier as `callTracer`, but output size is *significantly* larger)
> *   **Output Size:** Can generate **very large** outputs (potentially hundreds of thousands of steps/lines, consuming significant memory/browser resources) for even moderately complex transactions.
> *   **Use Case:** Best suited for highly specific debugging tasks, deep gas optimization analysis at the opcode level, or verifying precise execution paths, rather than general transaction overview. **Use with caution.**

> **🚀 Leveraging GCP's Premium RPC Capabilities**
>
> *   **GCP Advantage:** While the base multiplier is `50x`, the sheer volume of data returned by `structLog` makes it extremely demanding on RPC node resources. GCP's infrastructure and generous quotas make it feasible to retrieve these detailed traces where other providers might time out, restrict output size, or charge heavily.
> *   **PYUSD Insight:** `structLog` can be used for:
>     *   **Fine-grained Gas Profiling:** Identifying exactly which opcodes consume the most gas during PYUSD function execution (e.g., `SSTORE` during transfers/approvals).
>     *   **Debugging Reverts:** Pinpointing the exact opcode and state (stack/memory) where a PYUSD transaction failed.
>     *   **Security Analysis:** Examining low-level execution for potential vulnerabilities or unexpected behavior within PYUSD or interacting contracts.

**Analysis Workflow:**

1.  **Conditional Execution:** The code includes a flag (`RUN_STRUCTLOG_TRACE`) which defaults to `False`. Set this to `True` only if you specifically need this level of detail.
2.  **Fetch Trace:** Calls `debug_traceTransaction` using `TARGET_TX_HASH` and the `structLog` configuration.
3.  **Parse Steps:** The `parse_struct_log` function processes the potentially massive list of execution steps.
4.  **Summarize & Visualize:**
    *   **Overview:** Displays total steps, gas cost, call depth, and highlights PYUSD execution percentage.
    *   **Gas Analysis:** Generates plots showing gas cost distribution by opcode *category* and by individual *opcode*.
    *   **Execution Timeline:** Plots gas remaining over execution steps, highlighting sections executed within PYUSD contracts.
    *   **PYUSD Focus:** Analyzes opcode frequency and gas usage specifically within PYUSD contract execution steps.
    *   **Data Sample:** Displays the first few steps of the detailed execution trace DataFrame.
    *   **Export Options:** Allows downloading the full (potentially large) execution step data.

**💡 What to Look For:**
*   **Gas Plots:** Identify which opcode categories (e.g., `STORAGE`, `MEMORY`, `CALL`) and specific opcodes (e.g., `SSTORE`, `CALL`, `KECCAK256`) dominate gas consumption. Compare this within PYUSD vs. non-PYUSD sections.
*   **Execution Timeline:** Observe gas depletion patterns. Look for sharp drops corresponding to expensive operations. Note the percentage of time spent executing PYUSD code.
*   **Data Sample:** Understand the structure of the per-opcode data (PC, gas, stack, memory).
*   **(If Debugging):** Search the full trace data (if exported) for `REVERT` opcodes or unexpected stack/memory states near the point of failure.

In [ ]:
# =============================================================================================
# 🔬 Trace Transaction using debug_traceTransaction (structLog Tracer Opcode-Level Detail)
# =============================================================================================
# This cell performs detailed opcode-level transaction analysis using the debug_traceTransaction method with the structLog tracer.
# It prepares the data and presents insights by:
# - Conditionally executing the trace based on the RUN_STRUCTLOG_TRACE flag.
# - Parsing the raw structLog output, calculating step-by-step gas costs, categorizing opcodes, and tracking contract execution context (including PYUSD).
# - Generating interactive Plotly visualizations for overall gas usage (by category, by opcode), execution timeline (highlighting PYUSD), and PYUSD-specific gas analysis.
# - Displaying summary statistics (Panel, Rich tables) and a sample of the processed trace data.
# - Providing interactive buttons for exporting the detailed trace data to CSV, JSON, or Google Sheets using helper functions.

import base64
import json
from datetime import datetime
from IPython.display import HTML, display
import ipywidgets as widgets
from IPython.display import clear_output

def download_csv_direct(df, filename=None):
    """Creates a direct download for CSV without intermediate display."""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"pyusd_data_{timestamp}.csv"

    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()

    # Create direct download HTML
    html = f'''
    <script>
    function download(filename, data) {{
        const a = document.createElement('a');
        a.href = data;
        a.download = filename;
        document.body.appendChild(a);
        a.click();
        document.body.removeChild(a);
    }}

    download("{filename}", "data:text/csv;base64,{payload}");
    </script>
    <div>Downloading {filename}...</div>
    '''
    return HTML(html)

def download_json_direct(data, filename=None):
    """Creates a direct download for JSON without intermediate display."""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"pyusd_data_{timestamp}.json"

    # Convert to JSON string (handling non-serializable objects)
    json_str = json.dumps(data, default=str, indent=2)
    b64 = base64.b64encode(json_str.encode()).decode()

    # Create direct download HTML
    html = f'''
    <script>
    function download(filename, data) {{
        const a = document.createElement('a');
        a.href = data;
        a.download = filename;
        document.body.appendChild(a);
        a.click();
        document.body.removeChild(a);
    }}

    download("{filename}", "data:application/json;base64,{b64}");
    </script>
    <div>Downloading {filename}...</div>
    '''
    return HTML(html)

def export_to_google_sheets(df, data_dict, tx_hash):
    """Export StructLog analysis data to Google Sheets with rich formatting."""
    # Show loading message
    console.print("[cyan3]Exporting to Google Sheets...", style="info")

    try:
        # Create a new Google Sheet with meaningful title
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        sheet_title = f"PYUSD StructLog Analysis {tx_hash[:10]} {timestamp}"

        # Use the global gc_sheets client that's already authenticated
        spreadsheet = gc_sheets.create(sheet_title)

        # Get the default worksheet and rename it
        worksheet = spreadsheet.get_worksheet(0)
        worksheet.update_title("Execution Steps")

        # Set up a header with transaction info
        header_values = [
            ["PYUSD StructLog Analysis"],
            [f"Transaction: {tx_hash}"],
            [f"Analysis Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"],
            [""],  # Empty row for spacing
        ]
        worksheet.update("A1", header_values)

        # Format the header with bold text and colored background
        worksheet.format("A1:A1", {
            "textFormat": {"bold": True, "fontSize": 14},
            "backgroundColor": {"red": 0.9, "green": 0.9, "blue": 1.0}
        })

        worksheet.format("A2:A3", {
            "textFormat": {"bold": True, "fontSize": 12}
        })

        current_row = 5  # Start after header

        # Add transaction stats summary
        if "summary" in data_dict:
            stats = data_dict["summary"]

            # Add section title
            worksheet.update(f"A{current_row}", [["Analysis Summary"]])
            worksheet.format(f"A{current_row}:A{current_row}", {
                "textFormat": {"bold": True, "fontSize": 12},
                "backgroundColor": {"red": 0.8, "green": 0.9, "blue": 1.0}
            })
            current_row += 1

            # Add stats data
            stats_rows = []
            stats_rows.append(["Metric", "Value"])  # Header row
            for key, value in stats.items():
                # Format keys and values appropriately
                formatted_key = key.replace("_", " ").title()

                # Try to format numerical values with commas
                try:
                    if isinstance(value, (int, float)):
                        formatted_value = f"{value:,}"
                    else:
                        formatted_value = str(value)
                except:
                    formatted_value = str(value)

                stats_rows.append([formatted_key, formatted_value])

            # Add stats table
            stats_start_row = current_row
            worksheet.update(f"A{stats_start_row}", stats_rows)

            # Format stats table header
            worksheet.format(f"A{stats_start_row}:B{stats_start_row}", {
                "textFormat": {"bold": True},
                "backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}
            })

            current_row += len(stats_rows) + 2  # Add extra space after table

        # Add opcode distribution reference
        worksheet.update(f"A{current_row}", [["Opcode Distribution Analysis"]])
        worksheet.format(f"A{current_row}:A{current_row}", {
            "textFormat": {"bold": True, "fontSize": 12},
            "backgroundColor": {"red": 0.7, "green": 0.9, "blue": 1.0}
        })
        current_row += 1

        worksheet.update(f"A{current_row}", [["📊 Opcode distribution visualizations are available in the notebook"]])
        current_row += 2

        # Add gas usage reference
        worksheet.update(f"A{current_row}", [["Gas Usage Analysis"]])
        worksheet.format(f"A{current_row}:A{current_row}", {
            "textFormat": {"bold": True, "fontSize": 12},
            "backgroundColor": {"red": 1.0, "green": 0.9, "blue": 0.7}
        })
        current_row += 1

        worksheet.update(f"A{current_row}", [["📊 Gas usage visualizations are available in the notebook"]])
        current_row += 2

        # Add main DataFrame data
        if not df.empty:
            # Add a section title
            worksheet.update(f"A{current_row}", [["Execution Steps Data"]])
            worksheet.format(f"A{current_row}:A{current_row}", {
                "textFormat": {"bold": True, "fontSize": 12},
                "backgroundColor": {"red": 0.8, "green": 0.8, "blue": 1.0}
            })
            current_row += 1

            # For StructLog data, select most important columns for readability
            # if dataframe has too many columns
            if len(df.columns) > 10:
                key_cols = ["pc", "op", "gas", "gasCost", "depth", "stack", "memory", "storage"]
                display_cols = [col for col in key_cols if col in df.columns]

                # Add any custom columns that might contain analysis results
                other_important_cols = []
                for col in df.columns:
                    if col not in display_cols and any(x in col.lower() for x in ["pyusd", "token", "contract", "note", "category"]):
                        other_important_cols.append(col)

                display_cols.extend(other_important_cols)
                display_df = df[display_cols]
            else:
                display_df = df

            # Convert DataFrame to list of lists for the worksheet
            df_values = [display_df.columns.tolist()] + display_df.values.tolist()

            # Format values for better readability
            for i in range(1, len(df_values)):
                for j, col in enumerate(display_df.columns):
                    val = df_values[i][j]

                    # Format different column types appropriately
                    if pd.isnull(val):
                        df_values[i][j] = ""
                    elif col in ["gas", "gasCost"] and isinstance(val, (int, float)):
                        df_values[i][j] = f"{val:,}"
                    elif col in ["stack", "memory", "storage"] and isinstance(val, (list, dict)):
                        # Truncate long data structures to prevent huge cells
                        df_values[i][j] = str(val)[:100] + "..." if len(str(val)) > 100 else str(val)
                    else:
                        df_values[i][j] = str(val)

            worksheet.update(f"A{current_row}", df_values)

            # Format the DataFrame header
            worksheet.format(f"A{current_row}:{chr(65+len(display_df.columns)-1)}{current_row}", {
                "textFormat": {"bold": True},
                "backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}
            })

            # Add alternating row colors for readability
            data_rows = len(df_values)
            for i in range(2, data_rows + 1, 2):
                row_num = current_row + i - 1
                worksheet.format(f"A{row_num}:{chr(65+len(display_df.columns)-1)}{row_num}", {
                    "backgroundColor": {"red": 0.97, "green": 0.97, "blue": 1.0}
                })

        # Try to auto-resize columns for better readability
        try:
            worksheet.columns_auto_resize(0, 10)  # Attempt to resize first 10 columns
        except:
            pass  # Ignore if not supported

        # Clear loading message and show success message
        clear_output()
        console.print("✓ Successfully exported to Google Sheets", style="spring_green3")

        # Open the spreadsheet in a new tab
        spreadsheet_url = f"https://docs.google.com/spreadsheets/d/{spreadsheet.id}"
        html = f'''
        <script>
        window.open("{spreadsheet_url}", "_blank");
        </script>
        <div>Spreadsheet created and opened: <a href="{spreadsheet_url}" target="_blank">{sheet_title}</a></div>
        '''
        return HTML(html)

    except Exception as e:
        # Clear loading message and show error
        clear_output()
        console.print(f"❌ Error creating Google Sheet: {str(e)}", style="error")
        return HTML(f"<div style='color:red'>Error creating Google Sheet: {str(e)}</div>")

def parse_struct_log(struct_logs_list, tx_hash):
    """Parses structLog output for analysis and visualization."""
    if not struct_logs_list or not isinstance(struct_logs_list, list):
        console.print("[warning]No structLog data provided or invalid format.", style="warning")
        return None

    # --- Initialize tracking variables ---
    log_data = []
    total_gas_cost = 0
    last_gas = 0
    current_contracts = {}  # Map depth -> contract address
    pyusd_execution_steps = 0

    # --- Define OPCODE categories for better analysis ---
    OPCODE_CATEGORIES = {
        "arithmetic": ["ADD", "MUL", "SUB", "DIV", "SDIV", "MOD", "SMOD", "ADDMOD", "MULMOD", "EXP", "SIGNEXTEND"],
        "comparison": ["LT", "GT", "SLT", "SGT", "EQ", "ISZERO"],
        "bitwise": ["AND", "OR", "XOR", "NOT", "BYTE", "SHL", "SHR", "SAR"],
        "memory": ["MLOAD", "MSTORE", "MSTORE8", "MSIZE", "MCOPY"],
        "storage": ["SLOAD", "SSTORE"],
        "flow": ["JUMP", "JUMPI", "JUMPDEST", "PC", "STOP", "RETURN", "REVERT"],
        "stack": ["POP", "PUSH1", "PUSH2", "PUSH3", "PUSH4", "PUSH5", "PUSH6", "PUSH7", "PUSH8",
                 "PUSH9", "PUSH10", "PUSH11", "PUSH12", "PUSH13", "PUSH14", "PUSH15", "PUSH16",
                 "PUSH17", "PUSH18", "PUSH19", "PUSH20", "PUSH21", "PUSH22", "PUSH23", "PUSH24",
                 "PUSH25", "PUSH26", "PUSH27", "PUSH28", "PUSH29", "PUSH30", "PUSH31", "PUSH32",
                 "DUP1", "DUP2", "DUP3", "DUP4", "DUP5", "DUP6", "DUP7", "DUP8",
                 "DUP9", "DUP10", "DUP11", "DUP12", "DUP13", "DUP14", "DUP15", "DUP16",
                 "SWAP1", "SWAP2", "SWAP3", "SWAP4", "SWAP5", "SWAP6", "SWAP7", "SWAP8",
                 "SWAP9", "SWAP10", "SWAP11", "SWAP12", "SWAP13", "SWAP14", "SWAP15", "SWAP16"],
        "environment": ["ADDRESS", "BALANCE", "ORIGIN", "CALLER", "CALLVALUE", "CALLDATALOAD",
                        "CALLDATASIZE", "CALLDATACOPY", "CODESIZE", "CODECOPY", "GASPRICE",
                        "EXTCODESIZE", "EXTCODECOPY", "RETURNDATASIZE", "RETURNDATACOPY",
                        "EXTCODEHASH", "BLOCKHASH", "COINBASE", "TIMESTAMP", "NUMBER",
                        "DIFFICULTY", "GASLIMIT", "CHAINID", "SELFBALANCE", "BASEFEE"],
        "contract": ["CREATE", "CREATE2", "CALL", "CALLCODE", "DELEGATECALL", "STATICCALL", "SELFDESTRUCT"],
        "logging": ["LOG0", "LOG1", "LOG2", "LOG3", "LOG4"],
        "gas": ["GAS"],
        "other": []
    }

    def get_opcode_category(opcode):
        """Determine the category of an opcode based on predefined categories."""
        for category, opcodes in OPCODE_CATEGORIES.items():
            if opcode in opcodes:
                return category
        return "other"

    console.print(f"[info]Parsing {len(struct_logs_list):,} structLog steps for {shorten_address(tx_hash)}\n\n", style="info")

    # Get initial gas from the first step if available
    if struct_logs_list and isinstance(struct_logs_list[0], dict) and 'gas' in struct_logs_list[0]:
        last_gas = struct_logs_list[0].get('gas', 0)

    # --- Process each execution step in the struct logs ---
    for i, step in enumerate(struct_logs_list):
        # Ensure step is a dictionary
        if not isinstance(step, dict):
            continue

        # Calculate gas cost for this step
        current_gas = step.get('gas', last_gas)
        gas_cost = last_gas - current_gas
        total_gas_cost += gas_cost if gas_cost > 0 else 0

        # Get basic step information
        depth = step.get('depth', 0)
        op = step.get('op', 'N/A')

        # Track contract context changes on CALL instructions
        is_pyusd_related = False
        if op in ['CALL', 'STATICCALL', 'DELEGATECALL']:
            try:
                stack = step.get('stack', [])
                if len(stack) >= 2:  # Need at least 2 stack items for call address
                    # Address is the second stack item for CALL, STATICCALL
                    address_raw = stack[1]
                    if address_raw.startswith('0x'):
                        address = '0x' + address_raw[-40:]
                    else:
                        # Handle numeric representation
                        try:
                            address = '0x' + hex(int(address_raw, 16))[-40:].zfill(40)
                        except ValueError:
                            address = None

                    if address:
                        current_contracts[depth+1] = address.lower()
                        is_pyusd_related = is_pyusd_contract(address)
            except Exception:
                pass

        # Check if we're currently in a PYUSD contract
        current_contract = current_contracts.get(depth, None)
        is_in_pyusd = is_pyusd_contract(current_contract) if current_contract else False

        if is_in_pyusd:
            pyusd_execution_steps += 1

        # Categorize the opcode
        opcode_category = get_opcode_category(op)

        # Build the execution data record
        log_data.append({
            'step': i,
            'pc': step.get('pc', 0),
            'op': op,
            'opcode_category': opcode_category,
            'gas': current_gas,
            'gasCost': gas_cost if gas_cost >= 0 else 0,  # Ensure non-negative cost
            'depth': depth,
            'stack_depth': len(step.get('stack', [])),
            'mem_size_bytes': len(step.get('memory', [])) * 32,  # Memory size in bytes
            'current_contract': current_contract,
            'is_pyusd_contract': is_in_pyusd,
            'is_pyusd_related': is_pyusd_related or is_in_pyusd
        })
        last_gas = current_gas

    if not log_data:
        console.print("[warning]No valid steps found in structLog data after parsing.", style="warning")
        return None

    # --- Convert to DataFrame for analysis ---
    df = pd.DataFrame(log_data)

    # --- Display Summary ---
    pyusd_percentage = (pyusd_execution_steps / len(df) * 100) if len(df) > 0 else 0

    console.print(Panel(f"""
[bold]structLog Trace Summary for {shorten_address(tx_hash)}[/bold]
Total Steps Parsed: {len(df):,}
Total Gas Cost (calc): {total_gas_cost:,}
Max Depth: {df['depth'].max() if not df.empty else 'N/A'}
Max Stack Depth: {df['stack_depth'].max() if not df.empty else 'N/A'}
Max Memory (bytes): {df['mem_size_bytes'].max() if not df.empty else 'N/A'}
Steps in PYUSD Contracts: {pyusd_execution_steps:,} ({pyusd_percentage:.1f}% of execution)""",
        title="structLog Overview", border_style="cyan3", expand=False))

    # --- Generate Visualizations ---
    if not df.empty:
        # --- Plot Gas Cost by Opcode Category ---
        try:
            # Visualization Header
            console.print("\n\n[bold yellow3]⛽ Gas Cost by Opcode Category[/bold yellow3]")
            console.print("────────────────────────────────", style="yellow3")

            # Calculate opcode category gas costs
            category_gas = df.groupby('opcode_category')['gasCost'].sum().reset_index()
            if not category_gas.empty:
                # Sort categories by gas usage for better visualization
                category_gas = category_gas.sort_values(by='gasCost', ascending=False)

                # Add percentage to the name
                total_gas = category_gas['gasCost'].sum()
                category_gas['name_with_pct'] = category_gas.apply(
                    lambda x: f"{x['opcode_category']} ({x['gasCost']/total_gas*100:.1f}%)",
                    axis=1
                )

                fig_cat_gas = px.pie(
                    category_gas,
                    values='gasCost',
                    names='name_with_pct',  # Use the new column with percentages
                    title=' '  # Empty title, we'll add it in layout
                )

                # Custom settings for better visualization
                fig_cat_gas.update_traces(
                    textposition='inside',
                    textinfo='percent',
                    insidetextorientation='radial'
                )

                # Position the title with extra space and make it bold
                fig_cat_gas.update_layout(
                    title={
                        'text': f'<b>Gas Cost by Opcode Category ({shorten_address(tx_hash)})</b>',  # Bold title
                        'y': 0.95,  # Position higher for more space
                        'x': 0.5,
                        'xanchor': 'center',
                        'yanchor': 'top',
                        'font': {'size': 16}
                    },
                    margin=dict(t=100, b=50, l=50, r=50),  # Increased top margin for title
                )
                fig_cat_gas.show()
        except Exception as plot_err:
            console.print(f"[warning]Could not generate opcode category gas plot: {plot_err}", style="warning")

        # --- Plot Gas Cost per Opcode ---
        try:
            # Visulization Header
            console.print("\n\n[bold yellow3]⛽ Top 30 Opcodes by Gas Cost[/bold yellow3]")
            console.print("──────────────────────────────", style="yellow3")
            # Calculate opcode gas costs
            opcode_gas = df.groupby('op')['gasCost'].sum().sort_values(ascending=False).reset_index()
            # Only show opcodes with non-zero gas cost (as requested)
            opcode_gas = opcode_gas[opcode_gas['gasCost'] > 0]

            if not opcode_gas.empty:
                # Show all significant opcodes, not just top 30
                significant_opcodes = len(opcode_gas) if len(opcode_gas) <= 30 else 30

                fig_op_gas = px.bar(
                    opcode_gas.head(significant_opcodes),
                    x='op',
                    y='gasCost',
                    labels={'op': '<b>Opcode</b>', 'gasCost': '<b>Total Gas Cost</b>'}  # Bold axis labels
                )

                # Bold title
                fig_op_gas.update_layout(
                    title={
                        'text': f'<b>Top {significant_opcodes} Opcodes by Gas Cost ({shorten_address(tx_hash)})</b>',
                        'y': 0.95,
                        'x': 0.5,
                        'xanchor': 'center',
                        'yanchor': 'top',
                        'font': {'size': 16}
                    },
                    margin=dict(t=100, b=50, l=50, r=50)  # Increased top margin
                )
                fig_op_gas.show()
            else:
                 console.print("[info]No significant gas costs found per opcode.", style="info")
        except Exception as plot_err:
             console.print(f"[warning]Could not generate opcode gas plot: {plot_err}", style="warning")

        # --- Plot Gas Remaining Over Steps with PYUSD Highlighting ---
        try:
            # Downsample if too many steps to prevent browser freezing
            # ⚠️ WARNING: if there are too many steps it will consume lots of computes browser could crash or freeze.
            max_points_plot = 5000
            if len(df) > max_points_plot:
                # Ensure consistent sampling, e.g., take every Nth point
                indices = np.round(np.linspace(0, len(df) - 1, max_points_plot)).astype(int)
                plot_df = df.iloc[indices]
                plot_title = f'<b>Gas Remaining Over Execution Steps (Sampled) ({shorten_address(tx_hash)})</b>'
                xaxis_title = '<b>Execution Step (Sampled)</b>'
                console.print(
                    f"\n\n[bold yellow3]⛽ Plotting downsampled gas remaining[/bold yellow3] [info]({len(plot_df)} points out of {len(df)}).[/info]\n"
                    f"[yellow3]────────────────────────────────────────────────────────────────────[/yellow3]"
                )
            else:
                plot_df = df
                plot_title = f'<b>Gas Remaining Over Execution Steps ({shorten_address(tx_hash)})</b>'
                xaxis_title = '<b>Execution Step</b>'

            # Create a more informative plot with PYUSD sections highlighted
            fig_gas_steps = go.Figure()

            # Add base gas trace
            fig_gas_steps.add_trace(go.Scatter(
                x=plot_df['step'],
                y=plot_df['gas'],
                mode='lines',
                name='Gas Remaining',
                line=dict(color='blue')
            ))

            # Highlight PYUSD contract execution sections
            if 'is_pyusd_contract' in plot_df.columns:
                pyusd_sections = []
                current_section = None

                for i, row in plot_df.iterrows():
                    if row['is_pyusd_contract'] and current_section is None:
                        # Start a new PYUSD section
                        current_section = {'start': row['step']}
                    elif not row['is_pyusd_contract'] and current_section is not None:
                        # End the current PYUSD section
                        current_section['end'] = plot_df.iloc[i-1]['step'] if i > 0 else row['step']
                        pyusd_sections.append(current_section)
                        current_section = None

                # Handle case where the last section is a PYUSD section
                if current_section is not None:
                    current_section['end'] = plot_df.iloc[-1]['step']
                    pyusd_sections.append(current_section)

                # Add highlighted areas for PYUSD execution
                for section in pyusd_sections:
                    fig_gas_steps.add_shape(
                        type="rect",
                        x0=section['start'], x1=section['end'],
                        y0=0, y1=plot_df['gas'].max(),
                        fillcolor="rgba(0,255,0,0.1)",
                        line=dict(width=0),
                        layer="below"
                    )

            fig_gas_steps.update_layout(
                title={
                    'text': plot_title,
                    'y': 0.95,
                    'x': 0.5,
                    'xanchor': 'center',
                    'yanchor': 'top',
                    'font': {'size': 16}
                },
                xaxis_title=xaxis_title,
                yaxis_title='<b>Gas</b>',  # Bold y-axis label
                showlegend=True,
                margin=dict(t=100, b=60, l=60, r=60)  # Increased top margin
            )

            # Add annotation for PYUSD sections - MOVED TO BOTTOM LEFT
            if pyusd_percentage > 0:
                fig_gas_steps.add_annotation(
                    x=0.02, y=0.02,  # Bottom left
                    xref="paper", yref="paper",
                    text=f"Green sections: PYUSD contract execution ({pyusd_percentage:.1f}%)",
                    showarrow=False,
                    font=dict(color="green"),
                    bgcolor="white",
                    bordercolor="green",
                    borderwidth=1,
                    align="left"
                )

            fig_gas_steps.show()
        except Exception as plot_err:
             console.print(f"[warning]Could not generate gas remaining plot: {plot_err}", style="warning")

        # --- PYUSD-Specific Analysis ---
        if 'is_pyusd_contract' in df.columns and df['is_pyusd_contract'].any():
            console.print("\n[bold green3]🧩 PYUSD Contract Execution Analysis[/bold green3]")

            # Filter for PYUSD execution steps
            pyusd_df = df[df['is_pyusd_contract']]

            # Analyze PYUSD opcodes
            pyusd_opcodes = pyusd_df.groupby('op').size().sort_values(ascending=False)

            pyusd_table = Table(title="Top PYUSD Contract Operations", show_header=True, header_style="bold green3")
            pyusd_table.add_column("Opcode", style="dim")
            pyusd_table.add_column("Count", justify="right")
            pyusd_table.add_column("% of PYUSD Ops", justify="right")

            for op, count in pyusd_opcodes.head(10).items():
                percentage = (count / len(pyusd_df) * 100)
                pyusd_table.add_row(op, str(count), f"{percentage:.1f}%")

            console.print(pyusd_table)

            # Analyze PYUSD gas usage by category
            pyusd_gas_by_cat = pyusd_df.groupby('opcode_category')['gasCost'].sum().sort_values(ascending=False)

            try:
                # Create a pie chart for PYUSD gas usage by category
                gas_cat_df = pd.DataFrame({'category': pyusd_gas_by_cat.index, 'gas_used': pyusd_gas_by_cat.values})

                # Add percentage to the name
                total_gas = gas_cat_df['gas_used'].sum()
                gas_cat_df['name_with_pct'] = gas_cat_df.apply(
                    lambda x: f"{x['category']} ({x['gas_used']/total_gas*100:.1f}%)",
                    axis=1
                )

                fig_pyusd_gas = px.pie(
                    gas_cat_df,
                    values='gas_used',
                    names='name_with_pct',
                    title=' '  # Empty title, we'll add it in layout
                )

                # Custom settings for better visualization
                fig_pyusd_gas.update_traces(
                    textposition='inside',
                    textinfo='percent',
                    insidetextorientation='radial'
                )

                # Bold title
                fig_pyusd_gas.update_layout(
                    title={
                        'text': f'<b>PYUSD Contract Gas Usage by Category ({shorten_address(tx_hash)})</b>',
                        'y': 0.95,
                        'x': 0.5,
                        'xanchor': 'center',
                        'yanchor': 'top',
                        'font': {'size': 16}
                    },
                    margin=dict(t=100, b=50, l=50, r=50)  # Increased top margin
                )

                fig_pyusd_gas.show()
            except Exception as plot_err:
                console.print(f"[warning]Could not generate PYUSD gas category plot: {plot_err}", style="warning")

        # Display DataFrame sample first
        console.print("\n\n[bold cyan3]📊 structLog Data Sample (First 10 steps)[/bold cyan3]")
        console.print("──────────────────────────────────────────", style="cyan3")

        # Get a more informative view by focusing on key columns
        display_cols = ['step', 'op', 'opcode_category', 'gas', 'gasCost', 'depth', 'is_pyusd_contract']
        display(df[display_cols].head(10))

        console.print(f"[info]Full structLog DataFrame has {len(df)} rows. Displaying only head.", style="info")

        # PYUSD-specific summary
        pyusd_steps = df['is_pyusd_contract'].sum()
        if pyusd_steps > 0:
            pyusd_pct = (pyusd_steps / len(df)) * 100
            console.print(f"[success]PYUSD-specific execution: {pyusd_steps:,} steps ({pyusd_pct:.1f}% of total)", style="success")

            # Top Gas-Consuming PYUSD Operations
            if 'is_pyusd_contract' in df.columns:
                pyusd_ops_gas = df[df['is_pyusd_contract']].groupby('op')['gasCost'].sum().sort_values(ascending=False)

                if not pyusd_ops_gas.empty:
                    gas_table = Table(title="Top Gas-Consuming PYUSD Operations", show_header=True, header_style="bold green")
                    gas_table.add_column("Operation", style="dim")
                    gas_table.add_column("Gas Used", justify="right")
                    gas_table.add_column("% of PYUSD Gas", justify="right")

                    total_pyusd_gas = pyusd_ops_gas.sum()
                    for op, gas in pyusd_ops_gas.head(10).items():
                        percentage = (gas / total_pyusd_gas * 100)
                        gas_table.add_row(op, f"{gas:,}", f"{percentage:.1f}%")

                    console.print(gas_table)

        # Export options
        console.print("\n\n[bold cyan3]📤 Export Options:[/bold cyan3]")
        console.print("──────────────────", style="cyan3")

        # Create export output area
        export_output = widgets.Output()

        # Create export buttons with proper styling
        export_buttons = widgets.HBox([
            widgets.Button(
                description='Export to CSV',
                button_style='primary',  # Green
                layout=widgets.Layout(width='150px')
            ),
            widgets.Button(
                description='Export as JSON',
                button_style='warning',  # Orange
                layout=widgets.Layout(width='150px')
            ),
            widgets.Button(
                description='Export to Google Sheets',
                button_style='info',     # Blue
                layout=widgets.Layout(width='200px')
            )
        ])

        # Define export handlers
        def export_csv(b):
            with export_output:
                clear_output()
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"structlog_trace_{tx_hash[:10]}_{timestamp}.csv"
                display(download_csv_direct(df, filename))

        def export_json(b):
            with export_output:
                clear_output()
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"structlog_trace_{tx_hash[:10]}_{timestamp}.json"
                # Prepare export data with basic DataFrame stats to avoid reference issues
                export_data = {
                    "transaction_hash": tx_hash,
                    "analysis_type": "structLog",
                    "summary": {
                        "total_steps": len(df),
                        "total_gas_cost": total_gas_cost,
                        "max_depth": df['depth'].max() if not df.empty else None,
                        "max_stack_depth": df['stack_depth'].max() if not df.empty else None,
                        "max_memory_bytes": df['mem_size_bytes'].max() if not df.empty else None,
                        "pyusd_steps": pyusd_execution_steps,
                        "pyusd_percentage": pyusd_percentage
                    },
                    "opcode_categories": category_gas.to_dict('records') if 'category_gas' in locals() and not category_gas.empty else []
                }
                display(download_json_direct(export_data, filename))

        def export_to_sheets(b):
            with export_output:
                clear_output()
                try:
                    # Only use variables that are in scope and defined
                    export_data = {
                        "transaction_hash": tx_hash,
                        "summary": {
                            "total_steps": len(df),
                            "total_gas_cost": total_gas_cost,
                            "max_depth": df['depth'].max() if not df.empty else None,
                            "max_stack_depth": df['stack_depth'].max() if not df.empty else None,
                            "max_memory_bytes": df['mem_size_bytes'].max() if not df.empty else None,
                            "pyusd_steps": pyusd_execution_steps,
                            "pyusd_percentage": pyusd_percentage
                        }
                    }

                    # Add opcode categories if available
                    if 'category_gas' in locals() and not category_gas.empty:
                        export_data["opcode_categories"] = category_gas.to_dict('records')

                    display(export_to_google_sheets(df, export_data, tx_hash))
                except Exception as e:
                    html = f"<div style='color:red'>Error exporting to Google Sheets: {str(e)}</div>"
                    display(HTML(html))

                    # Fallback to CSV
                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                    filename = f"structlog_trace_{tx_hash[:10]}_{timestamp}.csv"
                    display(download_csv_direct(df, filename))
                    display(HTML("<div>Falling back to CSV download due to Google Sheets error.</div>"))

        # Connect handlers to buttons
        export_buttons.children[0].on_click(export_csv)
        export_buttons.children[1].on_click(export_json)
        export_buttons.children[2].on_click(export_to_sheets)

        # Display button container and output area
        display(export_buttons)
        display(export_output)

    return df


# --- Execute Tracing ---
# WARNING: structLog can be VERY large and slow. Default to False.
RUN_STRUCTLOG_TRACE = True  # <<< SET TO TRUE TO RUN THIS EXPENSIVE TRACE

if not validate_tx_hash:  # Check validation flag from setup cell
    console.print("[warning]TARGET_TX_HASH not set or invalid. Cannot run structLog analysis.", style="warning")
elif RUN_STRUCTLOG_TRACE:
    console.print("\n\n[bold]🎯 Using structLog on Mainnet[/bold]", style="cyan3")
    console.print("───────────────────────────────", style="cyan3")
    console.print(f"Target Transaction : {TARGET_TX_HASH}")

    # Use the default tracer configuration (the one that worked)
    tracer_config = {"tracerConfig": TRACE_CONFIGS["structLog"]}

    # Request trace on Mainnet
    trace_result_struct = make_rpc_request("debug_traceTransaction", [TARGET_TX_HASH, tracer_config], network='mainnet')

    # Process result if we got one
    if trace_result_struct:
        # Check where structLogs might be in the response
        struct_logs = None
        if 'structLogs' in trace_result_struct:
            struct_logs = trace_result_struct['structLogs']
        elif 'result' in trace_result_struct and isinstance(trace_result_struct['result'], dict) and 'structLogs' in trace_result_struct['result']:
            struct_logs = trace_result_struct['result']['structLogs']

        if struct_logs and isinstance(struct_logs, list):
            struct_log_df = parse_struct_log(struct_logs, TARGET_TX_HASH)
        else:
            console.print("[warning]Got response, but structLogs format was not as expected.", style="warning")
    else:
        console.print(f"[warning]Failed to get trace data for {TARGET_TX_HASH}.", style="warning")

elif not RUN_STRUCTLOG_TRACE:
     console.print("\n[info]Skipping trace analysis as RUN_STRUCTLOG_TRACE is False.", style="info")


## 1.3 📜 `eth_getLogs`: Efficiently Fetching PYUSD Events
---

This section focuses on retrieving specific **event logs** emitted by the PYUSD smart contract. Events are a crucial mechanism for contracts to signal occurrences (like transfers or approvals) to the outside world. `eth_getLogs` provides a highly efficient way to query these events directly from the blockchain's indexed log data, avoiding the need to process non-relevant transactions or blocks.

Here, we specifically target the standard ERC-20 `Transfer(address,address,uint256)` event emitted by the PYUSD contract to track token movements.

> **🚀 Leveraging GCP's Blockchain Node Engine**
>
> *   **Method:** `eth_getLogs`
> *   **Quota Multiplier:** `50x` (Indicates this method consumes 50x the request quota compared to a basic call like `eth_getBlockByNumber` under GCP's Blockchain Node Engine pricing model).
> *   **GCP Integration & Limits:** While `eth_getLogs` is standard, node performance and limitations vary. Public or lower-tier nodes often impose strict limits on the query block range (e.g., 10,000 blocks) or total logs returned (e.g., 10,000 logs). Furthermore, specific endpoints, like the Google Blockchain Node Engine endpoint potentially used here (`blockchain.googleapis.com`), may enforce even tighter constraints (like the **5-block range limit** handled explicitly in the code). GCP's advantage often lies in providing reliable performance, consistent availability, and potentially higher throughput or more permissive limits on its paid tiers compared to shared public endpoints.
> *   **PYUSD Insight:** `eth_getLogs` enables us to:
>     *   Quickly find **all PYUSD `Transfer` events** within a specific block or a (potentially limited) range of blocks.
>     *   Efficiently track PYUSD **velocity and volume** over time by fetching logs incrementally.
>     *   Identify **top senders and receivers** of PYUSD within a given period.
>     *   Monitor other PYUSD events (like `Approval`, `Paused`, `Unpaused`) by changing the queried `topic`.

**Analysis Workflow:**

1.  **Define Filter:** The code configures a filter targeting the main PYUSD contract address (`PYUSD_CONFIG['ethereum']['address']`) and the specific topic hash for the `Transfer` event. It also specifies the target block(s) (e.g., `TARGET_BLOCK_IDENTIFIER`, 'latest', or a range).
2.  **Fetch Logs:** Calls `eth_getLogs` via the `web3.py` client. If a Google API endpoint is detected, it uses direct JSON-RPC requests to handle potential specific constraints (like the 5-block limit).
3.  **Parse & Decode:** The returned raw log data is parsed, extracting the `from` address, `to` address, and `value` (amount) for each `Transfer` event. PYUSD amounts are formatted using the correct 6 decimals (`PYUSD_CONFIG['ethereum']['decimals']`). Timestamps are fetched for context.
4.  **Analyze & Visualize:**
    *   **Data Table:** Displays a sample of the fetched transfer events (showing full addresses).
    *   **Statistics:** Calculates and shows key metrics (total transfers, volume, unique addresses) in a summary table.
    *   **Top Movers:** Identifies and tables the top senders and receivers by volume (showing full addresses).
    *   **Visualizations:** Generates plots showing transfer size distribution, volume over time (if timestamps available), and a network graph (Sankey diagram) of top transfer flows.
    *   **Export Options:** Provides buttons to download the parsed transfer data (CSV, JSON) or export to Google Sheets.

**💡 What to Look For:**
*   **Transfer Volume & Count:** Assess the PYUSD activity level within the queried block(s)/range.
*   **Top Senders/Receivers:** Identify major players interacting with PYUSD. Are they exchanges, bridges, protocols, or individual wallets?
*   **Network Graph:** Visualize the primary flow of PYUSD between addresses for the top flows.
*   **Timestamp Data (if available):** Observe any time-based patterns in transfer activity.

In [ ]:
# =============================================================================================
# 📜 PYUSD Transfer Logs using eth_getLogs
# =============================================================================================
# This cell retrieves and analyzes PYUSD token transfer events using the eth_getLogs RPC method.
#  Functionality includes:
# - Fetching PYUSD ERC-20 Transfer events respecting the 5-block maximum range limit.
# - Calculating key statistical metrics (total volume, average transfer size, unique participants).
# - Generating detailed visualizations: transfer size distribution, volume timeline, and token flow networks.
# - Network visualization using a Sankey diagram showing the top 50 transfer flows between addresses.
# - Displaying ranked tables of top senders and receivers with volume and transaction counts.
# - Providing interactive exports to CSV, JSON, or Google Sheets with complete analysis data.
# - Handling both individual block analysis and small block ranges for detailed examination.

import base64
import json
from datetime import datetime
from IPython.display import HTML, display
import ipywidgets as widgets
from IPython.display import clear_output

def download_csv_direct(df, filename=None):
    """Creates a direct download for CSV without intermediate display."""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"pyusd_data_{timestamp}.csv"

    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()

    # Create direct download HTML
    html = f'''
    <script>
    function download(filename, data) {{
        const a = document.createElement('a');
        a.href = data;
        a.download = filename;
        document.body.appendChild(a);
        a.click();
        document.body.removeChild(a);
    }}

    download("{filename}", "data:text/csv;base64,{payload}");
    </script>
    <div>Downloading {filename}...</div>
    '''
    return HTML(html)

def download_json_direct(data, filename=None):
    """Creates a direct download for JSON without intermediate display."""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"pyusd_data_{timestamp}.json"

    # Convert to JSON string (handling non-serializable objects)
    json_str = json.dumps(data, default=str, indent=2)
    b64 = base64.b64encode(json_str.encode()).decode()

    # Create direct download HTML
    html = f'''
    <script>
    function download(filename, data) {{
        const a = document.createElement('a');
        a.href = data;
        a.download = filename;
        document.body.appendChild(a);
        a.click();
        document.body.removeChild(a);
    }}

    download("{filename}", "data:application/json;base64,{b64}");
    </script>
    <div>Downloading {filename}...</div>
    '''
    return HTML(html)

def export_to_google_sheets(df, data_dict, title_prefix):
    """Export analysis data to Google Sheets using the existing gspread client.

    Args:
        df: DataFrame with main data
        data_dict: Dictionary with additional structured data
        title_prefix: Prefix for sheet title
    """
    try:
        # Create a new Google Sheet with meaningful title
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        sheet_title = f"PYUSD Transfer Analysis {title_prefix} {timestamp}"

        # Use the global gc_sheets client that's already authenticated
        spreadsheet = gc_sheets.create(sheet_title)

        # Get the default worksheet and rename it
        worksheet = spreadsheet.get_worksheet(0)
        worksheet.update_title("Transfer Data")

        # Set up a header with analysis info
        header_values = [
            ["PYUSD Transfer Analysis"],
            [f"Analysis Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"],
            [""],  # Empty row for spacing
        ]
        worksheet.update("A1", header_values)

        # Format the header
        worksheet.format("A1:A1", {"textFormat": {"bold": True, "fontSize": 14}})

        # Add summary stats if available
        if "statistics" in data_dict:
            stats = data_dict["statistics"]
            stats_rows = [["Analysis Summary"], [""]]
            for key, value in stats.items():
                stats_rows.append([key.replace("_", " ").title(), str(value)])

            # Add stats after the header (row 4)
            worksheet.update("A4", stats_rows)
            stats_end_row = 4 + len(stats_rows)
        else:
            stats_end_row = 4

        # Add main DataFrame data below the stats
        if not df.empty:
            start_row = stats_end_row + 2  # Leave a gap

            # Add a section title
            worksheet.update(f"A{start_row}", [["Transfer Data"]])
            worksheet.format(f"A{start_row}:A{start_row}", {"textFormat": {"bold": True}})

            # Convert DataFrame to list of lists for the worksheet
            df_values = [df.columns.tolist()] + df.values.tolist()
            worksheet.update(f"A{start_row+1}", df_values)

        # Open the spreadsheet in a new tab
        spreadsheet_url = f"https://docs.google.com/spreadsheets/d/{spreadsheet.id}"
        html = f'''
        <script>
        window.open("{spreadsheet_url}", "_blank");
        </script>
        <div>Spreadsheet created and opened: <a href="{spreadsheet_url}" target="_blank">{sheet_title}</a></div>
        '''
        return HTML(html)

    except Exception as e:
        return HTML(f"<div style='color:red'>Error creating Google Sheet: {str(e)}</div>")

def fetch_pyusd_transfer_logs(from_block='latest', to_block='latest', network='mainnet'):
    """Fetches PYUSD Transfer events using eth_getLogs on the specified network.

    Handles both direct Web3.py calls and JSON-RPC requests for compatibility with Google Blockchain API,
    which has a 5-block maximum range limitation.
    """
    w3_client = w3_clients.get(network)
    if not w3_client or not w3_client.is_connected():
        console.print(f"[error]Web3 client for '{network}' not available or not connected.", style="error")
        return None

    pyusd_checksum_address = Web3.to_checksum_address(PYUSD_CONFIG['ethereum']['address'])
    transfer_topic = PYUSD_CONFIG['ethereum']['transfer_event_topic']

    console.print(f"\n\n[bold cyan3]Fetching PYUSD Transfer logs...[/bold cyan3]", style="info")
    console.print(f"[bold]Network[/bold]  : [yellow3]{network.capitalize()}[/yellow3]", style="info")
    console.print(f"[bold]Contract[/bold] : [bold magenta3]{pyusd_checksum_address}[/bold magenta3]", style="info")
    console.print(f"[bold]Blocks[/bold]   : [bold green3]{from_block} → {to_block}[/bold green3]", style="info")
    console.print(f"[bold]Topic[/bold]    : [dim]{transfer_topic}[/dim]", style="info")

    try:
        # Detect if we're using Google Blockchain API
        is_google_api = False
        if hasattr(w3_client, 'provider') and hasattr(w3_client.provider, 'endpoint_uri'):
            endpoint_uri = str(w3_client.provider.endpoint_uri)
            is_google_api = "blockchain.googleapis.com" in endpoint_uri

        if is_google_api:
            import requests

            # Format block parameters for JSON-RPC
            def format_block_param_json(block_id):
                if block_id == 'latest' or block_id == 'pending' or block_id == 'earliest':
                    return block_id
                elif isinstance(block_id, int):
                    return hex(block_id)
                elif isinstance(block_id, str):
                    if block_id.startswith("0x"):
                        return block_id
                    else:
                        try:
                            return hex(int(block_id))
                        except ValueError:
                            raise ValueError(f"Invalid block identifier: {block_id}")
                else:
                    raise ValueError(f"Invalid block identifier type: {type(block_id)}")

            # Prepare the JSON-RPC payload
            payload = {
                "jsonrpc": "2.0",
                "method": "eth_getLogs",
                "params": [{
                    "fromBlock": format_block_param_json(from_block),
                    "toBlock": format_block_param_json(to_block),
                    "address": pyusd_checksum_address,
                    "topics": [transfer_topic]
                }],
                "id": 1
            }

            headers = {
                "Content-Type": "application/json",
                "Accept": "application/json"
            }

            # Extract API key from the endpoint URI if present
            api_key = None
            if "?key=" in endpoint_uri:
                api_key = endpoint_uri.split("?key=")[1]
                # Remove the key from the URI for the request
                endpoint_uri = endpoint_uri.split("?key=")[0]

            # Make direct request to the API
            response = requests.post(
                endpoint_uri,
                json=payload,
                headers=headers,
                params={"key": api_key} if api_key else None
            )

            if response.status_code != 200:
                console.print(f"[error]API request failed with status {response.status_code}: {response.text}", style="error")
                return None

            result = response.json()

            if "error" in result:
                console.print(f"[error]JSON-RPC error: {result['error']}", style="error")
                return None

            # Parse the logs from the response
            logs = result.get("result", [])

            if not logs:
                console.print(f"[info]No PYUSD Transfer logs found in the specified range ({from_block} - {to_block}) on {network.capitalize()}.", style="info")
                return pd.DataFrame()

            console.print(f"[success]Found {len(logs)} PYUSD Transfer logs on {network.capitalize()}.", style="success")
        else:
            # Use standard web3.py method for non-Google providers
            # eth_getLogs expects block numbers as hex strings or integer/tags
            def format_block_param(block_id):
                """Formats block identifier for eth_getLogs."""
                if isinstance(block_id, int):
                    return hex(block_id) # Prefer hex for consistency with RPC spec
                elif isinstance(block_id, str):
                     # Check if it's already hex or a known tag
                    if block_id.startswith("0x") or block_id in ["latest", "pending", "earliest"]:
                        return block_id
                    else: # Try converting potential numeric strings
                        try:
                            return hex(int(str(block_id)))
                        except ValueError:
                            console.print(f"[error]Invalid block identifier format for eth_getLogs: {block_id}. Use int, hex string, or tag.", style="error")
                            raise ValueError(f"Invalid block identifier: {block_id}")
                else: # Handle None or other types
                     raise ValueError(f"Invalid block identifier type: {type(block_id)}")

            log_filter = {
                "fromBlock": format_block_param(from_block),
                "toBlock": format_block_param(to_block),
                "address": pyusd_checksum_address,
                "topics": [transfer_topic] # Topic0 is the event signature
            }

            # Make the eth_getLogs request using standard web3.py method on the correct client
            logs = w3_client.eth.get_logs(log_filter)

            if not logs:
                console.print(f"[info]No PYUSD Transfer logs found in the specified range ({from_block} - {to_block}) on {network.capitalize()}.", style="info")
                return pd.DataFrame()

            console.print(f"[success]Found {len(logs)} PYUSD Transfer logs on {network.capitalize()}.", style="success")

        # Parse the logs (common for both methods)
        parsed_logs = []
        timestamps = {}  # Cache for block timestamps

        for log in logs:
            # Ensure log is a dictionary-like object
            if isinstance(log, dict):
                # For JSON-RPC response, convert hex strings to int where needed
                log_dict = log
                topics = log_dict.get('topics', [])
                data = log_dict.get('data', '')
                block_number = int(log_dict.get('blockNumber', '0x0'), 16)
                tx_hash = log_dict.get('transactionHash', '')
                log_index = int(log_dict.get('logIndex', '0x0'), 16)
            else:
                # For web3.py response
                if not hasattr(log, 'topics') or not hasattr(log, 'data'): continue
                topics = log['topics']
                data = log['data']
                block_number = log['blockNumber']
                tx_hash = log['transactionHash']
                log_index = log['logIndex']
                # Convert bytes to hex strings for consistency
                if isinstance(topics[0], bytes):
                    topics = [t.hex() if isinstance(t, bytes) else t for t in topics]
                if isinstance(data, bytes):
                    data = data.hex()
                if isinstance(tx_hash, bytes):
                    tx_hash = tx_hash.hex()

            try:
                # Check for standard ERC20 Transfer signature (3 topics)
                if len(topics) == 3:
                    # Extract addresses from topics (remove 0x if present and take last 40 chars)
                    topic1 = topics[1].replace('0x', '') if isinstance(topics[1], str) else topics[1].hex()
                    topic2 = topics[2].replace('0x', '') if isinstance(topics[2], str) else topics[2].hex()

                    from_addr = Web3.to_checksum_address('0x' + topic1[-40:])
                    to_addr = Web3.to_checksum_address('0x' + topic2[-40:])

                    # Extract value from data
                    data_hex = data if isinstance(data, str) else data.hex()
                    data_hex = data_hex.replace('0x', '')
                    value_raw = int(data_hex, 16)
                    value_pyusd = value_raw / (10**PYUSD_CONFIG['ethereum']['decimals'])

                    # Get block timestamp (cache to avoid redundant queries)
                    if block_number not in timestamps:
                        try:
                            block = w3_client.eth.get_block(block_number)
                            timestamps[block_number] = block.timestamp
                        except Exception:
                            timestamps[block_number] = None

                    # Ensure transaction hash has 0x prefix
                    tx_hash_hex = tx_hash if isinstance(tx_hash, str) else tx_hash.hex()
                    if not tx_hash_hex.startswith('0x'):
                        tx_hash_hex = '0x' + tx_hash_hex

                    parsed_logs.append({
                        "blockNumber": block_number,
                        "transactionHash": tx_hash_hex,
                        "logIndex": log_index,
                        "from": from_addr,
                        "from_short": shorten_address(from_addr),
                        "to": to_addr,
                        "to_short": shorten_address(to_addr),
                        "value_pyusd": value_pyusd,
                        "value_raw": value_raw,
                        "timestamp": timestamps.get(block_number)
                    })
                else:
                    # Log unexpected topic count for debugging
                    tx_hash_str = tx_hash if isinstance(tx_hash, str) else tx_hash.hex()
                    console.print(f"[warning]Log index {log_index} tx {shorten_address(tx_hash_str)} has {len(topics)} topics (expected 3 for Transfer). Skipping.", style="warning")
            except Exception as e:
                # Handle parsing errors
                tx_hash_str = tx_hash if isinstance(tx_hash, str) else tx_hash.hex()
                console.print(f"[warning]Could not parse log index {log_index} tx {shorten_address(tx_hash_str)}: {e}", style="warning")

        return pd.DataFrame(parsed_logs)

    except ValueError as ve: # Catch specific invalid block identifier error
        console.print(f"[error]Invalid block parameter for eth_getLogs: {ve}", style="error")
        return None
    except Exception as e:
        console.print(f"[error]Error fetching logs with eth_getLogs on {network.capitalize()}: {e}", style="error")
        # Add hints for common errors
        if "filter not found" in str(e):
            console.print("[info]Hint: This might happen with very large block ranges on some nodes.", style="info")
        elif "exceeds block range limit" in str(e) or "block range is too large" in str(e):
            console.print("[info]Hint: Reduce the block range (e.g., fetch logs for smaller chunks of blocks).", style="info")
        elif "invalid topic" in str(e).lower():
             console.print(f"[error]Hint: Check if transfer_topic '{transfer_topic}' is correct.", style="error")
        return None

def analyze_pyusd_transfers(df):
    """Generates enhanced analytics for PYUSD transfer data"""
    if df is None or df.empty:
        console.print("[warning]No transfer data to analyze.", style="warning")
        return

    # Basic statistics
    stats = {
        "total_transfers": len(df),
        "total_volume": df['value_pyusd'].sum(),
        "avg_transfer": df['value_pyusd'].mean(),
        "median_transfer": df['value_pyusd'].median(),
        "max_transfer": df['value_pyusd'].max(),
        "min_transfer": df['value_pyusd'].min(),
        "unique_senders": df['from'].nunique(),
        "unique_receivers": df['to'].nunique()
    }

    # Create statistics table
    console.print("\n\n[bold cyan3]PYUSD Transfer Metrices[/bold cyan3]")
    console.print("───────────────────────", style="cyan3")
    stats_table = Table(title="", header_style="bold cyan3")
    stats_table.add_column("Metric")
    stats_table.add_column("Value", justify="right")

    decimals = PYUSD_CONFIG['ethereum']['decimals']
    stats_table.add_row("Total Transfers", f"{stats['total_transfers']:,}")
    stats_table.add_row("Total Volume", f"{stats['total_volume']:,.{decimals}f} PYUSD")
    stats_table.add_row("Average Transfer", f"{stats['avg_transfer']:,.{decimals}f} PYUSD")
    stats_table.add_row("Median Transfer", f"{stats['median_transfer']:,.{decimals}f} PYUSD")
    stats_table.add_row("Maximum Transfer", f"{stats['max_transfer']:,.{decimals}f} PYUSD")
    stats_table.add_row("Minimum Transfer", f"{stats['min_transfer']:,.{decimals}f} PYUSD")
    stats_table.add_row("Unique Senders", f"{stats['unique_senders']:,}")
    stats_table.add_row("Unique Receivers", f"{stats['unique_receivers']:,}")

    console.print(stats_table)

    # Top senders analysis
    if len(df) > 0:
        console.print("\n\n[bold cyan3]Top PYUSD Senders[/bold cyan3]")
        console.print("──────────────────", style="cyan3")
        top_senders = df.groupby('from').agg({
            'value_pyusd': ['sum', 'count'],
            'from_short': 'first'
        }).reset_index()

        top_senders.columns = ['address', 'total_value', 'transactions', 'address_short']
        top_senders = top_senders.sort_values('total_value', ascending=False).head(10)

        sender_table = Table(show_header=True, header_style="bold cyan")
        sender_table.add_column("Sender")
        sender_table.add_column("Total Sent (PYUSD)", justify="right")
        sender_table.add_column("Transactions", justify="right")
        sender_table.add_column("% of Volume", justify="right")

        for _, row in top_senders.iterrows():
            pct_volume = (row['total_value'] / stats['total_volume'] * 100)
            sender_table.add_row(
                row['address'],
                f"{row['total_value']:,.{decimals}f}",
                f"{row['transactions']:,}",
                f"{pct_volume:.1f}%"
            )

        console.print(sender_table)

        # Top receivers analysis
        console.print("\n\n[bold cyan3]Top PYUSD Receivers[/bold cyan3]")
        console.print("───────────────────", style="cyan3")
        top_receivers = df.groupby('to').agg({
            'value_pyusd': ['sum', 'count'],
            'to_short': 'first'
        }).reset_index()

        top_receivers.columns = ['address', 'total_value', 'transactions', 'address_short']
        top_receivers = top_receivers.sort_values('total_value', ascending=False).head(10)

        receiver_table = Table(show_header=True, header_style="bold cyan")
        receiver_table.add_column("Receiver")
        receiver_table.add_column("Total Received (PYUSD)", justify="right")
        receiver_table.add_column("Transactions", justify="right")
        receiver_table.add_column("% of Volume", justify="right")

        for _, row in top_receivers.iterrows():
            pct_volume = (row['total_value'] / stats['total_volume'] * 100)
            receiver_table.add_row(
                row['address'],
                f"{row['total_value']:,.{decimals}f}",
                f"{row['transactions']:,}",
                f"{pct_volume:.1f}%"
            )

        console.print(receiver_table)

    # Visualizations
    try:
        console.print("\n\n[bold]PYUSD Transfer Size Distribution[/bold]", style="magenta3")
        console.print("─────────────────────────────────", style="magenta3")
        # 1. Transfer Size Distribution visualization

        fig_dist = px.histogram(
            df, x='value_pyusd',
            title='PYUSD Transfer Size Distribution',
            labels={'value_pyusd': 'Transfer Size (PYUSD)', 'count': 'Number of Transfers'},
            nbins=50,
            opacity=0.75,
            color_discrete_sequence=['rgba(0, 123, 255, 0.8)']
        )

        # Configure histogram layout
        fig_dist.update_layout(
            template="plotly_white",
            bargap=0.1,
            plot_bgcolor='white',
            margin=dict(l=50, r=50, t=80, b=50),
            title_font=dict(size=20),
            yaxis_title="Number of Transfers",
            xaxis_title="Transfer Size (PYUSD)"
        )

        # Add range slider to better handle outliers in the distribution
        q1 = df['value_pyusd'].quantile(0.25)
        q3 = df['value_pyusd'].quantile(0.75)
        iqr = q3 - q1
        upper_bound = q3 + (1.5 * iqr)

        if df['value_pyusd'].max() > upper_bound:
            fig_dist.update_layout(
                xaxis=dict(
                    rangeslider=dict(visible=True),
                    type='linear'
                )
            )

        fig_dist.show()

        # 2. Transfer Volume Over Time visualization
        console.print("\n\n[bold]PYUSD Transfer Volume Over Time[/bold]", style="magenta3")
        console.print("───────────────────────────────", style="magenta3")

        if 'timestamp' in df.columns and df['timestamp'].notna().any():
            df_with_time = df[df['timestamp'].notna()].copy()
            df_with_time['datetime'] = pd.to_datetime(df_with_time['timestamp'], unit='s')

            # Aggregate by hour for cleaner visualization
            df_with_time['hour'] = df_with_time['datetime'].dt.floor('H')
            hourly_volume = df_with_time.groupby('hour')['value_pyusd'].sum().reset_index()

            # Ensure data exists before creating visualization
            if not hourly_volume.empty:
                fig_time = px.line(
                    hourly_volume, x='hour', y='value_pyusd',
                    title='PYUSD Transfer Volume Over Time',
                    labels={'hour': 'Time', 'value_pyusd': 'Volume (PYUSD)'}
                )

                # Configure time series chart layout
                fig_time.update_layout(
                    template="plotly_white",
                    plot_bgcolor='white',
                    margin=dict(l=50, r=50, t=80, b=50),
                    title_font=dict(size=20),
                    xaxis_title="Time",
                    yaxis_title="Volume (PYUSD)",
                    hovermode="x unified"
                )

                # Style the line for better visibility
                fig_time.update_traces(
                    line=dict(width=3, color='rgb(0, 123, 255)'),
                    mode='lines+markers',
                    marker=dict(size=8, color='rgb(0, 123, 255)')
                )

                fig_time.show()
            else:
                console.print("[warning]No time-based data available for visualization.", style="warning")

        # 3. Network graph using Sankey diagram
        if len(df) > 0:
            console.print("\n\n[bold magenta3]PYUSD Transfer Flow - Top 50 Transfers[/bold magenta3]")
            console.print("───────────────────────────────────────", style="magenta3")
            # Aggregate transfers between address pairs
            pair_transfers = df.groupby(['from', 'to'])['value_pyusd'].sum().reset_index()
            pair_transfers = pair_transfers.sort_values('value_pyusd', ascending=False).head(50)

            # Prepare data for Sankey diagram
            labels = []
            address_to_idx = {}

            # Create a unique index for each address
            for _, row in pair_transfers.iterrows():
                if row['from'] not in address_to_idx:
                    address_to_idx[row['from']] = len(labels)
                    labels.append(shorten_address(row['from']))
                if row['to'] not in address_to_idx:
                    address_to_idx[row['to']] = len(labels)
                    labels.append(shorten_address(row['to']))

            # Create source, target, and value arrays for Sankey
            sources = [address_to_idx[row['from']] for _, row in pair_transfers.iterrows()]
            targets = [address_to_idx[row['to']] for _, row in pair_transfers.iterrows()]
            values = pair_transfers['value_pyusd'].tolist()

            # Create Sankey diagram with flow direction
            fig_sankey = go.Figure(data=[go.Sankey(
                node=dict(
                    pad=15,
                    thickness=20,
                    line=dict(color="black", width=0.5),
                    label=labels,
                    color="blue"
                ),
                link=dict(
                    source=sources,
                    target=targets,
                    value=values,
                    color="rgba(0,100,200,0.3)"
                )
            )])

            # Configure Sankey layout
            fig_sankey.update_layout(
                title_text="PYUSD Transfer Flow - Top 50 Transfers",
                font=dict(size=12),
                width=1000,
                height=800
            )

            fig_sankey.show()

    except Exception as viz_error:
        console.print(f"[warning]Visualization error: {viz_error}", style="warning")

    # --- Add Export Options Section ---
    console.print("\n\n[bold cyan3]📤 Export Options:[/bold cyan3]")
    console.print("─────────────────", style="cyan3")

    # Create export output area
    export_output = widgets.Output()

    # Create export buttons with proper styling
    export_buttons = widgets.HBox([
        widgets.Button(
            description='Export to CSV',
            button_style='primary',
            layout=widgets.Layout(width='150px')
        ),
        widgets.Button(
            description='Export as JSON',
            button_style='warning',
            layout=widgets.Layout(width='150px')
        ),
        widgets.Button(
            description='Export to Google Sheets',
            button_style='info',
            layout=widgets.Layout(width='200px')
        )
    ])

    # Define export handlers
    def export_csv(b):
        with export_output:
            clear_output()
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"pyusd_transfers_{timestamp}.csv"
            display(download_csv_direct(df, filename))

    def export_json(b):
        with export_output:
            clear_output()
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"pyusd_transfers_{timestamp}.json"
            # Prepare export data with basic DataFrame stats
            export_data = {
                "analysis_type": "PYUSD Transfers",
                "analysis_time": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                "statistics": stats,
                "top_senders": top_senders.to_dict('records') if 'top_senders' in locals() else [],
                "top_receivers": top_receivers.to_dict('records') if 'top_receivers' in locals() else []
            }
            display(download_json_direct(export_data, filename))

    def export_to_sheets(b):
        with export_output:
            clear_output()
            try:
                # Prepare export data
                export_data = {
                    "statistics": stats,
                    "top_senders": top_senders.to_dict('records') if 'top_senders' in locals() else [],
                    "top_receivers": top_receivers.to_dict('records') if 'top_receivers' in locals() else []
                }

                display(export_to_google_sheets(df, export_data, "Transfers"))
            except Exception as e:
                html = f"<div style='color:red'>Error exporting to Google Sheets: {str(e)}</div>"
                display(HTML(html))

                # Fallback to CSV
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"pyusd_transfers_{timestamp}.csv"
                display(download_csv_direct(df, filename))
                display(HTML("<div>Falling back to CSV download due to Google Sheets error.</div>"))

    # Connect handlers to buttons
    export_buttons.children[0].on_click(export_csv)
    export_buttons.children[1].on_click(export_json)
    export_buttons.children[2].on_click(export_to_sheets)

    # Display button container and output area
    display(export_buttons)
    display(export_output)

    return df

# --- Execute Transfer Log Analysis ---
# Set parameters for fetching logs
FETCH_TRANSFERS = True
BLOCKS_TO_FETCH = 1000

if FETCH_TRANSFERS:
    if BLOCKS_TO_FETCH == 'latest':
        # Just fetch latest block
        transfers_df = fetch_pyusd_transfer_logs(from_block='latest', to_block='latest')
    else:
        # Always respect the 5-block limit for Google Blockchain API
        try:
            latest_block = w3_clients['mainnet'].eth.block_number

            # Strictly respect the 5-block maximum range
            start_block = latest_block - 4  # This gives exactly 5 blocks including the latest
            console.print("\n\n[bold]📜 Fetching Logs via eth_getLogs[/bold]", style="cyan3")
            console.print("─────────────────────────────────", style="cyan3")
            console.print(f"\n\n[bold chartreuse1]📡 Querying Recent PYUSD Transfers [Block {start_block:,} to {latest_block:,} (latest)] (Max 5-Block Limit)[/bold chartreuse1]")
            console.print("────────────────────────────────────────────────────────────────────────────────────────────────", style="chartreuse1")
            transfers_df = fetch_pyusd_transfer_logs(from_block=start_block, to_block=latest_block)
        except Exception as e:
            console.print(f"[warning]Error fetching latest blocks: {e}. Falling back to single block query.", style="warning")
            transfers_df = fetch_pyusd_transfer_logs(from_block='latest', to_block='latest')

    if transfers_df is not None and not transfers_df.empty:
        # Display a sample of the data
        console.print("\n\n[bold cyan3]Transfer Data Sample (First 5 rows)[/bold cyan3]")
        console.print("───────────────────────────────────", style="cyan3")
        sample_display_cols = ['blockNumber', 'transactionHash', 'from', 'to', 'value_pyusd', 'timestamp']
        display(transfers_df[sample_display_cols].head())

        # Run analysis
        analyze_pyusd_transfers(transfers_df)
    else:
        console.print("[warning]No PYUSD transfers found or error occurred.", style="warning")

# --- Execute Log Fetching for Target Block on Mainnet ---
if 'TARGET_BLOCK_IDENTIFIER' in locals() and TARGET_BLOCK_IDENTIFIER is not None:
    block_id_getlogs = TARGET_BLOCK_IDENTIFIER
    console.print(f"\n\n[bold chartreuse1]📡 Querying PYUSD logs for Target Block Identifier: {block_id_getlogs} on Mainnet[/bold chartreuse1]")
    console.print("───────────────────────────────────────────────────────────────────────────────", style="chartreuse1")

    # Fetch for a single block by setting from_block and to_block to the same identifier
    pyusd_logs_df = fetch_pyusd_transfer_logs(from_block=block_id_getlogs, to_block=block_id_getlogs, network='mainnet')

    if pyusd_logs_df is not None and not pyusd_logs_df.empty:
        console.print("\n\n[bold cyan3]📊 PYUSD Transfer Logs (Mainnet Block)[/bold cyan3]")
        console.print("───────────────────────────────────────", style="cyan3")
        # Show full addresses instead of shortened ones
        display_cols = ['blockNumber', 'from', 'to', 'value_pyusd', 'transactionHash']
        display(pyusd_logs_df[display_cols])

        # Run analysis on block data
        analyze_pyusd_transfers(pyusd_logs_df)

    elif pyusd_logs_df is not None:
        console.print(f"[info]No PYUSD transfers found in block {block_id_getlogs} on Mainnet.", style="info")

else:
    console.print("[warning]TARGET_BLOCK_IDENTIFIER not set. Skipping Mainnet eth_getLogs analysis.", style="warning")

# --- Optional: Query for a Block Range Instead of Single Block ---
RUN_BLOCK_RANGE_FETCH = True

if RUN_BLOCK_RANGE_FETCH and 'TARGET_BLOCK_IDENTIFIER' in locals() and TARGET_BLOCK_IDENTIFIER is not None:
    # Use TARGET_BLOCK_IDENTIFIER as the center of a smaller range
    block_center = TARGET_BLOCK_IDENTIFIER if isinstance(TARGET_BLOCK_IDENTIFIER, int) else int(TARGET_BLOCK_IDENTIFIER, 16) if isinstance(TARGET_BLOCK_IDENTIFIER, str) and TARGET_BLOCK_IDENTIFIER.startswith('0x') else None

    if block_center:
        # Respect the 5-block limit for Google Blockchain API
        # Use center block plus 2 on each side (5 blocks total)
        range_start = block_center - 2
        range_end = block_center + 2
        console.print(f"[info]Fetching PYUSD logs with 5-block range: {range_start}-{range_end}", style="info")

        range_logs_df = fetch_pyusd_transfer_logs(from_block=range_start, to_block=range_end, network='mainnet')

        if range_logs_df is not None and not range_logs_df.empty:
            console.print(f"\n\n[bold chartreuse1]📡 Querying PYUSD Transfer Logs (Block Range {range_start}-{range_end})[/bold chartreuse1]")
            console.print("──────────────────────────────────────────────────────────────────────", style="chartreuse1")
            # Show full addresses in this display too
            display_cols = ['blockNumber', 'from', 'to', 'value_pyusd', 'transactionHash']
            display(range_logs_df[display_cols])

            # Run analysis on range data
            analyze_pyusd_transfers(range_logs_df)

        elif range_logs_df is not None:
            console.print(f"[info]No PYUSD transfers found in block range {range_start}-{range_end} on Mainnet.", style="info")
    else:
        console.print("[warning]Could not determine a numeric block center for range query.", style="warning")

## 1.4 📄 `eth_getCode`: Fetching and Analyzing Contract Bytecode
---

This section uses the `eth_getCode` RPC method to retrieve the **runtime bytecode** associated with a specific Ethereum address at a given block state. This allows us to inspect the actual executable code deployed on the blockchain.

Analyzing bytecode is essential for:

*   **Verification:** Confirming whether an address belongs to a smart contract or an Externally Owned Account (EOA). EOAs have no code.
*   **Proxy Analysis:** Comparing the minimal bytecode of a proxy contract (like `PYUSD_PROXY`) with the extensive logic bytecode of its implementation contract (`PYUSD_IMPLEMENTATION`).
*   **Basic Functionality Identification:** Detecting potential function signatures (like standard ERC-20 functions or *potentially* PYUSD-specific ones *if included in the tool's known signature set*) present within the bytecode. *Note that this relies on matching known 4-byte selectors found via bytecode patterns and is a heuristic approach, not a full decompilation.*
*   **Pattern Recognition:** Identifying common contract patterns (e.g., Ownable, Pausable, Proxy types) based on embedded function selectors.

> **🚀 Leveraging GCP's Capabilities**
>
> *   **Method:** `eth_getCode`
> *   **Multiplier:** `10x` (More efficient than tracing, but still higher than basic calls)
> *   **GCP Advantage:** While a standard call, reliable access via GCP ensures consistent retrieval of bytecode for analysis, even for large contracts or during high network load.
> *   **PYUSD Insight:** Allows us to:
>     *   Directly compare the lean **PYUSD proxy** contract code with its feature-rich **implementation** code.
>     *   Verify the bytecode of the **Supply Controller** contract.
>     *   Potentially identify known PYUSD function signatures *if available in the analysis tool's predefined signature dictionary* directly from the bytecode of any interacting contract.

**Analysis Workflow & Features:**

This section provides an interactive bytecode analysis tool with multiple modes accessed via tabs:

1.  **PYUSD Analysis Tab:**
    *   Automatically fetches and analyzes the bytecode for the official PYUSD Proxy, Implementation, and Supply Controller contracts on Mainnet.
    *   Performs individual analysis (`analyze_bytecode`) showing size, detected standards/patterns, and known functions.
    *   Compares the Proxy and Implementation contracts (`compare_proxy_implementation`), highlighting differences in size and function signatures.
    *   Visualizes the proxy architecture and contract relationships using Graphviz (*Note: Requires Graphviz installation and PATH configuration for diagrams to display*).
2.  **From Transaction Tab:**
    *   Takes a transaction hash as input.
    *   Identifies all contract addresses involved (target, event emitters, created contracts) using `get_contracts_from_tx`.
    *   Fetches and analyzes the bytecode for each identified contract.
    *   If multiple contracts are found, performs a similarity comparison (`compare_multiple_contracts`) based on known function signatures.
3.  **Custom Contracts Tab:**
    *   Allows entering up to three arbitrary contract addresses.
    *   Fetches and analyzes the bytecode for each provided address.
    *   Performs comparisons (Proxy vs. Impl if 2 addresses, Multi-contract if 3 addresses).
4.  **Export Options:** Available within each tab to download the analysis results (JSON) or export to Google Sheets (Colab).

**💡 What to Look For:**
*   **Bytecode Size:** Note the significant size difference between proxy and implementation contracts.
*   **Detected Patterns/Standards:** See if contracts adhere to ERC standards or implement common patterns like Ownable/Pausable.
*   **Proxy Analysis:** Understand the implementation address linked to the PYUSD proxy.
*   **Function Signatures:** Observe which known functions are detected within the bytecode (*remembering this is based on matching predefined signatures found via bytecode heuristics and is not a complete list of all functions*).
*   **Comparisons:** Identify shared vs. unique functions between related contracts.

In [ ]:
# =============================================================================================
# 📄 Advanced Contract Bytecode Analysis using eth_getCode
# =============================================================================================
# This cell provides comprehensive bytecode analysis for Ethereum smart contracts through multiple approaches.
# Functionality includes:
# - Fetching and analyzing contract bytecode with detection of ERC standards, proxy patterns, and security features
# - Comparing proxy contracts with their implementations to visualize delegation patterns
# - Analyzing multiple contracts from transactions to understand their relationships
# - Comparing different stablecoins (PYUSD, USDC, USDT) or other contract types side-by-side
# - Generating interactive visualizations of contract sizes, function distributions, and similarity metrics
# - Creating proxy architecture diagrams and contract relationship visualizations
# - Exporting complete analysis results to JSON or Google Sheets for further investigation
# - Supporting mainnet, sepolia, and holesky networks with flexible contract selection options

import base64
import json
import os
import re
from datetime import datetime
from IPython.display import HTML, display, clear_output
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from web3 import Web3
from rich.console import Console
from rich.table import Table
from rich.text import Text
from rich.theme import Theme
from graphviz import Digraph

# Auto-adapting color theme that works well in both light and dark terminals
custom_theme = Theme({
    "info": "cyan3",
    "error": "red3",
    "warning": "yellow3",
    "success": "green3",
    "spring_green3": "spring_green3",
    "dim": "grey50",
})

# Ensure console is created with the theme
console = Console(theme=custom_theme)

# --- Globals (Ensure these are defined/initialized in a prior cell) ---

# Some known signatures into a single dictionary for efficient lookup
ALL_KNOWN_SIGNATURES = {
    **(ERC20_SIGNATURES if 'ERC20_SIGNATURES' in globals() else {}),
    **(ERC721_SIGNATURES if 'ERC721_SIGNATURES' in globals() else {}),
    **(ERC1155_SIGNATURES if 'ERC1155_SIGNATURES' in globals() else {}),
    **(PROXY_PATTERNS if 'PROXY_PATTERNS' in globals() else {}),
    **(UUPS_PATTERNS if 'UUPS_PATTERNS' in globals() else {}),
    **(DIAMOND_PATTERNS if 'DIAMOND_PATTERNS' in globals() else {}),
    **(SECURITY_PATTERNS if 'SECURITY_PATTERNS' in globals() else {}),
    **(DEFI_PATTERNS if 'DEFI_PATTERNS' in globals() else {}),
    **(GAS_PATTERNS if 'GAS_PATTERNS' in globals() else {})
}
if 'PYUSD_SIGNATURES' in globals():
    ALL_KNOWN_SIGNATURES.update(PYUSD_SIGNATURES)

# --- Helper Functions ---

def shorten_address(address, chars=4):
    """Displays full blockchain address, adding line breaks if too long."""
    if not isinstance(address, str) or not address.startswith("0x"):
        return str(address)  # Return original if not a valid address string

    # uncomment if you want to truncate addresses
    # # If address is very long (like more than 25 chars), add a line break
    # if len(address) > 25:
    #     return f"{address[:14]}\n{address[14:]}"
    return address

# --- Core Analysis Functions ---

def get_contract_code(contract_address, block_identifier="latest", network='mainnet'):
    """Fetches the runtime bytecode for a contract address using the configured Web3 client."""
    if 'w3_clients' not in globals() or network not in w3_clients:
        print(f"[error]Web3 client for '{network}' network not configured.", style="error")
        return None

    w3_client = w3_clients.get(network)
    if not w3_client or not w3_client.is_connected():
        print(f"[error]Web3 client for '{network}' is not available or not connected.", style="error")
        return None

    try:
        checksum_address = Web3.to_checksum_address(contract_address)
    except ValueError:
        console.print(f"[error]Invalid contract address format provided: {contract_address}", style="error")
        return None

    console.print(f"[info]Fetching bytecode for {checksum_address} at block '{block_identifier}' on {network.capitalize()}.", style="info")

    try:
        code_bytes = w3_client.eth.get_code(checksum_address, block_identifier=block_identifier)
        hex_code = code_bytes.hex() # Includes '0x' prefix

        if hex_code == "0x":
            console.print(f"[warning]No bytecode found at {checksum_address} (Block: {block_identifier}, Network: {network.capitalize()}). It might be an EOA or a destroyed contract.", style="warning")
            return None
        else:
            bytecode_size = len(code_bytes)
            console.print(f"[success]Successfully retrieved bytecode for {checksum_address}.", style="success")
            # Display summary using Rich Table
            console.print("\n\n[bold]🔬 Contract Bytecode Information[/bold]", style="magenta3")
            console.print("─────────────────────────────────", style="magenta3")
            table = Table(show_header=False, box=None, padding=(0,1), title="")
            table.add_column("Field")
            table.add_column("Value")
            table.add_row("[bold]Address:[/bold]", checksum_address)
            table.add_row("[bold]Network:[/bold]", network.capitalize())
            table.add_row("[bold]Block:[/bold]", str(block_identifier))
            table.add_row("[bold]Bytecode Size:[/bold]", f"{bytecode_size:,} bytes")
            table.add_row("[bold]Preview:[/bold]", f"{hex_code[:100]}...")
            console.print(table)
            return hex_code

    except Exception as e:
        console.print(f"[error]Error fetching bytecode for {checksum_address} on {network.capitalize()}: {e}", style="error")
        return None


def get_contracts_from_tx(tx_hash, network='mainnet'):
    """Extracts contract addresses involved in a transaction (to, logs, creation) and verifies they have bytecode."""
    if 'w3_clients' not in globals() or network not in w3_clients:
         print(f"[error]Web3 client for '{network}' network not configured.", style="error")
         return None

    w3_client = w3_clients.get(network)
    if not w3_client or not w3_client.is_connected():
        console.print(f"[error]Web3 client for '{network}' not available or not connected.", style="error")
        return None

    try:
        console.print(f"[info]Fetching transaction details for {tx_hash} on {network.capitalize()}...", style="info")
        tx = w3_client.eth.get_transaction(tx_hash)
        if not tx:
             console.print(f"[error]Transaction {tx_hash} not found on {network}.", style="error")
             return None
        receipt = w3_client.eth.get_transaction_receipt(tx_hash)
        if not receipt:
             # This can happen if the transaction hasn't been mined yet
             console.print(f"[warning]Transaction receipt for {tx_hash} not found on {network}. Transaction might be pending.", style="warning")
             return None

        contracts = {} # {address: name}
        processed_addresses = set() # Avoid duplicate lookups

        # Helper to check address for bytecode and add to contracts dict
        def check_and_add_contract(address, name_prefix):
            if not address or address in processed_addresses:
                return
            try:
                code = w3_client.eth.get_code(address)
                # Check if code exists and is not empty bytecode marker
                if code and code != b'\x00' and code.hex() != '0x':
                    contract_name = f"{name_prefix} ({shorten_address(address)})"
                    contracts[address] = contract_name
                processed_addresses.add(address)
            except Exception as e:
                console.print(f"[warning]Could not check code for address {address}: {e}", style="warning")

        # Check 'to' address
        check_and_add_contract(tx.to, "Target Contract")

        # Check addresses from event logs
        for log in receipt.logs:
            check_and_add_contract(log.address, "Event Emitter")

        # Check contract creation address
        check_and_add_contract(receipt.contractAddress, "Created Contract")

        if contracts:
            console.print(f"[success]Found {len(contracts)} address(es) with bytecode involved in transaction {tx_hash}.", style="success")
        else:
            console.print(f"[warning]Found 0 addresses with bytecode involved in transaction {tx_hash}. Transaction might not interact with contracts or contracts might be destroyed.", style="warning")
        return contracts

    except Exception as e:
        console.print(f"[error]Error processing transaction {tx_hash}: {str(e)}", style="error")
        return None


def analyze_bytecode(bytecode, contract_name="Contract"):
    """Analyzes contract bytecode for size, standards, known functions, and common patterns."""
    if not bytecode or bytecode == "0x":
        console.print(f"[warning]No bytecode provided for analysis of '{contract_name}'.", style="warning")
        return None

    code = bytecode[2:] if bytecode.startswith('0x') else bytecode
    if not code:
        console.print(f"[warning]Empty bytecode provided for analysis of '{contract_name}'.", style="warning")
        return None

    results = {
        "contract_name": contract_name,
        "address": contract_name.split("(")[-1].strip(")") if "(" in contract_name and "..." in contract_name else "N/A",
        "size_bytes": len(code) // 2,
        "standards": [],
        "erc20_functionality": {"compatible": False, "functions": []},
        "erc721_functionality": {"compatible": False, "functions": []},
        "erc1155_functionality": {"compatible": False, "functions": []},
        "proxy_functionality": {"is_proxy": False, "proxy_type": None, "functions": []},
        "security_functionality": {"has_security_controls": False, "functions": []},
        "defi_functionality": {"has_defi_features": False, "functions": []},
        "gas_optimization": {"optimized": False, "features": []},
        "detected_patterns": [],
        "all_detected_functions": [], # Stores tuples (signature, name, category) for known functions
        "bytecode_metrics": {
            "size": len(code) // 2,
            "complexity_estimate": max(1, len(code) // 200), # Avoid 0, estimate based on size
            "method_count_estimate": 0, # Based on known methods found
            "has_loops_or_recursion": "Unknown" # Requires deeper analysis
        }
    }

    # Find potential 4-byte signatures using PUSH4 opcode heuristic
    potential_sigs_hex = set(re.findall(r'63([0-9a-f]{8})', code))
    potential_sigs = set(f"0x{s}" for s in potential_sigs_hex) # Add '0x' prefix

    found_known_sigs = set() # Track signatures already categorized

    # Check against known signature categories
    categories_config = {
        "ERC20": {"signatures": ERC20_SIGNATURES if 'ERC20_SIGNATURES' in globals() else {}, "threshold": 5, "result_key": "erc20_functionality"},
        "ERC721": {"signatures": ERC721_SIGNATURES if 'ERC721_SIGNATURES' in globals() else {}, "threshold": 4, "result_key": "erc721_functionality"},
        "ERC1155": {"signatures": ERC1155_SIGNATURES if 'ERC1155_SIGNATURES' in globals() else {}, "threshold": 4, "result_key": "erc1155_functionality"},
        "Proxy": {"signatures": {**(PROXY_PATTERNS if 'PROXY_PATTERNS' in globals() else {}), **(UUPS_PATTERNS if 'UUPS_PATTERNS' in globals() else {}), **(DIAMOND_PATTERNS if 'DIAMOND_PATTERNS' in globals() else {})}, "threshold": 1, "result_key": "proxy_functionality"},
        "Security": {"signatures": SECURITY_PATTERNS if 'SECURITY_PATTERNS' in globals() else {}, "threshold": 1, "result_key": "security_functionality"},
        "DeFi": {"signatures": DEFI_PATTERNS if 'DEFI_PATTERNS' in globals() else {}, "threshold": 1, "result_key": "defi_functionality"},
        "Gas Opt": {"signatures": GAS_PATTERNS if 'GAS_PATTERNS' in globals() else {}, "threshold": 1, "result_key": "gas_optimization"}
    }

    for category_name, config in categories_config.items():
        count = 0
        signatures_dict = config["signatures"]
        result_key = config["result_key"]
        functions_list = []

        for signature, func_name in signatures_dict.items():
            if signature in potential_sigs:
                count += 1
                functions_list.append(func_name)
                # Add to the overall list if not already seen
                if signature not in found_known_sigs:
                    results["all_detected_functions"].append((signature, func_name, category_name))
                    found_known_sigs.add(signature)

        # Store identified functions for this category
        if "functions" in results[result_key]:
            results[result_key]["functions"] = sorted(list(set(functions_list))) # Unique sorted list

        # Determine compatibility/presence based on threshold
        if count >= config["threshold"]:
            if "compatible" in results[result_key]: results[result_key]["compatible"] = True
            elif "is_proxy" in results[result_key]: results[result_key]["is_proxy"] = True
            elif "has_security_controls" in results[result_key]: results[result_key]["has_security_controls"] = True
            elif "has_defi_features" in results[result_key]: results[result_key]["has_defi_features"] = True
            elif "optimized" in results[result_key]: results[result_key]["optimized"] = True

    # Update total known method count estimate
    results["all_detected_functions"].sort(key=lambda x: (x[2], x[1])) # Sort by category, then name
    results["bytecode_metrics"]["method_count_estimate"] = len(results["all_detected_functions"])

    # Identify standards
    if results["erc20_functionality"]["compatible"]: results["standards"].append("ERC20")
    if results["erc721_functionality"]["compatible"]: results["standards"].append("ERC721")
    if results["erc1155_functionality"]["compatible"]: results["standards"].append("ERC1155")

    # Refine proxy type identification
    if results["proxy_functionality"]["is_proxy"]:
        proxy_funcs_found = {f[0] for f in results["all_detected_functions"] if f[2] == 'Proxy'}
        # Check based on signature sets defined globally
        is_eip1967 = any(s in proxy_funcs_found for s in (PROXY_PATTERNS if 'PROXY_PATTERNS' in globals() else {}))
        is_uups = any(s in proxy_funcs_found for s in (UUPS_PATTERNS if 'UUPS_PATTERNS' in globals() else {}))
        is_diamond = any(s in proxy_funcs_found for s in (DIAMOND_PATTERNS if 'DIAMOND_PATTERNS' in globals() else {}))

        proxy_type = "Basic/Unknown Proxy" # Default
        pattern_desc = "Proxy Pattern: Basic/Unknown"
        if is_diamond:
            proxy_type = "Diamond Proxy (EIP-2535)"
            pattern_desc = "Proxy Pattern: Diamond"
        elif is_uups: # UUPS often uses EIP-1967 storage, so check first
            proxy_type = "UUPS Proxy (EIP-1822 / EIP-1967)"
            pattern_desc = "Proxy Pattern: UUPS"
        elif is_eip1967:
            proxy_type = "Transparent Proxy (EIP-1967)"
            pattern_desc = "Proxy Pattern: Transparent"

        results["proxy_functionality"]["proxy_type"] = proxy_type
        results["detected_patterns"].append(pattern_desc)

    # Check for common library patterns based on security functions found
    sec_funcs = set(results["security_functionality"]["functions"])
    if "pause()" in sec_funcs and "unpause()" in sec_funcs: results["detected_patterns"].append("Pattern: Pausable")
    if "owner()" in sec_funcs and "transferOwnership(address)" in sec_funcs: results["detected_patterns"].append("Pattern: Ownable")

    # Check for other bytecode string patterns
    # Note: Simple string checks are prone to false positives but can be indicative
    if re.search(r'e3010170.{68}0033', code): results["detected_patterns"].append("Metadata: IPFS Hash (CBOR)")
    if "create2" in code: results["detected_patterns"].append("Opcode Hint: CREATE2")
    if "selfdestruct" in code: results["detected_patterns"].append("Opcode Hint: SELFDESTRUCT")

    # --- Output Results ---
    console.print(f"\n\n[bold yellow3]📊 Bytecode Analysis Results: {contract_name}[/bold yellow3]")
    console.print("──────────────────────────────", style="yellow3")

    # Summary Table using Rich
    analysis_table = Table(show_header=False, box=None, padding=(0,1))
    analysis_table.add_column("Property")
    analysis_table.add_column("Value")
    analysis_table.add_row("Size:", f"{results['size_bytes']:,} bytes")
    if results["standards"]: analysis_table.add_row("Standards:", ", ".join(results["standards"]))
    if results["proxy_functionality"]["is_proxy"]:
        analysis_table.add_row("Proxy Contract:", "✅ Yes")
        if results["proxy_functionality"]["proxy_type"]: analysis_table.add_row("Proxy Type:", results["proxy_functionality"]["proxy_type"])
    if results["security_functionality"]["has_security_controls"]: analysis_table.add_row("Security Controls:", "✅ Yes")
    if results["defi_functionality"]["has_defi_features"]: analysis_table.add_row("DeFi Features:", "✅ Yes")
    if results["detected_patterns"]: analysis_table.add_row("Other Patterns/Hints:", Text(", ".join(results["detected_patterns"]), overflow="fold"))
    complexity_val = results["bytecode_metrics"]["complexity_estimate"]
    complexity_label = "Low" if complexity_val < 10 else "Medium" if complexity_val < 50 else "High"
    analysis_table.add_row("[bold]Complexity Estimate:[/bold]", complexity_label)
    analysis_table.add_row("[bold]Known Methods Found:[/bold]", str(results["bytecode_metrics"]["method_count_estimate"]))
    console.print(analysis_table)

    # Overview DataFrame using Pandas
    result_df_data = [
        {"Property": "Size", "Value": f"{results['size_bytes']:,} bytes"},
        {"Property": "Standards", "Value": ", ".join(results["standards"]) if results["standards"] else "None detected"},
        {"Property": "Is Proxy", "Value": "Yes" if results["proxy_functionality"]["is_proxy"] else "No"},
        {"Property": "Proxy Type", "Value": results["proxy_functionality"]["proxy_type"] if results["proxy_functionality"]["is_proxy"] else "N/A"},
        {"Property": "Security Controls", "Value": "Yes" if results["security_functionality"]["has_security_controls"] else "No"},
        {"Property": "Complexity", "Value": complexity_label},
        {"Property": "Known Method Count", "Value": results["bytecode_metrics"]["method_count_estimate"]}
    ]
    result_df = pd.DataFrame(result_df_data)
    display(HTML("<h4>Contract Overview</h4>"))
    display(result_df)

    # Detected Known Functions DataFrame using Pandas
    if results["all_detected_functions"]:
        func_df_data = [{"Category": cat, "Signature": sig, "Function Name": name}
                        for sig, name, cat in results["all_detected_functions"]]
        func_df = pd.DataFrame(func_df_data)
        display(HTML(f"<h4>Detected Known Functions ({len(func_df)} entries)</h4>"))
        with pd.option_context('display.max_rows', 100): # Limit display length in notebook
             display(func_df)

    return results


def compare_proxy_implementation(proxy_code, impl_code, proxy_name="Proxy", impl_name="Implementation"):
    """Compares proxy and implementation contracts focusing on size and known function signatures."""
    if not proxy_code or proxy_code == '0x' or not impl_code or impl_code == '0x':
        console.print("[warning]Missing or empty bytecode provided for proxy-implementation comparison.", style="warning")
        return None

    proxy_size = len(proxy_code[2:]) // 2 if proxy_code.startswith('0x') else len(proxy_code) // 2
    impl_size = len(impl_code[2:]) // 2 if impl_code.startswith('0x') else len(impl_code) // 2

    console.print("\n\n[bold]🔄 Proxy vs. Implementation Comparison:[/bold]", style="chartreuse1")
    console.print("─────────────────────────────────────────", style="chartreuse1")
    console.print(f"\n\n{proxy_name} & {impl_name}", style="chartreuse1")

    # --- Size Comparison ---
    display(HTML("<h4>Size Comparison</h4>"))
    sizes_df = pd.DataFrame({
        'Contract': [proxy_name, impl_name],
        'Size (bytes)': [proxy_size, impl_size],
        'Notes': ["Delegates calls, minimal logic", "Contains main business logic"]
    })
    display(sizes_df)

    try:
        console.print("\n\n[bold]Contract Size Comparison (Bytes)[/bold]", style="magenta3")
        console.print("──────────────────────────────", style="magenta3")

        # Create size comparison chart using matplotlib
        plt.figure(figsize=(12, 6))
        bars = plt.bar(sizes_df['Contract'], sizes_df['Size (bytes)'], color=['skyblue', 'lightcoral'])

        # Add data labels on top of bars
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                    f'{int(height):,}', ha='center', va='bottom')

        plt.title('Contract Size Comparison (Bytes)')
        plt.ylabel('Size (bytes)')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        display(plt.gcf())
        plt.close()
    except Exception as e:
        console.print(f"[warning]Could not generate size comparison chart: {e}", style="warning")

    comparison_results = {
        "proxy_name": proxy_name, "implementation_name": impl_name,
        "proxy_size": proxy_size, "implementation_size": impl_size,
        "size_ratio": impl_size / proxy_size if proxy_size > 0 else float('inf')
    }

    # Add spacing between visualizations
    display(HTML("<div style='height: 20px;'></div>"))

    # --- Function Signature Analysis (Known Signatures Only) ---
    console.print("\n[bold]Function Signature Analysis (Known Signatures)[/bold]", style="magenta3")
    console.print("────────────────────────────────────────────────", style="magenta3")

    # Helper to extract known signatures from bytecode
    def get_known_signatures(bytecode):
        code_str = bytecode[2:] if bytecode.startswith('0x') else bytecode
        sigs_hex = set(re.findall(r'63([0-9a-f]{8})', code_str)) # PUSH4 heuristic
        return set(f"0x{s}" for s in sigs_hex if f"0x{s}" in ALL_KNOWN_SIGNATURES)

    proxy_known_sigs = get_known_signatures(proxy_code)
    impl_known_sigs = get_known_signatures(impl_code)

    shared_sigs = proxy_known_sigs.intersection(impl_known_sigs)
    proxy_unique = proxy_known_sigs - shared_sigs
    impl_unique = impl_known_sigs - shared_sigs

    console.print(f"Analysis based on signatures found in the known dictionary:")
    console.print(f"[bold]Shared Known Signatures:[/bold] {len(shared_sigs)}")
    console.print(f"[bold]Proxy-Only Known Signatures:[/bold] {len(proxy_unique)}")
    console.print(f"[bold]Implementation-Only Known Signatures:[/bold] {len(impl_unique)}")

        # --- Generate Function Distribution Chart (Excluding 0% Categories) ---
    try:
        # Define original categories, their corresponding values, and consistent colors
        all_categories = ['Shared Known', 'Proxy Only Known', 'Implementation Only Known']
        all_values = [len(shared_sigs), len(proxy_unique), len(impl_unique)]
        # Use a map to ensure colors stay consistent with categories after filtering
        color_map = {
            'Shared Known': 'mediumseagreen',
            'Proxy Only Known': 'skyblue',
            'Implementation Only Known': 'salmon'
        }

        # Filter out categories with zero signatures to avoid cluttering the chart
        filtered_categories = []
        filtered_values = []
        filtered_colors = []
        for category, value in zip(all_categories, all_values):
            if value > 0:
                filtered_categories.append(category)
                filtered_values.append(value)
                filtered_colors.append(color_map[category])

        # Proceed only if there are non-zero categories to display
        if filtered_values:
            try:
                # Display header for the chart section
                console.print("\n\n[bold]Distribution of Known Function Signatures (Non-Zero)[/bold]", style="magenta3")
                console.print("───────────────────────────────────────────────────", style="magenta3")

                # Create the pie chart using the filtered data
                plt.figure(figsize=(8, 4)) # Use smaller dimensions to reduce rendering size
                plt.pie(
                    filtered_values,
                    labels=filtered_categories,
                    autopct='%1.1f%%',
                    colors=filtered_colors,
                    startangle=90,
                    pctdistance=0.85, # Place percentage labels slightly inside slices (helps with overlap)
                )
                plt.axis('equal') # Ensure the pie chart is drawn as a circle

                # Add a legend showing counts for each displayed category
                plt.legend(
                    [f"{cat} ({val})" for cat, val in zip(filtered_categories, filtered_values)],
                    loc="best" # Position legend automatically
                )

                plt.tight_layout() # Adjust plot to prevent labels from clipping
                display(plt.gcf()) # Display the plot in the notebook
                plt.close() # Close the plot object to release memory

                # --- Display Filtered Data in a Table ---
                # Calculate percentages based on the filtered data sum
                func_dist_df_data = []
                total_sum = sum(filtered_values)
                for cat, val in zip(filtered_categories, filtered_values):
                     func_dist_df_data.append({
                         'Category': cat,
                         'Count': val,
                         'Percentage': f"{val/total_sum*100:.1f}%" if total_sum > 0 else "0.0%"
                     })
                func_dist_df = pd.DataFrame(func_dist_df_data)

                # Display the DataFrame with a clear title
                display(HTML("<h5>Known Signature Distribution (Non-Zero)</h5>"))
                display(func_dist_df)

            except Exception as e:
                console.print(f"[error]Error creating pie chart: {e}", style="error")
                # Fallback: Display the DataFrame even if the chart fails, if data exists
                if 'func_dist_df' in locals() and not func_dist_df.empty:
                    display(HTML("<h5>Known Signature Distribution (Chart Generation Failed)</h5>"))
                    display(func_dist_df)

        else:
            # If all categories filtered out, display a notification message
             display(HTML("""
            <div style="margin-top: 15px; padding: 10px; border: 1px solid #ddd; background-color: #00005f; font-size: 0.9em; color:white;">
                <strong>Note:</strong> No known function signatures were found in either contract for comparison.
            </div>
            """))

    except Exception as e:
        # Catch any unexpected errors during the analysis process
        console.print(f"[error]Error in signature distribution analysis: {e}", style="error")

    # Add spacing between visualizations
    display(HTML("<div style='height: 20px;'></div>"))

    # Prepare DataFrame with identified functions for display
    try:
        # Convert signatures to function names for better readability
        shared_funcs = [ALL_KNOWN_SIGNATURES.get(sig, sig) for sig in shared_sigs]
        proxy_funcs = [ALL_KNOWN_SIGNATURES.get(sig, sig) for sig in proxy_unique]
        impl_funcs = [ALL_KNOWN_SIGNATURES.get(sig, sig) for sig in impl_unique]

        # Create DataFrames for each category
        shared_df = pd.DataFrame({'Signature': list(shared_sigs), 'Function': shared_funcs, 'Category': 'Shared'}) if shared_sigs else None
        proxy_only_df = pd.DataFrame({'Signature': list(proxy_unique), 'Function': proxy_funcs, 'Category': 'Proxy Only'}) if proxy_unique else None
        impl_only_df = pd.DataFrame({'Signature': list(impl_unique), 'Function': impl_funcs, 'Category': 'Implementation Only'}) if impl_unique else None

        # Display each DataFrame if it exists
        if shared_df is not None and not shared_df.empty:
            display(HTML("<h5>Shared Known Functions</h5>"))
            display(shared_df)
        if proxy_only_df is not None and not proxy_only_df.empty:
            display(HTML("<h5>Proxy-Only Known Functions</h5>"))
            display(proxy_only_df)
        if impl_only_df is not None and not impl_only_df.empty:
            display(HTML("<h5>Implementation-Only Known Functions</h5>"))
            display(impl_only_df)
    except Exception as e:
        console.print(f"[error]Error displaying function details: {e}", style="error")

    display(HTML("""
    <div style="margin-top: 15px; padding: 10px; border: 1px solid #ddd; background-color: #00005f; font-size: 0.9em; color:white;">
    <strong>Note on Function Identification:</strong> This analysis focuses on function signatures present in our predefined dictionaries (ERC20, Proxy, Security, etc.), identified using bytecode patterns. Bytecode contains many 4-byte sequences; those not matching known signatures or patterns are not listed here. Full identification requires comprehensive databases (e.g., <a href="https://www.4byte.directory/" target="_blank">4byte.directory</a>) or decompilation tools.
    </div>
    """))

    # Add signature data to comparison results (as list of dicts for easier JSON export)
    comparison_results["shared_functions"] = shared_df.to_dict('records') if shared_df is not None else []
    comparison_results["proxy_only_functions"] = proxy_only_df.to_dict('records') if proxy_only_df is not None else []
    comparison_results["implementation_only_functions"] = impl_only_df.to_dict('records') if impl_only_df is not None else []

    return comparison_results


def compare_multiple_contracts(contract_codes, contract_names):
    """Compares multiple contracts based on size and similarity of known function signatures."""
    if len(contract_codes) < 2:
        console.print("[warning]Need at least two contracts with valid bytecode for comparison.", style="warning")
        return None
    if len(contract_codes) != len(contract_names):
        console.print("[error]Internal error: Number of contract codes and names provided for comparison do not match.", style="error")
        return None

    console.print(f"\n\n[bold cyan3]📊 Multi-Contract Comparison ({len(contract_names)} Contracts)[/bold cyan3]")
    console.print("──────────────────────────────────────────", style="cyan3")

    # --- Size Comparison ---
    display(HTML("<h4>Size Comparison</h4>"))
    sizes = [len(code[2:]) // 2 if code.startswith('0x') else len(code) // 2 for code in contract_codes]
    sizes_df = pd.DataFrame({'Contract': contract_names, 'Size (bytes)': sizes})
    display(sizes_df.sort_values('Size (bytes)', ascending=False))

    try:
        console.print("\n\n[bold]Contract Size Comparison (Bytes)[/bold]", style="magenta3")
        console.print("─────────────────────────────────", style="magenta3")

        # Create size comparison chart using matplotlib
        plt.figure(figsize=(12, max(6, 0.5 * len(contract_names))))

        # Sort data for better visualization
        sorted_sizes_df = sizes_df.sort_values('Size (bytes)', ascending=True)

        # Create horizontal bar chart for better fit with many contracts
        bars = plt.barh(sorted_sizes_df['Contract'], sorted_sizes_df['Size (bytes)'],
                      color=plt.cm.Paired(np.linspace(0, 1, len(sorted_sizes_df))))

        # Add text labels next to bars
        for bar in bars:
            width = bar.get_width()
            plt.text(width + 0.1, bar.get_y() + bar.get_height()/2, f"{int(width):,}",
                    ha='left', va='center')

        plt.title('Contract Size Comparison (Bytes)')
        plt.xlabel('Size (bytes)')
        plt.grid(axis='x', linestyle='--', alpha=0.7)
        plt.tight_layout()
        display(plt.gcf())
        plt.close()
    except Exception as e:
        console.print(f"[warning]Could not generate multi-contract size chart: {e}", style="warning")

    # Add spacing between visualizations
    display(HTML("<div style='height: 20px;'></div>"))

    # --- Function Signature Similarity (Known Signatures Only) ---
    display(HTML("<h4>Function Signature Similarity (Based on Known Signatures)</h4>"))
    console.print("[dim]Comparing contracts based on the Jaccard similarity of their known function signatures.", style="dim")

    # Helper to extract known signatures
    def get_known_signatures(bytecode):
        code_str = bytecode[2:] if bytecode.startswith('0x') else bytecode
        sigs_hex = set(re.findall(r'63([0-9a-f]{8})', code_str))
        return set(f"0x{s}" for s in sigs_hex if f"0x{s}" in ALL_KNOWN_SIGNATURES)

    all_signatures_known = {name: get_known_signatures(code) for name, code in zip(contract_names, contract_codes)}

    similarity_data = []
    for i in range(len(contract_names)):
        for j in range(i + 1, len(contract_names)):
            name_i, name_j = contract_names[i], contract_names[j]
            sigs_i, sigs_j = all_signatures_known[name_i], all_signatures_known[name_j]

            intersection_size = len(sigs_i.intersection(sigs_j))
            union_size = len(sigs_i.union(sigs_j))
            similarity = (intersection_size / union_size * 100) if union_size > 0 else 0

            similarity_data.append({
                'Contract Pair': f"{name_i} & {name_j}",
                'Shared Known Signatures': intersection_size,
                'Total Unique Known Signatures': union_size,
                'Similarity (%)': similarity
            })

    if not similarity_data:
        console.print("[info]No contract pairs to compare or no known signatures found in multiple contracts.", style="info")
        # Return basic info even if comparison fails
        return {"contract_names": contract_names, "sizes": dict(zip(contract_names, sizes)), "similarity_data": []}

    similarity_df = pd.DataFrame(similarity_data).sort_values('Similarity (%)', ascending=False)
    similarity_df['Similarity (%)'] = similarity_df['Similarity (%)'].round(2)

    display(HTML("<h5>Pairwise Similarity (Jaccard Index of Known Signatures)</h5>"))
    display(similarity_df)

    # Similarity Bar Chart
    try:
        console.print("\n\n[bold]Contract Similarity (% Shared Known Signatures)[/bold]", style="magenta3")
        console.print("─────────────────────────────────────────────────", style="magenta3")
        console.print("[italic]Enlarge small visualization previews by clicking.[/italic]", style="cyan")

        # Create similarity bar chart using matplotlib
        plt.figure(figsize=(10, max(6, 0.6 * len(similarity_df))))

        # Sort data by similarity for better visualization
        sorted_df = similarity_df.sort_values('Similarity (%)')

        # Create horizontal bar chart with color gradient
        cmap = plt.cm.viridis
        norm = plt.Normalize(min(sorted_df['Similarity (%)']), max(sorted_df['Similarity (%)']))
        colors = cmap(norm(sorted_df['Similarity (%)']))

        bars = plt.barh(sorted_df['Contract Pair'], sorted_df['Similarity (%)'], color=colors)

        # Add percentage labels
        for bar in bars:
            width = bar.get_width()
            plt.text(width + 1, bar.get_y() + bar.get_height()/2, f"{width:.1f}%",
                    ha='left', va='center')

        plt.title('Contract Similarity (% Shared Known Signatures)')
        plt.xlabel('Similarity (%)')
        plt.grid(axis='x', linestyle='--', alpha=0.7)
        plt.tight_layout()
        display(plt.gcf())
        plt.close()
    except Exception as e:
        console.print(f"[error]Could not generate similarity chart: {str(e)}", style="error")
        # Try to display the data in a simpler format as fallback
        display(HTML(f"<h5>Contract Similarity Data (Chart Generation Failed)</h5>"))
        display(similarity_df)

    # Use light-theme friendly version for the note
    display(HTML("""
    <div style="margin-top: 15px; padding: 10px; border: 1px solid #ddd; background-color: #00005f; font-size: 0.9em; color:white;">
    <strong>Note:</strong> Similarity calculation uses only function signatures found in the predefined dictionaries. Higher similarity indicates more shared known functionality.
    </div>
    """))

    return {
        "contract_names": contract_names,
        "sizes": dict(zip(contract_names, sizes)),
        "signature_count_known": {name: len(sigs) for name, sigs in all_signatures_known.items()},
        "similarity_data": similarity_df.to_dict('records')
    }


# --- Export Functions ---

def export_to_google_sheets(analysis_results, comparison_results=None, multi_contract_comparison=None):
    """Exports the analysis data to a new Google Sheet."""
    if 'gc_sheets' not in globals() or gc_sheets is None:
         console.print("[error]Google Sheets client (gc_sheets) is not initialized. Cannot export.", style="error")
         return HTML("<div style='color:red'>Error: Google Sheets client not initialized.</div>")

    try:
        console.print("[cyan3]Exporting analysis results to Google Sheets...", style="info")
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        sheet_title = f"Smart Contract Bytecode Analysis {timestamp}"
        spreadsheet = gc_sheets.create(sheet_title)
        worksheet = spreadsheet.get_worksheet(0)
        worksheet.update_title("Bytecode Analysis Report")

        # --- Populate Sheet (Adapt based on required detail level) ---
        current_row = 1
        header_values = [
            [f"Smart Contract Bytecode Analysis Report"],
            [f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"], [""]
        ]
        worksheet.update(f"A{current_row}", header_values)
        worksheet.format(f"A1:A1", {"textFormat": {"bold": True, "fontSize": 14}})
        current_row += len(header_values)

        # Individual Contract Analysis Sections
        for contract_name, results in analysis_results.items():
            worksheet.update(f"A{current_row}", [[f"Analysis: {contract_name}"]])
            worksheet.format(f"A{current_row}", {"textFormat": {"bold": True, "fontSize": 12}, "backgroundColor": {"red": 0.9, "green": 0.9, "blue": 1.0}})
            current_row += 1

            info_table = [["Property", "Value"]]
            info_table.append(["Size", f"{results.get('size_bytes', 0):,} bytes"])
            if results.get("standards"): info_table.append(["Standards", ", ".join(results["standards"])])
            if results.get("proxy_functionality", {}).get("is_proxy"):
                info_table.append(["Is Proxy", "Yes"])
                info_table.append(["Proxy Type", results["proxy_functionality"].get("proxy_type", "N/A")])
            # Add more fields as needed...
            worksheet.update(f"A{current_row}", info_table)
            worksheet.format(f"A{current_row}:B{current_row}", {"textFormat": {"bold": True}}) # Header bold
            current_row += len(info_table) + 1 # Add spacing

            # Detected Functions Table
            if results.get("all_detected_functions"):
                 func_table = [["Category", "Signature", "Function Name"]]
                 for sig, name, cat in results["all_detected_functions"][:30]: # Limit rows in export
                     func_table.append([cat, sig, name])
                 if len(results["all_detected_functions"]) > 30:
                      func_table.append(["...", "...", f"(and {len(results['all_detected_functions']) - 30} more)"])

                 worksheet.update(f"A{current_row}", [["Detected Known Functions"]])
                 worksheet.format(f"A{current_row}", {"textFormat": {"bold": True}})
                 current_row +=1
                 worksheet.update(f"A{current_row}", func_table)
                 worksheet.format(f"A{current_row}:C{current_row}", {"textFormat": {"bold": True}}) # Header bold
                 current_row += len(func_table) + 1


        # Proxy-Implementation Comparison Section
        if comparison_results:
             worksheet.update(f"A{current_row}", [["Proxy vs. Implementation Comparison"]])
             worksheet.format(f"A{current_row}", {"textFormat": {"bold": True, "fontSize": 12}, "backgroundColor": {"red": 0.9, "green": 1.0, "blue": 0.9}})
             current_row += 1
             comp_table = [["Contract", "Size (bytes)"],
                           [comparison_results.get("proxy_name", "Proxy"), f"{comparison_results.get('proxy_size', 0):,}"],
                           [comparison_results.get("implementation_name", "Impl"), f"{comparison_results.get('implementation_size', 0):,}"]]
             worksheet.update(f"A{current_row}", comp_table)
             worksheet.format(f"A{current_row}:B{current_row}", {"textFormat": {"bold": True}}) # Header bold
             current_row += len(comp_table) + 1
             # Add function comparison tables if needed...

        # Multi-Contract Comparison Section
        if multi_contract_comparison:
             worksheet.update(f"A{current_row}", [["Multi-Contract Comparison"]])
             worksheet.format(f"A{current_row}", {"textFormat": {"bold": True, "fontSize": 12}, "backgroundColor": {"red": 1.0, "green": 0.9, "blue": 0.9}})
             current_row += 1
             # Add size table...
             # Add similarity table...
             sim_table = [["Contract Pair", "Similarity (%)", "Shared Known Sigs"]]
             for item in multi_contract_comparison.get("similarity_data", [])[:30]: # Limit rows
                 sim_table.append([item.get("Contract Pair"), f"{item.get('Similarity (%)', 0):.1f}%", item.get("Shared Known Signatures")])
             if len(multi_contract_comparison.get("similarity_data", [])) > 30:
                 sim_table.append(["...", "...", "..."])

             worksheet.update(f"A{current_row}", [["Similarity (Known Signatures)"]])
             worksheet.format(f"A{current_row}", {"textFormat": {"bold": True}})
             current_row += 1
             worksheet.update(f"A{current_row}", sim_table)
             worksheet.format(f"A{current_row}:C{current_row}", {"textFormat": {"bold": True}}) # Header bold
             current_row += len(sim_table) + 1

        # Auto-resize columns for readability (optional, might fail on complex sheets)
        try: worksheet.columns_auto_resize(0, 5)
        except: pass

        # --- Finalize and Return Link ---
        clear_output()
        console.print("✓ Successfully exported analysis to Google Sheets!", style="spring_green3")
        spreadsheet_url = f"https://docs.google.com/spreadsheets/d/{spreadsheet.id}"
        html = f'''
        <script> window.open("{spreadsheet_url}", "_blank"); </script>
        <div>Spreadsheet created: <a href="{spreadsheet_url}" target="_blank">{sheet_title}</a> (Opened in new tab)</div>
        '''
        return HTML(html)

    except Exception as e:
        clear_output()
        console.print(f"❌ Error exporting to Google Sheets: {str(e)}", style="error")
        return HTML(f"<div style='color:red; font-weight:bold;'>Error exporting to Google Sheets: {str(e)}</div>")


def download_analysis_json(analysis_results, comparison_results=None, multi_contract_comparison=None, filename=None):
    """Creates a JSON file containing the analysis results for download."""
    console.print("[cyan3]Preparing JSON export...", style="info")

    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"bytecode_analysis_{timestamp}.json"

    export_data = {
        "analysis_metadata": {
             "export_time": datetime.now().isoformat(),
             "tool_version": "1.0" # Example version
        },
        "contract_analysis": analysis_results,
    }
    if comparison_results: export_data["proxy_implementation_comparison"] = comparison_results
    if multi_contract_comparison: export_data["multi_contract_comparison"] = multi_contract_comparison

    try:
        json_str = json.dumps(export_data, default=str, indent=2)
        b64 = base64.b64encode(json_str.encode()).decode()

        clear_output() # Clear the "preparing" message
        console.print(f"✓ Successfully prepared JSON file: {filename}", style="spring_green3")

        # Create HTML for direct download link/trigger
        html = f'''
        <a download="{filename}" href="data:application/json;base64,{b64}" style="text-decoration:none;">
            <button style="padding: 8px 12px; background-color: #007bff; color: white; border: none; border-radius: 4px; cursor: pointer;">
                Download {filename}
            </button>
        </a>
        <script>
        // Optional: uncomment to auto-trigger download immediately
        /*
        (function() {{
            const link = document.createElement('a');
            link.href = "data:application/json;base64,{b64}";
            link.download = "{filename}";
            document.body.appendChild(link);
            link.click();
            document.body.removeChild(link);
        }})();
        */
        </script>
        '''
        return HTML(html)
    except Exception as e:
         clear_output()
         console.print(f"❌ Error creating JSON export: {str(e)}", style="error")
         return HTML(f"<div style='color:red; font-weight:bold;'>Error creating JSON: {str(e)}</div>")


# --- UI Setup and Tab Definitions ---

# Global containers for results (cleared by analysis functions)
analysis_results = {}
comparison_data = None
multi_comparison_data = None

# Main Tab Interface
analysis_tabs = widgets.Tab()

# Apply Custom CSS for better UI (ensure this matches your notebook theme)
display(HTML("""
<style>
.jupyter-widgets-output-area .jupyter-widgets .widget-tab > .p-TabBar-tabLabel {
    font-weight: bold;
}
.jupyter-widgets-output-area .jupyter-widgets .widget-tab-contents {
    border: 1px solid #ccc;
    padding: 15px;
}
.jupyter-widgets-output-area .jupyter-widgets .jupyter-button {
    color: white;
}
</style>
"""))


def show_export_options(parent_output=None):
    """Displays export buttons (JSON, Google Sheets) using global result variables."""
    global analysis_results, comparison_data, multi_comparison_data

    # Check if there are any results to export
    has_analysis = isinstance(analysis_results, dict) and analysis_results
    has_comparison = isinstance(comparison_data, dict) and comparison_data
    has_multi = isinstance(multi_comparison_data, dict) and multi_comparison_data

    if not has_analysis:
        # Display message within the provided output widget or directly
        message = "[warning]No analysis results currently available to export.[/warning]"
        if parent_output:
             with parent_output: console.print(message, style="warning")
        else: console.print(message, style="warning")
        return

    # Create buttons and output area for export actions
    export_buttons = widgets.HBox([
        widgets.Button(description='Export as JSON', button_style='warning', icon='download', layout=widgets.Layout(width='auto')),
        widgets.Button(description='Export to Google Sheets', button_style='info', icon='table', layout=widgets.Layout(width='auto'))
    ])
    export_output = widgets.Output()

    # Define button click handlers
    def export_json_handler(b):
        with export_output:
            clear_output()
            result_html = download_analysis_json(
                analysis_results if has_analysis else {},
                comparison_data if has_comparison else None,
                multi_comparison_data if has_multi else None
            )
            display(result_html)

    def export_sheets_handler(b):
        with export_output:
            clear_output()
            result_html = export_to_google_sheets(
                analysis_results if has_analysis else {},
                comparison_data if has_comparison else None,
                multi_comparison_data if has_multi else None
            )
            display(result_html)

    # Assign handlers
    export_buttons.children[0].on_click(export_json_handler)
    export_buttons.children[1].on_click(export_sheets_handler)

    header_output = widgets.Output()
    with header_output: # Capture prints into this widget
      console.print("\n\n[bold]Export Analysis Results:[/bold]", style="magenta3")
      console.print("────────────────────────", style="magenta3")

    # Combine into a VBox for display
    export_widget = widgets.VBox([
        header_output,
        export_buttons,
        export_output
    ])

    # Display within the parent output widget or directly
    if parent_output:
        with parent_output: display(export_widget)
    else: display(export_widget)


# --- Tab Content Generation Functions ---

def analyze_pyusd_tab():
    """Creates the UI and logic for the PYUSD analysis tab."""
    pyusd_tab = widgets.VBox()
    pyusd_output = widgets.Output()
    analyze_button = widgets.Button(description='Re-Analyze PYUSD Contracts', button_style='primary', icon='refresh', layout=widgets.Layout(width='auto'))

    def analyze_pyusd_action(b):
        # Access and reset global result containers
        global analysis_results, comparison_data, multi_comparison_data
        analysis_results = {}
        comparison_data = None
        multi_comparison_data = None

        with pyusd_output:
            clear_output()
            display(HTML("<h4 style='color: #00d7af;'>Analyzing Official PYUSD Contracts on Mainnet</h4>"))
            console.print("──────────────────────────────────────", style="cyan3")

            # Define contracts to analyze (ensure these globals exist)
            contracts_to_analyze = {}
            if 'PYUSD_PROXY' in globals(): contracts_to_analyze["PYUSD Proxy"] = PYUSD_PROXY
            if 'PYUSD_IMPLEMENTATION' in globals(): contracts_to_analyze["PYUSD Implementation"] = PYUSD_IMPLEMENTATION
            if 'SUPPLY_CONTROL_PROXY' in globals(): contracts_to_analyze["Supply Controller Proxy"] = SUPPLY_CONTROL_PROXY
            # Add Supply Controller Impl if needed for specific comparisons

            codes = {}
            analyses = {}

            # Fetch and analyze each contract
            for name, address in contracts_to_analyze.items():
                if not address:
                    console.print(f"[warning]Skipping {name}: Address not defined.", style="warning")
                    continue
                code = get_contract_code(address, network='mainnet')
                if code:
                    codes[name] = code
                    contract_display_name = f"{name} ({shorten_address(address)})"
                    analysis = analyze_bytecode(code, contract_display_name)
                    if analysis:
                        analyses[name] = analysis
                        analysis_results[contract_display_name] = analysis # Use display name as key

            # Perform Proxy-Implementation comparison if both analyzed
            proxy_name = "PYUSD Proxy"
            impl_name = "PYUSD Implementation"
            if proxy_name in analyses and impl_name in analyses:
                comparison_data = compare_proxy_implementation(
                    codes[proxy_name], codes[impl_name],
                    analyses[proxy_name]["contract_name"], analyses[impl_name]["contract_name"]
                )

            # Generate architecture/relationship diagrams (using Graphviz)
            # Ensure Graphviz is installed and on the system PATH
            try:
                # Proxy Architecture Diagram
                if proxy_name in analyses and impl_name in analyses and analyses[proxy_name]["proxy_functionality"]["is_proxy"]:
                    display(HTML("<h4>PYUSD Proxy Architecture Diagram</h4>"))
                    arch_graph = Digraph(comment="PYUSD Proxy Architecture", format='png')
                    arch_graph.attr(rankdir='LR', bgcolor='transparent', node_sep='0.5', rank_sep='1')
                    arch_graph.attr('node', shape='box', style='filled', fontname='helvetica', fontsize='10')
                    arch_graph.node('user', 'User / DApp', fillcolor='lightblue')
                    arch_graph.node('proxy', analyses[proxy_name]["contract_name"], fillcolor='palegreen')
                    arch_graph.node('impl', analyses[impl_name]["contract_name"], fillcolor='lightcoral')
                    # Optionally add Admin Proxy node if known
                    # arch_graph.node('admin', 'Proxy Admin\n(Upgrade Control)', fillcolor='lightyellow')
                    arch_graph.edge('user', 'proxy', label='Calls')
                    arch_graph.edge('proxy', 'impl', label='delegatecall')
                    # arch_graph.edge('admin', 'proxy', label='Upgrades')
                    display(arch_graph)

                # Add spacing between diagrams
                display(HTML("<div style='height: 20px;'></div>"))

                # Contract Relationship Diagram (PYUSD + Supply Controller)
                supply_proxy_name = "Supply Controller Proxy"
                if proxy_name in analyses and supply_proxy_name in analyses:

                     console.print("\n\n[bold]PYUSD Contract Relationships Diagram[/bold]", style="magenta3")
                     console.print("───────────────────────────────────", style="magenta3")
                     rel_graph = Digraph(comment="PYUSD Contract Relationships", format='png')
                     rel_graph.attr(bgcolor='transparent', node_sep='0.5', rank_sep='0.7')
                     rel_graph.attr('node', shape='box', style='filled', fontname='helvetica', fontsize='10')
                     pyusd_node_name = analyses[proxy_name]["contract_name"]
                     supply_node_name = analyses[supply_proxy_name]["contract_name"]
                     rel_graph.node('pyusd', pyusd_node_name, fillcolor='palegreen')
                     if impl_name in analyses:
                         rel_graph.node('pyusd_impl', analyses[impl_name]["contract_name"], fillcolor='lightcyan')
                         rel_graph.edge('pyusd', 'pyusd_impl', label='delegates to')
                     rel_graph.node('supply', supply_node_name, fillcolor='lightsalmon')
                     # Add Supply Impl if analyzed
                     # rel_graph.node('supply_impl', 'Supply Impl\n(...)', fillcolor='lightpink')
                     # rel_graph.edge('supply', 'supply_impl', label='delegates to')
                     rel_graph.edge('supply', 'pyusd', label='controls supply (mint/burn)')
                     display(rel_graph)

            except Exception as viz_err:
                 console.print(f"[warning]Could not render architecture/relationship diagram. Ensure Graphviz is installed and in PATH. Error: {viz_err}", style="warning")

            # Show export options if analysis produced results
            if analysis_results:
                show_export_options(pyusd_output)
            else:
                console.print("\n[error]No analysis results were generated for PYUSD contracts. Check network connection and contract addresses.", style="error")

    analyze_button.on_click(analyze_pyusd_action)

    # Layout for the tab
    pyusd_tab.children = [
        widgets.HTML("<h3>Analyze Official PYUSD Contracts</h3>"),
        widgets.HTML("<p>Analyzes the main PYUSD token proxy, implementation, and supply controller contracts on Ethereum Mainnet. Fetches bytecode, performs analysis, and visualizes relationships.</p>"),
        analyze_button,
        pyusd_output
    ]

    # Automatically run the analysis when the tab is first created
    analyze_pyusd_action(None)

    return pyusd_tab


def analyze_tx_tab():
    """Creates the UI and logic for the Transaction-based analysis tab."""
    tx_tab = widgets.VBox()
    tx_input = widgets.Text(placeholder='Enter transaction hash (e.g., 0xabc...)', description='Tx Hash:', layout=widgets.Layout(width='90%'))
    network_selector = widgets.Dropdown(options=['mainnet', 'sepolia', 'holesky'], value='mainnet', description='Network:', layout=widgets.Layout(width='auto'))
    analyze_button = widgets.Button(description='Analyze Contracts in Tx', button_style='primary', icon='search', layout=widgets.Layout(width='auto'))
    tx_output = widgets.Output()

    def analyze_tx_action(b, tx_hash_override=None): # Allow passing hash for auto-run
        # Access and reset global result containers
        global analysis_results, multi_comparison_data, comparison_data
        analysis_results = {}
        multi_comparison_data = None
        comparison_data = None # Reset single comparison too

        with tx_output:
            clear_output()
            console.print("\n\n[bold]Analyzing Transaction for Contracts[/bold]", style="cyan3")
            console.print("─────────────────────────────────", style="cyan3")


            # Determine transaction hash to use
            current_tx_hash = tx_hash_override if tx_hash_override else tx_input.value.strip()

            # Validate Tx Hash format
            if not current_tx_hash or not re.match(r'^0x[0-9a-fA-F]{64}$', current_tx_hash):
                console.print("[error]Invalid input: Please enter a valid 64-character transaction hash starting with 0x.", style="error")
                return

            # Update input field if hash was passed programmatically
            if tx_hash_override: tx_input.value = tx_hash_override

            selected_network = network_selector.value
            # Fetch contracts involved in the transaction
            contracts_in_tx = get_contracts_from_tx(current_tx_hash, selected_network)

            if not contracts_in_tx:
                console.print(f"[warning]No contracts with bytecode found in transaction {shorten_address(current_tx_hash)} on {selected_network}, or an error occurred during lookup.", style="warning")
                return # Exit if no contracts to analyze

            # Prepare lists for analysis and comparison
            contract_codes_list = []
            contract_names_list = []
            analyzed_contract_count = 0

            console.print(f"\n\n[bold cyan3]Analyzing {len(contracts_in_tx)} Contract(s) Found in Transaction[/bold cyan3]")
            console.print("───────────────────────────────────────────", style="cyan3")
            # Analyze each contract found
            for address, name in contracts_in_tx.items():
                code = get_contract_code(address, network=selected_network)
                if code:
                    analysis = analyze_bytecode(code, name) # Name includes address
                    if analysis:
                        analysis_results[analysis["contract_name"]] = analysis
                        contract_codes_list.append(code)
                        contract_names_list.append(analysis["contract_name"])
                        analyzed_contract_count += 1

            # Perform comparison if multiple contracts were successfully analyzed
            if analyzed_contract_count > 1:
                 multi_comparison_data = compare_multiple_contracts(contract_codes_list, contract_names_list)
            elif analyzed_contract_count == 1:
                 console.print("[info]Only one contract was successfully analyzed from the transaction. Skipping comparison.", style="info")
            else:
                 console.print("[warning]Could not successfully analyze bytecode for any contracts found in the transaction.", style="warning")

            # Display export options if results were generated
            if analysis_results:
                show_export_options(tx_output)

    analyze_button.on_click(analyze_tx_action)

    # Layout for the tab
    tx_tab.children = [
        widgets.HTML("<h3>Analyze Contracts from Transaction</h3>"),
        widgets.HTML("<p>Extracts contract addresses involved in a transaction (target, event emitters, created contracts), fetches their bytecode, analyzes them individually, and performs a multi-contract comparison if applicable.</p>"),
        widgets.HBox([tx_input, network_selector, analyze_button]),
        tx_output
    ]

    console.print("\n\n[bold]📄 Advanced Bytecode Analysis via eth_getCode[/bold]", style="cyan3")
    console.print("──────────────────────────────────────────────", style="cyan3")

    # Automatically analyze TARGET_TX_HASH if defined globally and valid
    if 'TARGET_TX_HASH' in globals() and TARGET_TX_HASH and re.match(r'^0x[0-9a-fA-F]{64}$', TARGET_TX_HASH):
        console.print(f"\n\n[info]➡️Auto-analyzing Target transaction hash: {TARGET_TX_HASH}", style="info")
        analyze_tx_action(None, TARGET_TX_HASH)

    return tx_tab


def analyze_custom_tab():
    """Creates the UI and logic for the Custom Contract analysis tab."""
    custom_tab = widgets.VBox()

    # Example stablecoin addresses for comparison
    example_addresses = [
        "0x6c3ea9036406852006290770BEdFcAbA0e23A0e8",  # PYUSD Proxy
        "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48",  # USDC Proxy
        "0xdac17f958d2ee523a2206206994597c13d831ec7"   # USDT
    ]
    example_names = ["PYUSD", "USDC", "USDT"]

    # Input fields for addresses and names
    address_inputs = [widgets.Text(placeholder=f'e.g., {example_addresses[i]}',
                                  description=f'Addr {i+1}:',
                                  layout=widgets.Layout(width='60%')) for i in range(3)]
    name_inputs = [widgets.Text(description=f'Name {i+1}:',
                               layout=widgets.Layout(width='30%')) for i in range(3)]

    # Set default names
    for i in range(3):
        name_inputs[i].value = f"Contract {chr(65+i)}"

    # Network selector and buttons
    network_selector = widgets.Dropdown(options=['mainnet', 'sepolia', 'holesky'], value='mainnet', description='Network:', layout=widgets.Layout(width='auto'))
    analyze_button = widgets.Button(description='Analyze Custom Contracts', button_style='primary', icon='cogs', layout=widgets.Layout(width='auto'))
    example_button = widgets.Button(description='Load Stablecoin Examples', button_style='info', icon='info', layout=widgets.Layout(width='auto'))

    custom_output = widgets.Output()

    def load_examples(b):
        """Loads example stablecoin addresses into the input fields."""
        for i in range(3):
            address_inputs[i].value = example_addresses[i]
            name_inputs[i].value = example_names[i]

    example_button.on_click(load_examples)

    def analyze_custom_action(b):
        # Access and reset global result containers
        global analysis_results, comparison_data, multi_comparison_data
        analysis_results = {}
        comparison_data = None
        multi_comparison_data = None

        with custom_output:
            clear_output()
            console.print("\n\n[bold]⚙️ Analyzing Custom Contracts[/bold]", style="cyan3")
            console.print("──────────────────────────────────────────────", style="cyan3")

            # Collect valid addresses and corresponding names
            addresses_to_analyze = []
            names_to_analyze = []
            for i in range(len(address_inputs)):
                addr = address_inputs[i].value.strip()
                if addr: # Only process if address field is not empty
                    if re.match(r'^0x[0-9a-fA-F]{40}$', addr):
                        addresses_to_analyze.append(addr)
                        # Use provided name or default
                        names_to_analyze.append(name_inputs[i].value.strip() or f"Contract {chr(65+i)}")
                    else:
                        console.print(f"[warning]Skipping invalid address format entered for Contract {i+1}: '{addr}'", style="warning")

            if not addresses_to_analyze:
                console.print("[error]No valid contract addresses entered. Please enter at least one valid Ethereum address.", style="error")
                return

            selected_network = network_selector.value
            contract_codes_list = []
            contract_names_list = []
            analyzed_contract_count = 0

            # Analyze each valid contract
            for i, address in enumerate(addresses_to_analyze):
                contract_display_name = f"{names_to_analyze[i]} ({shorten_address(address)})"
                code = get_contract_code(address, network=selected_network)
                if code:
                    analysis = analyze_bytecode(code, contract_display_name)
                    if analysis:
                        analysis_results[analysis["contract_name"]] = analysis
                        contract_codes_list.append(code)
                        contract_names_list.append(analysis["contract_name"])
                        analyzed_contract_count += 1

            # Perform comparisons based on the number of successfully analyzed contracts
            if analyzed_contract_count == 2:
                 # Assume the two might be a proxy/implementation pair
                 comparison_data = compare_proxy_implementation(
                     contract_codes_list[0], contract_codes_list[1],
                     contract_names_list[0], contract_names_list[1]
                 )
            elif analyzed_contract_count > 2:
                 # Perform multi-contract comparison
                 multi_comparison_data = compare_multiple_contracts(contract_codes_list, contract_names_list)
            elif analyzed_contract_count == 1:
                 console.print("[info]Only one contract was analyzed successfully. Skipping comparison.", style="info")
            else: # analyzed_contract_count == 0
                 console.print("[warning]Could not successfully analyze bytecode for any of the provided contracts.", style="warning")

            # Show export options if any results were generated
            if analysis_results:
                show_export_options(custom_output)

    analyze_button.on_click(analyze_custom_action)

    # Warning about addresses
    address_note = widgets.HTML("""
    <div style="margin-top: 15px; padding: 10px; border: 1px solid #ddd; background-color: #00005f; font-size: 0.9em; color:white;">
        <strong>Important:</strong> Enter only smart contract addresses, not user wallet addresses (EOAs).
        User wallet addresses will show "No bytecode found" error. Try the example stablecoin addresses for a demonstration.
    </div>
    """)

    # Layout for the tab using HBox for alignment
    input_rows = [widgets.HBox([address_inputs[i], name_inputs[i]]) for i in range(3)]
    custom_tab.children = [
        widgets.HTML("<h3>Analyze Custom Contracts by Address</h3>"),
        widgets.HTML("<p>Enter up to three contract addresses and optional custom names. The tool will fetch bytecode, analyze each contract, and perform comparisons (Proxy vs. Impl for two, Multi-Contract for three).</p>"),
        address_note,
        widgets.HBox([example_button], layout=widgets.Layout(margin='10px 0')),
        *input_rows, # Unpack the list of HBox widgets
        widgets.HBox([network_selector, analyze_button]),
        custom_output
    ]

    return custom_tab


# --- Main Execution: Assemble and Display Tabs ---

# Create the tab instances
tab1 = analyze_pyusd_tab()
tab2 = analyze_tx_tab()
tab3 = analyze_custom_tab()

# Assign tabs to the Tab widget
analysis_tabs.children = [tab1, tab2, tab3]

# Set descriptive titles for each tab
analysis_tabs.set_title(0, 'PYUSD Analysis')
analysis_tabs.set_title(1, 'From Transaction')
analysis_tabs.set_title(2, 'Custom Contracts')

# Display the main title and the tab interface
console.print("\n\nSelect an analysis mode below to fetch and examine contract bytecode:\n\n")
display(analysis_tabs)

## 1.5 🧱 `trace_block`: Analyzing All Transactions in a Block
---

This section utilizes the `trace_block` RPC method to retrieve summary traces for **all transactions** executed within a single target block (`TARGET_BLOCK_IDENTIFIER`). This provides a high-level overview of the block's activity, allowing us to understand the context surrounding specific PYUSD transactions and identify broader patterns.

> **🚀 Leveraging GCP's Premium RPC Capabilities**
>
> *   **Method:** `trace_block`
> *   **Multiplier:** `50x` (Available on Mainnet via GCP)
> *   **GCP Advantage:** Tracing an entire block is computationally demanding. GCP provides this capability on Mainnet, often restricted or unavailable elsewhere, allowing for comprehensive block-level analysis without needing to trace each transaction individually first.
> *   **PYUSD Insight:** `trace_block` helps us:
>     *   Quickly identify **all transactions interacting with PYUSD** within a specific block.
>     *   Analyze the **gas consumption patterns** across different transaction types (PYUSD transfers, DeFi interactions, contract creations) within the block.
>     *   Understand the immediate **on-chain environment** surrounding significant PYUSD events.
>     *   Detect **potential indicators** of MEV activity or unusual transaction sequences involving PYUSD by examining the full block context (transaction order, gas usage, specific calls).

**Analysis Workflow:**

1.  **Fetch Block Trace:** Calls `trace_block` using the target block identifier.
2.  **Process Traces:** Iterates through the list of transaction traces returned for the block.
3.  **Categorize & Analyze (within `analyze_block_trace`):**
    *   Each transaction trace is categorized (e.g., ETH Transfer, Contract Call, PYUSD Transfer, PYUSD Approval) using `categorize_transaction`.
    *   PYUSD-specific details (function calls, parameters, events) are extracted using `extract_pyusd_details`.
    *   Gas usage is aggregated by transaction type (`analyze_gas_distribution`).
    *   PYUSD token flow within the block is analyzed (`analyze_pyusd_flow`).
4.  **Visualize & Summarize:**
    *   **Block Summary Panel:** Displays key statistics (total transactions, gas, errors, PYUSD interactions) using `rich`.
    *   **Gas Analysis:** Shows tables (`rich`) and plots (`plotly`) of gas usage distribution by transaction type.
    *   **PYUSD Activity:** Tables (`rich`) detailing PYUSD interactions, function calls, and token flow graphs (`graphviz`).
    *   **Transaction Table:** An interactive table (`pandas`, `ipywidgets`) summarizing all transactions in the block, with options to filter for PYUSD interactions.
    *   **Export Options:** Download block transaction summaries.

**💡 What to Look For:**
*   **PYUSD Transaction Density:** How many transactions in the block interact with PYUSD?
*   **Gas Usage Patterns:** Which types of transactions consume the most gas in this block? Are PYUSD interactions relatively efficient or expensive compared to others?
*   **PYUSD Flow:** Observe the primary PYUSD movements within the block using the flow diagram.
*   **Context:** Examine the transactions immediately before and after significant PYUSD events **using the full transaction table**.

In [ ]:
# =============================================================================================
# 🧱 Block Tracing and Analysis using trace_block (Google Cost-Efficient API)
# =============================================================================================
# This cell utilizes the trace_block RPC method to analyze all transactions within a specific block.
# Functionality includes:
# - Fetching summary traces for every transaction in the target block via trace_block.
# - Categorizing each transaction (e.g., ETH Transfer, Contract Call, PYUSD Transfer, PYUSD Approval).
# - Extracting detailed information for PYUSD-related transactions (function calls, parameters, amounts, events).
# - Analyzing and visualizing gas consumption patterns across different transaction types within the block.
# - Analyzing and visualizing the flow of PYUSD tokens (transfers, approvals) within the block context using Graphviz.
# - Displaying comprehensive summary statistics and detailed tables using rich console formatting.
# - Providing an interactive table of all block transactions with filtering for PYUSD interactions.
# - Offering direct export options for the analysis results to CSV, JSON, and Google Sheets.
# - Leveraging GCP's premium RPC capabilities for the compute-intensive trace_block method on Mainnet.

def categorize_transaction(trace_item):
    """Categorizes a transaction trace based on its characteristics."""
    action = trace_item.get('action', {})
    to_addr = action.get('to', action.get('address', ''))
    input_data = action.get('input', '0x')

    # Initialize category data
    category = {
        'type': 'other',
        'subtype': 'unknown',
        'description': 'Other transaction',
        'is_pyusd': False,
        'function_name': None,
        'function_category': None
    }

    # Check contract creation
    if action.get('init') and not action.get('to'):
        category.update({
            'type': 'contract_creation',
            'subtype': 'deployment',
            'description': 'Contract Deployment'
        })
        return category

    # Check if address is any of the PYUSD contracts
    is_pyusd_contract = False
    if to_addr:
        to_addr_lower = to_addr.lower()
        for contract_addr in PYUSD_CONTRACTS:
            if to_addr_lower == contract_addr:
                is_pyusd_contract = True
                category.update({
                    'type': 'token',
                    'subtype': 'pyusd',
                    'description': f'PYUSD: {PYUSD_CONTRACTS[contract_addr]}',
                    'is_pyusd': True,
                    'contract_name': PYUSD_CONTRACTS[contract_addr]
                })
                break

    # If it's a PYUSD contract, try to decode function
    if is_pyusd_contract and input_data and len(input_data) >= 10:
        function_name = decode_pyusd_function(input_data)
        function_category = get_function_category(input_data)
        category.update({
            'function_name': function_name,
            'function_category': function_category,
            'description': f'PYUSD: {function_name}'
        })

        # Further refine subtype based on function category
        if function_category == 'token_movement':
            category['subtype'] = 'transfer'
        elif function_category == 'allowance':
            category['subtype'] = 'approval'
        elif function_category == 'supply_change':
            category['subtype'] = 'supply_change'
        elif function_category == 'control':
            category['subtype'] = 'control'

    # Check for ETH transfers (no input data)
    elif input_data == '0x' or len(input_data) <= 2:
        value = int(action.get('value', '0x0'), 16)
        if value > 0:
            category.update({
                'type': 'eth_transfer',
                'subtype': 'transfer',
                'description': 'ETH Transfer'
            })

    # Other contract interactions
    elif to_addr and len(input_data) >= 10:
        # This is a contract interaction, but not with PYUSD
        category.update({
            'type': 'contract_interaction',
            'subtype': 'call',
            'description': 'Contract Interaction',
            'function_selector': input_data[:10]
        })

    return category

def extract_pyusd_details(trace_item):
    """Extracts PYUSD-specific details from a trace item if present."""
    action = trace_item.get('action', {})
    result = trace_item.get('result', {})
    input_data = action.get('input', '0x')
    to_addr = action.get('to', '')

    details = {
        'is_pyusd_tx': False,
        'function': None,
        'function_category': None,
        'params': {},
        'events': [],
        'amount': None,
        'from_addr': None,
        'to_addr': None,
        'decoded': {}
    }

    # Check if this is a PYUSD contract
    is_pyusd_contract = False
    if to_addr:
        to_addr_lower = to_addr.lower()
        for contract_addr in PYUSD_CONTRACTS:
            if to_addr_lower == contract_addr:
                is_pyusd_contract = True
                details['contract_name'] = PYUSD_CONTRACTS[contract_addr]
                break

    if not is_pyusd_contract:
        return details

    # It's a PYUSD transaction
    details['is_pyusd_tx'] = True

    # Try to decode function call
    if len(input_data) >= 10:
        method_sig = input_data[:10]
        details['function'] = decode_pyusd_function(input_data)
        details['function_category'] = get_function_category(input_data)

        # Attempt basic parameter decoding for common functions
        if method_sig == '0xa9059cbb':  # transfer(address,uint256)
            try:
                to_address_hex = '0x' + input_data[10:74].lstrip('0')
                amount_hex = '0x' + input_data[74:138].lstrip('0')
                details['params']['to'] = Web3.to_checksum_address(to_address_hex)
                amount_raw = int(amount_hex, 16)
                details['amount'] = amount_raw / (10**PYUSD_CONFIG['ethereum']['decimals'])
                details['to_addr'] = details['params']['to']
                details['from_addr'] = action.get('from')

                details['decoded'] = {
                    'operation': 'transfer',
                    'from': action.get('from'),
                    'to': details['params']['to'],
                    'amount': f"{details['amount']} PYUSD"
                }
            except Exception as e:
                details['decode_error'] = str(e)

        elif method_sig == '0x095ea7b3':  # approve(address,uint256)
            try:
                spender_hex = '0x' + input_data[10:74].lstrip('0')
                amount_hex = '0x' + input_data[74:138].lstrip('0')
                details['params']['spender'] = Web3.to_checksum_address(spender_hex)
                amount_raw = int(amount_hex, 16)
                details['amount'] = amount_raw / (10**PYUSD_CONFIG['ethereum']['decimals'])

                details['decoded'] = {
                    'operation': 'approve',
                    'owner': action.get('from'),
                    'spender': details['params']['spender'],
                    'amount': f"{details['amount']} PYUSD"
                }
            except Exception as e:
                details['decode_error'] = str(e)

    # Extract any logs (events) from the result
    if 'logs' in result and isinstance(result['logs'], list):
        for log in result['logs']:
            if log.get('address', '').lower() == PYUSD_ADDRESS_LOWER_ETH and log.get('topics'):
                topic0 = log['topics'][0]

                if topic0 in PYUSD_EVENTS:
                    event_data = log.get('data', '0x')
                    decoded_event = decode_pyusd_event(topic0, log['topics'], event_data)
                    details['events'].append(decoded_event)

                    # For Transfer events, extract additional information
                    if topic0 == TRANSFER_EVENT_TOPIC and 'decoded' in decoded_event:
                        event_from = decoded_event['decoded'].get('from')
                        event_to = decoded_event['decoded'].get('to')
                        event_value = decoded_event['decoded'].get('value')

                        if not details['amount'] and event_value is not None:
                            details['amount'] = event_value / (10**PYUSD_CONFIG['ethereum']['decimals'])
                        if not details['from_addr'] and event_from:
                            details['from_addr'] = event_from
                        if not details['to_addr'] and event_to:
                            details['to_addr'] = event_to

    return details

def analyze_gas_distribution(traces_with_categories):
    """Analyzes gas distribution across different transaction types."""
    gas_by_category = {}
    gas_by_function = {}
    pyusd_vs_other = {
        'pyusd': {'count': 0, 'gas': 0},
        'other': {'count': 0, 'gas': 0}
    }

    for trace in traces_with_categories:
        category = trace.get('category', {})
        gas_used = trace.get('gas_used', 0)

        # By main category
        cat_type = category.get('type', 'other')
        if cat_type not in gas_by_category:
            gas_by_category[cat_type] = {'count': 0, 'gas': 0}
        gas_by_category[cat_type]['count'] += 1
        gas_by_category[cat_type]['gas'] += gas_used

        # PYUSD vs other
        if category.get('is_pyusd', False):
            pyusd_vs_other['pyusd']['count'] += 1
            pyusd_vs_other['pyusd']['gas'] += gas_used

            # By PYUSD function
            function_name = category.get('function_name', 'unknown')
            if function_name not in gas_by_function:
                gas_by_function[function_name] = {'count': 0, 'gas': 0}
            gas_by_function[function_name]['count'] += 1
            gas_by_function[function_name]['gas'] += gas_used
        else:
            pyusd_vs_other['other']['count'] += 1
            pyusd_vs_other['other']['gas'] += gas_used

    return {
        'by_category': gas_by_category,
        'by_function': gas_by_function,
        'pyusd_vs_other': pyusd_vs_other
    }

def visualize_gas_distribution(gas_analysis, title="Gas Usage Distribution in Block"):
    """Creates visualization for gas distribution across transaction types."""
    # Prepare data for visualization
    categories = []
    gas_values = []
    counts = []

    for category, data in gas_analysis['by_category'].items():
        if data['gas'] > 0:  # Only include categories with gas usage
            categories.append(category)
            gas_values.append(data['gas'])
            counts.append(data['count'])

    # Create figure with subplots in a container div
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=("", ""),  # Empty subplot titles, we'll add custom ones
        specs=[[{"type": "pie"}, {"type": "pie"}]],
        horizontal_spacing=0.05  # Reduce spacing between charts
    )

    # Add gas usage pie chart
    fig.add_trace(
        go.Pie(
            labels=categories,
            values=gas_values,
            textinfo='percent+label',
            hole=.3,
            marker_colors=px.colors.qualitative.Pastel
        ),
        row=1, col=1
    )

    # Add transaction count pie chart
    fig.add_trace(
        go.Pie(
            labels=categories,
            values=counts,
            textinfo='percent+label',
            hole=.3,
            marker_colors=px.colors.qualitative.Pastel
        ),
        row=1, col=2
    )

    # Create a container-like layout with proper spacing
    fig.update_layout(
        # Main title with padding
        title={
            'text': f"<b>{title}</b>",  # Bold with HTML
            'y': 0.98,                  # Position higher
            'x': 0.5,                   # Center align
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {
                'size': 24,             # Larger font
                'family': 'Arial, sans-serif'
            },
            'pad': {'b': 30}            # Add bottom padding to title
        },

        # Chart container with padding
        margin=dict(t=120, b=120, l=40, r=40),  # Extra top and bottom margin
        height=650,  # Increased height for better spacing

        # Add chart subtitles directly under each chart
        annotations=[
            dict(
                text="<b>Gas Usage by Transaction Type</b>",
                x=0.1,
                y=-0.10,        # Position directly under first chart
                xref="paper",
                yref="paper",
                showarrow=False,
                font=dict(size=14)
            ),
            dict(
                text="<b>Transaction Count by Type</b>",
                x=0.9,
                y=-0.10,        # Position directly under second chart
                xref="paper",
                yref="paper",
                showarrow=False,
                font=dict(size=14)
            )
        ]
    )

    return fig

def analyze_pyusd_flow(traces_with_details, block_number):
    """Analyzes PYUSD token flow within the block."""
    # Initialize flow data
    flow_data = {
        'total_transferred': 0,
        'unique_senders': set(),
        'unique_receivers': set(),
        'transfers': [],
        'approvals': [],
        'other_operations': []
    }

    for trace in traces_with_details:
        # Get PYUSD details safely
        pyusd_details = trace.get('pyusd_details')

        # Skip if no PYUSD details or not a PYUSD transaction
        if not pyusd_details or not pyusd_details.get('is_pyusd_tx'):
            continue

        function = pyusd_details.get('function', '')

        if function and 'transfer' in function.lower() and pyusd_details.get('amount') and pyusd_details.get('from_addr') and pyusd_details.get('to_addr'):
            # This is a transfer
            flow_data['total_transferred'] += pyusd_details['amount']
            flow_data['unique_senders'].add(pyusd_details['from_addr'])
            flow_data['unique_receivers'].add(pyusd_details['to_addr'])

            flow_data['transfers'].append({
                'tx_hash': trace.get('tx_hash'),
                'from': pyusd_details['from_addr'],
                'to': pyusd_details['to_addr'],
                'amount': pyusd_details['amount'],
                'failed': trace.get('failed', False)
            })

        elif function and 'approve' in function.lower() and pyusd_details.get('amount'):
            # This is an approval
            flow_data['approvals'].append({
                'tx_hash': trace.get('tx_hash'),
                'owner': pyusd_details.get('from_addr') or trace.get('from'),
                'spender': pyusd_details.get('params', {}).get('spender'),
                'amount': pyusd_details['amount'],
                'failed': trace.get('failed', False)
            })

        else:
            # Other PYUSD operations
            flow_data['other_operations'].append({
                'tx_hash': trace.get('tx_hash'),
                'function': function,
                'from': trace.get('from'),
                'failed': trace.get('failed', False)
            })

    # Calculate statistics
    flow_data['unique_senders_count'] = len(flow_data['unique_senders'])
    flow_data['unique_receivers_count'] = len(flow_data['unique_receivers'])
    flow_data['transfer_count'] = len(flow_data['transfers'])
    flow_data['approval_count'] = len(flow_data['approvals'])
    flow_data['other_count'] = len(flow_data['other_operations'])

    return flow_data

def create_pyusd_flow_diagram(flow_data, max_flows=10):
    """Creates a visualization of PYUSD token flows."""
    if not flow_data['transfers']:
        return None

    # Sort transfers by amount
    sorted_transfers = sorted(flow_data['transfers'], key=lambda x: x['amount'], reverse=True)

    # Take top flows based on amount
    top_flows = sorted_transfers[:max_flows]

    # Create flow diagram
    dot = Digraph(comment='PYUSD Token Flow', format='png')
    dot.attr(rankdir='LR', overlap='false')
    dot.attr('node', shape='box', style='filled', fontname='helvetica', fontsize='10')

    # Add nodes and edges
    added_nodes = set()

    for transfer in top_flows:
        from_addr = transfer['from']
        from_addr_short = from_addr[:6] + '...' + from_addr[-4:]
        to_addr = transfer['to']
        to_addr_short = to_addr[:6] + '...' + to_addr[-4:]
        amount = f"{transfer['amount']:.2f} PYUSD"

        # Add sender node if not already added
        node_id_from = from_addr
        if node_id_from not in added_nodes:
            dot.node(node_id_from, from_addr_short, fillcolor='lightblue', tooltip=from_addr)
            added_nodes.add(node_id_from)

        # Add receiver node if not already added
        node_id_to = to_addr
        if node_id_to not in added_nodes:
            dot.node(node_id_to, to_addr_short, fillcolor='palegreen', tooltip=to_addr)
            added_nodes.add(node_id_to)

        # Add edge for the transfer
        edge_style = 'dashed' if transfer.get('failed') else 'solid'
        dot.edge(node_id_from, node_id_to, label=amount, style=edge_style)

    return dot

# Dedicated Export Functions
def download_csv_direct(df, filename=None):
    """Creates a direct download for CSV without intermediate display."""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"pyusd_data_{timestamp}.csv"

    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()

    # Create direct download HTML
    html = f'''
    <script>
    function download(filename, data) {{
        const a = document.createElement('a');
        a.href = data;
        a.download = filename;
        document.body.appendChild(a);
        a.click();
        document.body.removeChild(a);
    }}

    download("{filename}", "data:text/csv;base64,{payload}");
    </script>
    <div>Downloading {filename}...</div>
    '''
    return HTML(html)

def download_json_direct(data, filename=None):
    """Creates a direct download for JSON without intermediate display."""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"pyusd_data_{timestamp}.json"

    # Convert to JSON string (handling non-serializable objects)
    json_str = json.dumps(data, default=str, indent=2)
    b64 = base64.b64encode(json_str.encode()).decode()

    # Create direct download HTML
    html = f'''
    <script>
    function download(filename, data) {{
        const a = document.createElement('a');
        a.href = data;
        a.download = filename;
        document.body.appendChild(a);
        a.click();
        document.body.removeChild(a);
    }}

    download("{filename}", "data:application/json;base64,{b64}");
    </script>
    <div>Downloading {filename}...</div>
    '''
    return HTML(html)

def export_to_google_sheets_direct(df, sheet_name=None):
    """Exports DataFrame directly to Google Sheets using authenticated session."""
    if sheet_name is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        sheet_name = f"PYUSD Analysis {timestamp}"

    # Use Google Colab integration for direct export
    html = f'''
    <script src="https://apis.google.com/js/platform.js" async defer></script>
    <script>
    function createSheet() {{
        const csv = `{df.to_csv(index=False).replace('"', '""')}`;

        // Create sheet using Google Colab API
        google.colab.kernel.invokeFunction('notebook.createSheet', [csv, '{sheet_name}'], {{}});
    }}

    // Execute immediately
    setTimeout(createSheet, 100);
    </script>
    <div>Creating Google Sheet "{sheet_name}"...</div>
    '''

    # Register the Python callback that the JavaScript will call
    from google.colab import output

    @output.register_callback('notebook.createSheet')
    def create_sheet_callback(csv_data, name):
        try:
            from google.colab import auth
            from googleapiclient.discovery import build
            from googleapiclient.http import MediaInMemoryUpload
            import io

            # Ensure authentication
            auth.authenticate_user()

            # Create Drive API client
            drive_service = build('drive', 'v3')

            # File metadata
            file_metadata = {
                'name': name,
                'mimeType': 'application/vnd.google-apps.spreadsheet'
            }

            # Create CSV upload
            media = MediaInMemoryUpload(
                io.BytesIO(csv_data.encode('utf-8')),
                mimetype='text/csv',
                resumable=True
            )

            # Create the Sheet
            file = drive_service.files().create(
                body=file_metadata,
                media_body=media,
                fields='id,webViewLink'
            ).execute()

            # Return success with link
            return {
                'status': 'success',
                'file_id': file.get('id'),
                'link': file.get('webViewLink')
            }
        except Exception as e:
            return {
                'status': 'error',
                'message': str(e)
            }

    return HTML(html)

def analyze_block_trace(block_trace_results_list, block_identifier):
    """Analyzes the output list from trace_block with PYUSD-focused analysis."""
    if not isinstance(block_trace_results_list, list): # trace_block should return a list
        console.print(f"[error]Expected a list from trace_block for block {block_identifier}, got {type(block_trace_results_list)}.", style="error")
        display_json(block_trace_results_list, "Unexpected Result Structure")
        return None
    if not block_trace_results_list:
        console.print(f"[warning]No block trace results (empty list) provided for block {block_identifier}.", style="warning")
        return None

    console.print(f"\n\n[bold cyan3]🧱 Analysis of {len(block_trace_results_list)} Traces for Block {block_identifier}[/bold cyan3]")
    console.print("[info]Extracting and categorizing transaction data...", style="info")

    # Initialize counters and trackers
    pyusd_interactions_count = 0
    total_gas_used_in_traces = 0
    failed_traces_count = 0
    pyusd_transfer_volume = 0
    traces_with_categories = []  # Store traces with their categories

    # Process each trace
    for i, trace in enumerate(block_trace_results_list):
        # Default values
        tx_hash = 'N/A'
        from_addr = 'N/A'
        to_addr = 'N/A'
        value_eth = 0
        value_eth_str = '0 ETH'
        input_data = '0x'
        gas_used = 0
        error = None
        tx_pos = i # Fallback index

        try:
            # Extract basic trace info
            action = trace.get('action', {})
            result = trace.get('result', {})
            tx_hash = trace.get('transactionHash', action.get('transactionHash', f'trace_{i}'))
            tx_pos = trace.get('transactionPosition', tx_pos)

            from_addr = action.get('from', 'N/A')
            # Handle contract creation ('address') vs. call ('to')
            to_addr = action.get('to', action.get('address', 'N/A'))
            value_wei_hex = action.get('value', '0x0')
            input_data = action.get('input', '0x')

            # Check both top-level and result for gasUsed and error
            gas_used_hex = result.get('gasUsed', trace.get('gasUsed', '0x0'))
            error = trace.get('error', result.get('error', None))

            gas_used = int(gas_used_hex, 16) if gas_used_hex and isinstance(gas_used_hex, str) else gas_used_hex or 0

            # Convert value from Wei to ETH
            if isinstance(value_wei_hex, str) and value_wei_hex.startswith('0x'):
                value_eth = int(value_wei_hex, 16) / 1e18
            else:
                value_eth = (value_wei_hex or 0) / 1e18

            value_eth_str = format_value_eth(value_wei_hex)

            # Categorize the transaction
            category = categorize_transaction(trace)

            # Extract PYUSD-specific details if relevant
            pyusd_details = extract_pyusd_details(trace) if category.get('is_pyusd') else None

            # Update PYUSD transfer volume if this is a PYUSD transfer
            if pyusd_details and pyusd_details.get('amount') and 'transfer' in (pyusd_details.get('function') or '').lower():
                pyusd_transfer_volume += pyusd_details['amount']

        except Exception as parse_err:
            console.print(f"[warning]Could not fully parse trace item {i} in block {block_identifier}: {parse_err}", style="warning")
            error = f"Parse Error: {parse_err}" # Mark as failed due to parsing
            category = {'type': 'unknown', 'description': 'Parse Error', 'is_pyusd': False}
            pyusd_details = None

        # Update counters
        if category.get('is_pyusd'):
            pyusd_interactions_count += 1
        if error:
            failed_traces_count += 1
        total_gas_used_in_traces += gas_used

        # Store trace with category and details
        trace_summary = {
            'tx_index': tx_pos,
            'tx_hash': tx_hash,
            'from': from_addr,
            'to': to_addr,
            'value_eth': value_eth,
            'value_eth_str': value_eth_str,
            'gas_used': gas_used,
            'failed': error is not None,
            'error': error,
            'category': category,
            'pyusd_details': pyusd_details,
            'input_preview': input_data[:10] + ('...' if len(input_data)>10 else ''),
        }

        traces_with_categories.append(trace_summary)

    # Sort traces by transaction index
    traces_with_categories.sort(key=lambda x: x['tx_index'] if isinstance(x['tx_index'], int) else float('inf'))

    # Create DataFrame for display - include full addresses for proper data analysis
    # Use only one index (not creating an extra 'index' column)
    summary_df = pd.DataFrame([{
        'tx_index': t['tx_index'],
        'tx_hash': t['tx_hash'],
        'from': t['from'],  # Full address
        'to': t['to'],      # Full address
        'value': t['value_eth_str'],
        'gas_used': t['gas_used'],
        'type': t['category']['description'],
        'failed': t['failed'],
        'pyusd': t['category'].get('is_pyusd', False)
    } for t in traces_with_categories])

    # Display Block Summary Panel
    block_num = block_identifier
    if isinstance(block_identifier, str) and block_identifier.startswith('0x'):
        try:
            block_num = int(block_identifier, 16)
        except:
            block_num = block_identifier

    # Add more spacing before summary panel
    console.print("\n")
    console.print(Panel(f"""
[bold]Block Trace Analysis (Block {block_num})[/bold]
Transactions Traced: {len(block_trace_results_list)}
Total Gas Used (Sum): {total_gas_used_in_traces:,}
Traces with Errors: {failed_traces_count}
PYUSD Interactions: {pyusd_interactions_count}
PYUSD Transfer Volume: {pyusd_transfer_volume:.2f} PYUSD""",
        title="🧱 trace_block Analysis", border_style="cyan3", expand=False))

    # Analyze gas distribution
    console.print("\n\n[bold chartreuse1]ANALYZING GAS USAGE PATTERNS ACROSS TRANSACTION TYPES[/bold chartreuse1]")
    console.print("─────────────────────────────────────────────────────", style="chartreuse1")
    gas_analysis = analyze_gas_distribution(traces_with_categories)

    # Add spacing before gas analysis
    console.print("\n\n[bold chartreuse1]📊 Gas Usage Analysis by Transaction Type[/bold chartreuse1]")
    console.print("──────────────────────────────────────────", style="chartreuse1")

    # Create and display gas usage tables
    gas_table = Table(title="", title_style="bold chartreuse1")
    gas_table.add_column("Transaction Type", style="chartreuse1")
    gas_table.add_column("Count", justify="right")
    gas_table.add_column("Total Gas", justify="right")
    gas_table.add_column("Avg Gas/Tx", justify="right")
    gas_table.add_column("% of Total", justify="right")

    for category, data in sorted(gas_analysis['by_category'].items(), key=lambda x: x[1]['gas'], reverse=True):
        if data['gas'] > 0:  # Only include categories with gas usage
            avg_gas = data['gas'] // data['count'] if data['count'] > 0 else 0
            pct_total = (data['gas'] / total_gas_used_in_traces * 100) if total_gas_used_in_traces > 0 else 0
            gas_table.add_row(
                category.replace('_', ' ').title(),
                f"{data['count']:,}",
                f"{data['gas']:,}",
                f"{avg_gas:,}",
                f"{pct_total:.1f}%"
            )

    console.print(gas_table)

    # PYUSD specific analysis - only if there are PYUSD interactions
    if pyusd_interactions_count > 0:
        console.print("\n\n[bold chartreuse1]🪙 PYUSD Transactions Analysis in Block[/bold chartreuse1]")
        console.print("───────────────────────────────────────", style="chartreuse1")

        # Create PYUSD-specific table
        pyusd_table = Table(title="", title_style="bold chartreuse1")
        pyusd_table.add_column("Tx Hash", style="dim")
        pyusd_table.add_column("Function")
        pyusd_table.add_column("From")
        pyusd_table.add_column("To")
        pyusd_table.add_column("Amount")
        pyusd_table.add_column("Gas Used")
        pyusd_table.add_column("Status")

        # Get all PYUSD transactions
        pyusd_txs = [t for t in traces_with_categories if t['category'].get('is_pyusd')]

        # Sort by gas used (descending)
        pyusd_txs.sort(key=lambda x: x['gas_used'], reverse=True)

        # Add rows for PYUSD transactions (limit to first 15 to avoid huge output)
        for tx in pyusd_txs[:15]:
            details = tx['pyusd_details'] or {}
            function = tx['category'].get('function_name', 'Unknown')

            # Format from/to addresses - showing full addresses on hover
            from_full = details.get('from_addr') or tx['from']
            from_addr = f"{from_full[:6]}...{from_full[-4:]}" if from_full and len(from_full) > 10 else from_full

            to_full = None
            if details.get('to_addr'):
                to_full = details['to_addr']
            elif 'approve' in function.lower() and details.get('params', {}).get('spender'):
                to_full = details['params']['spender']

            to_addr = f"{to_full[:6]}...{to_full[-4:]}" if to_full and len(to_full) > 10 else "N/A"

            # Format amount
            amount = "N/A"
            if details.get('amount') is not None:
                amount = f"{details['amount']:.2f} PYUSD"

            # Format status
            status_color = "red" if tx['failed'] else "green"
            status_text = "Failed" if tx['failed'] else "Success"

            # Format transaction hash
            tx_hash = tx['tx_hash']
            tx_hash_display = f"{tx_hash[:10]}..." if len(tx_hash) > 12 else tx_hash

            # Add row with full data in title attributes
            pyusd_table.add_row(
                tx_hash_display,
                function,
                from_addr,
                to_addr,
                amount,
                f"{tx['gas_used']:,}",
                f"[{status_color}]{status_text}[/{status_color}]"
            )

        if len(pyusd_txs) > 15:
            pyusd_table.add_row(
                "...",
                f"+ {len(pyusd_txs) - 15} more",
                "",
                "",
                "",
                "",
                ""
            )

        console.print(pyusd_table)

        # Analyze PYUSD token flow
        console.print("\n\n[bold yellow3]ANALYZING PYUSD TOKEN FLOW WITHIN THE BLOCK[/bold yellow3]")
        console.print("───────────────────────────────────────────", style="yellow3")
        flow_analysis = analyze_pyusd_flow(traces_with_categories, block_num)

        # Display flow summary
        flow_panel = Panel(f"""
[bold]PYUSD Token Flow in Block {block_num}[/bold]
Total PYUSD Transferred: {flow_analysis['total_transferred']:.2f} PYUSD
Transfer Transactions: {flow_analysis['transfer_count']}
Approval Transactions: {flow_analysis['approval_count']}
Other PYUSD Transactions: {flow_analysis['other_count']}
Unique Senders: {flow_analysis['unique_senders_count']}
Unique Receivers: {flow_analysis['unique_receivers_count']}""",
            title="🔄 PYUSD Flow Analysis", border_style="yellow3", expand=False)

        console.print(flow_panel)

        # Create and display flow diagram if there are transfers
        if flow_analysis['transfers']:
            console.print("\n\n[bold magenta3]PYUSD Token Flow Fiagram (Top Transfers)[/bold magenta3]")
            console.print("─────────────────────────────────────────", style="magenta3")
            try:
                flow_diagram = create_pyusd_flow_diagram(flow_analysis)
                if flow_diagram:
                    display(flow_diagram)
            except Exception as viz_err:
                console.print(f"[warning]Could not create flow visualization: {viz_err}", style="warning")

    # Visualize gas distribution with better spacing
    console.print("\n\n[bold magenta3]Gas Usage Distribution in Block[/bold magenta3]")
    console.print("────────────────────────────────", style="magenta3")
    try:
        gas_viz = visualize_gas_distribution(gas_analysis)
        display(gas_viz)
    except Exception as viz_err:
        console.print(f"[warning]Could not create gas visualization: {viz_err}", style="warning")

    # Show filter options first
    console.print("\n\n[bold cyan3]📋 Block Transaction Summary[/bold cyan3]")
    console.print("─────────────────────────────", style="cyan3")

    # Create filter controls and PYUSD-focused display
    if not summary_df.empty:
        # Create PYUSD-only and All transactions dataframes
        pyusd_df = summary_df[summary_df['pyusd'] == True].copy()

        # Filter options above the table
        display(widgets.HTML("<h4>Filter Options:</h4>"))

        # Keep both buttons - arranged in an HBox for better layout
        filter_buttons = widgets.HBox([
            widgets.Button(
                description='Show PYUSD Only',
                button_style='success',
                layout=widgets.Layout(width='180px')
            ),
            widgets.Button(
                description='Show All Transactions',
                button_style='info',
                layout=widgets.Layout(width='200px')
            )
        ])

        display(filter_buttons)

        # Output area for transactions
        transaction_output = widgets.Output()

        # Export buttons below the table with better styling
        export_buttons = widgets.HBox([
            widgets.Button(
                description='Export to CSV',
                button_style='primary',  # Green
                layout=widgets.Layout(width='150px')
            ),
            widgets.Button(
                description='Export as JSON',
                button_style='warning',  # Orange
                layout=widgets.Layout(width='150px')
            ),
            widgets.Button(
                description='Export to Google Sheets',
                button_style='info',     # Blue
                layout=widgets.Layout(width='200px')
            )
        ])

        export_output = widgets.Output()

        def show_all_txs(b):
            with transaction_output:
                clear_output()
                display(summary_df)

        def show_pyusd_txs(b):
            with transaction_output:
                clear_output()
                display(pyusd_df)

        # Export handlers
        def export_csv(b):
            with export_output:
                clear_output()
                # Use the currently displayed DataFrame
                current_df = summary_df if transaction_output.outputs and len(transaction_output.outputs[0]['text/plain']) > len(str(pyusd_df)) else pyusd_df
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"block_{block_num}_transactions_{timestamp}.csv"
                display(download_csv_direct(current_df, filename))

        def export_json(b):
            with export_output:
                clear_output()
                # Use the currently displayed DataFrame
                current_df = summary_df if transaction_output.outputs and len(transaction_output.outputs[0]['text/plain']) > len(str(pyusd_df)) else pyusd_df
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"block_{block_num}_transactions_{timestamp}.json"
                records = current_df.to_dict('records')
                display(download_json_direct(records, filename))

        def export_to_sheets(b):
            with export_output:
                clear_output()
                # Use the currently displayed DataFrame
                current_df = summary_df if transaction_output.outputs and len(transaction_output.outputs[0]['text/plain']) > len(str(pyusd_df)) else pyusd_df
                sheet_name = f"Block {block_num} Transactions {datetime.now().strftime('%Y%m%d_%H%M%S')}"
                display(export_to_google_sheets_direct(current_df, sheet_name))

        # Connect callbacks
        filter_buttons.children[0].on_click(show_pyusd_txs)   # PYUSD Only button
        filter_buttons.children[1].on_click(show_all_txs)     # Show All button
        export_buttons.children[0].on_click(export_csv)
        export_buttons.children[1].on_click(export_json)
        export_buttons.children[2].on_click(export_to_sheets)

        # Display transaction table area
        display(transaction_output)

        # Show PYUSD transactions by default
        show_pyusd_txs(None)

        # Export section below the table
        console.print("\n\n[bold]Export Options:[/bold]", style="cyan3")
        console.print("───────────────", style="cyan3")
        display(export_buttons)
        display(export_output)

    # Return both the basic DataFrame and the enhanced data
    return {
        'summary_df': summary_df,
        'traces_with_categories': traces_with_categories,
        'gas_analysis': gas_analysis,
        'pyusd_interactions': pyusd_interactions_count,
        'total_gas_used': total_gas_used_in_traces,
        'pyusd_transfer_volume': pyusd_transfer_volume
    }

# --- Execute Block Tracing ---
# WARNING: Tracing a full block can be slow. Default to False.
RUN_TRACE_BLOCK = True # <<< SET TO TRUE TO RUN THIS EXPENSIVE TRACE

if 'TARGET_BLOCK_IDENTIFIER' in locals() and TARGET_BLOCK_IDENTIFIER is not None:
    block_id_trace_block = TARGET_BLOCK_IDENTIFIER
    block_id_for_rpc = None

    # Format block identifier for trace_block (requires hex string or tag)
    if isinstance(block_id_trace_block, int):
        block_id_for_rpc = hex(block_id_trace_block)
    elif isinstance(block_id_trace_block, str):
        if block_id_trace_block.startswith("0x"):
            block_id_for_rpc = block_id_trace_block # Hash or Hex Number
        elif block_id_trace_block in ["latest", "pending", "earliest"]:
            block_id_for_rpc = block_id_trace_block
        else: # Try converting string int
             try:
                 block_id_for_rpc = hex(int(block_id_trace_block))
             except ValueError:
                 console.print(f"[error]Invalid TARGET_BLOCK_IDENTIFIER '{block_id_trace_block}' for trace_block. Needs hex string, int, or tag.", style="error")

    if RUN_TRACE_BLOCK and block_id_for_rpc:
        console.print("\n\n[bold]🧱 Tracing & Analyis via trace_block[/bold]", style="cyan3")
        console.print("────────────────────────────────────", style="cyan3")
        console.print(f"\n\n[warning]Attempting 'trace_block' for {block_id_for_rpc} on Mainnet (Can be SLOW!)...[/warning]", style="warning")

        start_time = time.time()
        block_trace_results = make_rpc_request("trace_block", [block_id_for_rpc], network='mainnet')
        end_time = time.time()

        trace_time = end_time - start_time
        console.print(f"[info]trace_block API call completed in {trace_time:.2f} seconds", style="info")

        if block_trace_results is not None: # Check for None explicitly, as empty list is valid
            console.print("[success]Successfully retrieved block trace data.", style="success")
            analysis_results = analyze_block_trace(block_trace_results, block_id_for_rpc)
        else:
            console.print(f"[error]Failed to get block trace for {block_id_for_rpc} using 'trace_block'. RPC call failed, timed out, or returned null.", style="error")

    elif not RUN_TRACE_BLOCK:
        console.print(f"\n[info]Skipping 'trace_block' as RUN_TRACE_BLOCK is False.", style="info")
        console.print(f"[info]Set RUN_TRACE_BLOCK = True to execute this cell with the target block {block_id_trace_block}.", style="info")
    elif not block_id_for_rpc:
         pass # Error already printed

else:
    console.print("[warning]TARGET_BLOCK_IDENTIFIER not set. Skipping trace_block analysis.", style="warning")
    console.print("[info]Set TARGET_BLOCK_IDENTIFIER to a block number or hash to use this cell.", style="info")

## 1.6 🧱 `debug_traceBlockByNumber` and `debug_traceBlockByHash`: Detailed Block Tracing
---

This section explores tracing all transactions within a block using methods from the `debug` namespace: `debug_traceBlockByNumber` (which takes a block number) **and** `debug_traceBlockByHash` (which takes a block hash).

Unlike `trace_block` (from the `trace` namespace), these `debug_` methods often provide **more detailed trace outputs for *each* transaction** within the block, typically defaulting to a format similar to `debug_traceTransaction`'s `callTracer` on GCP. This allows for a deeper analysis of the internal execution flow of every transaction in the target block.

> **🚀 Leveraging GCP's Premium RPC Capabilities**
>
> *   **Methods:** `debug_traceBlockByNumber`, `debug_traceBlockByHash`
> *   **Multiplier:** `50x` (Each method call)
> *   **GCP Advantage:** Tracing full blocks with this level of detail per transaction is highly resource-intensive. GCP's infrastructure and free quotas make it feasible to retrieve and process these potentially very large and numerous trace results efficiently.
> *   **PYUSD Insight:** These methods enable:
>     *   Detailed analysis of **internal calls** for *all* transactions interacting with PYUSD in a block, not just a single target transaction.
>     *   Identifying **indirect PYUSD interactions** where a contract called by the main transaction subsequently interacts with PYUSD.
>     *   Comparing gas efficiency and execution paths of multiple PYUSD transactions within the same block context.
>     *   Analyzing block-level **MEV patterns** (like sandwich attacks) by examining the detailed traces of surrounding transactions.

**Analysis Workflow:**

1.  **Fetch Detailed Block Trace:**
    *   The first part calls `debug_traceBlockByNumber` using the target block identifier (converted to a hex number or tag) and a tracer configuration (typically `callTracer`).
    *   The second part retrieves the corresponding block hash and then calls `debug_traceBlockByHash` with the hash and the tracer configuration.
2.  **Process Individual Traces:** The `analyze_debug_block_trace` function is reused for the output list from *both* methods. It iterates through the list, where each item usually contains the detailed trace result for one transaction.
3.  **Categorize & Analyze:**
    *   Extracts details (gas, status, function calls) for each transaction's trace.
    *   Specifically identifies PYUSD function calls and categories.
    *   Detects internal transactions involving PYUSD (`detect_pyusd_internal_transactions`).
4.  **Visualize & Summarize:** (Outputs are generated for results from both methods if run)
    *   **Enhanced Block Summary:** Includes counts for PYUSD transfers, mints, burns, and total volume within the block.
    *   **PYUSD Function Analysis:** Shows distribution and visualization of PYUSD functions called across the block.
    *   **Internal Call Table:** Lists detected internal calls to PYUSD contracts.
    *   **Gas & Transfer Visualizations:** Plots gas distribution and PYUSD transfer networks for the block.
    *   **Transaction Table:** Interactive summary table for all transactions.
    *   **Export Options:** Download block summary data.

**💡 What to Look For:**
*   Compare the outputs/analyses from `debug_traceBlockByNumber` and `debug_traceBlockByHash` for consistency (they should generally be the same if targeting the same block).
*   **PYUSD Function Distribution:** Which PYUSD functions were most commonly called in this block?
*   **Internal PYUSD Calls:** Did contracts *other than* the main target interact with PYUSD?
*   **Gas Comparison:** Compare the gas usage of similar PYUSD operations within the same block.
*   **Correlated Activity:** Examine traces of transactions near PYUSD interactions for related DeFi or MEV activity.

In [ ]:
# =============================================================================================
# 🧱 Trace Block using debug_traceBlockByNumber / debug_traceBlockByHash
# =============================================================================================
# Note: Output format can be complex and node-dependent. Often returns a list of objects,
# each containing a 'result' which holds the trace (e.g., callTracer format).
# This cell performs detailed tracing and analysis of all transactions within a specific block
# using the `debug_traceBlockByNumber` and `debug_traceBlockByHash` RPC methods, providing
# deep insights into block activity with a focus on PYUSD interactions.
#
# Functionality includes:
# - Fetching detailed execution traces (e.g., callTracer output) for every transaction
#   in the target block via both debug methods.
# - Processing each transaction's trace to extract key information (from, to, gas, status, calls).
# - Analyzing PYUSD-specific interactions: function decoding, volume calculation, internal call detection.
# - Generating comprehensive analysis outputs: summary panel, function category table/pie chart,
#   direct PYUSD interactions table, internal calls table.
# - Visualizing block-level data: PYUSD transfer network graph (Graphviz), gas usage distribution histogram (Plotly).
# - Providing an interactive table (ipywidgets/pandas) for all block transactions with PYUSD filtering.
# - Offering direct export options for the analysis results to CSV, JSON, and Google Sheets.
# - Leveraging GCP's premium RPC capabilities for these resource-intensive debug methods on Mainnet.

import base64
from datetime import datetime
from IPython.display import HTML, clear_output
import json
import ipywidgets as widgets

# Dedicated Export Functions
def download_csv_direct(df, filename=None):
    """Creates a direct download for CSV without intermediate display."""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"pyusd_data_{timestamp}.csv"

    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()

    # Create direct download HTML
    html = f'''
    <script>
    function download(filename, data) {{
        const a = document.createElement('a');
        a.href = data;
        a.download = filename;
        document.body.appendChild(a);
        a.click();
        document.body.removeChild(a);
    }}

    download("{filename}", "data:text/csv;base64,{payload}");
    </script>
    <div>Downloading {filename}...</div>
    '''
    return HTML(html)

def download_json_direct(data, filename=None):
    """Creates a direct download for JSON without intermediate display."""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"pyusd_data_{timestamp}.json"

    # Convert to JSON string (handling non-serializable objects)
    json_str = json.dumps(data, default=str, indent=2)
    b64 = base64.b64encode(json_str.encode()).decode()

    # Create direct download HTML
    html = f'''
    <script>
    function download(filename, data) {{
        const a = document.createElement('a');
        a.href = data;
        a.download = filename;
        document.body.appendChild(a);
        a.click();
        document.body.removeChild(a);
    }}

    download("{filename}", "data:application/json;base64,{b64}");
    </script>
    <div>Downloading {filename}...</div>
    '''
    return HTML(html)

def export_to_google_sheets_direct(df, sheet_name=None):
    """Exports DataFrame directly to Google Sheets using authenticated session."""
    if sheet_name is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        sheet_name = f"PYUSD Analysis {timestamp}"

    # Use Google Colab integration for direct export
    html = f'''
    <script src="https://apis.google.com/js/platform.js" async defer></script>
    <script>
    function createSheet() {{
        const csv = `{df.to_csv(index=False).replace('"', '""')}`;

        // Create sheet using Google Colab API
        google.colab.kernel.invokeFunction('notebook.createSheet', [csv, '{sheet_name}'], {{}});
    }}

    // Execute immediately
    setTimeout(createSheet, 100);
    </script>
    <div>Creating Google Sheet "{sheet_name}"...</div>
    '''

    # Register the Python callback that the JavaScript will call
    from google.colab import output

    @output.register_callback('notebook.createSheet')
    def create_sheet_callback(csv_data, name):
        try:
            from google.colab import auth
            from googleapiclient.discovery import build
            from googleapiclient.http import MediaInMemoryUpload
            import io

            # Ensure authentication
            auth.authenticate_user()

            # Create Drive API client
            drive_service = build('drive', 'v3')

            # File metadata
            file_metadata = {
                'name': name,
                'mimeType': 'application/vnd.google-apps.spreadsheet'
            }

            # Create CSV upload
            media = MediaInMemoryUpload(
                io.BytesIO(csv_data.encode('utf-8')),
                mimetype='text/csv',
                resumable=True
            )

            # Create the Sheet
            file = drive_service.files().create(
                body=file_metadata,
                media_body=media,
                fields='id,webViewLink'
            ).execute()

            # Return success with link
            return {
                'status': 'success',
                'file_id': file.get('id'),
                'link': file.get('webViewLink')
            }
        except Exception as e:
            return {
                'status': 'error',
                'message': str(e)
            }

    return HTML(html)

def detect_pyusd_internal_transactions(debug_trace_list):
    """Analyze traces for PYUSD internal transactions (contracts calling PYUSD)."""
    internal_txs = []

    for i, trace_item in enumerate(debug_trace_list):
        tx_hash = trace_item.get('txHash', f'tx_{i}')
        trace_result = trace_item.get('result', {})

        # Skip if there are no calls
        if not isinstance(trace_result.get('calls'), list):
            continue

        # Function to recursively process calls
        def process_calls(calls, depth=0, parent_from=None, parent_to=None):
            for call in calls:
                call_to = call.get('to', '')
                call_from = call.get('from', parent_from if parent_from else '')
                call_type = call.get('type', 'CALL')
                call_input = call.get('input', '0x')
                call_gas_used = call.get('gasUsed', '0x0')

                # Check if this call is to a PYUSD contract
                if call_to and call_to.lower() in PYUSD_CONTRACTS:
                    # Try to decode the function
                    function_name = "Unknown"
                    if call_input and len(call_input) >= 10:
                        method_sig = call_input[:10]
                        if method_sig in PYUSD_SIGNATURES:
                            function_name = PYUSD_SIGNATURES[method_sig]["name"]

                    # Record this internal transaction
                    internal_txs.append({
                        'tx_hash': tx_hash,
                        'from': call_from,
                        'to': call_to,
                        'to_contract': get_contract_name(call_to),
                        'function': function_name,
                        'call_type': call_type,
                        'gas_used': int(call_gas_used, 16) if isinstance(call_gas_used, str) else call_gas_used,
                        'depth': depth
                    })

                # Process nested calls recursively
                if isinstance(call.get('calls'), list):
                    process_calls(call['calls'], depth + 1, call_from, call_to)

        # Start processing from the top level calls
        if isinstance(trace_result.get('calls'), list):
            process_calls(trace_result['calls'])

    return internal_txs

def analyze_debug_block_trace(debug_trace_list, block_identifier):
    """Analyzes the output list from debug_traceBlockByNumber/Hash with PYUSD focus."""
    if not isinstance(debug_trace_list, list):
        console.print(f"[error]Expected a list from debug_traceBlock for {block_identifier}, got {type(debug_trace_list)}.", style="error")
        display_json(debug_trace_list, "Unexpected Result Structure")
        return None
    if not debug_trace_list:
        console.print(f"[warning]No trace results (empty list) from debug_traceBlock for {block_identifier}.", style="warning")
        return None

    console.print(f"\n\n[bold cyan3]Analyzing {len(debug_trace_list)} Traces from debug_traceBlock for {block_identifier}[/bold cyan3]")

    # Counters and aggregators
    pyusd_interactions_count = 0
    pyusd_transfer_count = 0
    pyusd_mint_count = 0
    pyusd_burn_count = 0
    total_gas_used = 0
    failed_traces_count = 0

    # Track PYUSD transfers within block
    pyusd_transfers = []
    pyusd_volume = 0

    # Track function categories
    function_categories = {
        "token_movement": 0,
        "supply_change": 0,
        "allowance": 0,
        "control": 0,
        "admin": 0,
        "view": 0,
        "other": 0
    }

    trace_summary_list = []

    for i, trace_item in enumerate(debug_trace_list):
        # Default values
        tx_hash = 'N/A'
        from_addr = 'N/A'
        to_addr = 'N/A'
        value_eth_str = '0 ETH'
        gas_used = 0
        error = None
        interacted_with_pyusd = False
        tx_pos = i  # Fallback index
        pyusd_function = None
        pyusd_function_category = "other"
        is_pyusd_transfer = False
        is_pyusd_mint = False
        is_pyusd_burn = False
        transfer_value = 0

        try:
            # debug_traceBlock* often nests the main trace info under 'result'
            trace_result = trace_item.get('result', {})  # Get the nested trace
            tx_hash = trace_item.get('txHash', f'tx_{i}')  # Hash might be at top level

            # Parse fields from the nested 'result' if it exists
            if trace_result:
                from_addr = trace_result.get('from', 'N/A')
                to_addr = trace_result.get('to', 'N/A')
                value_wei_hex = trace_result.get('value', '0x0')
                gas_used_hex = trace_result.get('gasUsed', '0x0')
                error = trace_result.get('error')  # Error within the execution
                input_data = trace_result.get('input', '0x')

                gas_used = int(gas_used_hex, 16) if gas_used_hex else 0
                value_eth_str = format_value_eth(value_wei_hex)

                # Check for PYUSD contract interactions
                interacted_with_pyusd = to_addr and to_addr.lower() in PYUSD_CONTRACTS

                # If this is a PYUSD interaction, analyze the function being called
                if interacted_with_pyusd and input_data and input_data != '0x':
                    method_sig = input_data[:10]

                    if method_sig in PYUSD_SIGNATURES:
                        sig_info = PYUSD_SIGNATURES[method_sig]
                        pyusd_function = sig_info["name"]
                        pyusd_function_category = sig_info["category"]

                        # Track function category
                        function_categories[pyusd_function_category] += 1

                        # Check for specific functions
                        if method_sig == '0xa9059cbb':  # transfer
                            is_pyusd_transfer = True
                            pyusd_transfer_count += 1

                            # Try to decode transfer parameters
                            try:
                                to_offset = 10
                                to_param = "0x" + input_data[to_offset+24:to_offset+64]
                                amount_offset = 74
                                amount = int(input_data[amount_offset:amount_offset+64], 16)
                                transfer_value = amount
                                pyusd_volume += amount

                                # Track transfer for visualization
                                pyusd_transfers.append({
                                    'from': from_addr,
                                    'to': to_param,
                                    'value': amount,
                                    'tx_hash': tx_hash
                                })
                            except Exception:
                                pass

                        elif method_sig == '0x40c10f19':  # mint
                            is_pyusd_mint = True
                            pyusd_mint_count += 1

                            # Try to decode mint amount
                            try:
                                amount_offset = 74
                                amount = int(input_data[amount_offset:amount_offset+64], 16)
                                transfer_value = amount
                                pyusd_volume += amount
                            except Exception:
                                pass

                        elif method_sig == '0x42966c68':  # burn
                            is_pyusd_burn = True
                            pyusd_burn_count += 1

                            # Try to decode burn amount
                            try:
                                amount_offset = 10
                                amount = int(input_data[amount_offset:amount_offset+64], 16)
                                transfer_value = amount
                                pyusd_volume += amount
                            except Exception:
                                pass

                # Check for sub-calls that might interact with PYUSD
                if 'calls' in trace_result and isinstance(trace_result['calls'], list):
                    for call in trace_result['calls']:
                        if call.get('to', '').lower() in PYUSD_CONTRACTS:
                            interacted_with_pyusd = True
                            break
            else:
                # Handle cases where 'result' is missing - trace might have failed earlier
                error = trace_item.get('error', 'Missing trace result')  # Check top level error

        except Exception as parse_err:
            console.print(f"[warning]Could not fully parse trace item {i} in debug block trace {block_identifier}: {parse_err}", style="warning")
            error = f"Parse Error: {parse_err}"

        if error:
            failed_traces_count += 1
        if interacted_with_pyusd:
            pyusd_interactions_count += 1
        total_gas_used += gas_used

        trace_summary_list.append({
            'tx_index': tx_pos,  # Using list index as fallback for tx position
            'tx_hash': tx_hash,
            'from': from_addr,
            'to': to_addr,
            'value_eth': value_eth_str,
            'gas_used': gas_used,
            'failed': error is not None,
            'pyusd_interaction': interacted_with_pyusd,
            'pyusd_function': pyusd_function,
            'pyusd_function_category': pyusd_function_category,
            'is_pyusd_transfer': is_pyusd_transfer,
            'is_pyusd_mint': is_pyusd_mint,
            'is_pyusd_burn': is_pyusd_burn,
            'transfer_value': transfer_value
        })

    summary_df = pd.DataFrame(trace_summary_list)

    # Calculate key metrics
    pyusd_volume_formatted = format_value_pyusd(pyusd_volume) if pyusd_volume > 0 else "0 PYUSD"
    pyusd_pct = (pyusd_interactions_count / len(debug_trace_list) * 100) if debug_trace_list else 0

    # Display Enhanced Summary Stats
    console.print(Panel(f"""
[bold cyan3]Enhanced debug_traceBlock Summary ({block_identifier})[/bold cyan3]
[bold cyan3]Transactions Traced:[/bold cyan3] {len(debug_trace_list)}
[bold cyan3]Total Gas Used:[/bold cyan3] {total_gas_used:,}
[bold cyan3]Traces with Errors:[/bold cyan3] {failed_traces_count}

[bold green3]PYUSD Activity[/bold green3]
[bold green3]PYUSD Interactions:[/bold green3] {pyusd_interactions_count} ({pyusd_pct:.1f}% of transactions)
[bold green3]PYUSD Transfers:[/bold green3] {pyusd_transfer_count}
[bold green3]PYUSD Mints:[/bold green3] {pyusd_mint_count}
[bold green3]PYUSD Burns:[/bold green3] {pyusd_burn_count}
[bold green3]PYUSD Volume:[/bold green3] {pyusd_volume_formatted}""",
        title="debug_traceBlock Analysis", border_style="cyan3", expand=False))

    # Display function category breakdown if any PYUSD activity
    console.print("\n\n[bold]PYUSD Function Categories[/bold]", style="cyan3")
    console.print("────────────────────────", style="cyan3")
    if pyusd_interactions_count > 0:
        category_table = Table(title="", show_header=True, header_style="bold green3")
        category_table.add_column("Category")
        category_table.add_column("Count", justify="right")
        category_table.add_column("Percentage", justify="right")

        for category, count in function_categories.items():
            if count > 0:
                percentage = (count / pyusd_interactions_count * 100)
                category_table.add_row(
                    category.replace('_', ' ').title(),
                    str(count),
                    f"{percentage:.1f}%"
                )

        console.print(category_table)

        # Create visualization of function categories
        try:
            console.print(f"\n\n[bold]PYUSD Function Categories in Block Visualization: [cyan3]{block_identifier}[/cyan3][/bold]", style="magenta3")
            console.print("─────────────────────────────────────────────────", style="magenta3")

            category_data = pd.DataFrame([
                {"category": cat.replace('_', ' ').title(), "count": count}
                for cat, count in function_categories.items() if count > 0
            ])

            if not category_data.empty:
                fig_categories = px.pie(
                    category_data, values='count', names='category',
                    title=f'PYUSD Function Categories in Block {block_identifier}'
                )
                fig_categories.update_layout(template="plotly_white")
                fig_categories.show()
        except Exception as viz_err:
            console.print(f"[warning]Could not create function category visualization: {viz_err}", style="warning")

    # Display PYUSD-specific transactions
    if pyusd_interactions_count > 0 and not summary_df.empty:
        console.print("\n\n[bold green3]📊 PYUSD Interactions in Block[/bold green3]")
        console.print("───────────────────────────────", style="green3")

        pyusd_txs = summary_df[summary_df['pyusd_interaction']].copy()

        if not pyusd_txs.empty:
            # Add readable addresses
            pyusd_txs['from_short'] = pyusd_txs['from'].apply(shorten_address)
            pyusd_txs['to_short'] = pyusd_txs['to'].apply(shorten_address)

            # Add transfer value formatting
            if 'transfer_value' in pyusd_txs.columns:
                pyusd_txs['value_pyusd'] = pyusd_txs['transfer_value'].apply(
                    lambda x: format_value_pyusd(x) if x > 0 else ""
                )

            # Display streamlined view
            display_cols = [
                'tx_index', 'from', 'to', 'pyusd_function',
                'value_pyusd', 'gas_used', 'failed'
            ]

            # Filter columns that exist
            display_cols = [col for col in display_cols if col in pyusd_txs.columns]

            # Create a table to display this data
            console.print("\n\n[bold]PYUSD Transactions in Block[/bold]", style="cyan3")
            pyusd_table = Table(title="", title_style="")
            pyusd_table.add_column("Tx Index")
            pyusd_table.add_column("From")
            pyusd_table.add_column("To")
            pyusd_table.add_column("Function")
            pyusd_table.add_column("Amount")
            pyusd_table.add_column("Gas Used")
            pyusd_table.add_column("Status")

            # Add rows (limit to first 15 to avoid huge output)
            for idx, row in pyusd_txs.head(15).iterrows():
                status_color = "red" if row['failed'] else "green"
                status_text = "Failed" if row['failed'] else "Success"

                pyusd_table.add_row(
                    str(row['tx_index']),
                    row['from'],
                    row['to'],
                    str(row['pyusd_function'] or ""),
                    row.get('value_pyusd', ""),
                    f"{row['gas_used']:,}",
                    f"[{status_color}]{status_text}[/{status_color}]"
                )

            if len(pyusd_txs) > 15:
                pyusd_table.add_row(
                    "...",
                    f"+ {len(pyusd_txs) - 15} more",
                    "", "", "", "", ""
                )

            console.print(pyusd_table)
        else:
            console.print("[info]No direct PYUSD interactions found.", style="info")

    # Visualize PYUSD transfer network if transfers exist
    if pyusd_transfers:
        console.print("\n\n[bold magenta3]🔄 PYUSD Transfer Network in Block[/bold magenta3]")
        console.print("───────────────────────────────────", style="magenta3")

        try:
            # Create transfer network visualization
            flow_graph = Digraph(comment=f"PYUSD Transfers in Block {block_identifier}", format='png')
            flow_graph.attr(rankdir='TB', bgcolor='transparent')
            flow_graph.attr('node', shape='box', style='filled', fontname='helvetica',
                           fontcolor='black', fillcolor='palegreen')

            # Track nodes we've added
            added_nodes = set()

            # Aggregate transfers between same addresses
            transfer_map = {}

            for transfer in pyusd_transfers:
                from_addr = transfer['from']
                to_addr = transfer['to']
                value = transfer['value']

                key = (from_addr, to_addr)
                if key in transfer_map:
                    transfer_map[key] += value
                else:
                    transfer_map[key] = value

            # Add nodes and edges
            for (from_addr, to_addr), total_value in transfer_map.items():

                if from_addr not in added_nodes:
                    flow_graph.node(from_addr, label=from_addr)
                    added_nodes.add(from_addr)

                if to_addr not in added_nodes:
                    flow_graph.node(to_addr, label=to_addr)
                    added_nodes.add(to_addr)

                value_str = format_value_pyusd(total_value)
                flow_graph.edge(from_addr, to_addr, label=value_str)

            display(flow_graph)
            console.print("[info]This graph shows PYUSD transfers within the block.", style="info")
        except Exception as viz_err:
            console.print(f"[warning]Could not create PYUSD transfer network visualization: {viz_err}", style="warning")

    # Analyze internal transactions
    internal_txs = detect_pyusd_internal_transactions(debug_trace_list)
    if internal_txs:
        console.print("\n\n[bold green3]🔄 PYUSD Internal Transactions[/bold green3]")
        console.print("─────────────────────────────", style="green3")
        console.print(f"[info]Found {len(internal_txs)} internal transactions involving PYUSD contracts.", style="info")

        # Create a table of internal transactions
        console.print("\n\n[bold cyan3]Internal PYUSD Calls[/bold cyan3]")

        internal_table = Table(title="", title_style="bold cyan")
        internal_table.add_column("From", style="dim")
        internal_table.add_column("To Contract", style="cyan")
        internal_table.add_column("Function")
        internal_table.add_column("Gas Used", justify="right")
        internal_table.add_column("Depth", justify="right")

        # Show a subset if there are many
        display_limit = min(10, len(internal_txs))
        for i in range(display_limit):
            tx = internal_txs[i]
            internal_table.add_row(
                tx['from'],
                tx['to_contract'],
                tx['function'],
                f"{tx['gas_used']:,}",
                str(tx['depth'])
            )

        if len(internal_txs) > display_limit:
            internal_table.add_row(
                f"+ {len(internal_txs) - display_limit} more...",
                "", "", "", ""
            )

        console.print(internal_table)

    # Display summary table for all transactions with enhanced interactive features
    if not summary_df.empty:
        console.print("\n[bold chartreuse1]📋 Block Transaction Summary[/bold chartreuse1]")
        console.print("─────────────────────────────", style="chartreuse1")

        # Create filter controls and PYUSD-focused display
        # Create PYUSD-only dataframe
        pyusd_df = summary_df[summary_df['pyusd_interaction'] == True].copy()

        # Filter options above the table
        display(widgets.HTML("<h4>Filter Options:</h4>"))

        # Keep both buttons - arranged in an HBox for better layout
        filter_buttons = widgets.HBox([
            widgets.Button(
                description='Show PYUSD Only',
                button_style='success',
                layout=widgets.Layout(width='180px')
            ),
            widgets.Button(
                description='Show All Transactions',
                button_style='info',
                layout=widgets.Layout(width='200px')
            )
        ])

        display(filter_buttons)

        # Output area for transactions
        transaction_output = widgets.Output()

        # Export buttons with better styling
        export_buttons = widgets.HBox([
            widgets.Button(
                description='Export to CSV',
                button_style='primary',  # Green
                layout=widgets.Layout(width='150px')
            ),
            widgets.Button(
                description='Export as JSON',
                button_style='warning',  # Orange
                layout=widgets.Layout(width='150px')
            ),
            widgets.Button(
                description='Export to Google Sheets',
                button_style='info',     # Blue
                layout=widgets.Layout(width='200px')
            )
        ])

        export_output = widgets.Output()

        # Display columns for better view
        display_cols = [
            'tx_index', 'from', 'to', 'gas_used',
            'pyusd_interaction', 'pyusd_function', 'failed'
        ]

        # Filter columns that exist
        display_cols = [col for col in display_cols if col in summary_df.columns]

        def show_all_txs(b):
            with transaction_output:
                clear_output()
                display(summary_df[display_cols])

        def show_pyusd_txs(b):
            with transaction_output:
                clear_output()
                if len(pyusd_df) > 0:
                    display(pyusd_df[display_cols])
                else:
                    display(widgets.HTML("<p>No PYUSD transactions in this block.</p>"))

        # Export handlers
        def export_csv(b):
            with export_output:
                clear_output()
                # Use the currently displayed DataFrame based on which filter is active
                current_df = pyusd_df if transaction_output.outputs and len(transaction_output.outputs[0]['text/plain']) <= len(str(pyusd_df)) else summary_df
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"block_{block_identifier}_transactions_{timestamp}.csv"
                display(download_csv_direct(current_df, filename))

        def export_json(b):
            with export_output:
                clear_output()
                # Use the currently displayed DataFrame based on which filter is active
                current_df = pyusd_df if transaction_output.outputs and len(transaction_output.outputs[0]['text/plain']) <= len(str(pyusd_df)) else summary_df
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"block_{block_identifier}_transactions_{timestamp}.json"
                records = current_df.to_dict('records')
                display(download_json_direct(records, filename))

        def export_to_sheets(b):
            with export_output:
                clear_output()
                # Use the currently displayed DataFrame based on which filter is active
                current_df = pyusd_df if transaction_output.outputs and len(transaction_output.outputs[0]['text/plain']) <= len(str(pyusd_df)) else summary_df
                sheet_name = f"Block {block_identifier} Transactions {datetime.now().strftime('%Y%m%d_%H%M%S')}"
                display(export_to_google_sheets_direct(current_df, sheet_name))

        # Connect callbacks
        filter_buttons.children[0].on_click(show_pyusd_txs)   # PYUSD Only button
        filter_buttons.children[1].on_click(show_all_txs)     # Show All button
        export_buttons.children[0].on_click(export_csv)
        export_buttons.children[1].on_click(export_json)
        export_buttons.children[2].on_click(export_to_sheets)

        # Display transaction table area
        display(transaction_output)

        # Show PYUSD transactions by default
        show_pyusd_txs(None)

        # Export section below the table
        console.print("\n\n[bold]Export Options:[/bold]", style="cyan3")
        console.print("──────────────", style="cyan3")
        display(export_buttons)
        display(export_output)

    # Return the DataFrame for further analysis
    return summary_df

# --- Execute debug_traceBlockByNumber with enhanced config ---
# WARNING: Can be slow. Default to False.
RUN_DEBUG_TRACE_BLOCK_NUM = True # <<< SET TO TRUE TO RUN THIS

if 'TARGET_BLOCK_IDENTIFIER' in locals() and TARGET_BLOCK_IDENTIFIER is not None:
    block_id_debug_trace_num = TARGET_BLOCK_IDENTIFIER
    block_param_num = None

    # Format block identifier for debug_traceBlockByNumber (requires hex number string or tag)
    if isinstance(block_id_debug_trace_num, int):
        block_param_num = hex(block_id_debug_trace_num)
    elif isinstance(block_id_debug_trace_num, str):
        if block_id_debug_trace_num.startswith("0x") and len(block_id_debug_trace_num) != 66: # Is hex number
             block_param_num = block_id_debug_trace_num
        elif block_id_debug_trace_num in ["latest", "pending", "earliest"]:
            block_param_num = block_id_debug_trace_num
        else: # Try converting string int
             try:
                 block_param_num = hex(int(block_id_debug_trace_num))
             except ValueError:
                 console.print(f"[error]Invalid TARGET_BLOCK_IDENTIFIER '{block_id_debug_trace_num}' for debug_traceBlockByNumber. Needs hex number string or tag.", style="error")

    if RUN_DEBUG_TRACE_BLOCK_NUM and block_param_num:
        console.print("[bold cyan3]🧱 Tracing & Analyis via debug_traceBlockByNumber[/bold cyan3]")
        console.print("─────────────────────────────────────────────────", style="cyan3")
        console.print(f"\n\n[warning]Attempting 'debug_traceBlockByNumber' for {block_param_num} on Mainnet (Can be SLOW!)...[/warning]", style="warning")

        # Use the enhanced tracer config for better insights
        enhanced_config = {
            "tracer": "callTracer",
            "tracerConfig": TRACE_CONFIGS["callTracer"]
        }

        debug_block_num_trace_results = make_rpc_request(
            "debug_traceBlockByNumber",
            [block_param_num, enhanced_config],
            network='mainnet'
        )

        if debug_block_num_trace_results is not None:
            block_analysis = analyze_debug_block_trace(debug_block_num_trace_results, block_param_num)

            # Add any post-analysis insights here
            if block_analysis is not None and 'pyusd_interaction' in block_analysis.columns:
                pyusd_txs = block_analysis[block_analysis['pyusd_interaction']]
                if not pyusd_txs.empty:
                    # Create a histogram of gas usage for PYUSD vs non-PYUSD transactions
                    try:
                        # Add interaction type column
                        block_analysis['interaction_type'] = block_analysis['pyusd_interaction'].apply(
                            lambda x: 'PYUSD Transaction' if x else 'Other Transaction'
                        )

                        console.print(f"\n\n[bold]Gas Usage Distribution in Block: [cyan3]{block_param_num}[/cyan3][/bold]", style="magenta3")
                        console.print("────────────────────────────────", style="magenta3")

                        # Create histogram
                        fig_gas = px.histogram(
                            block_analysis, x='gas_used', color='interaction_type',
                            title=f'Gas Usage Distribution in Block {block_param_num}',
                            labels={'gas_used': 'Gas Used', 'count': 'Number of Transactions'},
                            log_y=True  # Log scale for better visibility of distribution
                        )
                        fig_gas.update_layout(template="plotly_white")
                        fig_gas.show()
                    except Exception as viz_err:
                        console.print(f"[warning]Could not create gas usage histogram: {viz_err}", style="warning")
        else:
            console.print(f"[error]Failed to get block trace for {block_param_num} using 'debug_traceBlockByNumber'.", style="error")

    elif not RUN_DEBUG_TRACE_BLOCK_NUM:
        console.print(f"\n[info]Skipping 'debug_traceBlockByNumber' as RUN_DEBUG_TRACE_BLOCK_NUM is False.", style="info")
    elif not block_param_num:
         pass # Error already printed by validation logic

else:
    console.print("[warning]TARGET_BLOCK_IDENTIFIER not set. Skipping debug_traceBlockByNumber analysis.", style="warning")


# --- Execute debug_traceBlockByHash with enhanced config ---
# WARNING: Can be slow. Default to False.
RUN_DEBUG_TRACE_BLOCK_HASH = True # <<< SET TO TRUE TO RUN THIS

if 'TARGET_BLOCK_IDENTIFIER' in locals() and TARGET_BLOCK_IDENTIFIER is not None:
    block_id_debug_trace_hash = TARGET_BLOCK_IDENTIFIER
    block_hash_param = None

    # Need to get the block HASH for this method
    try:
        console.print("\n\n[bold]🧱 Tracing & Analyis via debug_traceBlockByHash[/bold]", style="cyan3")
        console.print("────────────────────────────────────────────────", style="cyan3")
        console.print(f"\n\n[bold cyan3]📡 Querying block info for identifier '{block_id_debug_trace_hash}' to get hash...[bold cyan3]")
        # Use the mainnet client to get block info
        if w3_mainnet:
            block_info = w3_mainnet.eth.get_block(block_id_debug_trace_hash)
            if block_info and 'hash' in block_info:
                block_hash_param = block_info['hash'].hex()
                console.print(f"\n\n[info]Found block hash: {block_hash_param}", style="info")
            else:
                console.print(f"[error]Could not retrieve block info or hash for identifier '{block_id_debug_trace_hash}'. Block might not exist.", style="error")
        else:
             console.print("[error]Mainnet client not available to fetch block hash.", style="error")

    except Exception as e:
         console.print(f"[error]Error retrieving block hash for '{block_id_debug_trace_hash}': {e}", style="error")


    if RUN_DEBUG_TRACE_BLOCK_HASH and block_hash_param:
        console.print(f"\n[warning]Attempting 'debug_traceBlockByHash' for {block_hash_param} on Mainnet (Can be SLOW!)...[/warning]", style="warning")

        # Use the enhanced tracer config
        enhanced_config = {
            "tracer": "callTracer",
            "tracerConfig": TRACE_CONFIGS["callTracer"]
        }

        debug_block_hash_trace_results = make_rpc_request(
            "debug_traceBlockByHash",
            [block_hash_param, enhanced_config],
            network='mainnet'
        )

        if debug_block_hash_trace_results is not None:
            # Analyze with the same function as debug_traceBlockByNumber
            block_analysis = analyze_debug_block_trace(debug_block_hash_trace_results, block_hash_param)
        else:
            console.print(f"[error]Failed to get block trace for hash {block_hash_param} using 'debug_traceBlockByHash'.", style="error")

    elif not RUN_DEBUG_TRACE_BLOCK_HASH:
        console.print(f"\n[info]Skipping 'debug_traceBlockByHash' as RUN_DEBUG_TRACE_BLOCK_HASH is False.", style="info")
    elif not block_hash_param:
        pass # Error already printed by validation logic

else:
    console.print("[warning]TARGET_BLOCK_IDENTIFIER not set. Skipping debug_traceBlockByHash analysis.", style="warning")

## 1.7 🧪 `debug_traceCall`, `eth.call`, `eth.estimate_gas`: Advanced Simulation of PYUSD Transactions
---

This section utilizes a **multi-stage simulation approach** to analyze potential PYUSD transactions *without* broadcasting them or requiring real gas/ETH. It intelligently combines standard and advanced RPC methods (`eth.call`, `eth.estimate_gas`, and optionally `debug_traceCall`/`trace_call`) to provide comprehensive insights based on a specific block state.

This hybrid strategy is incredibly useful for:

*   **Pre-flight Validation (`eth.call`):** Quickly checking if a transaction would likely revert due to basic errors (invalid parameters, immediate contract logic failure).
*   **Gas Estimation (`eth.estimate_gas`):** Predicting the gas cost *before* sending, allowing for optimization and budget planning. The code uses this as the primary gas source unless a trace provides a more accurate figure.
*   **Revert Debugging (`eth.call` Error + Trace):** Understanding *why* a potential transaction might fail by analyzing the initial revert reason or, if tracing is enabled, examining the detailed execution path and internal calls.
*   **State Change Analysis (Trace Events):** Observing the potential impact of operations like `transfer` or `approve` by decoding specific events (e.g., `Transfer` event) emitted during the simulated trace.
*   **"What-If" Analysis & Comparison:** Testing different parameters for PYUSD functions to compare outcomes, gas costs (`eth.estimate_gas`), and execution details (trace).

> **🚀 Leveraging GCP's Premium RPC Capabilities**
>
> *   **Methods:** `eth.call`, `eth.estimate_gas`, `debug_traceCall` (preferred for tracing), `trace_call` (fallback)
> *   **Multiplier for `debug_traceCall`/`trace_call`:** `50x` (Available on Mainnet via GCP)
> *   **GCP Advantage:** Simulating complex transactions and obtaining detailed execution traces via `debug_traceCall` (especially those interacting with multiple contracts) is resource-intensive. GCP allows these detailed simulations efficiently on Mainnet.
> *   **PYUSD Insight:** This advanced simulation enables:
>     *   Robust **pre-flight checks** for PYUSD transfers, approvals, mints, burns.
>     *   Comparing the gas cost (`eth.estimate_gas`) of different PYUSD transfer amounts or approval values.
>     *   Debugging potential integration issues by simulating calls *from* other contracts *to* the PYUSD contract and analyzing the trace.

**Analysis Workflow (as implemented in the code):**

1.  **Define Call & Pre-Check:**
    *   Constructs the transaction call dictionary (`from`, `to`, `data`, etc.).
    *   Performs an optional, preliminary PYUSD balance check (`estimate_pyusd_balance` using `eth.call`) for relevant operations to provide immediate context.
2.  **Stage 1: Basic Simulation (`eth.call`):**
    *   *Always* executes `eth.call` first to get an immediate success/fail status and potential revert data.
3.  **Stage 2: Gas Estimation (`eth.estimate_gas`):**
    *   If `eth.call` succeeded OR failed *only* due to insufficient balance (indicating the logic is otherwise sound), it attempts `eth.estimate_gas` to get the predicted gas cost. This value is stored as the primary `gas_used`.
4.  **Stage 3: Detailed Tracing (Optional - `debug_traceCall`/`trace_call`):**
    *   If `use_trace=True` and the simulation is plausible (success or hypothetical success from Stage 2), it attempts detailed tracing.
    *   It prioritizes `debug_traceCall`, using the `make_rpc_request` helper to try multiple parameter formats for maximum compatibility (essential for providers like GCP).
    *   If `debug_traceCall` fails across formats, it attempts `trace_call` as a fallback.
5.  **Analyze & Assemble Results:**
    *   The `simulate_pyusd_transaction` function gathers results into an `analysis` dictionary.
    *   It decodes known ERC-20/PYUSD errors from `eth.call` failures.
    *   It categorizes gas usage based on the estimated gas and function type.
    *   **If a trace was successful:** It parses the `trace_result` to potentially refine `gas_used` (if available in trace), extract internal calls, and identify `state_changes` by decoding specific emitted events (like PYUSD `Transfer` logs) found within the trace.
    *   It decodes the return `output` for view functions (like `balanceOf`).
6.  **Visualize & Export:**
    *   Uses `display_simulation_analysis` to present a structured summary: function details, execution results (status, gas, errors, decoded output), state changes (from trace events), and transaction flow diagrams.
    *   Leverages `compare_pyusd_transactions` and `batch_simulate_pyusd_transactions` for multi-run analysis with comparison tables and charts.
    *   Provides export options (CSV, JSON, Google Sheets) via helper functions.

**💡 What to Look For:**

*   **Status:** Did `eth.call` succeed? If not, what was the decoded error? Is it a "Hypothetical Success" (meaning `eth.estimate_gas` worked even if the initial balance was too low)?
*   **Gas Used:** What is the `eth.estimate_gas` value? Was it refined by a value from the trace? How does it compare across different parameters (in comparison mode)?
*   **Gas Profile:** How does the gas usage categorize (Efficient, Medium, High) for this type of operation?
*   **Decoded Output:** For view functions (`balanceOf`, `allowance`), what is the simulated return value?
*   **State Changes (from Trace):** If tracing was enabled, were any key events (like `Transfer`) detected in the execution trace, indicating simulated state modifications?
*   **Internal Calls (from Trace):** Does the trace reveal calls made by the PYUSD contract to other contracts?

In [ ]:
# =============================================================================================
# 🧪 Advanced PYUSD Transaction Simulation with Trace Analysis
# =============================================================================================
# This cell provides comprehensive simulation and analysis for PYUSD transactions on Ethereum.
# Features:
# - Hybrid simulation approach using eth.call and trace_call for reliability
# - Transaction parameter optimization and gas estimation
# - Rich visual analysis of transaction effects and gas usage
# - Multi-format export capabilities (CSV, JSON, Google Sheets)
# - Advanced error decoding and meaningful diagnostics
# - Support for multiple transaction types and batched operations

from web3 import Web3
import pandas as pd
import numpy as np
import json
import base64
from datetime import datetime
from IPython.display import HTML, display
import ipywidgets as widgets
from IPython.display import clear_output
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.layout import Layout
from rich.console import Group
from rich.rule import Rule
from rich import box
from graphviz import Digraph
import matplotlib.pyplot as plt
from rich.theme import Theme

# Auto-adapting color theme that works well in both light and dark terminals
custom_theme = Theme({
    "info": "cyan3",
    "success": "spring_green3",
    "warning": "gold3",
    "error": "red3",
    "highlight": "royal_blue1"
})

# Initialize Rich console with the theme
console = Console(theme=custom_theme)

# =============================================================================================
# 📊 Visualization Functions
# =============================================================================================

def create_gas_comparison_chart(results, function_name):
    """Create a matplotlib bar chart comparing gas usage.

    Args:
        results: List of simulation results
        function_name: Name of the function being compared

    Returns:
        Success status boolean
    """
    try:
        # Prepare data for visualization
        gas_data = []
        for result in results:
            if result["success"] or result.get("hypothetical_success", False):
                gas_data.append({
                    "variant": result["variant"],
                    "gas_used": result["gas_used"],
                    "category": result.get("gas_category", "Unknown")
                })

        if not gas_data:
            return False

        # Create dataframe
        gas_df = pd.DataFrame(gas_data)

        # Print header
        console.print("\n\n[bold]⛽ Gas Usage Comparison Chart:[/bold]", style="magenta3")
        console.print("──────────────────────────────", style="magenta3")

        # Create figure and axis
        fig, ax = plt.subplots(figsize=(10, 6))

        # Plot the bars
        bars = ax.bar(gas_df['variant'], gas_df['gas_used'], color='steelblue')

        # Add value labels on top of bars
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{int(height):,}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom')

        # Add chart styling
        ax.set_title(f'Gas Usage Comparison - {function_name}()', fontsize=14, pad=20)
        ax.set_xlabel('Variant(s)', fontsize=12)
        ax.set_ylabel('Gas Used', fontsize=12)
        ax.grid(axis='y', linestyle='--', alpha=0.7)

        # Adjust layout
        plt.tight_layout()

        # Display the chart
        plt.show()

        # If we have multiple results, also create a relative cost comparison
        if len(gas_data) > 1:
            # Find minimum gas cost as baseline
            min_gas = min(item["gas_used"] for item in gas_data)
            rel_data = []
            for item in gas_data:
                rel_data.append({
                    "variant": item["variant"],
                    "relative_cost": item["gas_used"] / min_gas
                })

            rel_df = pd.DataFrame(rel_data)

            # Create a new figure for relative comparison
            console.print("\n\n[bold]⛽ Relative Gas Cost Comparison:[/bold]", style="magenta3")
            console.print("─────────────────────────────────", style="magenta3")

            fig2, ax2 = plt.subplots(figsize=(10, 6))

            # Plot the relative bars
            rel_bars = ax2.bar(rel_df['variant'], rel_df['relative_cost'], color='lightcoral')

            # Add value labels
            for bar in rel_bars:
                height = bar.get_height()
                ax2.annotate(f'{height:.2f}x',
                            xy=(bar.get_x() + bar.get_width() / 2, height),
                            xytext=(0, 3),  # 3 points vertical offset
                            textcoords="offset points",
                            ha='center', va='bottom')

            # Add chart styling
            ax2.set_title('Relative Gas Cost Comparison', fontsize=14, pad=20)
            ax2.set_xlabel('Variant', fontsize=12)
            ax2.set_ylabel('Relative Cost (1.0 = Cheapest)', fontsize=12)
            ax2.grid(axis='y', linestyle='--', alpha=0.7)

            # Adjust layout
            plt.tight_layout()

            # Display the chart
            plt.show()

        return True

    except Exception as viz_err:
        console.print(f"[warning]Could not create gas comparison chart: {viz_err}", style="warning")
        import traceback
        console.print(f"[dim]{traceback.format_exc()}[/dim]")
        return False

def create_batch_gas_chart(results, total_gas):
    """Create a matplotlib bar chart for batch operations gas usage.

    Args:
        results: List of simulation results
        total_gas: Total gas used by the batch

    Returns:
        Success status boolean
    """
    try:
        if len(results) <= 1:
            return False

        # Prepare data
        gas_data = []
        for i, result in enumerate(results):
            op_name = f"{i+1}. {result['function']}"
            gas_data.append({
                "operation": op_name,
                "gas_used": result['gas_used']
            })

        # Create dataframe
        gas_df = pd.DataFrame(gas_data)

        # Print header
        console.print("\n\n[bold]⛽ Batch Operations Gas Usage:[/bold]", style="magenta3")
        console.print("────────────────────────────", style="magenta3")

        # Create figure and axis
        fig, ax = plt.subplots(figsize=(12, 6))

        # Plot the bars
        bars = ax.bar(gas_df['operation'], gas_df['gas_used'], color='steelblue')

        # Add value labels
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{int(height):,}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),
                        textcoords="offset points",
                        ha='center', va='bottom')

        # Add total gas line
        ax.axhline(y=total_gas, color='r', linestyle='--', linewidth=2)

        # Add total gas annotation
        ax.annotate(f'Total Gas: {total_gas:,}',
                    xy=(len(results)/2 - 0.5, total_gas),
                    xytext=(0, 10),
                    textcoords="offset points",
                    ha='center', va='bottom',
                    color='red', fontsize=12,
                    bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="red", alpha=0.8))

        # Add chart styling
        ax.set_title('Batch Operations Gas Usage', fontsize=14, pad=20)
        ax.set_xlabel('Operation', fontsize=12)
        ax.set_ylabel('Gas Used', fontsize=12)
        ax.grid(axis='y', linestyle='--', alpha=0.7)

        # Rotate x-axis labels for better readability
        plt.xticks(rotation=45, ha='right')

        # Adjust layout
        plt.tight_layout()

        # Display the chart
        plt.show()

        return True

    except Exception as viz_err:
        console.print(f"[warning]Could not create batch gas chart: {viz_err}", style="warning")
        import traceback
        console.print(f"[dim]{traceback.format_exc()}[/dim]")
        return False

# =============================================================================================
# 📋 Export Functions
# =============================================================================================

def download_csv_direct(df, filename=None):
    """Creates a direct download for CSV without intermediate display."""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"pyusd_simulation_{timestamp}.csv"

    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()

    # Create direct download HTML
    html = f'''
    <script>
    function download(filename, data) {{
        const a = document.createElement('a');
        a.href = data;
        a.download = filename;
        document.body.appendChild(a);
        a.click();
        document.body.removeChild(a);
    }}

    download("{filename}", "data:text/csv;base64,{payload}");
    </script>
    <div>Downloading {filename}...</div>
    '''
    return HTML(html)

def download_json_direct(data, filename=None):
    """Creates a direct download for JSON without intermediate display."""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"pyusd_simulation_{timestamp}.json"

    # Custom JSON encoder that handles NumPy types
    class NumpyEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, np.ndarray):
                return obj.tolist()
            if isinstance(obj, np.integer):
                return int(obj)
            if isinstance(obj, np.floating):
                return float(obj)
            if isinstance(obj, (np.bool_)):
                return bool(obj)
            return super().default(obj)

    # Convert to JSON string with custom encoder
    json_str = json.dumps(data, default=str, indent=2, cls=NumpyEncoder)
    b64 = base64.b64encode(json_str.encode()).decode()

    # Create direct download HTML
    html = f'''
    <script>
    function download(filename, data) {{
        const a = document.createElement('a');
        a.href = data;
        a.download = filename;
        document.body.appendChild(a);
        a.click();
        document.body.removeChild(a);
    }}

    download("{filename}", "data:application/json;base64,{b64}");
    </script>
    <div>Downloading {filename}...</div>
    '''
    return HTML(html)

def export_to_google_sheets(df, data_dict, simulation_title):
    """Export simulation data to Google Sheets with rich formatting."""
    # Show loading message
    console.print("[info]Exporting to Google Sheets...", style="info")

    try:
        # Create a new Google Sheet with meaningful title
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        sheet_title = f"PYUSD Simulation {simulation_title} {timestamp}"

        # Use the global gc_sheets client that's already authenticated
        spreadsheet = gc_sheets.create(sheet_title)

        # Get the default worksheet and rename it
        worksheet = spreadsheet.get_worksheet(0)
        worksheet.update_title("Simulation Results")

        # Set up a header with transaction info
        header_values = [
            ["PYUSD Transaction Simulation Analysis"],
            [f"Simulation: {simulation_title}"],
            [f"Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"],
            [""],  # Empty row for spacing
        ]
        worksheet.update("A1", header_values)

        # Format the header with bold text and colored background
        worksheet.format("A1:A1", {
            "textFormat": {"bold": True, "fontSize": 14},
            "backgroundColor": {"red": 0.9, "green": 0.9, "blue": 1.0}
        })

        worksheet.format("A2:A3", {
            "textFormat": {"bold": True, "fontSize": 12}
        })

        current_row = 5  # Start after header

        # Add transaction stats summary
        if "summary" in data_dict:
            stats = data_dict["summary"]

            # Add section title
            worksheet.update(f"A{current_row}", [["Analysis Summary"]])
            worksheet.format(f"A{current_row}:A{current_row}", {
                "textFormat": {"bold": True, "fontSize": 12},
                "backgroundColor": {"red": 0.8, "green": 0.9, "blue": 1.0}
            })
            current_row += 1

            # Add stats data
            stats_rows = []
            stats_rows.append(["Metric", "Value"])  # Header row
            for key, value in stats.items():
                # Format keys and values appropriately
                formatted_key = key.replace("_", " ").title()

                # Try to format numerical values with commas
                try:
                    if isinstance(value, (int, float)):
                        formatted_value = f"{value:,}"
                    else:
                        formatted_value = str(value)
                except:
                    formatted_value = str(value)

                stats_rows.append([formatted_key, formatted_value])

            # Add stats table
            stats_start_row = current_row
            worksheet.update(f"A{stats_start_row}", stats_rows)

            # Format stats table header
            worksheet.format(f"A{stats_start_row}:B{stats_start_row}", {
                "textFormat": {"bold": True},
                "backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}
            })

            current_row += len(stats_rows) + 2  # Add extra space after table

        # Add gas analysis reference
        worksheet.update(f"A{current_row}", [["Gas Usage Analysis"]])
        worksheet.format(f"A{current_row}:A{current_row}", {
            "textFormat": {"bold": True, "fontSize": 12},
            "backgroundColor": {"red": 1.0, "green": 0.9, "blue": 0.7}
        })
        current_row += 1

        worksheet.update(f"A{current_row}", [["📊 Gas usage visualizations are available in the notebook"]])
        current_row += 2

        # Add main DataFrame data
        if not df.empty:
            # Add a section title
            worksheet.update(f"A{current_row}", [["Simulation Data"]])
            worksheet.format(f"A{current_row}:A{current_row}", {
                "textFormat": {"bold": True, "fontSize": 12},
                "backgroundColor": {"red": 0.8, "green": 0.8, "blue": 1.0}
            })
            current_row += 1

            # For simulation data, select most important columns for readability
            if len(df.columns) > 10:
                key_cols = ["variant", "function", "from", "to", "amount", "gas_used", "status", "error"]
                display_cols = [col for col in key_cols if col in df.columns]

                # Add any custom columns that might contain analysis results
                other_important_cols = []
                for col in df.columns:
                    if col not in display_cols and any(x in col.lower() for x in ["pyusd", "token", "note", "category"]):
                        other_important_cols.append(col)

                display_cols.extend(other_important_cols)
                display_df = df[display_cols]
            else:
                display_df = df

            # Convert DataFrame to list of lists for the worksheet
            df_values = [display_df.columns.tolist()] + display_df.values.tolist()

            # Format values for better readability
            for i in range(1, len(df_values)):
                for j, col in enumerate(display_df.columns):
                    val = df_values[i][j]

                    # Format different column types appropriately
                    if pd.isnull(val):
                        df_values[i][j] = ""
                    elif col in ["gas_used", "amount"] and isinstance(val, (int, float)):
                        df_values[i][j] = f"{val:,}"
                    else:
                        df_values[i][j] = str(val)

            worksheet.update(f"A{current_row}", df_values)

            # Format the DataFrame header
            worksheet.format(f"A{current_row}:{chr(65+len(display_df.columns)-1)}{current_row}", {
                "textFormat": {"bold": True},
                "backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}
            })

            # Add alternating row colors for readability
            data_rows = len(df_values)
            for i in range(2, data_rows + 1, 2):
                row_num = current_row + i - 1
                worksheet.format(f"A{row_num}:{chr(65+len(display_df.columns)-1)}{row_num}", {
                    "backgroundColor": {"red": 0.97, "green": 0.97, "blue": 1.0}
                })

        # Try to auto-resize columns for better readability
        try:
            worksheet.columns_auto_resize(0, 10)  # Resize first 10 columns
        except:
            pass  # Ignore if not supported

        # Clear loading message and show success message
        clear_output()
        console.print("✓ Successfully exported to Google Sheets", style="success")

        # Open the spreadsheet in a new tab
        spreadsheet_url = f"https://docs.google.com/spreadsheets/d/{spreadsheet.id}"
        html = f'''
        <script>
        window.open("{spreadsheet_url}", "_blank");
        </script>
        <div>Spreadsheet created and opened: <a href="{spreadsheet_url}" target="_blank">{sheet_title}</a></div>
        '''
        return HTML(html)

    except Exception as e:
        # Clear loading message and show error
        clear_output()
        console.print(f"❌ Error creating Google Sheet: {str(e)}", style="error")
        return HTML(f"<div style='color:red'>Error creating Google Sheet: {str(e)}</div>")

def create_comparison_export(results, function_name, from_addr):
    """Creates export options for comparison results."""
    # Create a DataFrame from results
    comp_df = pd.DataFrame(results)

    # Format params for better readability
    comp_df['formatted_params'] = comp_df['params'].apply(lambda x: ', '.join([str(p) for p in x]))

    # Create export output area
    export_output = widgets.Output()

    # Create export buttons with styling
    export_buttons = widgets.HBox([
        widgets.Button(
            description='Export to CSV',
            button_style='primary',
            layout=widgets.Layout(width='180px')
        ),
        widgets.Button(
            description='Export as JSON',
            button_style='warning',
            layout=widgets.Layout(width='180px')
        ),
        widgets.Button(
            description='Export to Google Sheets',
            button_style='info',
            layout=widgets.Layout(width='200px')
        )
    ])

    # Define export handlers
    def export_csv(b):
        with export_output:
            clear_output()
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"pyusd_comparison_{function_name}_{timestamp}.csv"
            # Create a clean export DataFrame
            export_cols = ['variant', 'formatted_params', 'success', 'hypothetical_success', 'gas_used', 'gas_category', 'error']
            export_df = comp_df[export_cols]
            # Rename columns for clarity
            export_df.columns = ['Variant', 'Parameters', 'Success', 'Hypothetical Success', 'Gas Used', 'Gas Profile', 'Error']
            display(download_csv_direct(export_df, filename))

    def export_json(b):
        with export_output:
            clear_output()
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"pyusd_comparison_{function_name}_{timestamp}.json"

            # Create a clean export data structure
            export_data = {
                "comparison_type": "PYUSD Transaction Comparison",
                "function": function_name,
                "from_address": from_addr,
                "timestamp": datetime.now().isoformat(),
                "variants": []
            }

            # Format each variant result
            for result in results:
                variant_data = {
                    "variant": result["variant"],
                    "parameters": [str(p) for p in result["params"]],
                    "success": result["success"],
                    "hypothetical_success": result.get("hypothetical_success", False),
                    "gas_used": result["gas_used"],
                    "gas_category": result.get("gas_category", "Unknown")
                }

                if result["error"]:
                    variant_data["error"] = result["error"]

                export_data["variants"].append(variant_data)

            display(download_json_direct(export_data, filename))

    def export_to_sheets(b):
        with export_output:
            clear_output()
            try:
                # Create a clean export DataFrame
                export_cols = ['variant', 'formatted_params', 'success', 'hypothetical_success', 'gas_used', 'gas_category', 'error']
                export_df = comp_df[export_cols]
                # Rename columns for clarity
                export_df.columns = ['Variant', 'Parameters', 'Success', 'Hypothetical Success', 'Gas Used', 'Gas Profile', 'Error']

                # Create summary data
                summary_data = {
                    "summary": {
                        "function": function_name,
                        "from_address": from_addr,
                        "timestamp": datetime.now().isoformat(),
                        "variants_compared": len(results),
                        "successful_variants": sum(1 for r in results if r["success"] or r.get("hypothetical_success", False))
                    }
                }

                display(export_to_google_sheets(export_df, summary_data, f"Comparison {function_name}()"))
            except Exception as e:
                console.print(f"[error]Error exporting to Google Sheets: {e}", style="error")
                # Fallback to CSV export
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"pyusd_comparison_{function_name}_{timestamp}.csv"
                export_cols = ['variant', 'formatted_params', 'success', 'hypothetical_success', 'gas_used', 'gas_category', 'error']
                export_df = comp_df[export_cols]
                export_df.columns = ['Variant', 'Parameters', 'Success', 'Hypothetical Success', 'Gas Used', 'Gas Profile', 'Error']
                display(download_csv_direct(export_df, filename))
                display(HTML(f"<div style='color:orange'>Falling back to CSV download due to Google Sheets error: {str(e)}</div>"))

    # Connect handlers to buttons
    export_buttons.children[0].on_click(export_csv)
    export_buttons.children[1].on_click(export_json)
    export_buttons.children[2].on_click(export_to_sheets)

    # Display export section
    console.print("\n\n[bold]📤 Export Options:[/bold]", style="cyan3")
    console.print("─────────────────", style="cyan3")

    display(export_buttons)
    display(export_output)

def create_batch_export(results, from_addr):
    """Creates export options for batch simulation results."""
    # Create a DataFrame from results
    batch_data = []

    for i, result in enumerate(results):
        # Create a row for each operation
        batch_data.append({
            "operation": i+1,
            "function": result['function'],
            "params": str(result['params']),
            "status": "Success" if result['success'] else "Hypothetical" if result.get('hypothetical_success', False) else "Failed",
            "gas_used": result['gas_used'],
            "gas_category": result.get('gas_category', 'Unknown'),
            "error": result['error'] if result['error'] else "None"
        })

    batch_df = pd.DataFrame(batch_data)

    # Create export output area
    export_output = widgets.Output()

    # Create export buttons with styling
    export_buttons = widgets.HBox([
        widgets.Button(
            description='Export Batch to CSV',
            button_style='primary',
            layout=widgets.Layout(width='180px')
        ),
        widgets.Button(
            description='Export Batch as JSON',
            button_style='warning',
            layout=widgets.Layout(width='180px')
        ),
        widgets.Button(
            description='Export to Google Sheets',
            button_style='info',
            layout=widgets.Layout(width='190px')
        )
    ])

    # Define export handlers
    def export_csv(b):
        with export_output:
            clear_output()
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"pyusd_batch_simulation_{timestamp}.csv"
            display(download_csv_direct(batch_df, filename))

    def export_json(b):
        with export_output:
            clear_output()
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"pyusd_batch_simulation_{timestamp}.json"

            # Calculate batch statistics
            batch_size = len(results)
            successful_ops = sum(1 for r in results if r['success'] or r.get('hypothetical_success', False))
            total_gas = sum(r['gas_used'] for r in results)

            # Create a clean export data structure
            export_data = {
                "simulation_type": "PYUSD Batch Transaction",
                "from_address": from_addr,
                "timestamp": datetime.now().isoformat(),
                "summary": {
                    "batch_size": batch_size,
                    "successful_operations": successful_ops,
                    "success_rate": f"{(successful_ops/batch_size)*100:.1f}%",
                    "total_gas": total_gas
                },
                "operations": []
            }

            # Format each operation result
            for i, result in enumerate(results):
                op_data = {
                    "index": i+1,
                    "function": result["function"],
                    "parameters": [str(p) for p in result["params"]],
                    "success": result["success"],
                    "hypothetical_success": result.get("hypothetical_success", False),
                    "gas_used": result["gas_used"],
                    "gas_category": result.get("gas_category", "Unknown")
                }

                if result["error"]:
                    op_data["error"] = result["error"]

                export_data["operations"].append(op_data)

            display(download_json_direct(export_data, filename))

    def export_to_sheets(b):
        with export_output:
            clear_output()
            try:
                # Calculate batch statistics
                batch_size = len(results)
                successful_ops = sum(1 for r in results if r['success'] or r.get('hypothetical_success', False))
                total_gas = sum(r['gas_used'] for r in results)

                # Prepare export data
                simulation_title = f"Batch {batch_size} operations"

                # Create summary data
                summary_data = {
                    "summary": {
                        "from_address": from_addr,
                        "timestamp": datetime.now().isoformat(),
                        "batch_size": batch_size,
                        "successful_operations": successful_ops,
                        "success_rate": f"{(successful_ops/batch_size)*100:.1f}%",
                        "total_gas": total_gas
                    }
                }

                display(export_to_google_sheets(batch_df, summary_data, simulation_title))
            except Exception as e:
                console.print(f"[error]Error exporting to Google Sheets: {e}", style="error")
                # Fallback to CSV export
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"pyusd_batch_simulation_{timestamp}.csv"
                display(download_csv_direct(batch_df, filename))
                display(HTML(f"<div style='color:orange'>Falling back to CSV download due to Google Sheets error: {str(e)}</div>"))

    # Connect handlers to buttons
    export_buttons.children[0].on_click(export_csv)
    export_buttons.children[1].on_click(export_json)
    export_buttons.children[2].on_click(export_to_sheets)

    # Display export section
    console.print("\n\n[bold]📤 Batch Export Options:[/bold]", style="cyan3")
    console.print("───────────────────────", style="cyan3")
    display(export_buttons)
    display(export_output)

# =============================================================================================
# 📋 Simulation Helpers
# =============================================================================================

def decode_erc20_error(error_code):
    """Decodes common ERC-20 error codes into human-readable messages."""
    known_errors = {
        # Common ERC-20 error selectors
        "0x08c379a0": "Error string",
        "0x356680b7": "ERC20: transfer amount exceeds balance",
        "0x4e487b71": "Panic/Arithmetic error",
        "0x01336cea": "ERC20: transfer from the zero address",
        "0xbbc67f8f": "ERC20: transfer to the zero address",
        "0x7939f424": "ERC20: approve from the zero address",
        "0xd505accf": "ERC20: permit expired",
        "0xdab70cb7": "ERC20: insufficient allowance",
        "0xd1bebf0c": "ERC20: transfer to the zero address",
        "0x8baa579f": "ERC20: invalid signature",
        "0x0827a183": "ERC20Permit: expired deadline",
        "0x8f4eb604": "ERC20Permit: invalid signature",
        "0x3b8da488": "AccessControl: account is missing role",
        "0x219f5d17": "Token operation is paused",
        "0xf0019fe6": "Address is blacklisted",
        "0x1bb2a6b6": "ERC20: cannot approve from the zero address",
        "0x710086b0": "ERC20: cannot approve to the zero address"
    }

    if error_code in known_errors:
        return known_errors[error_code]
    return f"Unknown error code: {error_code}"

def categorize_gas_usage(function_name, gas_used):
    """Categorizes gas usage based on function type for better analysis."""
    # Define gas usage categories
    gas_categories = {
        "Basic Transfer": ["transfer"],
        "Authorization": ["approve", "increaseAllowance", "decreaseAllowance", "permit", "transferWithAuthorization"],
        "Advanced Transfer": ["transferFrom"],
        "Supply Management": ["mint", "burn"],
        "Administrative": ["pause", "unpause", "transferOwnership", "renounceOwnership"],
        "Query": ["balanceOf", "allowance", "totalSupply", "decimals", "name", "symbol", "paused", "owner"]
    }

    for category, functions in gas_categories.items():
        if function_name in functions:
            # Add context based on gas amount
            if gas_used > 80000:
                return f"{category} (High Gas)"
            elif gas_used > 40000:
                return f"{category} (Medium Gas)"
            else:
                return f"{category} (Efficient)"

    return "Other Operation"

def make_rpc_request(method, params, network='mainnet'):
    """Helper function to make raw RPC requests via the specified network's provider."""
    w3_client = w3_clients.get(network)
    if not w3_client or not w3_client.is_connected():
        console.print(f"[error]Web3 client for '{network}' not available or not connected.", style="error")
        return None
    try:
        response = w3_client.provider.make_request(method, params)
        if 'error' in response:
            console.print(f"[error]RPC Error ({method} on {network}): {response['error']['message']} (Code: {response['error']['code']})", style="error")
            # console.print(response['error']) # Uncomment for full error details
            return None
        return response.get('result') # Return None if 'result' key is missing
    except Exception as e:
        console.print(f"[error]Exception during RPC call ({method} on {network}): {str(e)}", style="error")
        return None

# =============================================================================================
# 🔄 Core Simulation Functions
# =============================================================================================

def create_pyusd_call_data(function_name, *params):
    """Creates encoded call data for PYUSD contract functions."""
    # Use PYUSD_SIGNATURES from the configuration cell
    signatures = {}
    for selector, sig_info in PYUSD_SIGNATURES.items():
        name = sig_info["name"].split("(")[0]  # Extract function name
        param_types = []
        param_section = sig_info["name"].split("(")[1].rstrip(")")
        if param_section:
            param_types = param_section.split(",")
        signatures[name] = {
            'selector': selector[2:] if selector.startswith("0x") else selector,
            'params': param_types
        }

    if function_name not in signatures:
        raise ValueError(f"Unsupported function: {function_name}")

    sig = signatures[function_name]
    if len(params) != len(sig['params']):
        raise ValueError(f"Expected {len(sig['params'])} parameters for {function_name}, got {len(params)}")

    # Start with function selector
    call_data = sig['selector']

    # Add encoded parameters
    for i, param_type in enumerate(sig['params']):
        param_value = params[i]

        if param_type == 'address':
            # Ensure it's a valid address
            addr = Web3.to_checksum_address(param_value)
            # Remove 0x prefix and pad to 32 bytes
            encoded_param = addr[2:].lower().zfill(64)
            call_data += encoded_param

        elif param_type == 'uint256':
            # Convert to raw integer value
            if isinstance(param_value, float) and function_name in ['transfer', 'approve', 'transferFrom', 'mint', 'burn']:
                # For amounts, convert from PYUSD to raw value
                decimals = PYUSD_CONFIG['ethereum']['decimals']
                raw_value = int(param_value * (10**decimals))
            else:
                raw_value = int(param_value)

            # Convert to hex without 0x prefix and pad to 32 bytes
            encoded_param = hex(raw_value)[2:].zfill(64)
            call_data += encoded_param

        elif param_type == 'bool':
            # Encode boolean as 0 or 1 padded to 32 bytes
            encoded_param = (1 if param_value else 0)
            encoded_param = hex(encoded_param)[2:].zfill(64)
            call_data += encoded_param

        elif param_type == 'bytes32':
            # Ensure bytes32 is properly formatted
            if isinstance(param_value, str):
                if param_value.startswith('0x'):
                    param_value = param_value[2:]
                encoded_param = param_value.zfill(64)
            else:
                encoded_param = hex(param_value)[2:].zfill(64)
            call_data += encoded_param

    return '0x' + call_data

def estimate_pyusd_balance(address, block="latest", network='mainnet'):
    """Estimates PYUSD balance for an address using eth.call."""
    w3_client = w3_clients.get(network)
    if not w3_client or not w3_client.is_connected():
        console.print(f"[error]Web3 client for '{network}' not available or not connected.", style="error")
        return None

    try:
        address = Web3.to_checksum_address(address)
        pyusd_address = PYUSD_CONFIG['ethereum']['address']

        # Create call data for balanceOf(address)
        call_data = create_pyusd_call_data('balanceOf', address)

        # Prepare call parameters
        call_params = {
            "to": pyusd_address,
            "data": call_data
        }

        # Execute call
        result = w3_client.eth.call(call_params, block_identifier=block)

        if result:
            # Decode uint256 result
            balance_raw = int(result.hex(), 16)
            balance_pyusd = balance_raw / (10**PYUSD_CONFIG['ethereum']['decimals'])
            return balance_pyusd

        return 0
    except Exception as e:
        console.print(f"[warning]Error estimating PYUSD balance for {address}: {e}", style="warning")
        return None

def simulate_pyusd_transaction(function_name, from_addr, *params, gas_limit=None, gas_price=None, value=0, block="latest", use_trace=True, network='mainnet'):
    """Simulates PYUSD transactions with an efficient hybrid approach."""
    w3_client = w3_clients.get(network)
    if not w3_client or not w3_client.is_connected():
        console.print(f"[error]Web3 client for '{network}' not available or not connected.", style="error")
        return None, None

    # Track operation category for better analysis
    operation_category = None
    for category, functions in GAS_CATEGORIES.items():
        if function_name in functions:
            operation_category = category
            break

    if not operation_category:
        operation_category = "other"

    try:
        # Validate addresses and create call data
        from_checksum_addr = Web3.to_checksum_address(from_addr)
        pyusd_checksum_address = PYUSD_CONFIG['ethereum']['address']
        call_data = create_pyusd_call_data(function_name, *params)
    except ValueError as e:
         console.print(f"[error]Invalid parameters for simulation: {e}", style="error")
         return None, None

    # Create a panel for transaction information
    tx_panel_content = f"[bold cyan3]Function:[/bold cyan3] [white]{function_name}()[/white]\n"
    tx_panel_content += f"[bold cyan3]From:[/bold cyan3] [white]{from_checksum_addr}[/white]\n"
    tx_panel_content += f"[bold cyan3]To:[/bold cyan3] [white]{pyusd_checksum_address}[/white]\n"
    tx_panel_content += f"[bold cyan3]Network:[/bold cyan3] [white]{network.capitalize()}[/white]"

    console.print(Panel(
        tx_panel_content,
        title="[bold]🔄 Simulating PYUSD Transaction[/bold]",
        border_style="cyan3",
        box=box.ROUNDED
    ))

    # Check PYUSD balance for relevant operations
    if function_name in ['transfer', 'transferFrom', 'burn', 'transferWithAuthorization', 'burnFrom']:
        # For transfer and burn, check sender's balance
        balance = None
        check_address = from_addr

        # For transferFrom and burnFrom, check the owner's balance (first param)
        if function_name in ['transferFrom', 'burnFrom']:
            check_address = params[0]

        balance = estimate_pyusd_balance(check_address, block=block, network=network)

        if balance is not None:
            # Get amount from parameters based on function
            amount_index = 1 if function_name in ['transfer', 'approve', 'transferWithAuthorization'] else 2 if function_name == 'transferFrom' else 0
            if amount_index < len(params):
                amount = params[amount_index]
                if isinstance(amount, float):
                    if balance < amount:
                        console.print(Panel(
                            f"Address [bold]{shorten_address(check_address)}[/bold] has [bold red]insufficient[/bold red] PYUSD balance ([bold]{balance:.6f}[/bold]) for this operation ([bold]{amount:.6f}[/bold]).",
                            border_style="yellow",
                            box=box.ROUNDED
                        ))
                    else:
                        console.print(Panel(
                            f"Address [bold]{shorten_address(check_address)}[/bold] has [bold green]sufficient[/bold green] PYUSD balance ([bold]{balance:.6f}[/bold]) for this operation ([bold]{amount:.6f}[/bold]).",
                            border_style="green",
                            box=box.ROUNDED
                        ))

    # Prepare transaction parameters
    call_params = {
        "from": from_checksum_addr,
        "to": pyusd_checksum_address,
        "data": call_data,
    }

    # Add optional parameters if provided
    if value > 0:
        call_params["value"] = hex(value)
    if gas_limit:
        call_params["gas"] = hex(gas_limit)
    if gas_price:
        call_params["gasPrice"] = hex(gas_price)

    # STEP 1: Always try eth.call first - guaranteed to work
    console.print("\n\n[info]Running basic transaction simulation...", style="info")
    try:
        result = w3_client.eth.call(call_params, block_identifier=block)
        success = True
        output = result.hex() if result else '0x'
    except Exception as e:
        success = False
        error_msg = str(e)
        console.print(f"[warning]Transaction simulation revealed error: {error_msg}", style="warning")
        output = None

    # STEP 2: Try to estimate gas if call succeeded, or even if it failed due to balance
    gas_used = 0
    hypothetical_success = False

    if success or (not success and error_msg == "0x356680b7"):  # If successful or just a balance issue
        try:
            # Try to estimate gas usage
            console.print("[info]Estimating gas usage...", style="info")
            gas_estimated = w3_client.eth.estimate_gas({
                **call_params,
                "value": 0  # Ensure no ETH is sent
            })
            gas_used = gas_estimated

            if not success and error_msg == "0x356680b7":
                hypothetical_success = True
                console.print("[info]Transaction would likely succeed with sufficient balance.", style="info")
        except Exception as gas_err:
            # Couldn't estimate gas
            console.print(f"[warning]Could not estimate gas: {gas_err}", style="warning")

    # STEP 3: Advanced trace analysis if requested
    trace_result = None
    if use_trace and (success or hypothetical_success):
        console.print("[info]Attempting detailed transaction trace analysis...", style="info")

        # Configure trace parameters based on mode
        tracer_config = TRACE_CONFIGS["callTracer"]

        # Try multiple trace call parameter formats to maximize compatibility
        trace_formats = [
            # Format 1: Simple params array
            [call_params, block],

            # Format 2: With tracer specified
            [call_params, block, "callTracer"],

            # Format 3: Full tracer config
            [call_params, block, {"tracer": "callTracer", "tracerConfig": tracer_config}],

            # Format 4: Object format for Google RPC
            {
                "transaction": call_params,
                "blockNumber": block if block != "latest" else "latest",
                "tracer": "callTracer"
            }
        ]

        for i, trace_params in enumerate(trace_formats):
            try:
                console.print(f"[info]Trying trace format {i+1}...", style="info")
                trace_result = make_rpc_request("debug_traceCall", trace_params, network=network)
                if trace_result:
                    console.print("[success]Successfully obtained transaction trace.", style="success")
                    break
            except Exception as e:
                console.print(f"[warning]Trace format {i+1} failed: {e}", style="warning")

        if not trace_result:
            # Try alternate RPC method as last resort
            try:
                console.print("[info]Trying alternate tracing method...", style="info")
                trace_result = make_rpc_request("trace_call", trace_formats[0], network=network)
                if trace_result:
                    console.print("[success]Successfully obtained transaction trace using alternate method.", style="success")
            except Exception as e:
                console.print(f"[warning]All trace methods failed: {e}", style="warning")

    # Create analysis from the available data
    analysis = {
        'function': function_name,
        'params': params,
        'success': success,
        'hypothetical_success': hypothetical_success,
        'gas_used': gas_used,
        'operation_category': operation_category,
        'gas_category': categorize_gas_usage(function_name, gas_used),
        'error': None,
        'state_changes': [],
        'output': output,
        'calls': [],
        'timestamp': datetime.now().isoformat()
    }

    # Handle error decoding
    if not success:
        # Decode error if it looks like a hex code
        if isinstance(error_msg, str) and error_msg.startswith("0x"):
            decoded_error = decode_erc20_error(error_msg)
            analysis['error'] = decoded_error

            # Special handling for balance errors
            if error_msg == "0x356680b7":
                analysis['note'] = "This transaction would succeed if the address had sufficient PYUSD balance."
        else:
            analysis['error'] = error_msg

    # Add trace data if available
    if trace_result:
        # Extract relevant data from trace if available
        if isinstance(trace_result, dict):
            # Extract gas usage from trace if available
            if 'gasUsed' in trace_result:
                gas_used_value = trace_result['gasUsed']
                analysis['gas_used'] = int(gas_used_value, 16) if isinstance(gas_used_value, str) and gas_used_value.startswith('0x') else int(gas_used_value)

            # Extract internal calls
            if 'calls' in trace_result and isinstance(trace_result['calls'], list):
                analysis['calls'] = trace_result['calls']

                # Extract state changes from internal calls
                for call in trace_result['calls']:
                    # Look for events, logs, or storage changes
                    if 'logs' in call and isinstance(call['logs'], list):
                        for log in call['logs']:
                            if 'address' in log and log['address'].lower() == PYUSD_CONFIG['ethereum']['address'].lower():
                                # This is a PYUSD log
                                if 'topics' in log and log['topics']:
                                    # Check if it's a Transfer event
                                    topic0 = log['topics'][0]
                                    if topic0 == PYUSD_CONFIG['ethereum']['transfer_event_topic']:
                                        try:
                                            from_addr = Web3.to_checksum_address('0x' + log['topics'][1][-40:])
                                            to_addr = Web3.to_checksum_address('0x' + log['topics'][2][-40:])
                                            value_raw = int(log['data'], 16)
                                            value_pyusd = value_raw / (10**PYUSD_CONFIG['ethereum']['decimals'])

                                            analysis['state_changes'].append({
                                                'type': 'transfer',
                                                'from': from_addr,
                                                'to': to_addr,
                                                'amount': value_pyusd
                                            })
                                        except Exception as e:
                                            console.print(f"[warning]Could not decode Transfer event: {e}", style="warning")

    # Decode function outputs for view functions
    if success and function_name in ['balanceOf', 'allowance', 'totalSupply', 'decimals']:
        try:
            if output and output != '0x':
                output_raw = int(output, 16)
                if function_name in ['balanceOf', 'allowance', 'totalSupply']:
                    # Convert raw value to PYUSD
                    decimals = PYUSD_CONFIG['ethereum']['decimals']
                    output_value = output_raw / (10**decimals)
                    analysis['decoded_output'] = output_value
                else:
                    analysis['decoded_output'] = output_raw
        except Exception as e:
            analysis['output_error'] = str(e)

    # Display analysis results
    display_simulation_analysis(analysis, function_name, params, from_address=from_checksum_addr)

    # Export simulation data if requested
    if success or hypothetical_success:
        # Create export options with detailed simulation data
        create_comparison_export([analysis], function_name, from_checksum_addr)

    return output, analysis

def display_simulation_analysis(analysis, function_name, params, from_address=None):
    """Displays the results of the simulation analysis in a user-friendly format."""
    # Create a visual divider
    console.print("\n\n[bold]🔍 PYUSD Transaction Simulation Analysis[/bold]", style="cyan3")
    console.print("──────────────────────────────────────────", style="cyan3")

    # Create function information panel
    function_info = []

    # Function name with styling
    function_info.append(f"[bold cyan3]Function:[/bold cyan3] [bold white]{function_name}()[/bold white]")

    # Format parameters based on function
    if function_name == 'transfer':
        to_addr = params[0]
        amount = params[1]
        function_info.append(f"[bold cyan3]From:[/bold cyan3] [white]{from_address}[/white]")
        function_info.append(f"[bold cyan3]To:[/bold cyan3] [white]{to_addr}[/white]")
        function_info.append(f"[bold cyan3]Amount:[/bold cyan3] [white]{amount:,.2f} PYUSD[/white]")
    elif function_name == 'approve':
        spender = params[0]
        amount = params[1]
        function_info.append(f"[bold cyan3]Owner:[/bold cyan3] [white]{from_address}[/white]")
        function_info.append(f"[bold cyan3]Spender:[/bold cyan3] [white]{spender}[/white]")
        function_info.append(f"[bold cyan3]Amount:[/bold cyan3] [white]{amount:,.2f} PYUSD[/white]")
    elif function_name == 'transferFrom':
        from_addr = params[0]
        to_addr = params[1]
        amount = params[2]
        function_info.append(f"[bold cyan3]Spender:[/bold cyan3] [white]{from_address}[/white]")
        function_info.append(f"[bold cyan3]From:[/bold cyan3] [white]{from_addr}[/white]")
        function_info.append(f"[bold cyan3]To:[/bold cyan3] [white]{to_addr}[/white]")
        function_info.append(f"[bold cyan3]Amount:[/bold cyan3] [white]{amount:,.2f} PYUSD[/white]")
    elif function_name == 'balanceOf':
        address = params[0]
        function_info.append(f"[bold cyan3]Address:[/bold cyan3] [white]{address}[/white]")
    elif function_name == 'allowance':
        owner = params[0]
        spender = params[1]
        function_info.append(f"[bold cyan3]Owner:[/bold cyan3] [white]{owner}[/white]")
        function_info.append(f"[bold cyan3]Spender:[/bold cyan3] [white]{spender}[/white]")
    elif function_name == 'mint':
        to_addr = params[0]
        amount = params[1]
        function_info.append(f"[bold cyan3]Minter:[/bold cyan3] [white]{from_address}[/white]")
        function_info.append(f"[bold cyan3]To:[/bold cyan3] [white]{to_addr}[/white]")
        function_info.append(f"[bold cyan3]Amount:[/bold cyan3] [white]{amount:,.2f} PYUSD[/white]")
    elif function_name == 'burn':
        amount = params[0]
        function_info.append(f"[bold cyan3]Burner:[/bold cyan3] [white]{from_address}[/white]")
        function_info.append(f"[bold cyan3]Amount:[/bold cyan3] [white]{amount:,.2f} PYUSD[/white]")
    elif function_name == 'transferWithAuthorization':
        to_addr = params[1]
        amount = params[2]
        function_info.append(f"[bold cyan3]Authorized From:[/bold cyan3] [white]{params[0]}[/white]")
        function_info.append(f"[bold cyan3]To:[/bold cyan3] [white]{to_addr}[/white]")
        function_info.append(f"[bold cyan3]Amount:[/bold cyan3] [white]{amount:,.2f} PYUSD[/white]")
        function_info.append(f"[bold cyan3]Executor:[/bold cyan3] [white]{from_address}[/white]")
    elif function_name == 'permit':
        owner = params[1]
        spender = params[1]
        amount = params[2]
        function_info.append(f"[bold cyan3]Owner:[/bold cyan3] [white]{owner}[/white]")
        function_info.append(f"[bold cyan3]Spender:[/bold cyan3] [white]{spender}[/white]")
        function_info.append(f"[bold cyan3]Amount:[/bold cyan3] [white]{amount:,.2f} PYUSD[/white]")
        function_info.append(f"[bold cyan3]Submitter:[/bold cyan3] [white]{from_address}[/white]")

    # Create transaction result information
    transaction_info = []

    # Status with appropriate coloring
    status_color = "green3" if analysis['success'] else "yellow3" if analysis['hypothetical_success'] else "red3"
    status_text = "Success" if analysis['success'] else "Hypothetical Success" if analysis['hypothetical_success'] else "Failed"
    transaction_info.append(f"[bold cyan3]Status:[/bold cyan3] [{status_color}]{status_text}[/{status_color}]")

    # Operation category
    transaction_info.append(f"[bold cyan3]Category:[/bold cyan3] [white]{analysis['operation_category'].title()}[/white]")

    # Error message if any
    if analysis['error']:
        transaction_info.append(f"[bold cyan3]Error:[/bold cyan3] [red3]{analysis['error']}[/red3]")

    # Additional notes if any
    if 'note' in analysis:
        transaction_info.append(f"[bold cyan3]Note:[/bold cyan3] [yellow3]{analysis['note']}[/yellow3]")

    # Gas usage
    transaction_info.append(f"[bold cyan3]Gas Used:[/bold cyan3] [white]{analysis['gas_used']:,}[/white]")
    transaction_info.append(f"[bold cyan3]Gas Profile:[/bold cyan3] [white]{analysis['gas_category']}[/white]")

    # Output data for view functions
    if 'decoded_output' in analysis:
        if function_name in ['balanceOf', 'allowance', 'totalSupply']:
            transaction_info.append(f"[bold cyan3]Result:[/bold cyan3] [green3]{analysis['decoded_output']:,.6f} PYUSD[/green3]")
        else:
            transaction_info.append(f"[bold cyan3]Result:[/bold cyan3] [green3]{analysis['decoded_output']}[/green3]")
    elif analysis['output']:
        transaction_info.append(f"[bold cyan3]Raw Output:[/bold cyan3] [dim white]{analysis['output']}[/dim white]")

    # Internal Calls Count if available
    if 'calls' in analysis and analysis['calls']:
        transaction_info.append(f"[bold cyan3]Internal Calls:[/bold cyan3] [white]{len(analysis['calls'])}[/white]")

    # Create panels for better visual separation
    function_panel = Panel(
        "\n".join(function_info),
        title="[bold]Transaction Details[/bold]",
        border_style="cyan3",
        box=box.ROUNDED
    )

    result_panel = Panel(
        "\n".join(transaction_info),
        title="[bold]Execution Results[/bold]",
        border_style="cyan3",
        box=box.ROUNDED
    )

    # Display panels side by side if possible, otherwise stacked
    console.print(Group(function_panel, result_panel))

    # Display state changes if any
    if analysis['state_changes']:
        console.print("\n[bold cyan3]State Changes:[/bold cyan3]")

        state_table = Table(show_header=True, header_style="bold cyan3", box=box.ROUNDED)
        state_table.add_column("Type", style="cyan3")
        state_table.add_column("Details", style="white")

        for change in analysis['state_changes']:
            if change['type'] == 'transfer':
                from_addr = shorten_address(change['from'])
                to_addr = shorten_address(change['to'])
                amount = change['amount']
                details = f"{amount:,.2f} PYUSD from {from_addr} to {to_addr}"
                state_table.add_row("Transfer", details)
            else:
                state_table.add_row(change['type'].title(), str(change))

        console.print(Panel(state_table, title="[bold]State Changes[/bold]", border_style="cyan", box=box.ROUNDED))

    # Create transaction flow diagram for transfers
    if from_address and function_name in ['transfer', 'transferFrom', 'mint', 'burn', 'transferWithAuthorization', 'permit'] and (analysis['success'] or analysis['hypothetical_success']):
        try:
            console.print("\n\n[bold magenta3]Transaction Flow Visualization:[/bold magenta3]")
            console.print("────────────────────────────────", style="magenta3")

            flow_graph = Digraph(comment=f"PYUSD {function_name} Flow", format='png')
            flow_graph.attr(rankdir='TB', bgcolor='transparent')
            flow_graph.attr('node', shape='box', style='filled', fontname='helvetica', fontsize='10')

            if function_name == 'transfer':
                sender_addr = from_address
                to_addr = params[0]
                amount = params[1]

                flow_graph.node('sender', f"Sender\n{sender_addr}", fillcolor='lightblue')
                flow_graph.node('receiver', f"Receiver\n{to_addr}", fillcolor='palegreen')
                flow_graph.edge('sender', 'receiver', label=f"{amount:,.2f} PYUSD")

            elif function_name == 'transferFrom':
                owner_addr = params[0]
                to_addr = params[1]
                amount = params[2]
                sender_addr = from_address

                flow_graph.node('owner', f"Owner\n{owner_addr}", fillcolor='lightblue')
                flow_graph.node('spender', f"Spender\n{sender_addr}", fillcolor='lightyellow')
                flow_graph.node('receiver', f"Receiver\n{to_addr}", fillcolor='palegreen')

                flow_graph.edge('spender', 'owner', label="1. Has Allowance")
                flow_graph.edge('owner', 'receiver', label=f"2. {amount:,.2f} PYUSD")

            elif function_name == 'mint':
                to_addr = params[0]
                amount = params[1]

                flow_graph.node('minter', f"Minter\n{shorten_address(from_address)}", fillcolor='lightcoral')
                flow_graph.node('receiver', f"Receiver\n{shorten_address(to_addr)}", fillcolor='palegreen')
                flow_graph.node('supply', "PYUSD Supply", fillcolor='lightcyan')

                flow_graph.edge('minter', 'supply', label=f"1. Increases by {amount:,.2f}")
                flow_graph.edge('supply', 'receiver', label=f"2. {amount:,.2f} PYUSD")

            elif function_name == 'burn':
                amount = params[0]

                flow_graph.node('burner', f"Burner\n{shorten_address(from_address)}", fillcolor='lightcoral')
                flow_graph.node('supply', "PYUSD Supply", fillcolor='lightcyan')

                flow_graph.edge('burner', 'supply', label=f"Burns {amount:,.2f} PYUSD")
                flow_graph.edge('supply', 'null', label=f"Decreases by {amount:,.2f}", style='dashed')

            elif function_name == 'transferWithAuthorization':
                from_addr = params[0]
                to_addr = params[1]
                amount = params[2]

                flow_graph.node('owner', f"Owner\n{from_addr}", fillcolor='lightblue')
                flow_graph.node('executor', f"Executor\n{from_address}", fillcolor='lightyellow')
                flow_graph.node('receiver', f"Receiver\n{to_addr}", fillcolor='palegreen')

                flow_graph.edge('owner', 'executor', label="1. Authorization")
                flow_graph.edge('executor', 'receiver', label=f"2. {amount:,.2f} PYUSD")

            elif function_name == 'permit':
                owner = params[0]
                spender = params[1]
                amount = params[2]

                flow_graph.node('owner', f"Owner\n{owner}", fillcolor='lightblue')
                flow_graph.node('spender', f"Spender\n{spender}", fillcolor='palegreen')
                flow_graph.node('submitter', f"Submitter\n{from_address}", fillcolor='lightyellow')

                flow_graph.edge('owner', 'submitter', label="1. Signed Permit")
                flow_graph.edge('submitter', 'spender', label=f"2. Approve {amount:,.2f} PYUSD")

            display(flow_graph)

        except Exception as viz_err:
            console.print(f"[warning]Could not create flow visualization: {viz_err}", style="warning")

def compare_pyusd_transactions(function_name, from_addr, param_sets, network='mainnet'):
    """Compares multiple PYUSD transaction simulations with different parameters."""
    # Create a visual divider for the comparison section
    console.print("\n\n[bold]🔄 PYUSD Transaction Comparison[/bold]", style="cyan3")
    console.print("───────────────────────────────", style="cyan3")

    # Display function being compared
    console.print(Panel(
        f"[bold white]Function:[/bold white] [cyan3]{function_name}()[/cyan3]\n"
        f"[bold white]From:[/bold white] [cyan3]{from_addr}[/cyan3]\n"
        f"[bold white]Network:[/bold white] [cyan3]{network.capitalize()}[/cyan3]\n"
        f"[bold white]Variants:[/bold white] [cyan3]{len(param_sets)}[/cyan3]",
        title="[bold]Comparison Parameters[/bold]",
        border_style="cyan3",
        box=box.ROUNDED
    ))

    results = []

    for i, params in enumerate(param_sets):
        # Create visual separator between variants
        console.print(f"\n\n[bold cyan3]Variant {i+1}[/bold cyan3]: {params}")
        sim_result, analysis = simulate_pyusd_transaction(function_name, from_addr, *params, network=network)

        if analysis:
            # Store result for comparison
            variant_name = f"Variant {i+1}"
            results.append({
                "variant": variant_name,
                "params": params,
                "success": analysis['success'],
                "hypothetical_success": analysis.get('hypothetical_success', False),
                "gas_used": analysis['gas_used'],
                "gas_category": analysis.get('gas_category', 'Unknown'),
                "error": analysis['error']
            })

    # Create comparison table
    if results:
        # Final comparison header
        console.print("\n\n[bold]📊 Comparison Results[/bold]", style="cyan3")
        console.print("──────────────────────", style="cyan3")

        comp_table = Table(show_header=True, header_style="bold cyan3", box=box.ROUNDED)
        comp_table.add_column("Variant", style="cyan3")

        # Add parameter columns based on function
        if function_name == 'transfer' or function_name == 'approve':
            comp_table.add_column("Recipient/Spender", style="white")
            comp_table.add_column("Amount", style="white")
        elif function_name == 'transferFrom':
            comp_table.add_column("From", style="white")
            comp_table.add_column("To", style="white")
            comp_table.add_column("Amount", style="white")
        elif function_name == 'mint':
            comp_table.add_column("To", style="white")
            comp_table.add_column("Amount", style="white")
        elif function_name == 'burn':
            comp_table.add_column("Amount", style="white")
        elif function_name == 'transferWithAuthorization':
            comp_table.add_column("From", style="white")
            comp_table.add_column("To", style="white")
            comp_table.add_column("Amount", style="white")
        elif function_name == 'permit':
            comp_table.add_column("Owner", style="white")
            comp_table.add_column("Spender", style="white")
            comp_table.add_column("Amount", style="white")

        comp_table.add_column("Status")
        comp_table.add_column("Gas Used", style="white")
        comp_table.add_column("Gas Profile", style="white")

        for result in results:
            row = [result["variant"]]

            # Add parameter values
            params = result["params"]
            if function_name == 'transfer' or function_name == 'approve':
                row.append(params[0])
                row.append(f"{params[1]:,.2f}")
            elif function_name == 'transferFrom':
                row.append(params[0])
                row.append(params[1])
                row.append(f"{params[2]:,.2f}")
            elif function_name == 'mint':
                row.append(params[0])
                row.append(f"{params[1]:,.2f}")
            elif function_name == 'burn':
                row.append(f"{params[0]:,.2f}")
            elif function_name == 'transferWithAuthorization':
                row.append(params[0])
                row.append(params[1])
                row.append(f"{params[2]:,.2f}")
            elif function_name == 'permit':
                row.append(params[0])
                row.append(params[1])
                row.append(f"{params[2]:,.2f}")

            # Add status and gas
            status_color = "green3" if result["success"] else "yellow3" if result["hypothetical_success"] else "red3"
            status_text = "Success" if result["success"] else "Hypothetical Success" if result["hypothetical_success"] else "Failed"
            row.append(f"[{status_color}]{status_text}[/{status_color}]")
            row.append(f"{result['gas_used']:,}")
            row.append(result["gas_category"])

            comp_table.add_row(*row)

        console.print(Panel(comp_table, border_style="cyan3", box=box.ROUNDED))

        # Create gas comparison chart
        create_gas_comparison_chart(results, function_name)

        # Create export options for comparison results
        create_comparison_export(results, function_name, from_addr)

    return results

def batch_simulate_pyusd_transactions(from_addr, batch_operations, network='mainnet'):
    """Simulates a batch of PYUSD transactions in sequence."""
    # Create a visual divider for the batch simulation
    console.print("\n\n[bold]🔄 PYUSD Batch Transaction Simulation[/bold]", style="cyan3")
    console.print("────────────────────────────────────", style="cyan3")

    # Display batch information
    console.print(Panel(
        f"[bold white]From Address:[/bold white] [cyan3]{from_addr}[/cyan3]\n"
        f"[bold white]Network:[/bold white] [cyan3]{network.capitalize()}[/cyan3]\n"
        f"[bold white]Batch Size:[/bold white] [cyan3]{len(batch_operations)}[/cyan3]",
        title="[bold]Batch Parameters[/bold]",
        border_style="cyan3",
        box=box.ROUNDED
    ))

    results = []
    total_gas = 0
    batch_success = True

    # Run each simulation in the batch
    for i, (function_name, params) in enumerate(batch_operations):
        console.print(f"\n\n[bold cyan3]Operation {i+1}/{len(batch_operations)}[/bold cyan3]: {function_name}{params}")

        # Convert params to a proper list if it's not already
        params_list = list(params) if isinstance(params, (list, tuple)) else [params]

        # Simulate this transaction
        sim_result, analysis = simulate_pyusd_transaction(function_name, from_addr, *params_list, network=network)

        # Store result
        results.append(analysis)

        # Update batch statistics
        if analysis['success'] or analysis.get('hypothetical_success', False):
            total_gas += analysis['gas_used']
        else:
            batch_success = False
            # Stop batch if a transaction fails
            console.print(Panel(
                f"[bold red]Batch simulation stopped at operation {i+1} due to failure[/bold red]\n"
                f"Function: {function_name}\n"
                f"Error: {analysis['error']}",
                title="[bold]Batch Failure[/bold]",
                border_style="red",
                box=box.ROUNDED
            ))
            break

    # Display batch summary
    status_color = "green" if batch_success else "red"
    status_text = "Success" if batch_success else "Failed"

    console.print(Panel(
        f"[bold white]Total Operations:[/bold white] [cyan]{len(results)}/{len(batch_operations)}[/cyan]\n"
        f"[bold white]Batch Status:[/bold white] [{status_color}]{status_text}[/{status_color}]\n"
        f"[bold white]Total Gas:[/bold white] [cyan]{total_gas:,}[/cyan]",
        title="[bold]Batch Summary[/bold]",
        border_style="cyan",
        box=box.ROUNDED
    ))

    # Create visualization for batch gas usage
    create_batch_gas_chart(results, total_gas)

    # Create export options for batch results
    create_batch_export(results, from_addr)

    return results

# =============================================================================================
# 🚀 Interactive Demo Section
# =============================================================================================

def run_pyusd_simulations():
    """Run a series of PYUSD transaction simulations with enhanced UI display."""
    # Addresses with PYUSD balance (publicly available on blockchain)
    PYUSD_HOLDER_ADDRESSES = [
        "0xf845a0A05Cbd91Ac15C3E59D126DE5dFbC2aAbb7",  # Primary simulation address
        "0xad6452a9b8F10b0fE084C83c396ABAe96411C761",
        "0x51C72848c68a965f66FA7a88855F9f7784502a7F",
        "0x4bb41165A817628992ee40ea8e92F8800f143FbD",
        "0x3519eE4d387150F230BaCa90f5C50E9d296164c6"
    ]

    # Use first address with balance for simulations
    SIM_FROM_ADDR = PYUSD_HOLDER_ADDRESSES[0]
    SIM_TO_ADDR = "0x5754284f345afc66a98fbB0a0Afe71e0F007B949"  # Example recipient
    SIM_AMOUNT = 100.0  # Amount of PYUSD

    # Create a visually appealing header
    console.print("\n\n[bold]🧪 Advanced PYUSD Transaction Simulator[/bold]", style="chartreuse1")
    console.print("──────────────────────────────────────", style="chartreuse1")

    console.print(Panel(
        "[bold chartreuse1]This Cell simulates PYUSD transactions against the Ethereum blockchain[/bold chartreuse1]\n"
        "[white]Test various ERC-20 operations without executing actual transactions[/white]\n\n"
        "[bold yellow3]Features:[/bold yellow3]\n"
        "• Reliable transaction simulation with advanced tracing\n"
        "• Gas usage analysis and optimization recommendations\n"
        "• Detailed state change tracking\n"
        "• Multi-format data export options\n"
        "• Batch transaction simulation\n"
        "• Support for advanced PYUSD operations",
        title="[bold]Transaction Simulator Information[/bold]",
        subtitle="[dim]Powered by Google Blockchain RPC[/dim]",
        border_style="chartreuse1",
        box=box.ROUNDED,
        padding=(1, 2)
    ))

    # Add disclaimer in a panel
    console.print("\n\n[bold]⚠️ Disclaimer[/bold]", style="yellow3")

    console.print(Panel(
        "• This is a [bold]read-only simulation tool[/bold]. No actual transactions are executed.\n"
        "• The addresses used contain real PYUSD balances found on the public blockchain.\n"
        "• This tool is for [bold]educational and testing purposes only[/bold].\n"
        "• All simulations are performed against blockchain data without modifying any state.\n"
        "• The trace_call API optimizations maximize Google's blockchain RPC capabilities.",
        title="[bold yellow3]DISCLAIMER[/bold yellow3]",
        border_style="yellow3",
        box=box.ROUNDED
    ))

    console.print("\n")  # Add spacing

    # 1. Transfer simulation
    console.print("\n\n[bold chartreuse1]1. PYUSD Transfer Simulation[/bold chartreuse1]")
    console.print("─────────────────────────────", style="chartreuse1")

    sim_result, sim_analysis = simulate_pyusd_transaction(
        'transfer', SIM_FROM_ADDR, SIM_TO_ADDR, SIM_AMOUNT, network='mainnet'
    )

    # 2. PYUSD Balance check simulation
    console.print("\n\n[bold chartreuse1]2. PYUSD Balance Check Simulation[/bold chartreuse1]")
    console.print("──────────────────────────────────", style="chartreuse1")

    sim_result_balance, sim_analysis_balance = simulate_pyusd_transaction(
        'balanceOf', SIM_FROM_ADDR, SIM_FROM_ADDR, network='mainnet'
    )

    # 3. PYUSD Transfer Amount Comparison
    console.print("\n\n[bold chartreuse1]3. PYUSD Transfer Amount Comparison[/bold chartreuse1]")
    console.print("───────────────────────────────────", style="chartreuse1")

    compare_results = compare_pyusd_transactions(
        'transfer',
        SIM_FROM_ADDR,
        [
            (SIM_TO_ADDR, 10000.0),
            (SIM_TO_ADDR, 3000.0),
            (SIM_TO_ADDR, 200.0)
        ],
        network='mainnet'
    )

    # 4. Batch Transaction Simulation
    console.print("\n\n[bold chartreuse1]4. PYUSD Batch Transaction Simulation[/bold chartreuse1]")
    console.print("──────────────────────────────────────", style="chartreuse1")

    batch_operations = [
        ('balanceOf', [SIM_FROM_ADDR]),
        ('transfer', [SIM_TO_ADDR, 50.0]),
        ('approve', [SIM_TO_ADDR, 200.0]),
    ]
    batch_results = batch_simulate_pyusd_transactions(
        SIM_FROM_ADDR,
        batch_operations,
        network='mainnet'
    )

    console.print(Panel(
        "[cyan3]Simulation completed successfully![/cyan3]\n\n"
        "[white]This cell demonstrates advanced PYUSD transaction simulation capabilities:[/white]\n"
        "• [bold]Basic Transfer[/bold]: Simple PYUSD transfer with gas estimation\n"
        "• [bold]Balance Check[/bold]: View operation with result decoding\n"
        "• [bold]Transaction Comparison[/bold]: Gas usage analysis across different amounts\n"
        "• [bold]Batch Operations[/bold]: Multiple operations simulated in sequence\n\n"
        "[white]All results can be exported in CSV, JSON, or Google Sheets formats.[/white]",
        border_style="green3",
        box=box.ROUNDED,
        title="[bold]🎉 Simulation Complete[/bold]"
    ))

# Execute the simulations
run_pyusd_simulations()

## 1.8 📑 `trace_transaction`: Alternative Transaction Trace Analysis

This section explores `trace_transaction`, another RPC method for retrieving an execution trace for an *existing, already mined* transaction (`TARGET_TX_HASH`). While similar in purpose to `debug_traceTransaction`, its output format and the level of detail can differ depending on the node implementation. It often provides a list of "actions" or "sub-traces" representing the top-level call and internal calls.

> **🚀 Leveraging GCP's Premium RPC Capabilities**
>
> *   **Method:** `trace_transaction`
> *   **Multiplier:** `50x` (Available on Mainnet via GCP)
> *   **GCP Advantage:** Like other tracing methods, this can be resource-intensive. GCP ensures reliable retrieval of these traces for analysis. Comparing its output with `debug_traceTransaction` can sometimes reveal different perspectives on the execution.
> *   **PYUSD Insight:** `trace_transaction` offers another way to:
>     *   Visualize the call stack involving PYUSD interactions.
>     *   Extract PYUSD function calls and **parameters** from the trace actions.
>     *   Analyze gas usage breakdowns across the different actions within the trace.
>     *   Identify common PYUSD transaction patterns (simple transfers, swaps, approvals).
>     *   Detect potential MEV activity or security concerns based on the trace structure.

**Analysis Workflow:**

1.  **Fetch Trace:** Calls `trace_transaction` using the `TARGET_TX_HASH`.
2.  **Process Actions:** The `analyze_trace_transaction` function iterates through the list of trace actions.
3.  **Extract & Decode:** For each action, it extracts details like `from`, `to`, `value`, `gasUsed`, `input`, `output`, and `error`. It specifically decodes calls targeting PYUSD contracts.
4.  **Analyze & Aggregate:**
    *   Calculates overall gas usage, errors, and call depth.
    *   Counts PYUSD interactions and categorizes PYUSD function calls.
    *   Identifies PYUSD transfers and calculates volume.
    *   Performs gas efficiency analysis against benchmarks.
    *   Applies heuristics to identify transaction patterns (e.g., swap, simple transfer) and potential MEV/security flags.
5.  **Visualize & Summarize:**
    *   **Enhanced Summary Panel:** Displays aggregated metrics, complexity score, identified pattern, MEV/security flags.
    *   **Security Concerns Table:** Lists any detected high-risk operations.
    *   **PYUSD Function/Category Tables & Plots:** Shows the distribution and gas usage related to PYUSD calls.
    *   **Transfer Table & Visualizations:** Details PYUSD token movements using tables and interactive **Plotly network graphs**.
    *   **Call Tree Visualization:** Generates an interactive **Plotly graph** of the execution path (potentially using Graphviz for layout).
    *   **Gas/Depth Plots:** Visualizes gas usage by contract type and call depth.
    *   **Interactive Replay:** Provides a slider/play button to step through the execution sequence.
    *   **Filtered Data Table:** Displays trace actions with interactive filtering.
    *   **Export Options:** Download detailed trace analysis data.

**💡 What to Look For:**
*   **Transaction Pattern:** What type of PYUSD activity does the analysis classify this as?
*   **Security/MEV Flags:** Are there any warnings related to high-risk functions or potential MEV?
*   **Gas Efficiency:** How does the gas usage for PYUSD calls compare to benchmarks?
*   **Call Tree & Replay:** Follow the execution flow, paying attention to PYUSD interactions and any errors.
*   **Token Flow:** Understand where PYUSD originated and ended up within this transaction.


In [ ]:
# =============================================================================================
# 📑 PYUSD Transaction Analysis with Interactive Visualization
# =============================================================================================
# This cell analyzes Ethereum transactions with a focus on PYUSD tokens, providing:
# - Detailed execution trace analysis with step-by-step replay
# - Visualization of contract interactions, call hierarchies, and token flows
# - Security analysis identifying MEV potential and transaction patterns
# - Gas usage optimization insights and export capabilities

import base64
from datetime import datetime
from IPython.display import HTML, clear_output, display, Javascript
import json
import ipywidgets as widgets
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import networkx as nx
import math
from plotly.subplots import make_subplots
from rich.panel import Panel
from rich.table import Table
import time
import re
from eth_utils import decode_hex, to_hex, to_int

# Transaction pattern definitions for classification
PYUSD_TRANSACTION_PATTERNS = {
    "simple_transfer": {
        "description": "Simple PYUSD transfer between addresses",
        "signatures": ["transfer(address,uint256)"],
        "contracts": 1,
        "calls_min": 1,
        "calls_max": 3
    },
    "swap_operation": {
        "description": "PYUSD swap through DEX",
        "signatures": ["transfer(address,uint256)", "swapExactTokensForTokens", "swapTokensForExactTokens"],
        "contracts_min": 2,
        "has_external_calls": True
    },
    "liquidity_provision": {
        "description": "Adding/removing liquidity with PYUSD",
        "signatures": ["transfer(address,uint256)", "mint", "addLiquidity"],
        "contracts_min": 2
    },
    "bridge_operation": {
        "description": "PYUSD bridge operation (cross-chain)",
        "signatures": ["transfer(address,uint256)", "deposit", "lock"],
        "gas_intensive": True
    },
    "multi_transfer": {
        "description": "Multiple PYUSD transfers in one transaction",
        "signatures": ["transfer(address,uint256)"],
        "min_transfers": 2
    },
    "approval_flow": {
        "description": "PYUSD approval for future spending",
        "signatures": ["approve(address,uint256)"],
        "calls_min": 1,
        "calls_max": 3
    },
    "supply_change": {
        "description": "Minting or burning of PYUSD supply",
        "signatures": ["mint(address,uint256)", "burn(uint256)"],
        "admin_operation": True
    }
}

# MEV patterns to detect
MEV_PATTERNS = {
    "sandwich_attack": {
        "description": "Transaction sandwiched between two related transactions",
        "indicators": ["swap before and after", "price impact"]
    },
    "arbitrage": {
        "description": "Multi-step operation exploiting price differences",
        "indicators": ["multiple DEX interactions", "circular flow"]
    },
    "front_running": {
        "description": "Transaction potentially front-run by a MEV bot",
        "indicators": ["unusual gas price", "similar operation before"]
    }
}

# Security risk levels for different operations
SECURITY_RISK_LEVELS = {
    "transferOwnership(address)": "high",
    "pause()": "medium",
    "unpause()": "medium",
    "blacklist": "medium",
    "upgrade": "high",
    "initialize": "high",
    "selfdestruct": "critical",
    "mint(address,uint256)": "high",
    "burn(uint256)": "medium",
    "renounceOwnership()": "high"
}

# Common gas costs for PYUSD operations for comparison - using full function names
PYUSD_GAS_BENCHMARKS = {
    "transfer(address,uint256)": {
        "median": 65000,
        "p25": 52000,
        "p75": 78000
    },
    "approve(address,uint256)": {
        "median": 46000,
        "p25": 42000,
        "p75": 58000
    },
    "transferFrom(address,address,uint256)": {
        "median": 75000,
        "p25": 65000,
        "p75": 90000
    },
    "mint(address,uint256)": {
        "median": 110000,
        "p25": 95000,
        "p75": 130000
    },
    "burn(uint256)": {
        "median": 90000,
        "p25": 80000,
        "p75": 105000
    }
}

# Export Functions
def download_csv_direct(df, filename=None):
    """Creates a direct download for CSV without intermediate display."""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"pyusd_data_{timestamp}.csv"
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = f'''
    <script>
    function download(filename, data) {{
        const a = document.createElement('a');
        a.href = data;
        a.download = filename;
        document.body.appendChild(a);
        a.click();
        document.body.removeChild(a);
    }}
    download("{filename}", "data:text/csv;base64,{payload}");
    </script>
    <div>Downloading {filename}...</div>
    '''
    return HTML(html)

def download_json_direct(data, filename=None):
    """Creates a direct download for JSON without intermediate display."""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"pyusd_data_{timestamp}.json"
    json_str = json.dumps(data, default=str, indent=2)
    b64 = base64.b64encode(json_str.encode()).decode()
    html = f'''
    <script>
    function download(filename, data) {{
        const a = document.createElement('a');
        a.href = data;
        a.download = filename;
        document.body.appendChild(a);
        a.click();
        document.body.removeChild(a);
    }}
    download("{filename}", "data:application/json;base64,{b64}");
    </script>
    <div>Downloading {filename}...</div>
    '''
    return HTML(html)

def export_to_google_sheets_direct(df, sheet_name=None):
    """Exports DataFrame directly to Google Sheets using authenticated session in Colab."""
    if sheet_name is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        sheet_name = f"PYUSD Analysis {timestamp}"

    try:
        from google.colab import auth
        from googleapiclient.discovery import build
        from googleapiclient.http import MediaInMemoryUpload
        import io
        from google.colab import output

        @output.register_callback('notebook.createSheet')
        def create_sheet_callback(csv_data, name):
            try:
                auth.authenticate_user()
                drive_service = build('drive', 'v3')
                file_metadata = {'name': name, 'mimeType': 'application/vnd.google-apps.spreadsheet'}
                media = MediaInMemoryUpload(
                    io.BytesIO(csv_data.encode('utf-8')),
                    mimetype='text/csv',
                    resumable=True
                )
                file = drive_service.files().create(
                    body=file_metadata,
                    media_body=media,
                    fields='id,webViewLink'
                ).execute()
                return {
                    'status': 'success',
                    'file_id': file.get('id'),
                    'link': file.get('webViewLink')
                }
            except Exception as e:
                return {'status': 'error', 'message': str(e)}

        csv_escaped = df.to_csv(index=False).replace('`', '\\`').replace('$', '\\$')

        html = f'''
        <script src="https://apis.google.com/js/platform.js" async defer></script>
        <script>
        function createSheet() {{
            const csv = `{csv_escaped}`;
            google.colab.kernel.invokeFunction('notebook.createSheet', [csv, '{sheet_name}'], {{}})
                .then(result => {{
                    const outputDiv = document.getElementById('gsheet-output');
                    if(result.data['application/json'].status === 'success') {{
                        outputDiv.innerHTML = '<div>✅ Google Sheet created: <a href="' + result.data['application/json'].link + '" target="_blank">{sheet_name}</a></div>';
                    }} else {{
                        outputDiv.innerHTML = '<div style="color:red">❌ Error creating Google Sheet: ' + result.data['application/json'].message + '</div>';
                    }}
                }})
                .catch(error => {{
                     document.getElementById('gsheet-output').innerHTML = '<div style="color:red">❌ Error invoking Google Sheet creation: ' + error + '</div>';
                }});
        }}
        setTimeout(createSheet, 100);
        </script>
        <div id="gsheet-output">Creating Google Sheet "{sheet_name}"...</div>
        '''
        return HTML(html)

    except ImportError:
         return HTML("<div style='color:red'>Google Colab specific export not available. Please run in Google Colab.</div>")
    except Exception as e:
         return HTML(f"<div style='color:red'>Error setting up Google Sheets export: {e}</div>")


# Plotly Visualization Functions
def create_plotly_contract_interaction_graph(contract_interactions):
    """Creates an interactive Plotly Network graph for contract interactions with directional arrows"""
    if not contract_interactions:
        return None

    G = nx.DiGraph()

    contracts_seen = set()
    for src, dst in contract_interactions:
        if not (isinstance(src, str) and src.startswith('0x') and len(src) == 42): continue
        if not (isinstance(dst, str) and dst.startswith('0x') and len(dst) == 42): continue

        if src not in contracts_seen:
            src_name = PYUSD_CONTRACTS.get(src, "External Contract")
            G.add_node(src, name=src_name, is_pyusd=(src in PYUSD_CONTRACTS))
            contracts_seen.add(src)

        if dst not in contracts_seen:
            dst_name = PYUSD_CONTRACTS.get(dst, "External Contract")
            G.add_node(dst, name=dst_name, is_pyusd=(dst in PYUSD_CONTRACTS))
            contracts_seen.add(dst)

        if src in G and dst in G:
            G.add_edge(src, dst)

    if not G.nodes():
        console.print("[warning]No valid contract interactions found to generate graph.", style="warning")
        return None

    try:
        import pygraphviz as pgv
        pos = nx.nx_agraph.graphviz_layout(G, prog='dot')
    except ImportError:
        console.print("[warning] pygraphviz import failed. Falling back to spring layout for contract interaction graph.", style="warning")
        pos = nx.spring_layout(G, seed=42, k=1.5)
    except Exception as layout_err:
        console.print(f"[warning] Hierarchical layout failed ({layout_err}). Falling back to spring layout.", style="warning")
        pos = nx.spring_layout(G, seed=42, k=1.5)

    edge_traces = []

    for edge in G.edges():
        src, dst = edge
        if src not in pos or dst not in pos: continue

        src_name = G.nodes[src].get('name', 'Unknown')
        dst_name = G.nodes[dst].get('name', 'Unknown')

        x0, y0 = pos[src]
        x1, y1 = pos[dst]

        dx = x1 - x0
        dy = y1 - y0
        length = math.sqrt(dx**2 + dy**2)
        if length > 0: udx, udy = dx / length, dy / length
        else: udx, udy = 0, 0

        arrow_ratio = 0.8
        arrow_x = x0 + arrow_ratio * dx
        arrow_y = y0 + arrow_ratio * dy
        angle = math.degrees(math.atan2(dy, dx))

        edge_trace = go.Scatter(
            x=[x0, x1], y=[y0, y1],
            line=dict(width=1.5, color='rgba(50, 50, 50, 0.8)'),
            hoverinfo='text',
            text=f"From: {src_name}<br>To: {dst_name}<br>From address: {src}<br>To address: {dst}",
            mode='lines', showlegend=False
        )
        arrow_trace = go.Scatter(
            x=[arrow_x], y=[arrow_y], mode='markers',
            marker=dict(symbol='triangle-right', size=12, color='rgba(50, 50, 50, 0.8)', angle=angle),
            hoverinfo='none', showlegend=False
        )
        edge_traces.append(edge_trace)
        edge_traces.append(arrow_trace)

    node_x, node_y, node_colors, node_sizes, hover_texts, node_addresses = [], [], [], [], [], []
    for node in G.nodes():
        if node not in pos: continue
        node_data = G.nodes[node]
        x, y = pos[node]
        node_x.append(x); node_y.append(y)
        is_pyusd = node_data.get('is_pyusd', False)
        node_name = node_data.get('name', 'Unknown')
        if is_pyusd:
            if "PYUSD Token" in node_name: node_colors.append('rgba(144, 238, 144, 0.9)')
            elif "Supply Control" in node_name: node_colors.append('rgba(135, 206, 250, 0.9)')
            else: node_colors.append('rgba(224, 255, 255, 0.9)')
        else: node_colors.append('rgba(211, 211, 211, 0.9)')
        node_sizes.append(25 if is_pyusd else 18)
        node_addresses.append(node)
        hover_texts.append(f"<b>{node_name}</b><br>Address: {node}")

    node_trace = go.Scatter(
        x=node_x, y=node_y, mode='markers+text', hoverinfo='text', text=node_addresses,
        textposition="bottom center", hovertext=hover_texts,
        marker=dict(showscale=False, color=node_colors, size=node_sizes, line=dict(width=1, color='#000')),
        textfont=dict(family="monospace", size=10, color="black")
    )

    legend_traces = [
        go.Scatter(x=[None], y=[None], mode='markers', marker=dict(size=15, color='rgba(144, 238, 144, 0.9)'), name='PYUSD Token', showlegend=True),
        go.Scatter(x=[None], y=[None], mode='markers', marker=dict(size=15, color='rgba(135, 206, 250, 0.9)'), name='Supply Control', showlegend=True),
        go.Scatter(x=[None], y=[None], mode='markers', marker=dict(size=15, color='rgba(211, 211, 211, 0.9)'), name='External Contract', showlegend=True)
    ]

    fig = go.Figure()
    for trace in edge_traces: fig.add_trace(trace)
    fig.add_trace(node_trace)
    for trace in legend_traces: fig.add_trace(trace)

    fig.update_layout(
        title='<b>Contract Interaction Overview</b>', titlefont=dict(size=16), showlegend=True,
        legend=dict(title="Contract Types", orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1, bgcolor="rgba(255, 255, 255, 0.8)"),
        hovermode='closest', margin=dict(b=20, l=5, r=5, t=60),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        template="plotly_white", height=600, paper_bgcolor='rgba(255,255,255,0.8)', plot_bgcolor='rgba(255,255,255,0.8)'
    )
    return fig


def create_plotly_call_graph(call_data_list):
    """Creates an interactive Plotly Network graph for the call trace hierarchy"""
    if not call_data_list:
        return None

    G = nx.DiGraph()
    id_map = {}
    node_list_for_graph = []

    for i, call in enumerate(call_data_list):
        trace_addr = call.get('trace_addr', [])
        trace_addr_str = "_".join(map(str, trace_addr)) if trace_addr else "root"
        node_id = f"node_{trace_addr_str}"
        count = 0
        original_node_id = node_id
        while node_id in id_map.values():
             count += 1
             node_id = f"{original_node_id}_{count}"

        call['id'] = node_id
        id_map[tuple(trace_addr)] = node_id
        node_list_for_graph.append(call)

    for call in node_list_for_graph:
        call_defaults = {
            'type': 'CALL', 'depth': 0, 'from': 'N/A', 'to': 'N/A', 'value_eth': 0.0, 'gasUsed': 0,
            'is_pyusd': False, 'contract': 'Other', 'function_category': 'other', 'error': None,
            'input_preview': '0x', 'output_preview': '0x', 'function': 'N/A', 'trace_addr': []
        }
        node_data = {**call_defaults, **call}
        G.add_node(call['id'], **node_data)

    for call in node_list_for_graph:
        trace_addr = call.get('trace_addr', [])
        if trace_addr:
            parent_trace_addr = tuple(trace_addr[:-1])
            parent_id = id_map.get(parent_trace_addr)
            if parent_id and parent_id in G and call['id'] in G:
                G.add_edge(parent_id, call['id'])

    if not G.nodes():
         console.print("[warning]No nodes created for Plotly call graph.", style="warning")
         return None

    try:
        import pygraphviz as pgv
        pos = nx.nx_agraph.graphviz_layout(G, prog='dot')
    except ImportError:
        console.print("[warning]pygraphviz not found. Using spring layout for call graph.", style="warning")
        pos = nx.spring_layout(G, seed=42, k=0.5)
    except Exception as layout_err:
         console.print(f"[warning]Layout calculation failed ({layout_err}). Using random layout.", style="warning")
         pos = nx.random_layout(G, seed=42)

    edge_traces_by_type = {}
    for edge in G.edges():
        source, target = edge
        if source not in G or target not in G or source not in pos or target not in pos: continue
        source_data = G.nodes[source]; target_data = G.nodes[target]
        call_type = target_data.get('type', 'CALL').upper()
        if call_type not in edge_traces_by_type:
            color, style, width = 'rgba(128, 128, 128, 0.7)', 'solid', 1
            if call_type == 'DELEGATECALL': color, style, width = 'rgba(0, 0, 255, 0.7)', 'dash', 2
            elif call_type == 'STATICCALL': color, style, width = 'rgba(0, 128, 0, 0.7)', 'dot', 1.5
            elif call_type == 'CREATE': color, style, width = 'rgba(255, 165, 0, 0.8)', 'solid', 1.5
            edge_traces_by_type[call_type] = {'x': [], 'y': [], 'text': [], 'color': color, 'style': style, 'width': width}

        x0, y0 = pos[source]; x1, y1 = pos[target]
        edge_traces_by_type[call_type]['x'].extend([x0, x1, None])
        edge_traces_by_type[call_type]['y'].extend([y0, y1, None])
        hover_text = f"<b>{call_type}</b><br>From: {shorten_address(source_data.get('from', 'N/A'))}<br>To: {shorten_address(target_data.get('to', 'N/A'))}"
        edge_traces_by_type[call_type]['text'].append(hover_text)

    edge_traces = []
    for call_type, trace_data in edge_traces_by_type.items():
        edge_traces.append(go.Scatter(x=trace_data['x'], y=trace_data['y'],
                           line=dict(width=trace_data['width'], color=trace_data['color'], dash=trace_data['style']),
                           hoverinfo='text', text=trace_data['text'], mode='lines', name=call_type))

    node_x, node_y, node_colors, node_sizes, node_text, hover_texts = [], [], [], [], [], []
    for node_id in G.nodes():
        if node_id not in pos: continue
        node_data = G.nodes[node_id]
        x, y = pos[node_id]
        node_x.append(x); node_y.append(y)

        function_name = node_data.get('function', 'N/A')
        if function_name == 'N/A':
            function_name = get_function_description(
                node_data.get('input_preview', '0x'), node_data.get('is_pyusd', False), node_data.get('contract', 'Other')
            )

        short_name = function_name.split('(')[0] if '(' in function_name else function_name
        node_text.append(short_name[:10] + '...' if len(short_name) > 12 else short_name)

        hover_text = f"<b>{node_data.get('type', 'CALL').upper()} Call</b><br>"
        hover_text += f"Depth: {node_data.get('depth','N/A')}, Gas: {node_data.get('gasUsed',0):,}<br>"
        hover_texts.append(hover_text)

        error = node_data.get('error')
        is_pyusd = node_data.get('is_pyusd', False)
        contract_name = node_data.get('contract', 'Other')
        color = 'rgba(211, 211, 211, 0.9)'
        if error: color = 'rgba(255, 99, 71, 0.9)'
        elif is_pyusd:
            if "PYUSD Token" in contract_name: color = 'rgba(144, 238, 144, 0.9)'
            elif "Supply Control" in contract_name: color = 'rgba(135, 206, 250, 0.9)'
            else: color = 'rgba(224, 255, 255, 0.9)'
        else:
             depth = node_data.get('depth', 0)
             intensity = min(95, max(70, 95 - depth * 5))
             rgb_val = intensity / 100.0
             color = f'rgba({int(rgb_val*255)}, {int(rgb_val*255)}, {int(rgb_val*255)}, 0.9)'
        node_colors.append(color)

        gas_used = node_data.get('gasUsed', 0)
        node_sizes.append(max(15, min(40, 15 + (gas_used / 50000))))

    node_trace = go.Scatter(
        x=node_x, y=node_y, mode='markers+text', hoverinfo='text', text=node_text, textposition="top center",
        hovertext=hover_texts,
        marker=dict(showscale=False, color=node_colors, size=node_sizes, line=dict(width=1, color='#000')),
        textfont=dict(family="Arial", size=9, color="#333")
    )

    node_color_legend = [
        go.Scatter(x=[None], y=[None], mode='markers', marker=dict(size=15, color='rgba(144, 238, 144, 0.9)'), name='PYUSD Token', showlegend=True),
        go.Scatter(x=[None], y=[None], mode='markers', marker=dict(size=15, color='rgba(135, 206, 250, 0.9)'), name='Supply Control', showlegend=True),
        go.Scatter(x=[None], y=[None], mode='markers', marker=dict(size=15, color='rgba(224, 255, 255, 0.9)'), name='Other PYUSD Contract', showlegend=True),
        go.Scatter(x=[None], y=[None], mode='markers', marker=dict(size=15, color='rgba(211, 211, 211, 0.9)'), name='Other Contract', showlegend=True),
        go.Scatter(x=[None], y=[None], mode='markers', marker=dict(size=15, color='rgba(255, 99, 71, 0.9)'), name='Error', showlegend=True)
    ]

    fig = go.Figure()
    for trace in edge_traces: fig.add_trace(trace)
    fig.add_trace(node_trace)
    for trace in node_color_legend: fig.add_trace(trace)

    fig.update_layout(
        title='<b>Detailed Call Graph Visualization (trace_transaction)</b>', titlefont=dict(size=16), showlegend=True,
        legend=dict(title="Legend", orientation="v", yanchor="top", y=0.99, xanchor="left", x=0.01, bgcolor="rgba(255, 255, 255, 0.8)"),
        hovermode='closest', margin=dict(b=40, l=5, r=5, t=60),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        template="plotly_white", height=700, paper_bgcolor='rgba(255,255,255,0.8)', plot_bgcolor='rgba(255,255,255,0.8)'
    )

    return fig


def create_plotly_flow_graph(transfers):
    """Creates an interactive Plotly Network graph for PYUSD token flows"""
    if not transfers:
        return None

    G = nx.DiGraph()
    transfer_totals = {}

    for transfer in transfers:
        from_addr = transfer.get('from')
        to_addr = transfer.get('to')
        amount_raw = transfer.get('amount', 0)
        if not isinstance(amount_raw, (int, float)): continue
        amount = int(amount_raw)
        if not from_addr or not to_addr or amount <= 0: continue

        edge_key = (from_addr.lower(), to_addr.lower())
        transfer_totals[edge_key] = transfer_totals.get(edge_key, 0) + amount

    nodes_added = set()
    for (from_addr, to_addr), total_amount in transfer_totals.items():
        if from_addr not in G:
            G.add_node(from_addr, address=from_addr, label=shorten_address(from_addr))
        if to_addr not in G:
            G.add_node(to_addr, address=to_addr, label=shorten_address(to_addr))
        G.add_edge(from_addr, to_addr, amount=total_amount, label=format_value_pyusd(total_amount))

    if not G.nodes():
        console.print("[warning]No valid transfer data found to create flow graph.", style="warning")
        return None

    try:
        pos = nx.spring_layout(G, k=1.0, seed=42)
    except Exception as layout_err:
        console.print(f"[warning]Flow graph layout failed ({layout_err}). Using random layout.", style="warning")
        pos = nx.random_layout(G, seed=42)

    edge_x, edge_y, edge_text = [], [], []
    label_x, label_y, label_text = [], [], []
    arrow_x, arrow_y, arrow_angles = [], [], []

    for edge in G.edges(data=True):
        source, target, data = edge
        if source not in pos or target not in pos: continue
        x0, y0 = pos[source]; x1, y1 = pos[target]

        edge_x.extend([x0, x1, None]); edge_y.extend([y0, y1, None])
        amount_str = format_value_pyusd(data['amount'])
        edge_text.append(f"Transfer: {amount_str}<br>From: {shorten_address(source)}<br>To: {shorten_address(target)}")

        label_x.append((x0 + x1) / 2); label_y.append((y0 + y1) / 2); label_text.append(amount_str)

        dx = x1 - x0; dy = y1 - y0
        arrow_ratio = 0.95
        arrow_x.append(x0 + dx * arrow_ratio); arrow_y.append(y0 + dy * arrow_ratio)
        arrow_angles.append(math.degrees(math.atan2(dy, dx)))

    edge_trace = go.Scatter(x=edge_x, y=edge_y, line=dict(width=2, color='rgba(50, 150, 50, 0.8)'),
                           hoverinfo='text', text=edge_text, mode='lines', name='Transfer')
    edge_label_trace = go.Scatter(x=label_x, y=label_y, text=label_text, mode='text', hoverinfo='none',
                                 showlegend=False, textfont=dict(size=10, color='darkgreen'))
    arrow_trace = go.Scatter(x=arrow_x, y=arrow_y, mode='markers', marker=dict(symbol='triangle-right', size=12,
                           color='rgba(50, 150, 50, 0.8)', angle=arrow_angles), hoverinfo='none', showlegend=False)

    node_x, node_y, node_text, node_hover = [], [], [], []
    for node in G.nodes():
        if node not in pos: continue
        x, y = pos[node]
        node_x.append(x); node_y.append(y)
        node_text.append(node)
        node_hover.append(f"<b>Address:</b> {node}")

    node_trace = go.Scatter(
        x=node_x, y=node_y, mode='markers+text', hoverinfo='text', text=node_text, textposition="bottom center",
        hovertext=node_hover,
        marker=dict(color='rgba(144, 238, 144, 0.8)', size=25, line=dict(width=1, color='darkgreen'), symbol='circle'),
        textfont=dict(family="monospace", size=9, color="black"), name='Address'
    )

    fig = go.Figure()
    fig.add_trace(edge_trace); fig.add_trace(edge_label_trace); fig.add_trace(arrow_trace); fig.add_trace(node_trace)
    fig.update_layout(
        title='<b>PYUSD Token Flow Analysis</b>', titlefont=dict(size=16), showlegend=True,
        legend=dict(title="Elements", orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1, bgcolor="rgba(255, 255, 255, 0.8)"),
        hovermode='closest', margin=dict(b=20, l=5, r=5, t=60),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        template="plotly_white", height=600, paper_bgcolor='rgba(255,255,255,0.8)', plot_bgcolor='rgba(255,255,255,0.8)'
    )
    return fig

# Helper Functions for Enhanced Analysis
def extract_params_from_input(input_data, method_sig, param_types):
    """Extract and decode parameters from input data based on function signature"""
    try:
        params_hex = input_data[10:]
        decoded_params = []
        pos = 0
        for param_type in param_types:
            if param_type == "address":
                param_val = "0x" + params_hex[pos+24:pos+64]
                decoded_params.append(param_val)
            elif param_type.startswith("uint"):
                param_val = int(params_hex[pos:pos+64], 16)
                decoded_params.append(param_val)
            elif param_type == "bool":
                param_val = int(params_hex[pos:pos+64], 16) != 0
                decoded_params.append(param_val)
            elif param_type == "bytes32":
                param_val = "0x" + params_hex[pos:pos+64]
                decoded_params.append(param_val)
            pos += 64
        return decoded_params
    except Exception as e:
        return []

def decode_pyusd_function(input_data):
    """Enhanced function to decode PYUSD function calls with human-readable parameters"""
    if not input_data or input_data == '0x':
        return {"name": "Unknown", "category": "other", "params": {}}

    method_sig = input_data[:10]

    if method_sig in PYUSD_SIGNATURES:
        sig_info = PYUSD_SIGNATURES[method_sig]
        function_name = sig_info["name"]
        function_category = sig_info["category"]
        params = {}

        if method_sig == '0xa9059cbb':  # transfer(address,uint256)
            try:
                decoded = extract_params_from_input(input_data, method_sig, ["address", "uint256"])
                if len(decoded) == 2:
                    params = {
                        "to": decoded[0], "to_address": shorten_address(decoded[0]),
                        "amount": decoded[1], "amount_formatted": format_value_pyusd(decoded[1])}
            except Exception: pass
        elif method_sig == '0x23b872dd':  # transferFrom(address,address,uint256)
            try:
                decoded = extract_params_from_input(input_data, method_sig, ["address", "address", "uint256"])
                if len(decoded) == 3:
                    params = {
                        "from": decoded[0], "from_address": shorten_address(decoded[0]),
                        "to": decoded[1], "to_address": shorten_address(decoded[1]),
                        "amount": decoded[2], "amount_formatted": format_value_pyusd(decoded[2])}
            except Exception: pass
        elif method_sig == '0x095ea7b3':  # approve(address,uint256)
            try:
                decoded = extract_params_from_input(input_data, method_sig, ["address", "uint256"])
                if len(decoded) == 2:
                    params = {
                        "spender": decoded[0], "spender_address": shorten_address(decoded[0]),
                        "amount": decoded[1], "amount_formatted": format_value_pyusd(decoded[1])}
            except Exception: pass
        elif method_sig == '0x40c10f19':  # mint(address,uint256)
            try:
                decoded = extract_params_from_input(input_data, method_sig, ["address", "uint256"])
                if len(decoded) == 2:
                    params = {
                        "to": decoded[0], "to_address": shorten_address(decoded[0]),
                        "amount": decoded[1], "amount_formatted": format_value_pyusd(decoded[1])}
            except Exception: pass
        elif method_sig == '0x42966c68':  # burn(uint256)
            try:
                decoded = extract_params_from_input(input_data, method_sig, ["uint256"])
                if len(decoded) == 1:
                    params = {"amount": decoded[0], "amount_formatted": format_value_pyusd(decoded[0])}
            except Exception: pass
        elif method_sig == '0x70a08231':  # balanceOf(address)
            try:
                decoded = extract_params_from_input(input_data, method_sig, ["address"])
                if len(decoded) == 1:
                    params = {"account": decoded[0], "account_address": shorten_address(decoded[0])}
            except Exception: pass
        elif method_sig == '0xdd62ed3e':  # allowance(address,address)
            try:
                decoded = extract_params_from_input(input_data, method_sig, ["address", "address"])
                if len(decoded) == 2:
                    params = {
                        "owner": decoded[0], "owner_address": shorten_address(decoded[0]),
                        "spender": decoded[1], "spender_address": shorten_address(decoded[1])}
            except Exception: pass
        elif method_sig == '0xf2fde38b':  # transferOwnership(address)
            try:
                decoded = extract_params_from_input(input_data, method_sig, ["address"])
                if len(decoded) == 1:
                    params = {"newOwner": decoded[0], "newOwner_address": shorten_address(decoded[0])}
            except Exception: pass

        return {"name": function_name, "category": function_category, "params": params}
    return {"name": "Unknown", "category": "other", "params": {}}


def analyze_gas_efficiency(gas_used, function_name):
    """Analyze gas efficiency compared to benchmarks"""
    if function_name in PYUSD_GAS_BENCHMARKS:
        benchmark = PYUSD_GAS_BENCHMARKS[function_name]
        median = benchmark["median"]
        p25 = benchmark["p25"]
        p75 = benchmark["p75"]
        pct_diff = ((gas_used - median) / median) * 100 if median > 0 else 0
        if gas_used <= p25: efficiency, color = "excellent", "green"
        elif gas_used <= median: efficiency, color = "good", "blue"
        elif gas_used <= p75: efficiency, color = "average", "yellow"
        else: efficiency, color = "poor", "red"
        return {"efficiency": efficiency, "color": color, "pct_diff": pct_diff, "compared_to_median": gas_used - median}
    return {"efficiency": "unknown", "color": "gray", "pct_diff": 0, "compared_to_median": 0}


def identify_transaction_pattern(summary_df, pyusd_transfers):
    """Identify common transaction patterns for PYUSD"""
    result = {"pattern": "unknown", "confidence": 0, "description": "Unknown transaction pattern", "matches": []}
    if summary_df.empty: return result

    pyusd_df = summary_df[summary_df['is_pyusd']] if 'is_pyusd' in summary_df.columns else pd.DataFrame()
    contract_count = len(pyusd_df['to'].unique()) if not pyusd_df.empty and 'to' in pyusd_df.columns else 0
    function_counts = pyusd_df['function'].value_counts().to_dict() if not pyusd_df.empty and 'function' in pyusd_df.columns else {}
    transfer_count = len(pyusd_transfers)
    matches = []

    if transfer_count == 1 and contract_count <= 1 and "transfer(address,uint256)" in function_counts and function_counts.get("transfer(address,uint256)",0) == 1:
        matches.append(("simple_transfer", 0.9, "Simple PYUSD transfer between addresses"))
    if transfer_count > 1 and "transfer(address,uint256)" in function_counts and function_counts.get("transfer(address,uint256)",0) > 1:
        matches.append(("multi_transfer", 0.8, "Multiple PYUSD transfers in one transaction"))
    if "approve(address,uint256)" in function_counts and function_counts.get("approve(address,uint256)",0) >= 1:
        matches.append(("approval_flow", 0.85, "PYUSD approval for future spending"))
    if ("mint(address,uint256)" in function_counts and function_counts.get("mint(address,uint256)",0) >= 1) or ("burn(uint256)" in function_counts and function_counts.get("burn(uint256)",0) >= 1):
        matches.append(("supply_change", 0.95, "Minting or burning of PYUSD supply"))

    external_contract_calls = summary_df[~summary_df['is_pyusd'] & (summary_df['type'].str.upper() == 'CALL')].shape[0] if 'is_pyusd' in summary_df.columns and 'type' in summary_df.columns else 0
    if transfer_count >= 1 and external_contract_calls >= 3:
        matches.append(("swap_operation", 0.7, "PYUSD swap through DEX"))
    if transfer_count >= 1 and not pyusd_df.empty and 'function' in pyusd_df.columns and 'mint' in ' '.join(pyusd_df['function'].astype(str).values):
        matches.append(("liquidity_provision", 0.6, "Adding/removing liquidity with PYUSD"))

    if matches:
        matches.sort(key=lambda x: x[1], reverse=True)
        best_match = matches[0]
        result = {"pattern": best_match[0], "confidence": best_match[1], "description": best_match[2], "matches": matches}
    return result


def detect_mev_potential(trace_list, summary_df, tx_hash):
    """Detect potential MEV activity in the transaction"""
    results = {"mev_detected": False, "type": None, "confidence": 0, "description": None, "indicators": []}
    if summary_df.empty: return results

    token_movements = len(summary_df[summary_df['function'].str.contains('transfer', na=False)]) if 'function' in summary_df.columns else 0
    unique_contracts = len(summary_df['to'].unique()) if 'to' in summary_df.columns else 0
    indicators = []
    has_swap = any(summary_df['function'].str.contains('swap', case=False, na=False)) if 'function' in summary_df.columns else False
    external_calls = summary_df[~summary_df['is_pyusd']].shape[0] if 'is_pyusd' in summary_df.columns else 0

    if has_swap and external_calls > 3:
        indicators.append(("potential_sandwich_target", 0.6, "Transaction contains swap with external calls"))
    if token_movements >= 3 and unique_contracts >= 3:
        indicators.append(("potential_arbitrage", 0.7, "Multiple token movements across different contracts"))

    if indicators:
        indicators.sort(key=lambda x: x[1], reverse=True)
        best_indicator = indicators[0]
        results = {"mev_detected": True, "type": best_indicator[0], "confidence": best_indicator[1], "description": best_indicator[2], "indicators": indicators}
    return results


def detect_security_concerns(trace_list, summary_df):
    """Detect potential security concerns in the transaction"""
    concerns = []
    if summary_df.empty: return concerns

    if 'function' in summary_df.columns:
        for _, row in summary_df.iterrows():
            function_name = row['function']
            if function_name in SECURITY_RISK_LEVELS:
                risk_level = SECURITY_RISK_LEVELS[function_name]
                concerns.append({
                    "level": risk_level,
                    "description": f"High-risk function '{function_name}' called",
                    "contract": row.get('contract', 'N/A'),
                    "from": row.get('from', 'N/A')
                })

    if 'function' in summary_df.columns and 'approve(address,uint256)' in summary_df['function'].unique():
         approval_rows = summary_df[summary_df['function'] == 'approve(address,uint256)']
         approval_amount_cols = [col for col in summary_df.columns if 'param_amount' in col and col != 'param_amount_formatted']
         if approval_amount_cols:
             amount_col = approval_amount_cols[0]
             for _, row in approval_rows.iterrows():
                 if pd.notna(row[amount_col]) and isinstance(row[amount_col], (int, float)):
                     amount_val = row[amount_col]
                     max_uint_approx = (2**256) - (2**128)
                     if amount_val >= max_uint_approx:
                          concerns.append({
                             "level": "medium",
                             "description": f"Potentially infinite approval granted",
                             "contract": row.get('contract', 'N/A'), "from": row.get('from', 'N/A')
                          })
                     elif amount_val >= (10**6) * 1000000:
                         concerns.append({
                             "level": "low",
                             "description": f"Large approval granted: {row.get('param_amount_formatted', format_value_pyusd(amount_val))}",
                             "contract": row.get('contract', 'N/A'), "from": row.get('from', 'N/A')
                         })

    if 'function' in summary_df.columns and any(summary_df['function'].str.contains('selfdestruct', case=False, na=False)):
        concerns.append({"level": "critical", "description": "Contract self-destruct called", "contract": "N/A", "from": "N/A"})

    return concerns

# Main Analysis Function
def analyze_trace_transaction(trace_list, tx_hash):
    """Analyzes the output list from trace_transaction with PYUSD focus."""
    if not isinstance(trace_list, list):
        console.print(f"[error]Expected a list from trace_transaction for {shorten_address(tx_hash)}, got {type(trace_list)}.", style="error")
        return None
    if not trace_list:
        console.print(f"[warning]No trace results (empty list) from trace_transaction for {shorten_address(tx_hash)}.", style="warning")
        return None

    console.print(f"\n\n[bold cyan3]Analyzing {len(trace_list)} Actions from trace_transaction for {shorten_address(tx_hash)}[/bold cyan3]")

    # Initialize tracking data
    summary = []
    total_gas_used = 0
    errors = 0
    pyusd_interactions = 0
    pyusd_calls_by_function = {}
    pyusd_calls_by_category = {cat: 0 for cat in ["token_movement", "supply_change", "allowance", "control", "admin", "view", "other"]}

    # Data for Plotly graphs
    call_data_list_for_plotly = []
    contract_interactions_list = []
    pyusd_transfers_for_plotly = []
    pyusd_transfers_display = []

    # Track call depth and sequence for replay visualization
    call_sequence = []
    max_depth = 0
    gas_by_depth = {}
    contract_interactions_agg = {}

    # Enhanced token flow tracking
    eth_flows = []

    # Process each trace action
    for i, trace_item in enumerate(trace_list):
        if not isinstance(trace_item, dict): continue

        action = trace_item.get('action', {})
        result = trace_item.get('result', {})
        trace_type = trace_item.get('type', 'N/A').upper()
        trace_addr = trace_item.get('traceAddress', [])
        error = trace_item.get('error')

        current_depth = len(trace_addr)
        max_depth = max(max_depth, current_depth)

        from_addr = action.get('from')
        from_addr_lower = from_addr.lower() if isinstance(from_addr, str) else None
        to_addr = action.get('to', result.get('address') if trace_type == 'CREATE' else None)
        to_addr_lower = to_addr.lower() if isinstance(to_addr, str) else None

        value_raw_wei = to_int(hexstr=action.get('value', '0x0')) if action.get('value') else 0
        value_eth = format_value_eth(value_raw_wei)
        value_eth_float = float(w3_mainnet.from_wei(value_raw_wei, 'ether')) if w3_mainnet and value_raw_wei is not None else 0.0

        gas_used_hex = result.get('gasUsed', trace_item.get('gasUsed', '0x0'))
        gas_used = int(gas_used_hex, 16) if gas_used_hex else 0

        depth_key = len(trace_addr)
        gas_by_depth[depth_key] = gas_by_depth.get(depth_key, 0) + gas_used

        call_data = action.get('input', '0x')
        output_data = result.get('output', '0x')

        # Check PYUSD based on address
        is_pyusd_call = False
        contract_name = "Other Contract"
        check_addr_lower = None
        if trace_type == 'CREATE':
            created_addr_lower = result.get('address','').lower() if result else ''
            if created_addr_lower in PYUSD_CONTRACTS:
                is_pyusd_call = True
                contract_name = PYUSD_CONTRACTS[created_addr_lower]
                check_addr_lower = created_addr_lower
            to_addr = result.get('address') if result else None
            to_addr_lower = to_addr.lower() if isinstance(to_addr, str) else None
        elif to_addr_lower:
            if to_addr_lower in PYUSD_CONTRACTS:
                is_pyusd_call = True
                contract_name = PYUSD_CONTRACTS[to_addr_lower]
                check_addr_lower = to_addr_lower

        # Track contract interactions
        if from_addr_lower and to_addr_lower:
            contract_interactions_list.append((from_addr_lower, to_addr_lower))
            interaction_key = f"{from_addr_lower}:{to_addr_lower}"
            contract_interactions_agg[interaction_key] = contract_interactions_agg.get(interaction_key, {"from": from_addr, "to": to_addr, "count": 0, "gas": 0})
            contract_interactions_agg[interaction_key]["count"] += 1
            contract_interactions_agg[interaction_key]["gas"] += gas_used

        if value_raw_wei > 0:
            eth_flows.append({"from": from_addr, "to": to_addr, "value": value_raw_wei, "value_formatted": value_eth, "trace_addr": trace_addr})

        # Function decoding
        function_decoded = {"name": "N/A", "category": "other", "params": {}}
        if trace_type in ['CALL', 'DELEGATECALL', 'STATICCALL'] and call_data and call_data != '0x':
            if is_pyusd_call:
                 function_decoded = decode_pyusd_function(call_data)
            else:
                 func_desc = get_function_description(call_data, is_pyusd_call, contract_name)
                 function_decoded['name'] = func_desc
        elif trace_type == 'CREATE':
             function_decoded['name'] = 'Constructor'

        function_name = function_decoded["name"]
        function_category = function_decoded["category"]

        if is_pyusd_call and function_name != "N/A" and function_name != "Unknown" and function_name != 'Constructor':
            pyusd_calls_by_function[function_name] = pyusd_calls_by_function.get(function_name, 0) + 1
            pyusd_calls_by_category[function_category] = pyusd_calls_by_category.get(function_category, 0) + 1

            method_sig = call_data[:10]
            if from_addr:
                if method_sig == '0xa9059cbb': # transfer
                    if 'amount' in function_decoded["params"] and 'to' in function_decoded["params"]:
                        transfer_info_plotly = {'from': from_addr, 'to': function_decoded["params"]["to"], 'amount': function_decoded["params"]["amount"], 'trace_addr': trace_addr }
                        pyusd_transfers_for_plotly.append(transfer_info_plotly)
                        transfer_info_display = {**transfer_info_plotly, 'value': transfer_info_plotly['amount'] / (10**PYUSD_CONFIG['ethereum']['decimals'])}
                        pyusd_transfers_display.append(transfer_info_display)
                elif method_sig == '0x23b872dd': # transferFrom
                    if all(k in function_decoded["params"] for k in ['from', 'to', 'amount']):
                        transfer_info_plotly = {'from': function_decoded["params"]["from"], 'to': function_decoded["params"]["to"], 'amount': function_decoded["params"]["amount"], 'trace_addr': trace_addr}
                        pyusd_transfers_for_plotly.append(transfer_info_plotly)
                        transfer_info_display = {**transfer_info_plotly, 'value': transfer_info_plotly['amount'] / (10**PYUSD_CONFIG['ethereum']['decimals'])}
                        pyusd_transfers_display.append(transfer_info_display)

        # Data for Plotly Call Graph
        call_info_plotly = {
            'id': f"node_{'_'.join(map(str, trace_addr)) if trace_addr else 'root'}",
            'trace_addr': trace_addr,
            'type': trace_type,
            'depth': current_depth,
            'from': from_addr if from_addr else "N/A",
            'to': to_addr if to_addr else ("CREATE" if trace_type == 'CREATE' else "N/A"),
            'value_eth': value_eth_float,
            'gasUsed': gas_used,
            'is_pyusd': is_pyusd_call,
            'contract': contract_name,
            'function_category': function_decoded['category'],
            'function': function_decoded['name'],
            'error': error,
            'input_preview': call_data[:10] + ('...' if len(call_data) > 10 else ''),
            'output_preview': output_data[:10] + ('...' if len(output_data) > 10 else ''),
        }
        call_data_list_for_plotly.append(call_info_plotly)

        if is_pyusd_call:
            pyusd_interactions += 1

        total_gas_used += gas_used
        if error: errors += 1

        call_sequence.append({
            'index': i, 'trace_addr': trace_addr, 'depth': len(trace_addr), 'type': trace_type,
            'from': from_addr, 'to': to_addr, 'is_pyusd': is_pyusd_call,
            'function': function_decoded["name"], 'params': function_decoded["params"],
            'gas_used': gas_used, 'value_eth': value_eth, 'error': error
        })

        # Prepare summary row
        gas_efficiency = analyze_gas_efficiency(gas_used, function_decoded["name"]) if is_pyusd_call and function_decoded["name"] in PYUSD_GAS_BENCHMARKS else None
        summary_row = {
            'index': i, 'trace_address': str(trace_addr), 'type': trace_type,
            'from': from_addr, 'to': to_addr if to_addr else ("CREATE" if trace_type == 'CREATE' else "N/A"),
            'is_pyusd': is_pyusd_call, 'contract': contract_name,
            'function': function_decoded["name"], 'category': function_decoded["category"],
            'value_eth': value_eth, 'gas_used': gas_used, 'gas_str': format_gas(gas_used),
            'gas_efficiency': gas_efficiency["efficiency"] if gas_efficiency else "N/A",
            'gas_pct_diff': gas_efficiency["pct_diff"] if gas_efficiency else 0,
            'error': error if error else "None", 'depth': len(trace_addr)
        }
        if function_decoded["params"]:
            for param_key, param_value in function_decoded["params"].items():
                if isinstance(param_value, (str, int, float, bool)) or param_value is None:
                    if isinstance(param_value, str) and param_value.startswith("0x"): continue
                    if param_key not in ['from', 'to']:
                        summary_row[f"param_{param_key}"] = param_value
                elif isinstance(param_value, list):
                    serializable_list = [str(item) if not isinstance(item, (str, int, float, bool)) else item for item in param_value]
                    summary_row[f"param_{param_key}"] = json.dumps(serializable_list)
                else:
                    summary_row[f"param_{param_key}"] = str(param_value)

        summary.append(summary_row)

    summary_df = pd.DataFrame(summary)

    pyusd_pct = (pyusd_interactions / len(trace_list) * 100) if trace_list else 0
    pyusd_gas = summary_df[summary_df['is_pyusd']]['gas_used'].sum() if 'is_pyusd' in summary_df.columns and 'gas_used' in summary_df.columns else 0
    pyusd_gas_pct = (pyusd_gas / total_gas_used * 100) if total_gas_used > 0 else 0

    tx_pattern = identify_transaction_pattern(summary_df, pyusd_transfers_display)
    mev_analysis = detect_mev_potential(trace_list, summary_df, tx_hash)
    security_concerns = detect_security_concerns(trace_list, summary_df)

    unique_contracts = len(summary_df['to'].unique()) if 'to' in summary_df.columns else 0
    complexity_score = min(100, (
        (len(trace_list) * 2) + (unique_contracts * 5) + (max_depth * 10) +
        (len(pyusd_transfers_display) * 3) + (errors * 5)
    ) / 3)

    # Generate Plotly Graphs
    contract_graph_plotly = None
    if contract_interactions_list:
        try:
            interactions_no_loops = list(set([(f, t) for f, t in contract_interactions_list if f != t]))
            if interactions_no_loops:
                 contract_graph_plotly = create_plotly_contract_interaction_graph(interactions_no_loops)
        except Exception as viz_err:
            console.print(f"[warning]Could not create Plotly contract interaction graph: {viz_err}", style="warning")
            print(traceback.format_exc())

    call_graph_plotly = None
    if call_data_list_for_plotly:
        try:
            call_graph_plotly = create_plotly_call_graph(call_data_list_for_plotly)
        except Exception as viz_err:
            console.print(f"[warning]Could not create Plotly call graph: {viz_err}", style="warning")
    else:
         console.print("[warning]No data collected for Plotly call graph.", style="warning")

    flow_graph_plotly = None
    if pyusd_transfers_for_plotly:
        try:
            flow_graph_plotly = create_plotly_flow_graph(pyusd_transfers_for_plotly)
        except Exception as viz_err:
            console.print(f"[warning]Could not create Plotly flow graph: {viz_err}", style="warning")

    # Display transaction summary panel
    console.print(Panel(f"""
[bold cyan3]PYUSD [green3]`trace_transaction`[/green3] Summary ({tx_hash})[/bold cyan3]
[bold cyan3]Total Actions:[/bold cyan3] {len(trace_list)}
[bold cyan3]Total Gas Used:[/bold cyan3] {total_gas_used:,}
[bold cyan3]Actions with Errors:[/bold cyan3] {errors}
[bold cyan3]Call Depth:[/bold cyan3] {max_depth} levels
[bold cyan3]Unique Contracts:[/bold cyan3] {unique_contracts}
[bold cyan3]Complexity Score:[/bold cyan3] {complexity_score:.1f}/100

[bold green3]PYUSD Activity:[/bold green3]
[bold cyan3]PYUSD Interactions:[/bold cyan3] {pyusd_interactions} ({pyusd_pct:.1f}% of actions)
[bold cyan3]PYUSD Gas Usage:[/bold cyan3] {pyusd_gas:,} ({pyusd_gas_pct:.1f}% of total gas)
[bold cyan3]PYUSD Transfers:[/bold cyan3] {len(pyusd_transfers_display)}

[bold yellow3]Transaction Pattern:[/bold yellow3]
[bold cyan3]Type:[/bold cyan3] {tx_pattern['pattern']}
[bold cyan3]Description:[/bold cyan3] {tx_pattern['description']}
[bold cyan3]Confidence:[/bold cyan3] {tx_pattern['confidence']:.0%}

[bold red3]Security Analysis:[/bold red3]
[bold cyan3]MEV Potential:[/bold cyan3] {"Yes" if mev_analysis["mev_detected"] else "No"}
{f"[bold cyan3]MEV Type:[/bold cyan3] {mev_analysis['type']} ({mev_analysis['confidence']:.0%})" if mev_analysis["mev_detected"] else ""}
[bold cyan3]Security Concerns:[/bold cyan3] {len(security_concerns)}""",
        title="PYUSD Transaction Analysis", border_style="cyan3", expand=False))

    # Display Plotly Graphs
    if contract_graph_plotly:
        console.print("\n\n[bold]📊 Contract Interaction Overview Chart:[/bold]", style="magenta3")
        console.print("───────────────────────────────────────", style="magenta3")

        display(contract_graph_plotly)
        console.print("[info]Interactive graph showing high-level contract interactions.", style="info")
    else:
        if not contract_interactions_list:
            console.print("\n[info]No contract interactions detected to generate graph.", style="info")

    console.print("\n\n[bold magenta3]📊 Detailed Call Graph Visualization Chart:[/bold magenta3]")
    console.print("───────────────────────────────────────────", style="magenta3")

    if call_graph_plotly:
        display(call_graph_plotly)
        console.print("[info]Interactive visualization of the detailed call hierarchy.", style="info")
    else:
        if not call_data_list_for_plotly:
             console.print("[warning]No call data available to generate call graph.", style="warning")

    if flow_graph_plotly:
        console.print("\n\n[bold magenta3]🔄 PYUSD Token Flow Analysis Chart:[/bold magenta3]")
        console.print("───────────────────────────────────", style="magenta3")

        display(flow_graph_plotly)
        console.print("[info]Interactive graph showing the movement of PYUSD tokens.", style="info")
    elif pyusd_transfers_display:
         console.print("\n\n[warning]PYUSD transfers detected, but flow graph generation failed.", style="warning")

    # Display security concerns if any
    if security_concerns:
        console.print("\n[bold red]⚠️ Security Concerns[/bold red]")
        security_table = Table(show_header=True, header_style="bold red")
        security_table.add_column("Level", style="dim")
        security_table.add_column("Description", style="bold")
        security_table.add_column("Contract")
        for concern in security_concerns:
            level_color = "red" if concern["level"] in ["high", "critical"] else "yellow"
            security_table.add_row(f"[{level_color}]{concern['level'].upper()}[/{level_color}]", concern["description"], concern.get("contract", "N/A"))
        console.print(security_table)

    # Display PYUSD function calls table if relevant
    if pyusd_calls_by_function:
        console.print("\n\n[bold green3]🪙 PYUSD Function Calls[/bold green3]")
        console.print("───────────────────────", style="green3")

        function_table = Table(show_header=True, header_style="bold green3")
        function_table.add_column("Function")
        function_table.add_column("Count", justify="right")
        function_table.add_column("Category", justify="center")
        function_table.add_column("Gas Used", justify="right")
        function_table.add_column("Gas Efficiency", justify="center")
        gas_by_function = {}
        if not summary_df.empty and 'function' in summary_df.columns and 'is_pyusd' in summary_df.columns and 'gas_used' in summary_df.columns:
             pyusd_rows = summary_df[summary_df['is_pyusd']]
             gas_by_function = pyusd_rows.groupby('function')['gas_used'].sum().to_dict()

        for func, count in sorted(pyusd_calls_by_function.items(), key=lambda x: x[1], reverse=True):
            category = next((info["category"] for sig, info in PYUSD_SIGNATURES.items() if info["name"] == func), "other")
            gas_used_func = gas_by_function.get(func, 0)
            efficiency, efficiency_color = "N/A", ""
            if func in PYUSD_GAS_BENCHMARKS and count > 0:
                eff_data = analyze_gas_efficiency(gas_used_func / count, func)
                efficiency = eff_data["efficiency"].title()
                efficiency_color = eff_data["color"]
            function_table.add_row(func, str(count), category.replace('_', ' ').title(), f"{gas_used_func:,}", f"[{efficiency_color}]{efficiency}[/{efficiency_color}]" if efficiency_color else efficiency)
        console.print(function_table)

        console.print("\n\n[bold]PYUSD Function Categories in Transaction Chart:[/bold]", style="magenta3")
        console.print("───────────────────────────────────────────────", style="magenta3")
        try:
            category_data = [{'category': cat.replace('_', ' ').title(), 'count': cnt} for cat, cnt in pyusd_calls_by_category.items() if cnt > 0]
            if category_data:
                category_df = pd.DataFrame(category_data)
                color_map = {'Token Movement': 'lightgreen', 'Supply Change': 'coral', 'Allowance': 'skyblue', 'Control': 'gold', 'Admin': 'darkred', 'View': 'lightgray', 'Other': 'silver'}
                fig_cat = px.pie(category_df, values='count', names='category', title=f'PYUSD Function Categories in Transaction {shorten_address(tx_hash)}', color='category', color_discrete_map=color_map)
                fig_cat.update_layout(template="plotly_white", legend_title_text='Function Category', legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
                fig_cat.update_traces(textposition='inside', textinfo='percent+label')
                fig_cat.show()
        except Exception as viz_err:
            console.print(f"[warning]Could not create function category chart: {viz_err}", style="warning")

    # Display PYUSD transfers table
    if pyusd_transfers_display:
        console.print("\n\n[bold yellow3]🪙 PYUSD Token Transfers[/bold yellow3]")
        console.print("─────────────────────────", style="yellow3")

        transfer_table = Table(show_header=True, header_style="bold yellow3")
        transfer_table.add_column("From")
        transfer_table.add_column("To")
        transfer_table.add_column("Amount", justify="right")
        transfer_table.add_column("Call Depth", justify="center")
        total_transferred = 0
        for transfer in pyusd_transfers_display:
            from_addr, to_addr = transfer.get('from'), transfer.get('to')
            amount = transfer.get('value',0.0)
            total_transferred += amount
            depth = len(transfer.get('trace_addr',[]))
            amount_str = f"{amount:,.6f} PYUSD"
            if amount >= 10000: amount_str = f"[bold green]{amount_str}[/bold green]"
            elif amount >= 1000: amount_str = f"[green]{amount_str}[/green]"
            transfer_table.add_row(from_addr, to_addr, amount_str, str(depth))
        console.print(transfer_table)
        console.print(f"[info]Total PYUSD transferred: {total_transferred:,.6f} PYUSD", style="info")

    # Display call stack depth visualization
    if gas_by_depth:
         console.print("\n\n[bold magenta3]Gas Usage by Call Depth Chart:[/bold magenta3]")
         console.print("──────────────────────────────", style="magenta3")
         try:
            depth_df = pd.DataFrame([{"depth": depth, "gas": gas} for depth, gas in gas_by_depth.items()])
            fig_depth = px.bar(depth_df, x='depth', y='gas', title=f'Gas Usage by Call Depth in {shorten_address(tx_hash)}', labels={'depth': 'Call Stack Depth', 'gas': 'Gas Used'}, color='gas', color_continuous_scale='Viridis')
            fig_depth.update_layout(template="plotly_white")
            fig_depth.show()
         except Exception as viz_err:
            console.print(f"[warning]Could not render call depth visualization: {viz_err}", style="warning")

    # Display gas usage by category visualizations
    try:
        console.print("\n\n[bold]Gas Usage by Contract Chart:[/bold]", style="magenta3")
        console.print("───────────────────────────", style="magenta3")

        if not summary_df.empty and 'gas_used' in summary_df.columns and total_gas_used > 0:
            contract_gas = summary_df.groupby('contract')['gas_used'].sum().reset_index()
            contract_gas['percentage'] = contract_gas['gas_used'] / total_gas_used * 100
            contract_colors = {"PYUSD Token": "green", "PYUSD Implementation": "darkgreen", "Supply Control": "blue", "Supply Control Impl": "darkblue", "Other Contract": "gray"}
            fig_contract_gas = px.pie(contract_gas, values='gas_used', names='contract', title=f'Gas Usage by Contract in {shorten_address(tx_hash)}', hover_data=['percentage'], color='contract', color_discrete_map=contract_colors)
            fig_contract_gas.update_layout(template="plotly_white")
            fig_contract_gas.show()

            if pyusd_interactions > 0 and pyusd_gas > 0:
                console.print("\n\n[bold]PYUSD Gas Usage by Function Category Chart:[/bold]", style="magenta3")
                console.print("───────────────────────────────────────────", style="magenta3")

                pyusd_gas_by_category = summary_df[summary_df['is_pyusd']].groupby('category')['gas_used'].sum().reset_index()
                if not pyusd_gas_by_category.empty:
                    pyusd_gas_by_category['percentage'] = pyusd_gas_by_category['gas_used'] / pyusd_gas * 100
                    category_colors = {"token_movement": "lightgreen", "supply_change": "coral", "allowance": "skyblue", "control": "gold", "admin": "darkred", "view": "lightgray", "other": "silver"}
                    pyusd_gas_by_category['category'] = pyusd_gas_by_category['category'].apply(lambda x: x.replace('_', ' ').title())
                    fig_category_gas = px.pie(pyusd_gas_by_category, values='gas_used', names='category', title=f'PYUSD Gas Usage by Function Category in {shorten_address(tx_hash)}', hover_data=['percentage'], color='category', color_discrete_map={k.replace('_', ' ').title(): v for k, v in category_colors.items()})
                    fig_category_gas.update_layout(template="plotly_white")
                    fig_category_gas.show()
    except Exception as viz_err:
        console.print(f"[warning]Could not create gas usage visualization: {viz_err}", style="warning")

    # Add gas efficiency analysis visualization
    try:
        console.print("\n\n[bold]Gas Efficiency by PYUSD Function Chart:[/bold]", style="magenta3")
        console.print("───────────────────────────────────────", style="magenta3")

        if not summary_df.empty and 'gas_efficiency' in summary_df.columns and 'function' in summary_df.columns:
            efficiency_df = summary_df[summary_df['gas_efficiency'] != 'N/A'].copy()
            if not efficiency_df.empty:
                efficiency_df['efficiency_level'] = pd.Categorical(efficiency_df['gas_efficiency'], categories=['excellent', 'good', 'average', 'poor'], ordered=True)
                efficiency_colors = {'excellent': 'green', 'good': 'lightgreen', 'average': 'gold', 'poor': 'coral'}
                fig_efficiency = px.bar(efficiency_df, x='function', y='gas_used', color='gas_efficiency', title=f'Gas Efficiency by PYUSD Function in {shorten_address(tx_hash)}', labels={'function': 'Function', 'gas_used': 'Gas Used', 'gas_efficiency': 'Efficiency'}, color_discrete_map=efficiency_colors)
                fig_efficiency.update_layout(template="plotly_white")
                fig_efficiency.show()
    except Exception as viz_err:
        console.print(f"[warning]Could not create gas efficiency visualization: {viz_err}", style="warning")

    # Create interactive transaction replay component
    if call_sequence:
        console.print("\n\n[bold yellow3]🎮 Interactive Transaction Replay[/bold yellow3]")
        console.print("──────────────────────────────────", style="yellow3")

        try:
            # Create interactive replay controls
            buttons_css = """
            <style>
            /* Play/Pause Button (single toggle button in the Play widget) */
            .widget-play .jupyter-button {
                color: white !important;
                background-color: #4285F4 !important; /* Blue */
                border-color: #4285F4 !important;
                font-weight: bold;
                width: 80px !important;
                height: 32px !important;
                position: relative;
            }

            /* Add text for Play/Pause states */
            .widget-play[playing="true"] .jupyter-button::after {
                content: "Pause";
                position: absolute;
                left: 10px;
            }

            .widget-play[playing="false"] .jupyter-button::after {
                content: "Play";
                position: absolute;
                left: 10px;
            }

            /* Hide the font-awesome icons that aren't showing */
            .widget-play .jupyter-button i {
                display: none !important;
            }

            /* Style for Next button */
            .next-button .jupyter-button {
                color: white !important;
                background-color: #F4B400 !important; /* Yellow */
                border-color: #F4B400 !important;
                font-weight: bold;
                width: 80px !important;
            }

            /* Style for Reset button */
            .reset-button .jupyter-button {
                color: white !important;
                background-color: #DB4437 !important; /* Red */
                border-color: #DB4437 !important;
                font-weight: bold;
                width: 80px !important;
            }
            </style>
            """
            display(HTML(buttons_css))

            # Create the controls with original functionality but with additional styling classes
            replay_output = widgets.Output()
            step_slider = widgets.IntSlider(value=0, min=0, max=len(call_sequence), step=1, description='Step:',
                                          continuous_update=False, orientation='horizontal', readout=True,
                                          readout_format='d', layout=widgets.Layout(width='500px'))

            # Use the original Play widget
            play_button = widgets.Play(value=0, min=0, max=len(call_sequence), step=1, interval=700)
            widgets.jslink((play_button, 'value'), (step_slider, 'value'))

            # Create custom buttons for Next and Reset with the specified class names
            next_button = widgets.Button(description="Next", layout=widgets.Layout(width='80px'),
                                      button_style='warning')
            next_button.add_class('next-button')  # Add class for CSS targeting

            reset_button = widgets.Button(description="Reset", layout=widgets.Layout(width='80px'),
                                        button_style='danger')
            reset_button.add_class('reset-button')  # Add class for CSS targeting

            # Button handlers for Next and Reset
            def next_clicked(b):
                if step_slider.value < len(call_sequence):
                    step_slider.value += 1

            def reset_clicked(b):
                play_button._playing = False  # Stop if playing
                step_slider.value = 0

            next_button.on_click(next_clicked)
            reset_button.on_click(reset_clicked)

            # Group controls together
            controls = widgets.HBox([play_button, next_button, reset_button, step_slider])

            def update_replay(change):
                step = change['new']
                with replay_output:
                    clear_output(wait=True)
                    if step == 0: display(HTML(f"<h3>Transaction Start</h3><p>Hash: {tx_hash}</p>"))
                    elif step <= len(call_sequence):
                        call = call_sequence[step-1]
                        html = f"<h3>Step {step} of {len(call_sequence)}</h3> <div style='display: flex; margin-bottom: 10px;'><div style='flex: 1;'><p><b>Call Type:</b> {call.get('type','N/A')}</p><p><b>From:</b> {call.get('from')}</p><p><b>To:</b> {call.get('to')}</p><p><b>Gas Used:</b> {call.get('gas_used',0):,}</p></div><div style='flex: 1;'><p><b>Call Depth:</b> {call.get('depth','N/A')}</p><p><b>PYUSD Call:</b> {'Yes' if call.get('is_pyusd') else 'No'}</p><p><b>Function:</b> {call.get('function') if call.get('function') else 'N/A'}</p><p><b>Status:</b> {'❌ Error: ' + call.get('error') if call.get('error') else '✅ Success'}</p></div></div>"
                        if call.get('params'):
                            html += "<div style='background-color: #050505; color:white; padding: 5px; border-radius: 5px;'><h3 style='color:white;'>Parameters:</h3><ul>"
                            for pk, pv in call['params'].items():
                                if pk in ['amount_formatted', 'to_address', 'from_address', 'spender_address']: html += f"<li><b>{pk.replace('_', ' ').title()}:</b> {pv}</li>"
                            html += "</ul></div>"
                        html += "<div style='margin-top: 15px;'><h4>Call Stack:</h4><div style='display: flex; align-items: center;'>"
                        current_call_depth = call.get('depth',0)
                        for i in range(current_call_depth + 1):
                            is_current = i == current_call_depth
                            is_pyusd_current = call.get('is_pyusd', False)
                            color = 'palegreen' if is_pyusd_current and is_current else ('lightgray' if is_current else '#e0e0e0')
                            border = f"2px solid {'green' if is_pyusd_current and is_current else ('gray' if is_current else 'darkgray')}"
                            size = '30px' if is_current else '25px'
                            html += f"<div style='width: {size}; height: {size}; margin-right: 5px; background-color: {color}; border: {border}; border-radius: 5px; display: flex; justify-content: center; align-items: center; font-size: 12px; color: black;'>{i}</div>"
                            if not is_current: html += "<div style='margin-right: 5px;'>→</div>"
                        html += "</div></div>"
                        display(HTML(html))
                        progress = step / len(call_sequence) * 100
                        display(HTML(f"""<div style="width: 100%; background-color: #f0f0f0; border-radius: 5px; margin-top: 20px;"><div style="width: {progress}%; height: 20px; background-color: #4CAF50; border-radius: 5px; text-align: center; color: white;">{progress:.1f}%</div></div>"""))
            step_slider.observe(update_replay, names='value')
            display(controls); display(replay_output); update_replay({'new': 0})
        except Exception as viz_err:
            console.print(f"[warning]Could not create transaction replay visualization: {viz_err}", style="warning")

    # Display summary table with interactive filtering
    if not summary_df.empty:
        console.print("\n\n[bold cyan3]📋 trace_transaction Details[/bold cyan3]")
        console.print("─────────────────────────────", style="cyan3")

        filter_output = widgets.Output(); table_output = widgets.Output()
        function_types = ['All'] + sorted(summary_df['function'].dropna().astype(str).unique().tolist())
        function_dropdown = widgets.Dropdown(options=function_types, value='All', description='Function:', layout=widgets.Layout(width='250px'))
        contract_types = ['All'] + sorted(summary_df['contract'].dropna().unique().tolist())
        contract_dropdown = widgets.Dropdown(options=contract_types, value='All', description='Contract:', layout=widgets.Layout(width='300px'))
        pyusd_checkbox = widgets.Checkbox(value=False, description='PYUSD Only')
        error_checkbox = widgets.Checkbox(value=False, description='Show Errors Only')
        filter_box = widgets.HBox([function_dropdown, contract_dropdown, pyusd_checkbox, error_checkbox])
        def update_table(change=None):
            with table_output:
                clear_output(wait=True)
                filtered_df = summary_df.copy()
                if function_dropdown.value != 'All': filtered_df = filtered_df[filtered_df['function'].astype(str) == function_dropdown.value]
                if contract_dropdown.value != 'All': filtered_df = filtered_df[filtered_df['contract'] == contract_dropdown.value]
                if pyusd_checkbox.value: filtered_df = filtered_df[filtered_df['is_pyusd']]
                if error_checkbox.value: filtered_df = filtered_df[filtered_df['error'] != 'None']

                def style_row(row):
                    styles = [''] * len(row)
                    if row['is_pyusd']:
                        styles = ['background-color: #dff0d8; color: black'] * len(row)
                    if row['error'] != 'None':
                         styles = ['background-color: #f2dede; color: #a94442'] * len(row)
                    return styles

                def color_gas_efficiency(val):
                    colors = {'excellent': 'green', 'good': 'darkgreen', 'average': 'orange', 'poor': 'red'}
                    return f'color: {colors.get(val)}'

                display_cols = ['type', 'from', 'to', 'is_pyusd', 'function', 'gas_str']
                if 'gas_efficiency' in filtered_df.columns: display_cols.append('gas_efficiency')
                display_cols.append('error')

                styled_df = filtered_df[display_cols].style.apply(style_row, axis=1)
                if 'gas_efficiency' in display_cols:
                    styled_df = styled_df.map(color_gas_efficiency, subset=['gas_efficiency'])

                display(HTML(f"<p>Showing {len(filtered_df)} of {len(summary_df)} actions</p>"))
                display(styled_df)

        function_dropdown.observe(update_table, names='value'); contract_dropdown.observe(update_table, names='value')
        pyusd_checkbox.observe(update_table, names='value'); error_checkbox.observe(update_table, names='value')
        display(widgets.HTML("<h4>Filter Options:</h4>")); display(filter_box); display(table_output); update_table()

        # Add export options
        console.print("\n\n[bold cyan3]📤 Export Options:[/bold cyan3]")
        console.print("──────────────────", style="cyan3")

        export_buttons = widgets.HBox([
            widgets.Button(description='Export to CSV', button_style='primary', layout=widgets.Layout(width='150px')),
            widgets.Button(description='Export as JSON', button_style='warning', layout=widgets.Layout(width='150px')),
            widgets.Button(description='Export to Google Sheets', button_style='info', layout=widgets.Layout(width='200px'))])
        export_output = widgets.Output()
        def export_csv(b):
            with export_output: clear_output(); ts = datetime.now().strftime("%Y%m%d_%H%M%S"); fname = f"transaction_{shorten_address(tx_hash)}_trace_{ts}.csv"; display(download_csv_direct(summary_df, fname))
        def export_json(b):
            with export_output:
                clear_output(); ts = datetime.now().strftime("%Y%m%d_%H%M%S"); fname = f"transaction_{shorten_address(tx_hash)}_trace_{ts}.json"
                records = summary_df.astype(str).to_dict('records')
                export_data = {"transaction_hash": tx_hash, "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                               "summary": {"total_actions": len(trace_list), "total_gas_used": total_gas_used, "errors": errors, "pyusd_interactions": pyusd_interactions, "pyusd_transfers": len(pyusd_transfers_display), "complexity_score": complexity_score, "transaction_pattern": tx_pattern['pattern'], "pattern_confidence": tx_pattern['confidence']},
                               "mev_analysis": mev_analysis, "security_concerns": security_concerns, "actions": records}
                display(download_json_direct(export_data, fname))
        def export_to_sheets(b):
            with export_output: clear_output(); sname = f"Tx {shorten_address(tx_hash)} Trace {datetime.now().strftime('%y%m%d_%H%M')}"; display(export_to_google_sheets_direct(summary_df, sname))
        export_buttons.children[0].on_click(export_csv); export_buttons.children[1].on_click(export_json); export_buttons.children[2].on_click(export_to_sheets)
        display(export_buttons); display(export_output)

    # Return the enhanced DataFrame with additional analysis metadata
    analysis_results = {
        "summary_df": summary_df,
        "contract_graph": contract_graph_plotly,
        "call_graph": call_graph_plotly,
        "flow_graph": flow_graph_plotly,
        "transaction_hash": tx_hash,
        "transaction_pattern": tx_pattern['pattern'],
        "complexity_score": complexity_score,
        "mev_potential": mev_analysis['mev_detected'],
        "security_concerns_count": len(security_concerns)
    }

    return analysis_results

# Execute trace_transaction with analysis
if 'TARGET_TX_HASH' in locals() and validate_tx_hash:
    console.print("\n\n[bold yellow3]📑 Tracing Transactions via 'trace_transaction'[/bold yellow3]")
    console.print("───────────────────────────────────────────────", style="yellow3")

    console.print(f"\n\n[info]Attempting enhanced 'trace_transaction' analysis for {TARGET_TX_HASH} on Mainnet...", style="info")

    trace_transaction_results = make_rpc_request("trace_transaction", [TARGET_TX_HASH], network='mainnet')

    if trace_transaction_results is not None:
        import traceback
        analysis_output = analyze_trace_transaction(trace_transaction_results, TARGET_TX_HASH)

        if analysis_output and isinstance(analysis_output, dict):
            trace_analysis_df = analysis_output.get("summary_df")

            if trace_analysis_df is not None and not trace_analysis_df.empty:
                 try:
                    tx_details = w3_mainnet.eth.get_transaction(TARGET_TX_HASH)
                    tx_receipt = w3_mainnet.eth.get_transaction_receipt(TARGET_TX_HASH)
                    if tx_details and tx_receipt:
                        console.print("\n\n[bold chartreuse1]📑 Transaction Context[/bold chartreuse1]")
                        console.print("───────────────────────", style="chartreuse1")

                        tx_table = Table(show_header=False, box=None, padding=(0, 1))
                        tx_table.add_column("Field"); tx_table.add_column("Value")
                        tx_table.add_row("[bold chartreuse1]Transaction Hash:[/bold chartreuse1]", TARGET_TX_HASH)
                        tx_table.add_row("[bold chartreuse1]Block:[/bold chartreuse1]", str(tx_receipt.blockNumber))
                        tx_table.add_row("[bold chartreuse1]From:[/bold chartreuse1]", shorten_address(tx_details['from']))
                        tx_table.add_row("[bold chartreuse1]To:[/bold chartreuse1]", shorten_address(tx_details.get('to')))
                        tx_table.add_row("[bold chartreuse1]Value:[/bold chartreuse1]", format_value_eth(tx_details['value']))
                        tx_table.add_row("[bold chartreuse1]Gas Price:[/bold chartreuse1]", f"{w3_mainnet.from_wei(tx_details['gasPrice'], 'gwei'):.2f} Gwei")
                        tx_table.add_row("[bold chartreuse1]Gas Used:[/bold chartreuse1]", f"{tx_receipt['gasUsed']:,}")
                        tx_table.add_row("[bold chartreuse1]Status:[/bold chartreuse1]", "[green3]Success[/green3]" if tx_receipt['status'] == 1 else "[red3]Failed[/red3]")
                        tx_position = None
                        try:
                            block = w3_mainnet.eth.get_block(tx_receipt.blockNumber, full_transactions=True)
                            for i, tx in enumerate(block.transactions):
                                if tx.hash.hex() == TARGET_TX_HASH: tx_position = i; break
                            if tx_position is not None:
                                tx_table.add_row("[bold chartreuse1]Position in Block:[/bold chartreuse1]", f"{tx_position + 1} of {len(block.transactions)}")
                                mev_risk, mev_color = "Low", "green3"
                                if 0 < tx_position < len(block.transactions) - 1: mev_risk, mev_color = "Medium", "yellow3"
                                if len(block.transactions) > 10 and 1 < tx_position < len(block.transactions) - 2: mev_risk, mev_color = "Higher", "red3"
                                tx_table.add_row("[bold chartreuse1]MEV Risk:[/bold chartreuse1]", f"[{mev_color}]{mev_risk}[/{mev_color}]")
                        except Exception as block_err:
                            console.print(f"[traceback]Could not get block details: {block_err}[/traceback]")
                        console.print(tx_table)
                        etherscan_url = f"https://etherscan.io/tx/{TARGET_TX_HASH}"
                        display(HTML(f'<p style="margin-top: 15px;"><a href="{etherscan_url}" target="_blank" style="color: #1E90FF; font-weight: bold; text-decoration: none;">🔗 View Transaction on Etherscan</a></p>'))
                 except Exception as tx_err:
                    console.print(f"[warning]Could not fetch additional transaction context: {tx_err}", style="warning")
            else:
                 console.print("[warning]Trace analysis failed or returned empty DataFrame, skipping context.", style="warning")
        else:
            console.print("[error]Analysis function failed to return expected results.", style="error")
    else:
        console.print(f"[error]Failed to get trace for {TARGET_TX_HASH} using 'trace_transaction'.", style="error")
else:
    console.print("[warning]TARGET_TX_HASH not set or invalid. Skipping trace_transaction analysis.", style="warning")

## 1.9 🔄 `trace_replayTransaction` and `trace_replayBlockTransactions`: State Replay Analysis (High Cost)
---

This section delves into the **most powerful (and computationally expensive)** tracing methods provided by GCP: `trace_replayTransaction` **and** `trace_replayBlockTransactions`. These allow re-executing a specific past transaction or all transactions in a past block *as if they were happening now* on the node, while enabling specific tracers to capture deep insights, most notably `stateDiff` for observing state changes.

*   **`trace_replayTransaction`:** Replays a single transaction (`TARGET_TX_HASH`).
*   **`trace_replayBlockTransactions`:** Replays all transactions in a block (`TARGET_BLOCK_IDENTIFIER`).

**Key Tracers Used Here:**

*   **`stateDiff`:** Records **every change** to account balances (ETH), nonces, code, and contract storage slots resulting from the execution. Essential for seeing the exact impact on PYUSD balances, allowances, supply, etc.
*   **`trace`:** Provides a call trace similar to `trace_transaction`, useful for correlating state changes with specific internal calls.
*   **`vmTrace`:** (Optional, can be selected) Offers an opcode-by-opcode execution trace like `structLog`, useful for extreme low-level debugging within the replay context.

> **⚠️ Extreme Cost Factor & GCP Value**
>
> *   **Methods:** `trace_replayTransaction`, `trace_replayBlockTransactions`
> *   **Multiplier:** `100x` (Each consumes 100x the quota/cost of a basic call)
> *   **GCP Advantage:** These methods are **extremely resource-intensive**, especially with the `stateDiff` tracer which generates significant output. GCP's infrastructure and generous free quotas make it feasible to run these replays on Mainnet, where they are often entirely unavailable or prohibitively expensive elsewhere.
> *   **PYUSD Insight:** Replay tracing enables:
>     *   **Precise State Change Tracking:** Observing exactly how PYUSD balances, total supply, allowances, and other contract state variables (like paused status) were modified by a transaction or across a whole block.
>     *   **Debugging Failed Transactions:** Understanding the exact state (balances, storage) that led to a revert, even if logs weren't emitted.
>     *   **Auditing Complex Operations:** Verifying the exact outcome and side effects of intricate PYUSD interactions (e.g., multi-step DeFi processes, bridge operations).
>     *   **Historical State Analysis:** Replaying transactions at different historical block heights to understand state evolution.

**Analysis Workflow (`trace_replayTransaction`):**

1.  **Request Replay:** Calls `trace_replayTransaction` with the `TARGET_TX_HASH` and selected tracers (controlled by `REPLAY_TRACERS`).
2.  **Analyze Result:** The `analyze_replay_transaction` function processes the returned dictionary.
3.  **Parse Tracers:**
    *   Analyzes `trace` output (if requested) for PYUSD calls, transfers, gas, and security flags.
    *   Analyzes `stateDiff` output (if requested) to identify and decode all state changes, specifically highlighting PYUSD contract modifications (balances, supply, storage).
    *   Analyzes `vmTrace` output (if requested) for opcode-level gas statistics.
4.  **Summarize & Visualize:** Presents findings through summary panels, tables detailing PYUSD interactions/transfers/state changes, security flag lists, and visualizations (token flow, gas usage). Includes export options.

**Analysis Workflow (`trace_replayBlockTransactions`):**

1.  **Request Block Replay:** Calls `trace_replayBlockTransactions` with the target block identifier (converted to hex) and selected tracers (controlled by `REPLAY_BLOCK_TRACERS`).
2.  **Process Transaction Results:** The `analyze_replay_block` function iterates through the list of replay results (one per transaction).
3.  **Aggregate & Analyze:** For each transaction's result, it extracts key PYUSD metrics (interactions, transfers, volume, state changes) and security flags by analyzing the chosen tracer outputs. It aggregates statistics across the entire block.
4.  **Visualize & Summarize:** Shows block-wide summaries, state change distributions, security flags, interactive transaction tables, PYUSD volume/activity plots, and export options.

**💡 What to Look For:**
*   **`stateDiff` Output:** This is the unique value. Focus on changes related to PYUSD: `totalSupply`, user `balances`, `allowances`, `owner`, `paused` state slots. Compare the `from` and `to` values for each change.
*   **Correlation:** Relate the state changes (`stateDiff`) back to the function calls observed in the `trace` output for the same transaction/block.
*   **Security Flags:** Pay attention to critical changes like ownership transfers, code modifications, large supply changes, or admin function calls detected in either tracer.
*   **(Block Replay):** Observe the *net effect* of the entire block on PYUSD state. Identify transactions with the largest impact on balances or supply.

In [ ]:
# =============================================================================================
# 🔄 PYUSD Replay Transaction using trace_replayTransaction and trace_replayBlockTransactions
# =============================================================================================
# These methods provide detailed execution tracing at significantly higher computational cost
# but deliver unparalleled insights into PYUSD transactions and state changes.
# This cell leverages high-cost replay methods (trace_replayTransaction, trace_replayBlockTransactions) for deep analysis:
# - Detailed state change analysis (`stateDiff`) showing PYUSD balance, storage, and supply modifications.
# - Execution tracing (`trace`) within the replay context to correlate calls with state changes.
# - Token flow visualization derived from trace or state diff, illustrating transfer patterns.
# - VM-level execution insights (if `vmTrace` option is used) for low-level debugging.
# - Security analysis identifying admin functions or critical state modifications (e.g., ownership, code changes).
# - Gas usage analysis within the replayed execution context.

import base64
from datetime import datetime
from IPython.display import HTML, clear_output, display
import json
import ipywidgets as widgets
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from graphviz import Digraph
from rich.panel import Panel
from rich.table import Table
import time
import re
from eth_utils import decode_hex, to_hex, to_int
from web3 import Web3

"""
IMPORTANT: Cost Warning ⚠️

trace_replayTransaction and trace_replayBlockTransactions are extremely computationally expensive
operations (approximately 100x the cost of regular trace methods). These methods are only available
for free through Google Cloud's Blockchain RPC service. On other providers, these would incur
significant costs and may even be unavailable.

Use these methods judiciously for detailed analysis rather than routine monitoring.

Key Features:
- Detailed execution tracing with different tracer options
- State change analysis showing how PYUSD balances and storage are modified
- Token flow visualization showing transfer patterns
- VM-level execution insights (with vmTrace option)

Available Tracers:
- "trace": Similar to trace_transaction but with additional context
- "stateDiff": Shows all state changes caused by the transaction (balances, storage, etc.)
- "vmTrace": Provides detailed VM execution trace with opcode-level information
"""

# Constants for PYUSD ERC20 Storage Layout Analysis
# These are the standard storage slots for PYUSD based on OpenZeppelin ERC20 implementation
# Actual slots may vary based on inheritance and implementation details
PYUSD_TOTAL_SUPPLY_SLOT = 0
PYUSD_BALANCES_SLOT = 1  # Mapping base - actual slot needs keccak256 hash of address + position
PYUSD_ALLOWANCES_SLOT = 2  # Mapping base - actual slot needs keccak256 hash
PYUSD_OWNER_SLOT = 3
PYUSD_PAUSED_SLOT = 4

# Map of known PYUSD operations to their gas profiles
PYUSD_GAS_PROFILE = {
    "transfer": {"low": 45000, "typical": 65000, "high": 80000},
    "transferFrom": {"low": 60000, "typical": 80000, "high": 100000},
    "mint": {"low": 90000, "typical": 120000, "high": 150000},
    "burn": {"low": 70000, "typical": 90000, "high": 120000},
    "approve": {"low": 40000, "typical": 46000, "high": 60000}
}

# Security colors for better dark theme visibility
SECURITY_COLORS = {
    "critical": "#ff5252", # Lighter red for dark themes
    "high": "#ff7070",     # Medium-light red
    "warning": "#ffd966",  # Amber yellow
    "info": "#80d8ff"      # Light blue
}

# DataTable pagination function for transaction displays
def display_transaction_dataframe(df, title="Transaction Data"):
    """
    Display a transaction DataFrame with pagination controls

    Args:
        df: DataFrame to display
        title: Title to show above the table
    """
    if df.empty:
        display(HTML(f"<h3>{title}</h3><p>No data available.</p>"))
        return

    # Create pagination controls
    page_size = widgets.Dropdown(
        options=[10, 20, 50, 100],
        value=20,
        description='No. of Rows:',
        disabled=False,
    )

    current_page = widgets.IntText(
        value=1,
        description='Page:',
        disabled=False,
        layout=widgets.Layout(width='120px')
    )

    prev_button = widgets.Button(
        description='Previous',
        disabled=True,
        button_style='info',
        layout=widgets.Layout(width='100px')
    )

    next_button = widgets.Button(
        description='Next',
        disabled=False,
        button_style='info',
        layout=widgets.Layout(width='100px')
    )

    controls = widgets.HBox([page_size, current_page, prev_button, next_button])
    output = widgets.Output()

    # Calculate total pages
    total_pages = max(1, (len(df) + page_size.value - 1) // page_size.value)
    page_info = widgets.HTML(f"<div>Page 1 of {total_pages} (Total rows: {len(df)})</div>")

    # Update function to refresh the table
    def update_table():
        with output:
            clear_output()
            start_idx = (current_page.value - 1) * page_size.value
            end_idx = min(start_idx + page_size.value, len(df))

            # Update button states
            prev_button.disabled = current_page.value <= 1
            next_button.disabled = current_page.value >= total_pages

            # Update page info
            page_info.value = f"<div>Page {current_page.value} of {total_pages} (Total rows: {len(df)})</div>"

            # Display slice of DataFrame
            display(df.iloc[start_idx:end_idx])

    # Button handlers
    def on_prev_clicked(b):
        current_page.value = max(1, current_page.value - 1)
        update_table()

    def on_next_clicked(b):
        current_page.value = min(total_pages, current_page.value + 1)
        update_table()

    def on_page_change(change):
        update_table()

    def on_page_size_change(change):
        nonlocal total_pages
        total_pages = max(1, (len(df) + change['new'] - 1) // change['new'])
        current_page.value = min(current_page.value, total_pages)
        update_table()

    # Connect event handlers
    prev_button.on_click(on_prev_clicked)
    next_button.on_click(on_next_clicked)
    current_page.observe(on_page_change, names='value')
    page_size.observe(on_page_size_change, names='value')

    # Display controls and initial table
    display(HTML(f"<h3>{title}</h3>"))
    display(widgets.VBox([controls, page_info, output]))
    update_table()

# Dedicated Export Functions
def download_csv_direct(df, filename=None):
    """Creates a direct download for CSV without intermediate display."""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"pyusd_data_{timestamp}.csv"

    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()

    # Create direct download HTML
    html = f'''
    <script>
    function download(filename, data) {{
        const a = document.createElement('a');
        a.href = data;
        a.download = filename;
        document.body.appendChild(a);
        a.click();
        document.body.removeChild(a);
    }}

    download("{filename}", "data:text/csv;base64,{payload}");
    </script>
    <div>Downloading {filename}...</div>
    '''
    return HTML(html)

def download_json_direct(data, filename=None):
    """Creates a direct download for JSON without intermediate display."""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"pyusd_data_{timestamp}.json"

    # Convert to JSON string (handling non-serializable objects)
    json_str = json.dumps(data, default=str, indent=2)
    b64 = base64.b64encode(json_str.encode()).decode()

    # Create direct download HTML
    html = f'''
    <script>
    function download(filename, data) {{
        const a = document.createElement('a');
        a.href = data;
        a.download = filename;
        document.body.appendChild(a);
        a.click();
        document.body.removeChild(a);
    }}

    download("{filename}", "data:application/json;base64,{b64}");
    </script>
    <div>Downloading {filename}...</div>
    '''
    return HTML(html)

def export_to_google_sheets_direct(df, sheet_name=None):
    """Exports DataFrame directly to Google Sheets using authenticated session."""
    if sheet_name is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        sheet_name = f"PYUSD Analysis {timestamp}"

    # Use Google Colab integration for direct export
    html = f'''
    <script src="https://apis.google.com/js/platform.js" async defer></script>
    <script>
    function createSheet() {{
        const csv = `{df.to_csv(index=False).replace('"', '""')}`;

        // Create sheet using Google Colab API
        google.colab.kernel.invokeFunction('notebook.createSheet', [csv, '{sheet_name}'], {{}});
    }}

    // Execute immediately
    setTimeout(createSheet, 100);
    </script>
    <div>Creating Google Sheet "{sheet_name}"...</div>
    '''

    # Register the Python callback that the JavaScript will call
    from google.colab import output

    @output.register_callback('notebook.createSheet')
    def create_sheet_callback(csv_data, name):
        try:
            from google.colab import auth
            from googleapiclient.discovery import build
            from googleapiclient.http import MediaInMemoryUpload
            import io

            # Ensure authentication
            auth.authenticate_user()

            # Create Drive API client
            drive_service = build('drive', 'v3')

            # File metadata
            file_metadata = {
                'name': name,
                'mimeType': 'application/vnd.google-apps.spreadsheet'
            }

            # Create CSV upload
            media = MediaInMemoryUpload(
                io.BytesIO(csv_data.encode('utf-8')),
                mimetype='text/csv',
                resumable=True
            )

            # Create the Sheet
            file = drive_service.files().create(
                body=file_metadata,
                media_body=media,
                fields='id,webViewLink'
            ).execute()

            # Return success with link
            return {
                'status': 'success',
                'file_id': file.get('id'),
                'link': file.get('webViewLink')
            }
        except Exception as e:
            return {
                'status': 'error',
                'message': str(e)
            }

    return HTML(html)

# Utility functions for replay analysis
def compute_storage_slot_for_address(mapping_position, address):
    """
    Calculate actual storage slot for an address in a mapping

    Args:
        mapping_position: Base position of the mapping in storage
        address: Ethereum address to calculate slot for

    Returns:
        String representation of the storage slot
    """
    # Convert address to bytes and pad to 32 bytes
    address_bytes = bytes.fromhex(address.replace('0x', '').lower())
    address_padded = address_bytes.rjust(32, b'\0')

    # Convert mapping position to bytes and pad to 32 bytes
    position_bytes = mapping_position.to_bytes(32, 'big')

    # Concatenate and hash to get the actual slot
    concatenated = address_padded + position_bytes
    slot_bytes = Web3.keccak(concatenated)

    # Convert to hex string
    return '0x' + slot_bytes.hex()

def categorize_gas_usage(gas_used, operation_type):
    """
    Categorize gas usage as low, typical, or high for a given operation

    Args:
        gas_used: Amount of gas used
        operation_type: Type of operation (transfer, mint, etc.)

    Returns:
        Category and efficiency rating
    """
    if operation_type not in PYUSD_GAS_PROFILE:
        return "unknown", 0

    profile = PYUSD_GAS_PROFILE[operation_type]

    if gas_used <= profile["low"]:
        return "excellent", 100
    elif gas_used <= profile["typical"]:
        efficiency = 100 - ((gas_used - profile["low"]) / (profile["typical"] - profile["low"])) * 20
        return "good", efficiency
    elif gas_used <= profile["high"]:
        efficiency = 80 - ((gas_used - profile["typical"]) / (profile["high"] - profile["typical"])) * 30
        return "average", efficiency
    else:
        excess = (gas_used - profile["high"]) / profile["high"]
        efficiency = max(0, 50 - (excess * 50))
        return "poor", efficiency

def interpret_pyusd_storage_slot(slot_hex, value_hex):
    """
    Interpret PYUSD storage slots and their values

    Args:
        slot_hex: Storage slot in hex
        value_hex: Value at that slot in hex

    Returns:
        Description, formatted value, and metadata
    """
    try:
        # Convert slot to integer
        slot_int = int(slot_hex, 16)
        value_int = int(value_hex, 16)

        # Standard ERC20 slots
        if slot_int == PYUSD_TOTAL_SUPPLY_SLOT:
            formatted_value = f"{value_int / (10**PYUSD_CONFIG['ethereum']['decimals']):,.6f} PYUSD"
            return "Total Supply", formatted_value, {"raw_value": value_int, "type": "total_supply"}

        elif slot_int == PYUSD_OWNER_SLOT:
            # Last 20 bytes should be the address
            address = '0x' + value_hex[-40:]
            return "Contract Owner", shorten_address(address), {"raw_value": address, "type": "owner"}

        elif slot_int == PYUSD_PAUSED_SLOT:
            is_paused = value_int > 0
            return "Paused State", "Paused" if is_paused else "Active", {"raw_value": is_paused, "type": "paused"}

        # Try to determine if it's a balance slot - this is complex and approximate
        else:
            # For demonstration - check if value looks like a balance (divisible by PYUSD decimals)
            if value_int % 10 == 0:  # Simple heuristic, could be improved
                formatted_value = f"{value_int / (10**PYUSD_CONFIG['ethereum']['decimals']):,.6f} PYUSD"
                return "Possible Balance", formatted_value, {"raw_value": value_int, "type": "balance"}

            return "Unknown Storage", f"0x{value_hex}", {"raw_value": value_int, "type": "unknown"}

    except Exception as e:
        return "Error Interpreting", str(e), {"error": str(e)}

# Main analysis functions
def analyze_replay_transaction(replay_result, tx_hash, requested_tracers):
    """
    Analyzes the output dictionary from trace_replayTransaction with PYUSD insights.

    Args:
        replay_result: The raw output from trace_replayTransaction
        tx_hash: The transaction hash being analyzed
        requested_tracers: List of tracer types that were requested in the RPC call

    Returns:
        Dictionary with PYUSD-relevant summary data
    """
    if not isinstance(replay_result, dict):
        console.print(f"[error]Expected a dict from trace_replayTransaction for {shorten_address(tx_hash)}, got {type(replay_result)}.", style="error")
        display_json(replay_result, "Unexpected Replay Result Structure")
        return None

    console.print(f"\n\n[bold cyan3]PYUSD Analysis of trace_replayTransaction for {shorten_address(tx_hash)}[/bold cyan3]")
    console.print(f"Requested Tracers: {', '.join(requested_tracers)}")

    summary_data = {
        'transaction_hash': tx_hash,
        'analysis_timestamp': datetime.now().isoformat(),
        'tracers_used': requested_tracers,
        'has_pyusd_interaction': False,
        'pyusd_state_changes': 0,
        'pyusd_balance_changes': [],
        'pyusd_transfers': [],
        'pyusd_operations': [],
        'gas_metrics': {},
        'security_flags': []
    }

    # DataFrames to store analysis results
    state_changes_df = pd.DataFrame()
    transfers_df = pd.DataFrame()
    vm_operations_df = pd.DataFrame()

    # --- Analyze 'trace' (if requested) ---
    if "trace" in requested_tracers and "trace" in replay_result:
        console.print("\n\n[bold cyan3]📊 PYUSD Trace Analysis[/bold cyan3]")
        console.print("────────────────────────", style="cyan3")

        # trace analysis looking for PYUSD transfers
        trace_list = replay_result["trace"]

        if isinstance(trace_list, list):
            pyusd_interactions = 0
            pyusd_calls_by_function = {}
            pyusd_transfers = []
            pyusd_operations = []
            total_gas_used = 0

            # Track sequence for visualization
            operation_sequence = []

            for i, trace_item in enumerate(trace_list):
                if not isinstance(trace_item, dict): continue

                action = trace_item.get('action', {})
                result = trace_item.get('result', {})
                trace_type = trace_item.get('type', 'N/A')
                trace_addr = trace_item.get('traceAddress', [])
                error = trace_item.get('error')

                to_addr = action.get('to', '')
                call_data = action.get('input', '0x')
                from_addr = action.get('from', '')
                gas_used_hex = result.get('gasUsed', '0x0')
                gas_used = int(gas_used_hex, 16) if gas_used_hex.startswith('0x') else int(gas_used_hex)
                total_gas_used += gas_used

                # Add to operation sequence for visualization
                operation_sequence.append({
                    'index': i,
                    'trace_type': trace_type,
                    'from': from_addr,
                    'to': to_addr,
                    'gas_used': gas_used,
                    'trace_addr': trace_addr,
                    'error': error
                })

                # Check if this is a PYUSD contract interaction
                is_pyusd_call = to_addr and to_addr.lower() in PYUSD_CONTRACTS
                contract_name = PYUSD_CONTRACTS.get(to_addr.lower(), "Other Contract") if is_pyusd_call else "External Contract"

                if is_pyusd_call:
                    pyusd_interactions += 1
                    summary_data['has_pyusd_interaction'] = True

                    # Analyze function call
                    if call_data and call_data != '0x':
                        method_sig = call_data[:10]
                        method_info = {"signature": method_sig, "name": "Unknown", "category": "unknown"}

                        if method_sig in PYUSD_SIGNATURES:
                            function_name = PYUSD_SIGNATURES[method_sig]["name"]
                            function_category = PYUSD_SIGNATURES[method_sig]["category"]
                            method_info = {
                                "signature": method_sig,
                                "name": function_name,
                                "category": function_category
                            }

                            # Track function call
                            if function_name in pyusd_calls_by_function:
                                pyusd_calls_by_function[function_name] += 1
                            else:
                                pyusd_calls_by_function[function_name] = 1

                            # Security analysis - check for admin functions
                            if function_category == "admin":
                                summary_data['security_flags'].append({
                                    'level': 'warning',
                                    'type': 'admin_function',
                                    'description': f"Admin function {function_name} called on {contract_name}",
                                    'details': {
                                        'function': function_name,
                                        'contract': contract_name,
                                        'from': from_addr
                                    }
                                })

                            # Track call with gas metrics
                            operation_info = {
                                'index': i,
                                'from': from_addr,
                                'to': to_addr,
                                'contract': contract_name,
                                'function': function_name,
                                'category': function_category,
                                'gas_used': gas_used,
                                'error': error,
                                'input_data': call_data,
                                'trace_addr': trace_addr
                            }

                            # Add gas efficiency analysis
                            if function_name.startswith("transfer") or function_name.startswith("mint") or function_name.startswith("burn") or function_name.startswith("approve"):
                                # Get base operation type (transfer, mint, etc.)
                                base_op = function_name.split("(")[0]
                                gas_category, efficiency = categorize_gas_usage(gas_used, base_op)
                                operation_info['gas_category'] = gas_category
                                operation_info['gas_efficiency'] = efficiency

                            pyusd_operations.append(operation_info)

                            # Extract PYUSD transfers
                            if method_sig == '0xa9059cbb':  # transfer
                                try:
                                    to_offset = 10
                                    to_param = "0x" + call_data[to_offset+24:to_offset+64]
                                    amount_offset = 74
                                    amount = int(call_data[amount_offset:amount_offset+64], 16)
                                    value_pyusd = amount / (10**PYUSD_CONFIG['ethereum']['decimals'])

                                    transfer_data = {
                                        'from': from_addr,
                                        'to': to_param,
                                        'value': value_pyusd,
                                        'value_raw': amount,
                                        'function': 'transfer',
                                        'trace_index': i,
                                        'trace_addr': trace_addr,
                                        'gas_used': gas_used,
                                        'error': error
                                    }
                                    pyusd_transfers.append(transfer_data)
                                    summary_data['pyusd_transfers'].append(transfer_data)
                                except Exception:
                                    pass
                            elif method_sig == '0x23b872dd':  # transferFrom
                                try:
                                    from_offset = 10
                                    from_param = "0x" + call_data[from_offset+24:from_offset+64]
                                    to_offset = 74
                                    to_param = "0x" + call_data[to_offset+24:to_offset+64]
                                    amount_offset = 138
                                    amount = int(call_data[amount_offset:amount_offset+64], 16)
                                    value_pyusd = amount / (10**PYUSD_CONFIG['ethereum']['decimals'])

                                    transfer_data = {
                                        'from': from_param,
                                        'to': to_param,
                                        'value': value_pyusd,
                                        'value_raw': amount,
                                        'function': 'transferFrom',
                                        'trace_index': i,
                                        'trace_addr': trace_addr,
                                        'gas_used': gas_used,
                                        'error': error
                                    }
                                    pyusd_transfers.append(transfer_data)
                                    summary_data['pyusd_transfers'].append(transfer_data)
                                except Exception:
                                    pass
                            elif method_sig == '0x40c10f19':  # mint
                                try:
                                    to_offset = 10
                                    to_param = "0x" + call_data[to_offset+24:to_offset+64]
                                    amount_offset = 74
                                    amount = int(call_data[amount_offset:amount_offset+64], 16)
                                    value_pyusd = amount / (10**PYUSD_CONFIG['ethereum']['decimals'])

                                    transfer_data = {
                                        'from': '0x0000000000000000000000000000000000000000',  # Mint from zero address
                                        'to': to_param,
                                        'value': value_pyusd,
                                        'value_raw': amount,
                                        'function': 'mint',
                                        'trace_index': i,
                                        'trace_addr': trace_addr,
                                        'gas_used': gas_used,
                                        'error': error
                                    }
                                    pyusd_transfers.append(transfer_data)
                                    summary_data['pyusd_transfers'].append(transfer_data)

                                    # Security flag for mint operations
                                    summary_data['security_flags'].append({
                                        'level': 'info',
                                        'type': 'supply_change',
                                        'description': f"PYUSD supply increased by {value_pyusd:,.2f}",
                                        'details': {
                                            'function': 'mint',
                                            'amount': value_pyusd,
                                            'to': to_param
                                        }
                                    })
                                except Exception:
                                    pass
                            elif method_sig == '0x42966c68':  # burn
                                try:
                                    amount_offset = 10
                                    amount = int(call_data[amount_offset:amount_offset+64], 16)
                                    value_pyusd = amount / (10**PYUSD_CONFIG['ethereum']['decimals'])

                                    transfer_data = {
                                        'from': from_addr,
                                        'to': '0x0000000000000000000000000000000000000000',  # Burn to zero address
                                        'value': value_pyusd,
                                        'value_raw': amount,
                                        'function': 'burn',
                                        'trace_index': i,
                                        'trace_addr': trace_addr,
                                        'gas_used': gas_used,
                                        'error': error
                                    }
                                    pyusd_transfers.append(transfer_data)
                                    summary_data['pyusd_transfers'].append(transfer_data)

                                    # Security flag for burn operations
                                    summary_data['security_flags'].append({
                                        'level': 'info',
                                        'type': 'supply_change',
                                        'description': f"PYUSD supply decreased by {value_pyusd:,.2f}",
                                        'details': {
                                            'function': 'burn',
                                            'amount': value_pyusd,
                                            'from': from_addr
                                        }
                                    })
                                except Exception:
                                    pass

            # Store operations for export
            summary_data['pyusd_operations'] = pyusd_operations
            if pyusd_operations:
                vm_operations_df = pd.DataFrame(pyusd_operations)

            # Store transfer data for export
            if pyusd_transfers:
                transfers_df = pd.DataFrame(pyusd_transfers)

            # Add gas metrics
            summary_data['gas_metrics'] = {
                'total_gas_used': total_gas_used,
                'pyusd_operations_count': len(pyusd_operations),
                'pyusd_transfers_count': len(pyusd_transfers),
                'pyusd_calls_by_function': pyusd_calls_by_function
            }

            # Display PYUSD function call summary
            if pyusd_calls_by_function:
                console.print(f"[success]Found {pyusd_interactions} PYUSD interactions in trace.", style="success")

                # Create function call table
                function_table = Table(show_header=True, header_style="bold green3")
                function_table.add_column("Function")
                function_table.add_column("Count", justify="right")
                function_table.add_column("Category", justify="center")
                function_table.add_column("Total Gas", justify="right")

                for func, count in sorted(pyusd_calls_by_function.items(), key=lambda x: x[1], reverse=True):
                    # Calculate total gas for this function
                    func_gas = sum([op['gas_used'] for op in pyusd_operations if op.get('function') == func])

                    # Get function category
                    category = "unknown"
                    for op in pyusd_operations:
                        if op.get('function') == func:
                            category = op.get('category', "unknown")
                            break

                    function_table.add_row(
                        func,
                        str(count),
                        category.replace('_', ' ').title(),
                        f"{func_gas:,}"
                    )

                console.print(function_table)
            else:
                console.print("[info]No PYUSD function calls detected in trace.", style="info")

            # Display PYUSD transfers
            if pyusd_transfers:
                console.print("\n\n[bold cyan3]🔄 PYUSD Token Movements Detected[/bold cyan3]")
                console.print("────────────────────────────────", style="cyan3")

                transfer_table = Table(show_header=True, header_style="bold cyan3")
                transfer_table.add_column("From")
                transfer_table.add_column("To")
                transfer_table.add_column("Amount", justify="right")
                transfer_table.add_column("Type", justify="center")
                transfer_table.add_column("Gas", justify="right")

                total_volume = 0
                for transfer in pyusd_transfers:
                    from_addr = transfer['from']
                    to_addr = transfer['to']
                    amount = transfer['value']
                    function = transfer['function']
                    gas = transfer['gas_used']
                    total_volume += amount

                    # Determine operation type for better readability
                    op_type = function
                    if function == 'transfer' or function == 'transferFrom':
                        op_type = "Transfer"
                    elif function == 'mint':
                        op_type = "Mint"
                        from_addr = "[italic]New Supply[/italic]"
                    elif function == 'burn':
                        op_type = "Burn"
                        to_addr = "[italic]Removed[/italic]"

                    transfer_table.add_row(
                        from_addr,
                        to_addr,
                        f"{amount:,.6f} PYUSD",
                        op_type,
                        f"{gas:,}"
                    )

                console.print(transfer_table)
                console.print(f"[info]Total PYUSD volume: {total_volume:,.6f} PYUSD", style="info")

                # Create enhanced token flow visualization
                try:
                    console.print("\n\n[bold magenta3]PYUSD Flow Chart:[/bold magenta3]")
                    console.print("──────────────────", style="magenta3")

                    # 1. Traditional graph visualization
                    flow_graph = Digraph(comment=f"PYUSD Flows in {tx_hash}", format='png')
                    flow_graph.attr(rankdir='TB', bgcolor='transparent')
                    flow_graph.attr('node', shape='box', style='filled', fontname='helvetica',
                                   fontcolor='black')

                    # Track nodes we've added
                    added_nodes = set()

                    # Aggregate transfers between same addresses
                    transfer_map = {}

                    for transfer in pyusd_transfers:
                        from_addr = transfer['from']
                        to_addr = transfer['to']
                        value = transfer['value']
                        function = transfer['function']

                        key = (from_addr, to_addr, function)
                        if key in transfer_map:
                            transfer_map[key] += value
                        else:
                            transfer_map[key] = value

                    # Add nodes and edges
                    for (from_addr, to_addr, function), total_value in transfer_map.items():
                        from_addr_short = shorten_address(from_addr)
                        to_addr_short = shorten_address(to_addr)

                        # Set node colors based on function
                        zero_addr = '0x0000000000000000000000000000000000000000'

                        # Special handling for mint/burn
                        if function == 'mint' and from_addr == zero_addr:
                            if from_addr not in added_nodes:
                                flow_graph.node(from_addr, label="New Supply", fillcolor="lightblue")
                                added_nodes.add(from_addr)
                        elif function == 'burn' and to_addr == zero_addr:
                            if to_addr not in added_nodes:
                                flow_graph.node(to_addr, label="Burned", fillcolor="lightsalmon")
                                added_nodes.add(to_addr)
                        else:
                            # Regular addresses
                            if from_addr not in added_nodes:
                                flow_graph.node(from_addr, label=from_addr_short, fillcolor="palegreen")
                                added_nodes.add(from_addr)

                            if to_addr not in added_nodes:
                                flow_graph.node(to_addr, label=to_addr_short, fillcolor="palegreen")
                                added_nodes.add(to_addr)

                        value_str = f"{total_value:,.2f} PYUSD"

                        # Edge style based on function type
                        edge_color = "black"
                        edge_style = "solid"
                        if function == 'mint':
                            edge_color = "blue"
                            edge_style = "dashed"
                        elif function == 'burn':
                            edge_color = "red"
                            edge_style = "dashed"

                        flow_graph.edge(from_addr, to_addr, label=value_str, color=edge_color, style=edge_style)

                    display(flow_graph)

                    # 2. Create Sankey diagram for better flow visualization
                    console.print("\n\n[bold magenta3]PYUSD Token Flow in Transaction Chart:[/bold magenta3]")
                    console.print("────────────────────────────────────", style="magenta3")

                    if len(pyusd_transfers) > 0:
                        # Prepare data for Sankey
                        unique_addresses = set()
                        for transfer in pyusd_transfers:
                            unique_addresses.add(transfer['from'])
                            unique_addresses.add(transfer['to'])

                        # Create mapping from address to index
                        address_to_idx = {addr: i for i, addr in enumerate(unique_addresses)}

                        # Prepare sources, targets, values
                        sources = []
                        targets = []
                        values = []
                        labels = []
                        colors = []

                        for addr in unique_addresses:
                            if addr == '0x0000000000000000000000000000000000000000':
                                labels.append('Zero Address')
                                colors.append('rgba(211, 211, 211, 0.8)')  # Light gray for zero address
                            else:
                                labels.append(shorten_address(addr))
                                colors.append('rgba(144, 238, 144, 0.8)')  # Light green for normal addresses

                        for transfer in pyusd_transfers:
                            sources.append(address_to_idx[transfer['from']])
                            targets.append(address_to_idx[transfer['to']])
                            values.append(transfer['value'])

                        # Create Sankey diagram
                        fig = go.Figure(data=[go.Sankey(
                            node=dict(
                                pad=15,
                                thickness=20,
                                line=dict(color="black", width=0.5),
                                label=labels,
                                color=colors
                            ),
                            link=dict(
                                source=sources,
                                target=targets,
                                value=values,
                                color=['rgba(143, 188, 143, 0.4)'] * len(sources)  # Semi-transparent green
                            )
                        )])

                        fig.update_layout(
                            title_text=f"PYUSD Token Flow in Transaction {shorten_address(tx_hash)}",
                            font_size=12
                        )

                        fig.show()

                    console.print("[info]These visualizations show PYUSD token flows in the transaction.", style="info")
                except Exception as viz_err:
                    console.print(f"[warning]Could not create PYUSD flow visualization: {viz_err}", style="warning")

                # Create a transaction replay visualization
                if operation_sequence:
                    console.print("\n\n[bold magenta3]📊 Transaction Execution Sequence Chart:[/bold magenta3]")
                    console.print("────────────────────────────────────────", style="magenta3")

                    try:
                        # Create a timeline visualization
                        timeline_df = pd.DataFrame(operation_sequence)

                        # Add colors based on trace type and if it's a PYUSD operation
                        timeline_df['color'] = 'lightgray'
                        for i, op in enumerate(operation_sequence):
                            # Check if this is a PYUSD operation
                            is_pyusd_op = False
                            for pyusd_op in pyusd_operations:
                                if pyusd_op['index'] == op['index']:
                                    is_pyusd_op = True
                                    break

                            if is_pyusd_op:
                                timeline_df.loc[i, 'color'] = 'lightgreen'
                            elif op['error']:
                                timeline_df.loc[i, 'color'] = 'lightcoral'

                        # Pre-format error values for better hover display
                        timeline_df['error_display'] = timeline_df['error'].fillna('None').astype(str)
                        timeline_df.loc[timeline_df['error_display'] == 'nan', 'error_display'] = 'None'

                        # Create the figure
                        fig = px.bar(
                            timeline_df,
                            x='index',
                            y='gas_used',
                            color='color',
                            color_discrete_map='identity',
                            title=f"Transaction Execution Sequence ({tx_hash})",
                            labels={'index': 'Operation Index', 'gas_used': 'Gas Used'},
                            hover_data=['trace_type', 'from', 'to', 'error_display']
                        )

                        # Improve layout
                        fig.update_layout(
                            showlegend=False,
                            xaxis_title="Operation Sequence",
                            yaxis_title="Gas Used",
                            template="plotly_white"
                        )

                        # Add custom hover template
                        fig.update_traces(
                            hovertemplate=(
                                "<b>Operation %{x}</b><br>" +
                                "Type: %{customdata[0]}<br>" +
                                "From: %{customdata[1]}<br>" +
                                "To: %{customdata[2]}<br>" +
                                "Gas: %{y:,}<br>" +
                                "Error: %{customdata[3]}"
                            )
                        )

                        fig.show()

                    except Exception as viz_err:
                        console.print(f"[warning]Could not create execution timeline visualization: {viz_err}", style="warning")

            summary_data['trace_actions'] = len(trace_list)
        else:
            console.print("[warning]Trace data is not in expected list format.", style="warning")
            display_json(trace_list, "Unexpected Trace Format")
            summary_data['trace_actions'] = 0

    # --- Analyze 'stateDiff' (if requested) with PYUSD focus ---
    if "stateDiff" in requested_tracers and "stateDiff" in replay_result:
        console.print("\n\n[bold cyan3]🔍 PYUSD State Change Analysis[/bold cyan3]")
        console.print("───────────────────────────────", style="cyan3")

        state_diff = replay_result["stateDiff"]
        state_changes = []
        pyusd_state_changes = []
        pyusd_balance_changes = []
        supply_changes = []

        if isinstance(state_diff, dict):
            # PYUSD token state changes counter
            pyusd_storage_changes = 0

            # Address of the PYUSD token contract (lowercase for comparison)
            pyusd_addr_lower = PYUSD_CONFIG['ethereum']['address'].lower()

            # Process all state changes in the diff
            for address, diffs in state_diff.items():
                address_lower = address.lower()
                is_pyusd_contract = address_lower == pyusd_addr_lower
                contract_name = PYUSD_CONTRACTS.get(address_lower, "Other Contract")

                # Process each type of state change
                if 'balance' in diffs and '*' in diffs['balance']:
                    bal_diff = diffs['balance']['*']
                    from_wei_hex = bal_diff.get('from', '0x0')
                    to_wei_hex = bal_diff.get('to', '0x0')

                    # Safe conversion to integers
                    from_wei = int(from_wei_hex, 16) if from_wei_hex.startswith('0x') else int(from_wei_hex)
                    to_wei = int(to_wei_hex, 16) if to_wei_hex.startswith('0x') else int(to_wei_hex)
                    change_wei = to_wei - from_wei

                    # Safe calculation of ETH value
                    if change_wei < 0:
                        change_eth = -float(w3_mainnet.from_wei(abs(change_wei), 'ether')) if w3_mainnet else -(abs(change_wei) / 1e18)
                    else:
                        change_eth = float(w3_mainnet.from_wei(change_wei, 'ether')) if w3_mainnet else (change_wei / 1e18)

                    change_entry = {
                        'address': address,
                        'type': 'balance',
                        'is_pyusd': is_pyusd_contract,
                        'contract': contract_name,
                        'slot/key': 'ETH',
                        'from_value': format_value_eth(from_wei_hex),
                        'to_value': format_value_eth(to_wei_hex),
                        'change': f"{change_eth:+.6f} ETH"
                    }
                    state_changes.append(change_entry)

                    if is_pyusd_contract:
                        pyusd_state_changes.append(change_entry)

                # Process nonce changes
                if 'nonce' in diffs and '*' in diffs['nonce']:
                    nonce_diff = diffs['nonce']['*']
                    from_nonce_hex = nonce_diff.get('from', '0x0')
                    to_nonce_hex = nonce_diff.get('to', '0x0')
                    from_nonce = int(from_nonce_hex, 16) if from_nonce_hex.startswith('0x') else int(from_nonce_hex)
                    to_nonce = int(to_nonce_hex, 16) if to_nonce_hex.startswith('0x') else int(to_nonce_hex)

                    change_entry = {
                        'address': address,
                        'type': 'nonce',
                        'is_pyusd': is_pyusd_contract,
                        'contract': contract_name,
                        'slot/key': 'Nonce',
                        'from_value': str(from_nonce),
                        'to_value': str(to_nonce),
                        'change': f"{to_nonce - from_nonce:+d}"
                    }
                    state_changes.append(change_entry)

                    if is_pyusd_contract:
                        pyusd_state_changes.append(change_entry)

                # Process code changes
                if 'code' in diffs and '*' in diffs['code']:
                    code_diff = diffs['code']['*']
                    change_type = "Code Created" if code_diff.get('to') and not code_diff.get('from') else \
                                 "Code Destroyed" if code_diff.get('from') and not code_diff.get('to') else \
                                 "Code Changed"

                    change_entry = {
                        'address': address,
                        'type': 'code',
                        'is_pyusd': is_pyusd_contract,
                        'contract': contract_name,
                        'slot/key': 'Bytecode',
                        'from_value': '(Exists)' if code_diff.get('from') else '(None)',
                        'to_value': '(Exists)' if code_diff.get('to') else '(None)',
                        'change': change_type
                    }
                    state_changes.append(change_entry)

                    if is_pyusd_contract:
                        pyusd_state_changes.append(change_entry)

                        # Add security flag for code changes
                        summary_data['security_flags'].append({
                            'level': 'critical',
                            'type': 'code_change',
                            'description': f"PYUSD contract code was modified - {change_type}",
                            'details': {
                                'contract': contract_name,
                                'address': address,
                                'change_type': change_type
                            }
                        })

                # Process storage changes - particularly important for PYUSD balances
                if 'storage' in diffs:
                    for slot, slot_diff_outer in diffs['storage'].items():
                        if '*' in slot_diff_outer:
                            slot_diff = slot_diff_outer['*']
                            from_val = slot_diff.get('from', '0x0')
                            to_val = slot_diff.get('to', '0x0')

                            # Enhanced storage slot interpretation
                            slot_interpretation, formatted_value, slot_metadata = interpret_pyusd_storage_slot(slot, to_val)

                            # Track PYUSD contract storage changes
                            if is_pyusd_contract:
                                pyusd_storage_changes += 1

                                # Check for important changes like total supply
                                if slot_metadata['type'] == 'total_supply':
                                    # Calculate value change
                                    from_supply = int(from_val, 16) / (10**PYUSD_CONFIG['ethereum']['decimals'])
                                    to_supply = int(to_val, 16) / (10**PYUSD_CONFIG['ethereum']['decimals'])
                                    supply_change = to_supply - from_supply

                                    supply_changes.append({
                                        'from': from_supply,
                                        'to': to_supply,
                                        'change': supply_change
                                    })

                                    # Add security flag for supply changes
                                    if abs(supply_change) > 0:
                                        summary_data['security_flags'].append({
                                            'level': 'info',
                                            'type': 'total_supply_change',
                                            'description': f"PYUSD totalSupply changed by {supply_change:+,.2f}",
                                            'details': {
                                                'from': from_supply,
                                                'to': to_supply,
                                                'change': supply_change
                                            }
                                        })

                                elif slot_metadata['type'] == 'balance':
                                    # Track balance changes
                                    from_balance = int(from_val, 16) / (10**PYUSD_CONFIG['ethereum']['decimals'])
                                    to_balance = int(to_val, 16) / (10**PYUSD_CONFIG['ethereum']['decimals'])
                                    balance_change = to_balance - from_balance

                                    pyusd_balance_changes.append({
                                        'slot': slot,
                                        'from': from_balance,
                                        'to': to_balance,
                                        'change': balance_change
                                    })
                                    summary_data['pyusd_balance_changes'].append({
                                        'slot': slot,
                                        'from': from_balance,
                                        'to': to_balance,
                                        'change': balance_change
                                    })

                                    slot_interpretation = f"Balance Change: {balance_change:+.6f} PYUSD"

                                elif slot_metadata['type'] == 'owner':
                                    # Add security flag for ownership changes
                                    summary_data['security_flags'].append({
                                        'level': 'critical',
                                        'type': 'ownership_change',
                                        'description': f"PYUSD contract ownership changed",
                                        'details': {
                                            'from': from_val,
                                            'to': to_val
                                        }
                                    })

                                elif slot_metadata['type'] == 'paused':
                                    # Add security flag for pause state changes
                                    new_state = "Paused" if slot_metadata['raw_value'] else "Unpaused"
                                    summary_data['security_flags'].append({
                                        'level': 'warning',
                                        'type': 'pause_state_change',
                                        'description': f"PYUSD contract is now {new_state}",
                                        'details': {
                                            'new_state': new_state,
                                            'raw_value': slot_metadata['raw_value']
                                        }
                                    })

                            change_entry = {
                                'address': address,
                                'type': 'storage',
                                'is_pyusd': is_pyusd_contract,
                                'contract': contract_name,
                                'slot/key': slot,
                                'from_value': from_val,
                                'to_value': to_val,
                                'change': slot_interpretation
                            }
                            state_changes.append(change_entry)

                            if is_pyusd_contract:
                                pyusd_state_changes.append(change_entry)

            # Update summary data
            summary_data['pyusd_state_changes'] = pyusd_storage_changes

            # Store state changes for export
            if state_changes:
                state_changes_df = pd.DataFrame(state_changes)

            # Display state changes with PYUSD focus
            if state_changes:
                state_diff_df = pd.DataFrame(state_changes)

                console.print(f"[info]Found {len(state_diff_df)} total state changes.", style="info")

                # Show PYUSD-specific changes first
                if pyusd_state_changes:
                    console.print(f"[success]Found {len(pyusd_state_changes)} PYUSD state changes.", style="success")
                    pyusd_df = pd.DataFrame(pyusd_state_changes)
                    display_cols = ['address', 'type', 'slot/key', 'from_value', 'to_value', 'change']

                    # Ensure columns exist
                    display_cols = [col for col in display_cols if col in pyusd_df.columns]

                    console.print("[bold green]PYUSD State Changes:[/bold green]")
                    display(pyusd_df[display_cols].head(20))

                    if len(pyusd_df) > 20:
                        console.print(f"[info]Showing first 20 of {len(pyusd_df)} PYUSD state changes.", style="info")

                    # Display PYUSD balance changes if found
                    if pyusd_balance_changes:
                        console.print(f"\n[bold green]PYUSD Balance Changes Detected ({len(pyusd_balance_changes)}):[/bold green]")
                        balance_df = pd.DataFrame(pyusd_balance_changes)
                        display(balance_df)

                    # Display supply changes if found
                    if supply_changes:
                        console.print("\n[bold green]PYUSD Supply Changes:[/bold green]")
                        for change in supply_changes:
                            console.print(f"Total Supply: {change['from']:,.6f} → {change['to']:,.6f} PYUSD (Change: {change['change']:+,.6f} PYUSD)")

                # Display all state changes with improved pagination
                console.print("\n\n[bold cyan3]📈 State Changes:[/bold cyan3]")
                console.print("─────────────────", style="cyan3")

                display_cols = ['address', 'contract', 'type', 'slot/key', 'from_value', 'to_value', 'change']

                # Ensure columns exist
                display_cols = [col for col in display_cols if col in state_diff_df.columns]

                # Use pagination display
                display_transaction_dataframe(state_diff_df[display_cols])

                summary_data['state_diff_entries'] = len(state_diff_df)
            else:
                console.print("[info]No state differences found.", style="info")
                summary_data['state_diff_entries'] = 0
        else:
            console.print("[warning]stateDiff format not as expected (expected dict).", style="warning")
            display_json(state_diff, "Unexpected stateDiff Structure")

    # --- Analyze 'vmTrace' (if requested) with PYUSD focus ---
    if "vmTrace" in requested_tracers and "vmTrace" in replay_result:
        console.print("\n[bold cyan3]🔍 VM Execution Analysis[/bold cyan3]")
        vm_trace = replay_result["vmTrace"]
        if isinstance(vm_trace, dict) and 'ops' in vm_trace:
            ops_list = vm_trace['ops']
            console.print(f"VM Trace contains {len(ops_list)} opcode steps.")

            # Analyze gas usage
            vm_gas_used = vm_trace.get('gasUsed', 'N/A')
            if isinstance(vm_gas_used, str) and vm_gas_used.startswith('0x'):
                vm_gas_used = int(vm_gas_used, 16)
            formatted_gas = format_gas(vm_gas_used)
            console.print(f"VM Trace Gas Used: {formatted_gas}")

            # Enhanced VM operations analysis
            pyusd_ops = 0
            sstore_count = 0
            gas_by_opcode = {}
            memory_operations = 0
            stack_operations = 0

            for op in ops_list:
                if 'op' in op:
                    opcode = op['op']
                    gas = op.get('gas', 0)

                    # Track gas usage by opcode
                    if opcode in gas_by_opcode:
                        gas_by_opcode[opcode]['count'] += 1
                        gas_by_opcode[opcode]['gas'] += gas
                    else:
                        gas_by_opcode[opcode] = {'count': 1, 'gas': gas}

                    # Count storage operations
                    if opcode == 'SSTORE':
                        sstore_count += 1

                    # Count memory operations
                    if 'MEM' in opcode or opcode in ['MLOAD', 'MSTORE', 'MSTORE8']:
                        memory_operations += 1

                    # Count stack operations
                    if opcode.startswith('PUSH') or opcode.startswith('DUP') or opcode.startswith('SWAP'):
                        stack_operations += 1

            # Create a table of top gas-using opcodes
            if gas_by_opcode:
                console.print("\n[bold green]Top Gas-Consuming Opcodes:[/bold green]")

                opcode_table = Table(show_header=True, header_style="bold green")
                opcode_table.add_column("Opcode", style="dim")
                opcode_table.add_column("Count", justify="right")
                opcode_table.add_column("Gas Used", justify="right")
                opcode_table.add_column("% of Total", justify="right")

                # Sort by gas usage
                sorted_opcodes = sorted(gas_by_opcode.items(), key=lambda x: x[1]['gas'], reverse=True)

                # Show top 10
                for opcode, data in sorted_opcodes[:10]:
                    gas_pct = (data['gas'] / vm_gas_used * 100) if isinstance(vm_gas_used, int) else 0
                    opcode_table.add_row(
                        opcode,
                        str(data['count']),
                        f"{data['gas']:,}",
                        f"{gas_pct:.1f}%"
                    )

                console.print(opcode_table)

            console.print(f"Storage write operations (SSTORE): {sstore_count}")
            console.print(f"Memory operations: {memory_operations}")
            console.print(f"Stack operations: {stack_operations}")

            summary_data['vm_trace_steps'] = len(ops_list)
            summary_data['vm_sstore_count'] = sstore_count
            summary_data['vm_operations'] = {
                'storage_writes': sstore_count,
                'memory_operations': memory_operations,
                'stack_operations': stack_operations,
                'top_gas_opcodes': [{'opcode': op, 'count': data['count'], 'gas': data['gas']}
                                  for op, data in sorted_opcodes[:5]] if 'sorted_opcodes' in locals() else []
            }
        else:
            console.print("[warning]vmTrace format not as expected (expected dict with 'ops').", style="warning")
            display_json(vm_trace, "Unexpected vmTrace Structure")

    # Final PYUSD summary with enhanced security insights
    console.print("\n\n[bold yellow3]PYUSD Transaction Summary[/bold yellow3]")
    console.print("─────────────────────────", style="yellow3")

    summary_table = Table(show_header=False, box=None, padding=(0, 1))
    summary_table.add_column("Metric")
    summary_table.add_column("Value")

    # PYUSD Interaction stats
    has_pyusd = summary_data['has_pyusd_interaction']
    summary_table.add_row("[bold yellow3]PYUSD Interaction:[/bold yellow3]", "[green3]Yes[/green3]" if has_pyusd else "[bright_cyan]No[/bright_cyan]")

    # PYUSD State Changes
    pyusd_state_changes = summary_data['pyusd_state_changes']
    summary_table.add_row("[bold yellow3]PYUSD State Changes:[/bold yellow3]", str(pyusd_state_changes))

    # PYUSD Transfers
    pyusd_transfers = len(summary_data['pyusd_transfers'])
    summary_table.add_row("[bold yellow3]PYUSD Transfers:[/bold yellow3]", str(pyusd_transfers))

    # PYUSD Balance Changes
    pyusd_balance_changes = len(summary_data['pyusd_balance_changes'])
    summary_table.add_row("[bold yellow3]PYUSD Balance Changes:[/bold yellow3]", str(pyusd_balance_changes))

    # Security flags
    security_flags = len(summary_data['security_flags'])
    security_color = "green3" if security_flags == 0 else SECURITY_COLORS["warning"] if security_flags < 2 else SECURITY_COLORS["critical"]
    summary_table.add_row("[bold yellow3]Security Flags:[/bold yellow3]", f"[{security_color}]{security_flags}[/{security_color}]")

    console.print(summary_table)

    # Display security flags if any
    if summary_data['security_flags']:
        console.print(f"\n\n[bold {SECURITY_COLORS['high']}]⚠️ Security Analysis[/bold {SECURITY_COLORS['high']}]")

        security_table = Table(show_header=True, header_style=f"bold {SECURITY_COLORS['high']}")
        security_table.add_column("Level", style="dim", justify="center")
        security_table.add_column("Type", style="dim")
        security_table.add_column("Description")

        for flag in summary_data['security_flags']:
            level = flag['level']
            level_color = SECURITY_COLORS["critical"] if level == "critical" else SECURITY_COLORS["high"] if level == "high" else SECURITY_COLORS["warning"] if level == "warning" else SECURITY_COLORS["info"]

            security_table.add_row(
                f"[{level_color}]{level.upper()}[/{level_color}]",
                flag['type'].replace('_', ' ').title(),
                flag['description']
            )

        console.print(security_table)

    # Add export options
    console.print("\n\n[bold cyan3]📤 Export Options:[/bold cyan3]")
    console.print("──────────────────", style="cyan3")

    # Create export buttons
    export_buttons = widgets.HBox([
        widgets.Button(
            description='Export to CSV',
            button_style='primary',
            layout=widgets.Layout(width='150px')
        ),
        widgets.Button(
            description='Export as JSON',
            button_style='warning',
            layout=widgets.Layout(width='150px')
        ),
        widgets.Button(
            description='Export to Google Sheets',
            button_style='info',
            layout=widgets.Layout(width='200px')
        )
    ])

    export_output = widgets.Output()

    # Add dropdown for export content selection
    export_content = widgets.Dropdown(
        options=['All Data', 'State Changes', 'Transfers', 'PYUSD Operations'],
        value='All Data',
        description='Export:',
        layout=widgets.Layout(width='250px')
    )

    # Display selection control
    display(export_content)

    # Export handlers
    def export_csv(b):
        with export_output:
            clear_output()
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

            # Select appropriate DataFrame based on user choice
            selected_content = export_content.value
            if selected_content == 'State Changes' and not state_changes_df.empty:
                df_to_export = state_changes_df
                filename = f"replay_tx_{shorten_address(tx_hash)}_state_changes_{timestamp}.csv"
            elif selected_content == 'Transfers' and not transfers_df.empty:
                df_to_export = transfers_df
                filename = f"replay_tx_{shorten_address(tx_hash)}_transfers_{timestamp}.csv"
            elif selected_content == 'PYUSD Operations' and not vm_operations_df.empty:
                df_to_export = vm_operations_df
                filename = f"replay_tx_{shorten_address(tx_hash)}_operations_{timestamp}.csv"
            else:
                # For 'All Data' or if specific DataFrame is empty, create a combined DataFrame
                combined_data = []

                # Add summary
                combined_data.append({
                    'section': 'Summary',
                    'key': 'transaction_hash',
                    'value': tx_hash
                })
                combined_data.append({
                    'section': 'Summary',
                    'key': 'has_pyusd_interaction',
                    'value': str(summary_data['has_pyusd_interaction'])
                })
                combined_data.append({
                    'section': 'Summary',
                    'key': 'pyusd_state_changes',
                    'value': str(summary_data['pyusd_state_changes'])
                })
                combined_data.append({
                    'section': 'Summary',
                    'key': 'pyusd_transfers_count',
                    'value': str(len(summary_data['pyusd_transfers']))
                })

                # Add security flags
                for i, flag in enumerate(summary_data['security_flags']):
                    combined_data.append({
                        'section': 'Security Flag',
                        'key': f"flag_{i+1}",
                        'value': f"{flag['level'].upper()} - {flag['description']}"
                    })

                df_to_export = pd.DataFrame(combined_data)
                filename = f"replay_tx_{shorten_address(tx_hash)}_summary_{timestamp}.csv"

            display(download_csv_direct(df_to_export, filename))

    def export_json(b):
        with export_output:
            clear_output()
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"replay_tx_{shorten_address(tx_hash)}_{timestamp}.json"

            # Full JSON export with all data
            display(download_json_direct(summary_data, filename))

    def export_to_sheets(b):
        with export_output:
            clear_output()
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

            # Select appropriate DataFrame based on user choice
            selected_content = export_content.value
            if selected_content == 'State Changes' and not state_changes_df.empty:
                df_to_export = state_changes_df
                sheet_name = f"Replay TX {shorten_address(tx_hash)} - State Changes"
            elif selected_content == 'Transfers' and not transfers_df.empty:
                df_to_export = transfers_df
                sheet_name = f"Replay TX {shorten_address(tx_hash)} - Transfers"
            elif selected_content == 'PYUSD Operations' and not vm_operations_df.empty:
                df_to_export = vm_operations_df
                sheet_name = f"Replay TX {shorten_address(tx_hash)} - Operations"
            else:
                # For 'All Data', create a summary DataFrame
                summary_rows = []
                for key, value in summary_data.items():
                    if key not in ['pyusd_transfers', 'pyusd_balance_changes', 'pyusd_operations', 'security_flags']:
                        if isinstance(value, dict):
                            for sub_key, sub_value in value.items():
                                summary_rows.append({
                                    'category': key,
                                    'key': sub_key,
                                    'value': str(sub_value)
                                })
                        else:
                            summary_rows.append({
                                'category': 'transaction',
                                'key': key,
                                'value': str(value)
                            })

                df_to_export = pd.DataFrame(summary_rows)
                sheet_name = f"Replay TX {shorten_address(tx_hash)} - Summary"

            display(export_to_google_sheets_direct(df_to_export, sheet_name))

    # Connect callbacks
    export_buttons.children[0].on_click(export_csv)
    export_buttons.children[1].on_click(export_json)
    export_buttons.children[2].on_click(export_to_sheets)

    display(export_buttons)
    display(export_output)

    return summary_data


def analyze_replay_block(replay_block_list, block_identifier, requested_tracers):
    """
    Analyzes the output list from trace_replayBlockTransactions with PYUSD focus.

    Args:
        replay_block_list: List of replay results for each transaction
        block_identifier: Block number or hash being analyzed
        requested_tracers: List of tracer types that were requested

    Returns:
        DataFrame with transaction-level PYUSD analysis
    """
    if not isinstance(replay_block_list, list):
        console.print(f"[error]Expected list from trace_replayBlockTransactions for {block_identifier}, got {type(replay_block_list)}.", style="error")
        display_json(replay_block_list, "Unexpected Replay Block Result")
        return None

    console.print(f"\n\n[bold cyan3]PYUSD Analysis of trace_replayBlockTransactions for {block_identifier}[/bold cyan3]")
    console.print(f"Results for {len(replay_block_list)} transactions. Requested Tracers: {', '.join(requested_tracers)}")

    block_summary = []
    total_state_changes = 0
    txs_with_pyusd_state_change = 0
    total_pyusd_transfers = 0
    total_pyusd_volume = 0
    pyusd_state_changes_by_type = {
        'balance': 0,
        'storage': 0,
        'code': 0,
        'other': 0
    }
    security_flags = []

    for i, tx_replay_result in enumerate(replay_block_list):
        tx_summary = {'tx_index': i}
        # Get tx hash if available (might be nested differently)
        tx_hash = tx_replay_result.get('txHash', tx_replay_result.get('transactionHash', f'tx_{i}'))
        tx_summary['tx_hash'] = tx_hash

        # Initialize PYUSD metrics
        tx_summary['has_pyusd_call'] = False
        tx_summary['pyusd_state_changes'] = 0
        tx_summary['pyusd_transfers'] = 0
        tx_summary['pyusd_volume'] = 0
        tx_summary['security_flags'] = []

        # Analyze trace data if available
        if "trace" in requested_tracers and "trace" in tx_replay_result:
            trace_list = tx_replay_result["trace"]
            if isinstance(trace_list, list):
                # Check for PYUSD interactions in the trace
                for trace_item in trace_list:
                    if not isinstance(trace_item, dict): continue

                    action = trace_item.get('action', {})
                    to_addr = action.get('to', '')
                    call_data = action.get('input', '0x')

                    # Check if this is a PYUSD contract interaction
                    if to_addr and to_addr.lower() in PYUSD_CONTRACTS:
                        tx_summary['has_pyusd_call'] = True

                        # Check for transfer functions
                        if call_data and call_data.startswith('0xa9059cbb'):  # transfer
                            try:
                                to_offset = 10
                                to_param = "0x" + call_data[to_offset+24:to_offset+64]
                                amount_offset = 74
                                amount = int(call_data[amount_offset:amount_offset+64], 16)
                                value_pyusd = amount / (10**PYUSD_CONFIG['ethereum']['decimals'])

                                tx_summary['pyusd_transfers'] += 1
                                tx_summary['pyusd_volume'] += value_pyusd
                                total_pyusd_transfers += 1
                                total_pyusd_volume += value_pyusd
                            except Exception:
                                pass
                        elif call_data and call_data.startswith('0x23b872dd'):  # transferFrom
                            try:
                                from_offset = 10
                                from_param = "0x" + call_data[from_offset+24:from_offset+64]
                                to_offset = 74
                                to_param = "0x" + call_data[to_offset+24:to_offset+64]
                                amount_offset = 138
                                amount = int(call_data[amount_offset:amount_offset+64], 16)
                                value_pyusd = amount / (10**PYUSD_CONFIG['ethereum']['decimals'])

                                tx_summary['pyusd_transfers'] += 1
                                tx_summary['pyusd_volume'] += value_pyusd
                                total_pyusd_transfers += 1
                                total_pyusd_volume += value_pyusd
                            except Exception:
                                pass

                        # Check for admin functions
                        method_sig = call_data[:10]
                        if method_sig in PYUSD_SIGNATURES:
                            function_info = PYUSD_SIGNATURES[method_sig]
                            if function_info["category"] == "admin":
                                # Add security flag
                                security_flag = {
                                    'tx_hash': tx_hash,
                                    'tx_index': i,
                                    'level': 'warning',
                                    'type': 'admin_function',
                                    'description': f"Admin function {function_info['name']} called"
                                }
                                tx_summary['security_flags'].append(security_flag)
                                security_flags.append(security_flag)

        # Analyze state diff if available
        if "stateDiff" in requested_tracers and "stateDiff" in tx_replay_result:
            state_diff = tx_replay_result["stateDiff"]
            tx_state_changes = 0
            tx_pyusd_changes = 0
            tx_pyusd_change = False

            if isinstance(state_diff, dict):
                for address, diffs in state_diff.items():
                    is_pyusd_contract = address.lower() == PYUSD_CONFIG['ethereum']['address'].lower()

                    # Count various types of changes
                    for change_type in ['balance', 'nonce', 'code', 'storage']:
                        if change_type in diffs:
                            tx_state_changes += 1
                            if change_type == 'storage':
                                tx_state_changes += len(diffs[change_type]) - 1

                            if is_pyusd_contract:
                                tx_pyusd_change = True
                                tx_pyusd_changes += 1
                                if change_type in pyusd_state_changes_by_type:
                                    pyusd_state_changes_by_type[change_type] += 1
                                else:
                                    pyusd_state_changes_by_type['other'] += 1

                                # Security check for critical changes
                                if change_type == 'code':
                                    security_flag = {
                                        'tx_hash': tx_hash,
                                        'tx_index': i,
                                        'level': 'critical',
                                        'type': 'code_change',
                                        'description': f"PYUSD contract code was modified"
                                    }
                                    tx_summary['security_flags'].append(security_flag)
                                    security_flags.append(security_flag)

            tx_summary['state_changes'] = tx_state_changes
            tx_summary['pyusd_state_changes'] = tx_pyusd_changes
            tx_summary['pyusd_state_change'] = tx_pyusd_change

            total_state_changes += tx_state_changes
            if tx_pyusd_change:
                txs_with_pyusd_state_change += 1

        block_summary.append(tx_summary)

    summary_df = pd.DataFrame(block_summary)

    # Calculate PYUSD metrics
    pyusd_txs_count = summary_df['has_pyusd_call'].sum() if 'has_pyusd_call' in summary_df.columns else 0
    pyusd_tx_percentage = (pyusd_txs_count / len(replay_block_list) * 100) if replay_block_list else 0

    # Block summary with PYUSD focus - enhanced colors for dark themes
    console.print(Panel(f"""
[bold cyan3]PYUSD Block Activity Summary ({block_identifier})[/bold cyan3]
[bold cyan3]Transactions Processed:[/bold cyan3] {len(replay_block_list)}
[bold cyan3]Total State Changes:[/bold cyan3] {total_state_changes:,}

[bold green3]PYUSD Activity[/bold green3]
[bold green3]PYUSD Transactions:[/bold green3] {pyusd_txs_count} ({pyusd_tx_percentage:.1f}% of transactions)
[bold green3]PYUSD State Changes:[/bold green3] {txs_with_pyusd_state_change} transactions affected
[bold green3]PYUSD Transfers:[/bold green3] {total_pyusd_transfers}
[bold green3]PYUSD Volume:[/bold green3] {total_pyusd_volume:,.6f} PYUSD

[bold {SECURITY_COLORS['high']}]Security:[/bold {SECURITY_COLORS['high']}]
Security Flags: {len(security_flags)}""",
        title="PYUSD Block Analysis", border_style="cyan3", expand=False))

    # Display security flags if any
    if security_flags:
        console.print(f"\n\n[bold {SECURITY_COLORS['high']}]🛡️ Security Concerns in Block[/bold {SECURITY_COLORS['high']}]")
        console.print(f"[{SECURITY_COLORS['high']}]─────────────────────────────[/{SECURITY_COLORS['high']}]")

        security_table = Table(show_header=True, header_style=f"bold {SECURITY_COLORS['high']}")
        security_table.add_column("TX Index", justify="right")
        security_table.add_column("TX Hash")
        security_table.add_column("Level", justify="center")
        security_table.add_column("Description")

        for flag in security_flags:
            level = flag['level']
            level_color = SECURITY_COLORS["critical"] if level == "critical" else SECURITY_COLORS["high"] if level == "high" else SECURITY_COLORS["warning"] if level == "warning" else SECURITY_COLORS["info"]

            security_table.add_row(
                str(flag['tx_index']),
                shorten_address(flag['tx_hash']),
                f"[{level_color}]{level.upper()}[/{level_color}]",
                flag['description']
            )

        console.print(security_table)

    # Display PYUSD state change distribution
    if txs_with_pyusd_state_change > 0:
        console.print("\n\n[bold green3]PYUSD State Change Types[/bold green3]")
        console.print("───────────────────────", style="green3")

        state_change_table = Table(title="", show_header=True, header_style="bold green3")
        state_change_table.add_column("Change Type")
        state_change_table.add_column("Count", justify="right")
        state_change_table.add_column("Description", style="italic")

        descriptions = {
            'balance': "ETH balance of PYUSD contracts",
            'storage': "Contract storage values (incl. balances, allowances, etc.)",
            'code': "Contract bytecode changes (upgrades, etc.)",
            'other': "Other state changes"
        }

        for change_type, count in pyusd_state_changes_by_type.items():
            if count > 0:
                state_change_table.add_row(
                    change_type.title(),
                    str(count),
                    descriptions.get(change_type, "")
                )

        console.print(state_change_table)

        # Create visualization for state changes
        console.print("\n\n[bold magenta3]📈 PYUSD State Change Types in Block Chart:[/bold magenta3]")
        console.print("────────────────────────────────────────────", style="magenta3")

        try:
            state_changes_data = pd.DataFrame([
                {"type": change_type.title(), "count": count}
                for change_type, count in pyusd_state_changes_by_type.items() if count > 0
            ])

            if not state_changes_data.empty:
                fig_state = px.pie(
                    state_changes_data, values='count', names='type',
                    title=f'PYUSD State Change Types in Block {block_identifier}',
                    color_discrete_sequence=px.colors.qualitative.Pastel
                )
                fig_state.update_layout(template="plotly_white")
                fig_state.show()
        except Exception as viz_err:
            console.print(f"[warning]Could not create state change visualization: {viz_err}", style="warning")

    # Transaction table with PYUSD focus
    if not summary_df.empty:
        console.print("\n\n[bold cyan3]📊 PYUSD Block Transaction Summary[/bold cyan3]")
        console.print("──────────────────────────────────", style="cyan3")

        # Filter and prepare display columns
        display_cols = ['tx_index', 'tx_hash']

        # Add PYUSD-specific columns if they exist
        for col in ['has_pyusd_call', 'pyusd_transfers', 'pyusd_volume', 'pyusd_state_changes', 'state_changes']:
            if col in summary_df.columns:
                display_cols.append(col)

        # Only include columns that exist in the DataFrame
        display_cols = [col for col in display_cols if col in summary_df.columns]

        # Use the new pagination display function
        display_transaction_dataframe(summary_df[display_cols])

        # Create PYUSD activity visualization if there's enough data
        console.print("\n\n[bold magenta3]🚚 PYUSD Transaction Volume Distribution in Block Chart:[/bold magenta3]")
        console.print("────────────────────────────────────────────────────────", style="magenta3")

        if pyusd_txs_count > 0 and 'pyusd_volume' in summary_df.columns:
            try:
                # Filter for PYUSD transactions
                pyusd_txs = summary_df[summary_df['pyusd_volume'] > 0].copy()

                if not pyusd_txs.empty:
                    # Create volume distribution chart
                    fig_volume = px.bar(
                        pyusd_txs, x='tx_index', y='pyusd_volume',
                        title=f'PYUSD Transaction Volume Distribution in Block {block_identifier}',
                        labels={'tx_index': 'Transaction Index', 'pyusd_volume': 'PYUSD Volume'},
                        color='pyusd_volume',
                        color_continuous_scale='Viridis'
                    )
                    fig_volume.update_layout(template="plotly_white")
                    fig_volume.show()

                    # Create heatmap of PYUSD activity
                    console.print("\n\n[bold magenta3]⬜ PYUSD Transaction Activity Heatmap in Block Chart:[/bold magenta3]")
                    console.print("─────────────────────────────────────────────────────", style="magenta3")

                    if len(pyusd_txs) >= 2:  # Only create heatmap if we have 2+ transactions
                        pyusd_txs['log_volume'] = np.log10(pyusd_txs['pyusd_volume'] + 1)  # log scale for better visualization
                        fig_heat = px.scatter(
                            pyusd_txs,
                            x='tx_index',
                            y='pyusd_state_changes',
                            size='log_volume',
                            color='pyusd_volume',
                            hover_data=['pyusd_transfers', 'pyusd_volume', 'tx_hash'],
                            title=f'PYUSD Transaction Activity Heatmap in Block {block_identifier}'
                        )
                        fig_heat.update_layout(template="plotly_white")
                        fig_heat.show()
            except Exception as viz_err:
                console.print(f"[warning]Could not create PYUSD volume visualization: {viz_err}", style="warning")

    # Add export options
    console.print("\n\n[bold cyan3]📤 Export Options:[/bold cyan3]")
    console.print("──────────────────", style="cyan3")

    # Export buttons with better styling
    export_buttons = widgets.HBox([
        widgets.Button(
            description='Export to CSV',
            button_style='primary',
            layout=widgets.Layout(width='150px')
        ),
        widgets.Button(
            description='Export as JSON',
            button_style='warning',
            layout=widgets.Layout(width='150px')
        ),
        widgets.Button(
            description='Export to Google Sheets',
            button_style='info',
            layout=widgets.Layout(width='200px')
        )
    ])

    export_output = widgets.Output()

    # Export handlers with block-specific data
    def export_csv(b):
        with export_output:
            clear_output()
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"replay_block_{block_identifier}_{timestamp}.csv"
            display(download_csv_direct(summary_df, filename))

    def export_json(b):
        with export_output:
            clear_output()
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"replay_block_{block_identifier}_{timestamp}.json"

            # Create rich JSON export
            export_data = {
                "block_identifier": block_identifier,
                "timestamp": datetime.now().isoformat(),
                "tracers_used": requested_tracers,
                "summary": {
                    "transactions_count": len(replay_block_list),
                    "pyusd_transactions_count": int(pyusd_txs_count),
                    "pyusd_transactions_percentage": float(pyusd_tx_percentage),
                    "total_state_changes": total_state_changes,
                    "txs_with_pyusd_state_change": txs_with_pyusd_state_change,
                    "total_pyusd_transfers": total_pyusd_transfers,
                    "total_pyusd_volume": float(total_pyusd_volume),
                    "security_flags_count": len(security_flags)
                },
                "state_changes_by_type": pyusd_state_changes_by_type,
                "security_flags": security_flags,
                "transactions": summary_df.to_dict('records')
            }

            display(download_json_direct(export_data, filename))

    def export_to_sheets(b):
        with export_output:
            clear_output()
            sheet_name = f"Block {block_identifier} Replay {datetime.now().strftime('%Y%m%d_%H%M%S')}"
            display(export_to_google_sheets_direct(summary_df, sheet_name))

    # Connect callbacks
    export_buttons.children[0].on_click(export_csv)
    export_buttons.children[1].on_click(export_json)
    export_buttons.children[2].on_click(export_to_sheets)

    display(export_buttons)
    display(export_output)

    return summary_df

# --- Execute trace_replayTransaction with PYUSD focus ---
RUN_REPLAY_TX = True # <<< SET TO TRUE TO RUN THIS (100x COST!)
REPLAY_TRACERS = ["trace", "stateDiff"] # Choose tracers: "trace", "vmTrace", "stateDiff"

if not validate_tx_hash:
    console.print("[warning]Skipping 'trace_replayTransaction': TARGET_TX_HASH invalid or not set.", style="warning")
elif RUN_REPLAY_TX:
    console.print("\n\n[bold chartreuse1]🔄 Replay Transaction with trace_replayTransaction[/bold chartreuse1]")
    console.print("─────────────────────────────────────────────────────", style="chartreuse1")

    console.print(f"\n\n[warning]Attempting replaying transaction using 'trace_replayTransaction' for {TARGET_TX_HASH} (100x cost!) with tracers: {REPLAY_TRACERS}...[/warning]", style="warning")
    params = [TARGET_TX_HASH, REPLAY_TRACERS]
    replay_result_data = make_rpc_request("trace_replayTransaction", params, network='mainnet')

    if replay_result_data:
        console.print("[success]trace_replayTransaction completed.", style="success")
        analysis_summary = analyze_replay_transaction(replay_result_data, TARGET_TX_HASH, REPLAY_TRACERS)
    else:
        console.print(f"[error]Failed to replay transaction {TARGET_TX_HASH}.", style="error")
else:
    console.print("\n[info]Skipping 'trace_replayTransaction' as RUN_REPLAY_TX is False.", style="info")


# --- Execute trace_replayBlockTransactions with PYUSD focus ---
RUN_REPLAY_BLOCK = True # <<< SET TO TRUE TO RUN THIS (EXTREME 100x COST!)
REPLAY_BLOCK_TRACERS = ["stateDiff", "trace"] # Choose tracers cautiously

if 'TARGET_BLOCK_IDENTIFIER' in locals() and TARGET_BLOCK_IDENTIFIER is not None:
    block_id_replay = TARGET_BLOCK_IDENTIFIER
    block_param_replay = None
    # Replay methods require specific block hash or number (hex), not tags
    if isinstance(block_id_replay, int):
        block_param_replay = hex(block_id_replay)
    elif isinstance(block_id_replay, str) and block_id_replay.startswith('0x') and len(block_id_replay) == 66:
        block_param_replay = block_id_replay # Assume block hash
    elif isinstance(block_id_replay, str) and block_id_replay.startswith('0x'):
        # Assume hex block number
        block_param_replay = block_id_replay
    elif isinstance(block_id_replay, str) and block_id_replay.isdigit():
        # Convert decimal string to hex
        block_param_replay = hex(int(block_id_replay))

    if RUN_REPLAY_BLOCK and block_param_replay:
        console.print("\n\n[bold chartreuse1]🔄 Replay Transaction with trace_replayTransaction[/bold chartreuse1]")
        console.print("───────────────────────────────────────────────────", style="chartreuse1")

        console.print(f"\n\n[warning]Attempting replaying transactions using 'trace_replayBlockTransactions' for {block_param_replay} (EXTREME COST!) with tracers: {REPLAY_BLOCK_TRACERS}...[/warning]", style="warning")

        params = [block_param_replay, REPLAY_BLOCK_TRACERS]
        replay_block_result = make_rpc_request("trace_replayBlockTransactions", params, network='mainnet')

        if replay_block_result:
            console.print("[success]trace_replayBlockTransactions completed.", style="success")
            analyze_replay_block(replay_block_result, block_param_replay, REPLAY_BLOCK_TRACERS)
        else:
            console.print(f"[error]Failed to replay block transactions for {block_param_replay}.", style="error")
    elif not RUN_REPLAY_BLOCK:
        console.print(f"\n\n[info]Skipping 'trace_replayBlockTransactions' as RUN_REPLAY_BLOCK is False.", style="info")
    elif not block_param_replay:
        console.print("[error]Could not convert TARGET_BLOCK_IDENTIFIER to a valid block parameter.", style="error")
else:
    console.print("[warning]TARGET_BLOCK_IDENTIFIER not set. Skipping block replay analysis.", style="warning")

## 1.10 💾 `debug_storageRangeAt`: Inspecting Raw Contract Storage
---

This section uses `debug_storageRangeAt` to directly query and retrieve the raw values stored in a contract's storage slots at a specific point in time (defined by a block hash and transaction index within that block). This provides a low-level snapshot of the contract's persistent state.

Interpreting the output requires understanding the contract's storage layout (how variables are packed into 256-bit slots). For complex contracts, this can be challenging, but it's invaluable for:

*   **Verifying State:** Directly checking values like `totalSupply`, `owner`, `paused` status if their slots are known.
*   **Debugging:** Examining storage values around the time a transaction failed.
*   **Proxy Analysis:** Identifying implementation/admin addresses stored in standard EIP-1967 slots.
*   **Mapping Analysis:** Calculating and querying specific slots within mappings (like balances or allowances) for targeted addresses.

> **🚀 Leveraging GCP's Premium RPC Capabilities**
>
> *   **Method:** `debug_storageRangeAt`
> *   **Multiplier:** `50x`
> *   **GCP Advantage:** Fetching storage ranges, especially large ones, requires significant node resources. GCP reliably provides this data access.
> *   **PYUSD Insight:** Enables us to:
>     *   Directly read the **PYUSD `totalSupply`**, **`owner`**, **`paused` state**, and **proxy implementation/admin addresses** by querying their known storage slots.
>     *   Investigate the **balance or allowance** for specific addresses by calculating and querying the corresponding mapping slots (demonstrated in helper functions).
>     *   Compare storage snapshots **before and after** a key transaction to pinpoint state changes (using the comparison function).
>     *   Analyze the **storage layout patterns** to confirm adherence to standards (ERC20, Proxy, Pausable).

**Analysis Workflow:**

1.  **Fetch Storage Dump:** Calls `debug_storageRangeAt` specifying the target contract address, block hash, transaction index (usually 0 for state before the block's transactions), starting slot key (often "0x0"), and the maximum number of slots to retrieve.
2.  **Parse & Interpret:** The `analyze_storage_dump` function processes the returned storage map:
    *   It identifies known slots (ERC20 standards, EIP-1967 proxy slots, PYUSD-specific slots) based on predefined patterns.
    *   It attempts to decode values into addresses, booleans, small integers, or formatted PYUSD amounts where possible.
    *   It categorizes slots based on their likely purpose (supply, balances, allowances, proxy, control, metadata, access control).
3.  **Visualize & Summarize:**
    *   **Contract Info Panel:** Displays key decoded values like total supply, paused state, implementation address.
    *   **Proxy Visualization:** If analyzing a proxy, shows a Graphviz diagram of the proxy-implementation(-admin) relationship.
    *   **Storage Table:** Presents the retrieved slots, decoded values, and interpretations in a filterable table (using `ipywidgets`).
    *   **Storage Layout Visualizations:** Generates plots (pie chart, bar chart) showing the distribution of storage slots by category and the layout of initial slots.
    *   **Security Analysis:** Flags potential security-relevant findings based on storage patterns (e.g., paused state, proxy details).
    *   **Pattern Detection:** Summarizes detected storage patterns (ERC20, Proxy, Pausable, etc.).
4.  **Advanced Features:**
    *   **Storage Comparison:** The `compare_storage` function fetches dumps from two different blocks and highlights the changed slots.
    *   **Mapping Analysis:** The `analyze_mapping_storage` function calculates and queries storage slots for specific keys within a mapping (e.g., checking balances for a list of addresses).
    *   **Balance Analysis:** The `analyze_pyusd_balances` function specifically focuses on decoding and visualizing the PYUSD balance mapping.
    *   **Historical Tracking:** The `analyze_storage_history` function tracks a single slot's value across multiple blocks.
5.  **Export Options:** Download parsed storage data, comparisons, or mapping analysis.

**💡 What to Look For:**
*   **Known Slots:** Verify expected values in slots like `totalSupply`, proxy implementation/admin, or paused state.
*   **Decoded Values:** Examine decoded addresses, PYUSD amounts, or boolean flags.
*   **Storage Categories:** Understand the composition of the contract's storage (how much is used for balances, allowances, control, etc.).
*   **(Comparison):** Focus on the highlighted rows showing changed values between blocks.
*   **(Mapping Analysis):** Check the balances or allowances for specific addresses of interest.

In [ ]:
# =============================================================================================
# 💾 Contract Storage Analysis with debug_storageRangeAt
# =============================================================================================
# This cell utilizes the `debug_storageRangeAt` RPC method for deep inspection of contract storage state:
# - Fetches raw storage slots for a specified contract at a given block hash.
# - Parses and decodes storage values into meaningful types (addresses, uints, bools, strings).
# - Applies specific interpretations for known PYUSD, ERC20, and OpenZeppelin patterns (e.g., totalSupply, balances, proxy slots, roles, paused state).
# - Analyzes and visualizes the storage layout, categorizing slots by function (supply, access control, proxy, metadata).
# - Compares storage snapshots between two blocks to highlight state changes, particularly useful for identifying supply/balance modifications.
# - Investigates mapping structures (like balances or allowances) by calculating storage keys and retrieving values.
# - Tracks the historical changes of specific storage slots across multiple blocks.
# - Automatically detects common contract design patterns (Proxy, Pausable, AccessControl) from the storage layout.
# - Generates comprehensive reports using Rich tables, Plotly/Matplotlib charts, and Graphviz diagrams.
# - Provides interactive filtering and direct export options (CSV, JSON, Google Sheets) for the analyzed data.
# =============================================================================================

import base64
from datetime import datetime
from IPython.display import HTML, clear_output, display
import json
import ipywidgets as widgets
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from graphviz import Digraph
from rich.panel import Panel
from rich.table import Table
import time
import re
from eth_utils import decode_hex, to_hex, to_int
from web3 import Web3


def analyze_storage_dump(storage_dump_result, contract_address, block_hash, is_proxy=None):
    """Parser for debug_storageRangeAt with PYUSD-specific interpretations."""
    if not storage_dump_result or 'storage' not in storage_dump_result:
        console.print("[error]Invalid or empty storage dump result received.", style="error")
        display_json(storage_dump_result, "Received Storage Dump Result")
        return None

    # Detect if target is proxy or implementation based on address if not specified
    if is_proxy is None:
        is_proxy = contract_address.lower() == PYUSD_PROXY.lower()

    contract_type = "Proxy" if is_proxy else "Implementation"
    storage_map = storage_dump_result['storage']
    console.print(f"\n\n[success]Retrieved {len(storage_map)} storage slots for {shorten_address(contract_address)} ({contract_type}) at block {shorten_address(block_hash)}.", style="success")

    # Known storage patterns for PYUSD (based on common ERC20 and OpenZeppelin patterns)
    known_slots = {
        # Common ERC20 slots
        0: "totalSupply (or part of it)",
        1: "name (string pointer or part)",
        2: "symbol (string pointer or part)",
        3: "decimals (or part of mapping pointer)",
        4: "balances mapping base",
        5: "allowances mapping base",
        # OpenZeppelin Proxy slots
        "0x360894a13ba1a3210667c828492db98dca3e2076cc3735a920a3ca505d382bbc": "EIP-1967 implementation slot",
        "0xb53127684a568b3173ae13b9f8a6016e243e63b6e8ee1178d6a717850b5d6103": "EIP-1967 admin slot",
        "0xa3f0ad74e5423aebfd80d3ef4346578335a9a72aeaee59ff6cb3582b35133d50": "EIP-1967 beacon slot",
        # OpenZeppelin AccessControl patterns
        "0xb09aa5aeb3702cfd50b6b62bc4532604938f21248a27a1d5ca736082b6819cc1": "Admin role hash",
        # OpenZeppelin Pausable pattern
        "0x5ac1dce9f7971a63e05025b10b44b6f3c868ae576a5e4a815201051d3eae29cb": "Paused state",
        # PYUSD-specific patterns
        "0x0000000000000000000000000000000000000000000000000000000000000014": "PYUSD version",
        # Role slots - more precise for PYUSD's specific structure
        "0x523a704056dcd17bcbde8daf7c077f098d4c0543350248342941a5f0bd09013b": "MINTER_ROLE hash",
        "0xe79898c174bd7837e39256eb383695fecfbd06b222fb859d684c784cbd5997bb": "PAUSER_ROLE hash",
        "0x7a8dc26796a1e50e6e190b70259f58f6a4edd5b21680169636c3b97720af2ffc": "TOKEN_CONTROLLER_ROLE hash",
    }

    # Add mapping prefix patterns
    balances_mapping_prefix = None
    allowances_mapping_prefix = None

    storage_list = []
    # Track key storage patterns
    total_supply = None
    paused_state = None
    implementation_address = None
    version = None
    roles = {}

    # First pass - identify key slots and extract basics
    for slot_hash, data in storage_map.items():
        try:
            # Convert slot hash to int when possible for better sorting
            if slot_hash.startswith('0x'):
                try:
                    slot_int = int(slot_hash, 16)
                    slot_display = slot_int
                except ValueError:
                    slot_int = None
                    slot_display = slot_hash
            else:
                slot_int = None
                slot_display = slot_hash

            # Get raw value
            value = data.get('value', 'N/A')

            # Store raw data first
            slot_entry = {
                'slot_int': slot_int,
                'slot_hex': slot_hash,
                'slot_display': slot_display,
                'raw_value': value,
                'decoded_value': None,
                'interpretation': None,
                'type': None,
                'category': 'unknown'  # Category for better organization
            }

            # Check for known slots by hash
            if slot_hash in known_slots:
                slot_entry['interpretation'] = known_slots[slot_hash]

                # Handle special slots with extra processing
                if slot_hash == "0x360894a13ba1a3210667c828492db98dca3e2076cc3735a920a3ca505d382bbc":
                    # Implementation slot
                    impl_addr = "0x" + value[-40:]
                    implementation_address = Web3.to_checksum_address(impl_addr)
                    slot_entry['decoded_value'] = implementation_address
                    slot_entry['type'] = "address"
                    slot_entry['category'] = 'proxy'

                # EIP-1967 admin slot
                elif slot_hash == "0xb53127684a568b3173ae13b9f8a6016e243e63b6e8ee1178d6a717850b5d6103":
                    admin_addr = "0x" + value[-40:]
                    slot_entry['decoded_value'] = Web3.to_checksum_address(admin_addr)
                    slot_entry['type'] = "address"
                    slot_entry['category'] = 'proxy'

                # Paused state slot
                elif slot_hash == "0x5ac1dce9f7971a63e05025b10b44b6f3c868ae576a5e4a815201051d3eae29cb":
                    paused_state = int(value, 16) == 1
                    slot_entry['decoded_value'] = str(paused_state)
                    slot_entry['type'] = "bool"
                    slot_entry['category'] = 'control'

                # Role related slots
                elif 'role' in slot_entry['interpretation'].lower():
                    role_name = slot_entry['interpretation'].replace(' hash', '')
                    slot_entry['category'] = 'access_control'

                    # Store for role visualization
                    roles[role_name] = {
                        'slot': slot_hash,
                        'value': value
                    }

            # Check for known slots by position
            elif slot_int is not None and slot_int in known_slots:
                slot_entry['interpretation'] = known_slots[slot_int]

                # Handle common ERC20 patterns
                if slot_int == 0 and value != '0x0':
                    # Likely total supply
                    try:
                        total_supply = int(value, 16)
                        slot_entry['decoded_value'] = format_value_pyusd(total_supply)
                        slot_entry['type'] = "uint256"
                        slot_entry['category'] = 'supply'
                    except ValueError:
                        pass

                # Check for balances mapping root
                elif slot_int == 4:
                    balances_mapping_prefix = slot_int
                    slot_entry['category'] = 'balances'

                # Check for allowances mapping root
                elif slot_int == 5:
                    allowances_mapping_prefix = slot_int
                    slot_entry['category'] = 'allowances'

                # Check for PYUSD version
                elif slot_int == 20:
                    try:
                        version = int(value, 16)
                        slot_entry['decoded_value'] = f"v{version}"
                        slot_entry['type'] = "uint256"
                        slot_entry['category'] = 'version'
                    except ValueError:
                        pass

            # Add to our results list
            storage_list.append(slot_entry)

        except Exception as e:
            console.print(f"[warning]Error processing slot {slot_hash}: {str(e)}", style="warning")
            # Still add the raw data
            storage_list.append({
                'slot_hex': slot_hash,
                'raw_value': data.get('value', 'N/A'),
                'error': str(e)
            })

    # Second pass - advanced decoding and interpretation
    for slot_entry in storage_list:
        if slot_entry.get('decoded_value') is not None:
            # Skip already processed entries
            continue

        value = slot_entry['raw_value']

        # Try to decode the value based on common patterns
        try:
            if len(value) == 66 and value.startswith('0x'):
                value_int = int(value, 16)

                # Small number - likely uint8/uint256
                if value_int < 1e12:
                    slot_entry['decoded_value'] = f"{value_int}"
                    slot_entry['type'] = "uint256"

                    # Check for boolean value
                    if value_int == 0 or value_int == 1:
                        slot_entry['interpretation'] = f"Possible boolean: {'true' if value_int == 1 else 'false'}"
                        slot_entry['type'] = "bool"

                        # Check for paused state in known paused slot
                        if slot_entry['slot_hex'] == "0x5ac1dce9f7971a63e05025b10b44b6f3c868ae576a5e4a815201051d3eae29cb":
                            paused_state = value_int == 1
                            slot_entry['interpretation'] = f"Paused state: {'true' if paused_state else 'false'}"
                            slot_entry['category'] = 'control'

                # Potential address
                elif value.startswith('0x000000000000000000000000'):
                    potential_addr = "0x" + value[-40:]
                    try:
                        checksummed_addr = Web3.to_checksum_address(potential_addr)
                        slot_entry['decoded_value'] = checksummed_addr
                        slot_entry['type'] = "address"

                        # Additional context if it's a PYUSD contract
                        if is_pyusd_contract(checksummed_addr):
                            contract_name = get_contract_name(checksummed_addr)
                            slot_entry['interpretation'] = f"PYUSD Contract: {contract_name}"
                            slot_entry['category'] = 'pyusd_contract'
                    except ValueError:
                        # Not a valid address
                        slot_entry['decoded_value'] = f"Possible address pattern: {potential_addr}"

                # Possibly part of a string or bytes
                else:
                    # Try to decode as ASCII/UTF-8 if it contains printable characters
                    bytes_value = bytes.fromhex(value[2:])
                    if all(32 <= b <= 126 for b in bytes_value if b != 0):
                        # Remove trailing zeros
                        bytes_value = bytes_value.rstrip(b'\x00')
                        try:
                            string_value = bytes_value.decode('utf-8')
                            if string_value and len(string_value) > 2:  # Only if we got something meaningful
                                slot_entry['decoded_value'] = f"'{string_value}'"
                                slot_entry['type'] = "string/bytes"
                                slot_entry['category'] = 'metadata'
                        except UnicodeDecodeError:
                            pass
        except Exception as e:
            # If decoding fails, just leave as is
            pass

        # If we haven't set a decoded value, use a simplified version of the raw
        if slot_entry.get('decoded_value') is None:
            if value.startswith('0x'):
                slot_entry['decoded_value'] = f"{value[:10]}...{value[-8:]}"
            else:
                slot_entry['decoded_value'] = value

        # If category not set, try to infer from type or interpretation
        if slot_entry.get('category') == 'unknown':
            if slot_entry.get('type') == 'address':
                slot_entry['category'] = 'address'
            elif slot_entry.get('type') == 'string/bytes':
                slot_entry['category'] = 'metadata'
            elif slot_entry.get('interpretation') and 'role' in slot_entry.get('interpretation').lower():
                slot_entry['category'] = 'access_control'

    # Categorize slots for better analysis
    pyusd_categories = {
        'supply': [],
        'balances': [],
        'allowances': [],
        'access_control': [],
        'proxy': [],
        'metadata': [],
        'paused': []
    }

    for slot_entry in storage_list:
        category = slot_entry.get('category', 'unknown')
        if category in pyusd_categories:
            pyusd_categories[category].append(slot_entry)
        elif category == 'control' and 'paused' in slot_entry.get('interpretation', '').lower():
            pyusd_categories['paused'].append(slot_entry)

    # Create DataFrame with better organization
    storage_df = pd.DataFrame(storage_list)

    # Add contextual summary
    console.print("\n\n[bold cyan3]📊 PYUSD Contract Storage Analysis[/bold cyan3]")
    console.print("──────────────────────────────────", style="cyan3")

    # Display contract info
    console.print(f"\n\n[bold cyan3]Contract Information - {contract_address}[/bold cyan3]")
    contract_info = Table(title="", title_style="bold cyan3")
    contract_info.add_column("Property", style="cyan3")
    contract_info.add_column("Value", style="green3")

    contract_info.add_row("Contract Type", contract_type)
    contract_info.add_row("Block", shorten_address(block_hash))

    if is_proxy and implementation_address:
        contract_info.add_row("Implementation", f"{implementation_address} ({get_contract_name(implementation_address)})")

    if total_supply is not None:
        contract_info.add_row("Total Supply", format_value_pyusd(total_supply))

    if paused_state is not None:
        contract_info.add_row("Paused State", str(paused_state))

    if version is not None:
        contract_info.add_row("PYUSD Version", f"v{version}")

    if roles:
        contract_info.add_row("Roles Detected", str(len(roles)))

    console.print(contract_info)

    # Display PYUSD proxy information if applicable
    if is_proxy and implementation_address:
        console.print("\n[bold cyan3]🔄 PYUSD Proxy Pattern Analysis[/bold cyan3]")

        # Enhanced proxy contract info
        proxy_table = Table(title="PYUSD Proxy Implementation", title_style="bold cyan")
        proxy_table.add_column("Property", style="cyan")
        proxy_table.add_column("Value", style="green")

        proxy_table.add_row("Proxy Contract", shorten_address(contract_address))
        proxy_table.add_row("Implementation", shorten_address(implementation_address))
        proxy_table.add_row("Implementation Name", get_contract_name(implementation_address))

        # Try to find admin slot
        admin_slot = "0xb53127684a568b3173ae13b9f8a6016e243e63b6e8ee1178d6a717850b5d6103"
        admin_entry = next((item for item in storage_list if item.get('slot_hex') == admin_slot), None)

        if admin_entry and 'decoded_value' in admin_entry:
            admin_address = admin_entry['decoded_value']
            proxy_table.add_row("Admin Address", shorten_address(admin_address))

        console.print(proxy_table)

        # Create proxy diagram
        try:
            proxy_graph = Digraph(comment='PYUSD Proxy Pattern')
            proxy_graph.attr(rankdir='LR', bgcolor='transparent')

            # Add nodes
            proxy_graph.node('proxy', f'Proxy\n{shorten_address(contract_address)}',
                           shape='box', style='filled', fillcolor='lightblue')
            proxy_graph.node('impl', f'Implementation\n{shorten_address(implementation_address)}',
                           shape='box', style='filled', fillcolor='lightgreen')

            # Add edge
            proxy_graph.edge('proxy', 'impl', label='delegates to')

            # Add admin if found
            if admin_entry and 'decoded_value' in admin_entry:
                proxy_graph.node('admin', f'Admin\n{shorten_address(admin_address)}',
                               shape='box', style='filled', fillcolor='lightsalmon')
                proxy_graph.edge('admin', 'proxy', label='controls')

            display(proxy_graph)
            console.print("[info]This diagram shows the PYUSD proxy pattern architecture.", style="info")
        except Exception as viz_err:
            console.print(f"[warning]Could not create proxy visualization: {viz_err}", style="warning")

    # Organize display of storage slots
    if 'slot_int' in storage_df.columns and storage_df['slot_int'].notna().any():
        # Sort by numeric slot first, then any non-numeric slots
        numeric_slots = storage_df[storage_df['slot_int'].notna()].sort_values('slot_int')
        non_numeric_slots = storage_df[storage_df['slot_int'].isna()].sort_values('slot_hex')
        storage_df = pd.concat([numeric_slots, non_numeric_slots])
    else:
        # Fallback sort if no numeric conversion
        storage_df = storage_df.sort_values('slot_hex')

    # Select and reorder columns for display
    display_columns = ['slot_display', 'decoded_value', 'interpretation', 'category', 'type', 'slot_hex']
    display_df = storage_df[[col for col in display_columns if col in storage_df.columns]]

    # Rename columns for better readability
    renamed_columns = {
        'slot_display': 'Slot',
        'decoded_value': 'Decoded Value',
        'interpretation': 'Interpretation',
        'type': 'Type',
        'slot_hex': 'Slot Hex',
        'category': 'Category'
    }
    display_df = display_df.rename(columns={col: renamed_columns.get(col, col) for col in display_df.columns})

    # Create filter buttons for categories
    filter_buttons = widgets.HBox([
        widgets.Button(
            description='All Slots',
            button_style='info',
            layout=widgets.Layout(width='120px')
        ),
        widgets.Button(
            description='Supply',
            button_style='success',
            layout=widgets.Layout(width='100px')
        ),
        widgets.Button(
            description='Proxy',
            button_style='warning',
            layout=widgets.Layout(width='100px')
        ),
        widgets.Button(
            description='Access Control',
            button_style='primary',
            layout=widgets.Layout(width='150px')
        )
    ])

    # Display the filter buttons
    display(widgets.HTML("<h3>Filter Storage Slots:</h3>"))
    display(filter_buttons)

    # Data table container
    data_table_container = widgets.Output()
    display(data_table_container)

    # Filter functions
    def show_all_slots(b):
        with data_table_container:
            clear_output()
            display(display_df)

    def show_supply_slots(b):
        with data_table_container:
            clear_output()
            supply_df = display_df[display_df['Category'].str.lower() == 'supply'] if 'Category' in display_df.columns else pd.DataFrame()
            if len(supply_df) > 0:
                display(supply_df)
            else:
                display(widgets.HTML("<p>No supply-related slots found.</p>"))

    def show_proxy_slots(b):
        with data_table_container:
            clear_output()
            proxy_df = display_df[display_df['Category'].str.lower() == 'proxy'] if 'Category' in display_df.columns else pd.DataFrame()
            if len(proxy_df) > 0:
                display(proxy_df)
            else:
                display(widgets.HTML("<p>No proxy-related slots found.</p>"))

    def show_access_control_slots(b):
        with data_table_container:
            clear_output()
            access_df = display_df[display_df['Category'].str.lower() == 'access_control'] if 'Category' in display_df.columns else pd.DataFrame()
            if len(access_df) > 0:
                display(access_df)
            else:
                display(widgets.HTML("<p>No access control slots found.</p>"))

    # Connect filter buttons
    filter_buttons.children[0].on_click(show_all_slots)
    filter_buttons.children[1].on_click(show_supply_slots)
    filter_buttons.children[2].on_click(show_proxy_slots)
    filter_buttons.children[3].on_click(show_access_control_slots)

    # Show all slots by default
    show_all_slots(None)

    # Create storage structure visualization
    console.print("\n\n[bold cyan3]💾 PYUSD Storage Structure Analysis[/bold cyan3]")
    console.print("───────────────────────────────────", style="cyan3")

    # Enhance slot categorization (keeping the same recategorize_slots function)
    def recategorize_slots(storage_df):
        """Improve categorization of storage slots to reduce unknowns."""
        if 'category' not in storage_df.columns:
            storage_df['category'] = 'unknown'

        for i, row in storage_df.iterrows():
            if row['category'] != 'unknown':
                continue

            slot_int = row.get('slot_int')
            interpretation = str(row.get('interpretation', ''))
            decoded_value = str(row.get('decoded_value', ''))

            if slot_int == 0:
                storage_df.at[i, 'category'] = 'supply'
            elif slot_int is not None and (slot_int == 1 or slot_int == 4):
                storage_df.at[i, 'category'] = 'balances'
            elif slot_int is not None and (slot_int == 2 or slot_int == 5):
                storage_df.at[i, 'category'] = 'allowances'
            elif row.get('type') == 'string/bytes':
                storage_df.at[i, 'category'] = 'metadata'
            elif 'role' in interpretation.lower():
                storage_df.at[i, 'category'] = 'access_control'
            elif interpretation and 'paused' in interpretation.lower():
                storage_df.at[i, 'category'] = 'control'
            elif "'" in decoded_value:
                storage_df.at[i, 'category'] = 'metadata'
            elif row.get('type') == 'address':
                storage_df.at[i, 'category'] = 'address'

        return storage_df

    # Apply enhanced categorization
    storage_df = recategorize_slots(storage_df)

    try:
        # Create storage category distribution table
        category_table = Table(title="", title_style="bold cyan3")
        category_table.add_column("Category", style="cyan3")
        category_table.add_column("Count", justify="right", style="green3")
        category_table.add_column("Description")

        category_descriptions = {
            'supply': 'Total PYUSD supply data',
            'balances': 'User token balance mapping',
            'allowances': 'Token spending approvals',
            'access_control': 'Role-based permission system',
            'proxy': 'Upgradeable contract implementation',
            'control': 'Contract control mechanisms like pause',
            'metadata': 'Token metadata like name and symbol',
            'address': 'Ethereum addresses storage',
            'pyusd_contract': 'References to PYUSD contracts',
            'unknown': 'Unidentified storage purpose'
        }

        category_counts = storage_df['category'].value_counts()
        for category, count in category_counts.items():
            category_table.add_row(
                category,
                str(count),
                category_descriptions.get(category, "")
            )

        console.print(category_table)

        # Function to format slot numbers
        def format_slot(slot):
            """Format slot numbers to be more compact"""
            if isinstance(slot, int):
                if slot < 20:  # Small numbers kept as is
                    return str(slot)
                else:  # Large numbers shown in hex
                    return f"0x{slot:x}"[:6] + "..."
            return str(slot)

        # 1. Pie chart of storage categories
        if not category_counts.empty:
            console.print("\n\n[bold magenta3]📊 Storage Category Distribution Chart:[/bold magenta3]")
            console.print("───────────────────────────────────────", style="magenta3")

            plt.figure(figsize=(8, 6))
            wedges, texts, autotexts = plt.pie(
                category_counts.values,
                labels=category_counts.index,
                autopct='%1.1f%%',
                startangle=90,
                shadow=False
            )

            # Make percentage text bold
            for autotext in autotexts:
                autotext.set_weight('bold')

            plt.axis('equal')  # Equal aspect ratio ensures pie is drawn as a circle
            plt.title('PYUSD Storage Categories', fontweight='bold', fontsize=14)
            plt.tight_layout()
            plt.show()
            plt.close()  # Close the figure to ensure separation

        # 2. Bar chart of first 15 numeric slots (reduced from 20 for cleaner display)
        numeric_slots = storage_df[storage_df['slot_int'].notna()].sort_values('slot_int').head(15)

        if not numeric_slots.empty:
            console.print("\n\n[bold magenta3]📊 Storage Slot Layout Chart:[/bold magenta3]")
            console.print("──────────────────────────────", style="magenta3")

            plt.figure(figsize=(12, 5))

            # Create colors dictionary for categories
            unique_categories = numeric_slots['category'].unique()
            colors = plt.cm.tab10(np.linspace(0, 1, len(unique_categories)))
            category_colors = dict(zip(unique_categories, colors))

            # Create bar colors based on category
            bar_colors = [category_colors[cat] for cat in numeric_slots['category']]

            # Format slot numbers for display
            slot_labels = [format_slot(slot) for slot in numeric_slots['slot_int']]

            # Create the bar chart
            bars = plt.bar(
                slot_labels,
                [1] * len(numeric_slots),  # All bars same height
                color=bar_colors
            )

            # Add annotations for important slots - only for slots with interpretations
            for i, (idx, row) in enumerate(numeric_slots.iterrows()):
                if row.get('interpretation'):
                    # Get first word only, truncate if too long
                    interp = row['interpretation'].split(' ')[0]
                    if len(interp) > 10:
                        interp = interp[:8] + '..'

                    plt.annotate(
                        interp,
                        xy=(i, 0.5),  # Use index instead of slot value
                        rotation=90,
                        fontsize=9,
                        fontweight='bold',
                        ha='center'
                    )

            # Add legend
            from matplotlib.patches import Patch
            legend_elements = [
                Patch(facecolor=category_colors[cat], label=cat)
                for cat in unique_categories
            ]
            plt.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=4)

            plt.title('PYUSD Storage Slot Layout', fontweight='bold', fontsize=14)
            plt.xlabel('Storage Slot', fontweight='bold')
            plt.ylabel('')
            plt.yticks([])  # Hide y-axis ticks
            plt.xticks(rotation=45)  # Rotate x-axis labels
            plt.tight_layout(pad=2.0)  # Add padding
            plt.show()
            plt.close()  # Close the figure to ensure separation

        # 3. Category distribution bar chart
        if not category_counts.empty:
            console.print("\n\n[bold magenta3]📊 Storage Composition Chart:[/bold magenta3]")
            console.print("────────────────────────────", style="magenta3")

            plt.figure(figsize=(10, 4))

            # Sort categories by count for better visualization
            sorted_categories = category_counts.sort_values(ascending=False)

            # Create horizontal bar chart
            bars = plt.barh(
                sorted_categories.index,
                sorted_categories.values,
                color=plt.cm.tab10(np.linspace(0, 1, len(sorted_categories)))
            )

            # Add count labels to the bars
            for bar in bars:
                width = bar.get_width()
                plt.text(
                    width + 0.3,
                    bar.get_y() + bar.get_height()/2,
                    f"{width}",
                    ha='left',
                    va='center',
                    fontweight='bold'
                )

            plt.xlabel('Number of Storage Slots', fontweight='bold')
            plt.title('PYUSD Storage Composition', fontweight='bold', fontsize=14)
            plt.tight_layout()
            plt.show()
            plt.close()  # Close the figure to ensure separation

        console.print("\n\n[info]Storage analysis visualization complete.", style="info")
    except Exception as viz_err:
        console.print(f"[warning]Could not create matplotlib visualization: {viz_err}", style="warning")

        # Always show text summary as backup
        console.print("\n\n[bold cyan3]PYUSD Storage Categories:[/bold cyan3]")
        for category, count in storage_df['category'].value_counts().items():
            console.print(f"  • {category}: {count} slots")

    # Check for security-relevant patterns
    console.print("\n\n[bold cyan3]🔒 PYUSD Security Analysis[/bold cyan3]")
    console.print("───────────────────────────", style="cyan3")

    security_flags = []

    # Check for proxy implementation
    implementation_slot = "0x360894a13ba1a3210667c828492db98dca3e2076cc3735a920a3ca505d382bbc"
    impl_entry = next((item for item in storage_list if item.get('slot_hex') == implementation_slot), None)
    if impl_entry and impl_entry.get('decoded_value'):
        security_flags.append({
            'level': 'info',
            'type': 'proxy_implementation',
            'description': f"PYUSD implementation: {impl_entry.get('decoded_value')}",
            'details': {'address': impl_entry.get('decoded_value')}
        })

    # Check for proxy admin
    admin_slot = "0xb53127684a568b3173ae13b9f8a6016e243e63b6e8ee1178d6a717850b5d6103"
    admin_entry = next((item for item in storage_list if item.get('slot_hex') == admin_slot), None)
    if admin_entry and admin_entry.get('decoded_value'):
        security_flags.append({
            'level': 'info',
            'type': 'proxy_admin',
            'description': f"PYUSD proxy admin: {admin_entry.get('decoded_value')}",
            'details': {'address': admin_entry.get('decoded_value')}
        })

    # Check for paused state
    paused_slot = "0x5ac1dce9f7971a63e05025b10b44b6f3c868ae576a5e4a815201051d3eae29cb"
    paused_entry = next((item for item in storage_list if item.get('slot_hex') == paused_slot), None)
    if paused_entry and paused_entry.get('raw_value'):
        paused_value = int(paused_entry.get('raw_value'), 16) if paused_entry.get('raw_value').startswith('0x') else 0
        if paused_value == 1:
            security_flags.append({
                'level': 'warning',
                'type': 'contract_paused',
                'description': "PYUSD contract is currently paused",
                'details': {'paused': True}
            })

    # Display security insights if any
    if security_flags:
        security_table = Table(show_header=True, header_style=f"bold {SECURITY_COLORS['high']}")
        security_table.add_column("Level", style="dim", justify="center")
        security_table.add_column("Type", style="dim")
        security_table.add_column("Description")

        for flag in security_flags:
            level = flag['level']
            level_color = SECURITY_COLORS["critical"] if level == "critical" else SECURITY_COLORS["high"] if level == "high" else SECURITY_COLORS["warning"] if level == "warning" else SECURITY_COLORS["info"]

            security_table.add_row(
                f"[{level_color}]{level.upper()}[/{level_color}]",
                flag['type'].replace('_', ' ').title(),
                flag['description']
            )

        console.print(security_table)
    else:
        console.print("[info]No security-relevant findings in storage analysis.", style="info")

    # Add export options
    console.print("\n\n[bold cyan3]📤 Export Options:[/bold cyan3]")
    console.print("──────────────────", style="cyan3")

    # Create export buttons
    export_buttons = widgets.HBox([
        widgets.Button(
            description='Export to CSV',
            button_style='primary',
            layout=widgets.Layout(width='150px')
        ),
        widgets.Button(
            description='Export as JSON',
            button_style='warning',
            layout=widgets.Layout(width='150px')
        ),
        widgets.Button(
            description='Export to Google Sheets',
            button_style='info',
            layout=widgets.Layout(width='200px')
        )
    ])

    export_output = widgets.Output()

    # Add dropdown for export content selection
    export_content = widgets.Dropdown(
        options=['All Storage', 'PYUSD Contract Info', 'Proxy Configuration', 'Critical Slots'],
        value='All Storage',
        description='Export:',
        layout=widgets.Layout(width='250px')
    )

    # Display selection control
    display(export_content)
    display(export_buttons)
    display(export_output)

    # Export handlers
    def export_csv(b):
        with export_output:
            clear_output()
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

            # Select appropriate data based on user choice
            selected_content = export_content.value
            if selected_content == 'PYUSD Contract Info':
                # Create contract info summary
                info_data = []
                for key, value in {
                    'address': contract_address,
                    'type': contract_type,
                    'block_hash': block_hash,
                    'implementation': implementation_address,
                    'total_supply': total_supply,
                    'paused_state': paused_state,
                    'version': version
                }.items():
                    if value is not None:
                        info_data.append({
                            'property': key,
                            'value': str(value)
                        })
                # Add categories summary
                for category, slots in pyusd_categories.items():
                    if slots:
                        info_data.append({
                            'property': f"{category} slots",
                            'value': str(len(slots))
                        })

                df_to_export = pd.DataFrame(info_data)
                filename = f"pyusd_contract_{shorten_address(contract_address)}_{timestamp}.csv"
            elif selected_content == 'Proxy Configuration' and contract_type == 'Proxy':
                # Filter to proxy-related data
                proxy_data = storage_df[storage_df['slot_hex'].isin([
                    "0x360894a13ba1a3210667c828492db98dca3e2076cc3735a920a3ca505d382bbc",  # implementation
                    "0xb53127684a568b3173ae13b9f8a6016e243e63b6e8ee1178d6a717850b5d6103"   # admin
                ])]
                df_to_export = proxy_data
                filename = f"pyusd_proxy_{shorten_address(contract_address)}_{timestamp}.csv"
            elif selected_content == 'Critical Slots':
                # Filter to slots with interpretations
                critical_data = storage_df[storage_df['interpretation'].notna()]
                df_to_export = critical_data
                filename = f"pyusd_critical_{shorten_address(contract_address)}_{timestamp}.csv"
            else:
                # All storage
                df_to_export = storage_df
                filename = f"pyusd_storage_{shorten_address(contract_address)}_{timestamp}.csv"

            display(download_csv_direct(df_to_export, filename))

    def export_json(b):
        with export_output:
            clear_output()
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"pyusd_storage_{shorten_address(contract_address)}_{timestamp}.json"

            # Create comprehensive export with metadata
            export_data = {
                "metadata": {
                    "contract_address": contract_address,
                    "contract_type": contract_type,
                    "block_hash": block_hash,
                    "analysis_timestamp": datetime.now().isoformat(),
                    "slots_analyzed": len(storage_df)
                },
                "contract_info": {
                    "implementation": implementation_address,
                    "total_supply": total_supply,
                    "paused_state": paused_state,
                    "version": version,
                    "roles": roles
                },
                "storage_data": storage_df.to_dict('records'),
                "pyusd_categories": {k: [s['slot_hex'] for s in v] for k, v in pyusd_categories.items() if v},
                "security_flags": security_flags
            }

            display(download_json_direct(export_data, filename))

    def export_to_sheets(b):
        with export_output:
            clear_output()
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

            # Select appropriate data based on user choice
            selected_content = export_content.value
            if selected_content == 'PYUSD Contract Info':
                # Create contract info summary
                info_data = []
                for key, value in {
                    'address': contract_address,
                    'type': contract_type,
                    'block_hash': block_hash,
                    'implementation': implementation_address,
                    'total_supply': total_supply,
                    'paused_state': paused_state,
                    'version': version
                }.items():
                    if value is not None:
                        info_data.append({
                            'property': key,
                            'value': str(value)
                        })
                # Add categories summary
                for category, slots in pyusd_categories.items():
                    if slots:
                        info_data.append({
                            'property': f"{category} slots",
                            'value': str(len(slots))
                        })

                df_to_export = pd.DataFrame(info_data)
                sheet_name = f"PYUSD Info - {shorten_address(contract_address)}"
            elif selected_content == 'Proxy Configuration' and contract_type == 'Proxy':
                proxy_data = storage_df[storage_df['slot_hex'].isin([
                    "0x360894a13ba1a3210667c828492db98dca3e2076cc3735a920a3ca505d382bbc",
                    "0xb53127684a568b3173ae13b9f8a6016e243e63b6e8ee1178d6a717850b5d6103"
                ])]
                df_to_export = proxy_data
                sheet_name = f"PYUSD Proxy - {shorten_address(contract_address)}"
            elif selected_content == 'Critical Slots':
                critical_data = storage_df[storage_df['interpretation'].notna()]
                df_to_export = critical_data
                sheet_name = f"PYUSD Critical - {shorten_address(contract_address)}"
            else:
                df_to_export = storage_df
                sheet_name = f"PYUSD Storage - {shorten_address(contract_address)}"

            display(export_to_google_sheets_direct(df_to_export, sheet_name))

    # Connect callbacks
    export_buttons.children[0].on_click(export_csv)
    export_buttons.children[1].on_click(export_json)
    export_buttons.children[2].on_click(export_to_sheets)

    # Return the organized data for further analysis
    return {
        'storage_df': storage_df,
        'contract_info': {
            'address': contract_address,
            'type': contract_type,
            'block_hash': block_hash,
            'implementation': implementation_address,
            'total_supply': total_supply,
            'paused_state': paused_state,
            'version': version,
            'roles': roles,
            'pyusd_categories': {k: len(v) for k, v in pyusd_categories.items() if v}
        },
        'security_flags': security_flags
    }


def compare_storage(address, block_hash1, block_hash2, tx_index1=0, tx_index2=0, slot_count=20, start_slot="0x0"):
    """Compare contract storage between two blocks to identify changes."""
    console.print(f"[info]Comparing storage for {shorten_address(address)} between two blocks...", style="info")

    # Get storage for first block
    params1 = [block_hash1, tx_index1, address, start_slot, slot_count]
    storage_dump1 = make_rpc_request("debug_storageRangeAt", params1, network='mainnet')

    # Get storage for second block
    params2 = [block_hash2, tx_index2, address, start_slot, slot_count]
    storage_dump2 = make_rpc_request("debug_storageRangeAt", params2, network='mainnet')

    if not storage_dump1 or not storage_dump2:
        console.print("[error]Failed to retrieve storage for comparison.", style="error")
        return None

    # Transform storage data for easier comparison
    def transform_storage(storage_dump):
        result = {}
        for slot, data in storage_dump.get('storage', {}).items():
            result[slot] = data.get('value')
        return result

    storage1 = transform_storage(storage_dump1)
    storage2 = transform_storage(storage_dump2)

    # Find all unique slots
    all_slots = set(list(storage1.keys()) + list(storage2.keys()))

    # Build comparison data
    comparison = []
    for slot in all_slots:
        value1 = storage1.get(slot, 'N/A')
        value2 = storage2.get(slot, 'N/A')
        changed = value1 != value2

        # Try to decode the values
        try:
            if value1 != 'N/A' and value2 != 'N/A':
                value1_int = int(value1, 16)
                value2_int = int(value2, 16)

                if abs(value1_int) < 1e12 and abs(value2_int) < 1e12:
                    # If small numbers, show the difference
                    diff = value2_int - value1_int
                    diff_str = f"{diff:+}" if changed else "No change"

                    # For PYUSD values, add formatted difference
                    if diff != 0 and abs(diff) < 1e9:  # Reasonable token amount
                        formatted_diff = format_value_pyusd(abs(diff))
                        diff_str += f" ({formatted_diff} {'decrease' if diff < 0 else 'increase'})"
                else:
                    diff_str = "Changed" if changed else "No change"
                    diff = None
            else:
                diff_str = "Added/Removed" if changed else "No change"
                diff = None
        except:
            diff_str = "Changed" if changed else "No change"
            diff = None

        # Check if this might be a balance/supply change
        is_balance_change = False
        is_supply_change = False

        # Look for potential balance slot (typically derived from keccak256(address + mapping_slot))
        if slot.startswith('0x') and len(slot) == 66 and changed and diff is not None:
            if abs(diff) < 1e12:  # Reasonable token amount
                is_balance_change = True

        # Check if this might be total supply slot
        if (slot == '0x0' or int(slot, 16) == 0 if slot.startswith('0x') else False) and changed and diff is not None:
            is_supply_change = True

        comparison.append({
            'slot': slot,
            'value_block1': value1,
            'value_block2': value2,
            'changed': changed,
            'diff': diff_str,
            'numeric_diff': diff,
            'is_balance_change': is_balance_change,
            'is_supply_change': is_supply_change
        })

    # Create DataFrame for display
    comparison_df = pd.DataFrame(comparison)
    comparison_df = comparison_df.sort_values(['changed', 'is_supply_change', 'is_balance_change'],
                                             ascending=[False, False, False])

    # Display results
    console.print(f"\n[bold cyan3]📊 Storage Comparison Results[/bold cyan3]")
    console.print(f"Block 1: {shorten_address(block_hash1)}")
    console.print(f"Block 2: {shorten_address(block_hash2)}")
    console.print(f"Changes detected: {comparison_df['changed'].sum()} of {len(comparison_df)} slots\n")

    # Highlight changes in display
    def highlight_changes(row):
        if row['changed']:
            # Different highlighting based on type of change
            if row.get('is_supply_change', False):
                return ['background-color: #ffcc99'] * len(row)  # Orange for supply
            elif row.get('is_balance_change', False):
                return ['background-color: #ccffcc'] * len(row)  # Green for balances
            else:
                return ['background-color: #ffcccc'] * len(row)  # Red for other changes
        return [''] * len(row)

    # Apply styling and display
    styled_df = comparison_df.style.apply(highlight_changes, axis=1)
    display(styled_df)

    # Create change visualization if changes detected
    if comparison_df['changed'].sum() > 0:
        console.print("\n[bold cyan3]📊 Storage Change Visualization[/bold cyan3]")

        try:
            # Filter to changed values
            changes = comparison_df[comparison_df['changed']].copy()

            # Add change category
            def categorize_change(row):
                if row.get('is_supply_change', False):
                    return 'Supply'
                elif row.get('is_balance_change', False):
                    return 'Balance'
                elif 'slot' in row and row['slot'] == "0x5ac1dce9f7971a63e05025b10b44b6f3c868ae576a5e4a815201051d3eae29cb":
                    return 'Pause State'
                elif 'slot' in row and row['slot'] == "0x360894a13ba1a3210667c828492db98dca3e2076cc3735a920a3ca505d382bbc":
                    return 'Implementation'
                else:
                    return 'Other'

            changes['category'] = changes.apply(categorize_change, axis=1)

            # Create bar chart of changes by category
            category_counts = changes['category'].value_counts().reset_index()
            category_counts.columns = ['Category', 'Count']

            fig = px.bar(
                category_counts,
                x='Category',
                y='Count',
                color='Category',
                title=f'PYUSD Storage Changes by Type - {shorten_address(address)}',
                labels={'Count': 'Number of Changes'}
            )

            fig.update_layout(template="plotly_white")
            fig.show()

            # If we have supply changes, create a specialized visualization
            supply_changes = changes[changes['category'] == 'Supply']
            if not supply_changes.empty:
                try:
                    # Get supply values
                    from_value = int(supply_changes.iloc[0]['value_block1'], 16) / 1e6  # PYUSD has 6 decimals
                    to_value = int(supply_changes.iloc[0]['value_block2'], 16) / 1e6
                    diff = to_value - from_value

                    # Create gauge chart
                    fig = go.Figure(go.Indicator(
                        mode = "gauge+number+delta",
                        value = to_value,
                        title = {'text': "PYUSD Supply"},
                        delta = {'reference': from_value, 'relative': False},
                        gauge = {
                            'axis': {'range': [None, max(from_value, to_value) * 1.1]},
                            'bar': {'color': "green" if diff >= 0 else "red"},
                            'steps': [
                                {'range': [0, from_value], 'color': "lightgray"}
                            ]
                        }
                    ))

                    fig.update_layout(
                        title=f"PYUSD Supply Change: {diff:+,.2f} ({diff/from_value*100:+.2f}%)",
                        template="plotly_white"
                    )

                    fig.show()
                except Exception as e:
                    console.print(f"[warning]Could not create supply change gauge: {e}", style="warning")

            console.print("[info]These visualizations highlight key changes in PYUSD storage between blocks.", style="info")
        except Exception as viz_err:
            console.print(f"[warning]Could not create change visualization: {viz_err}", style="warning")

    # Add export options
    console.print("\n[bold cyan3]📤 Export Options[/bold cyan3]")

    # Create export buttons
    export_buttons = widgets.HBox([
        widgets.Button(
            description='Export to CSV',
            button_style='primary',
            layout=widgets.Layout(width='150px')
        ),
        widgets.Button(
            description='Export as JSON',
            button_style='warning',
            layout=widgets.Layout(width='150px')
        ),
        widgets.Button(
            description='Export to Google Sheets',
            button_style='info',
            layout=widgets.Layout(width='200px')
        )
    ])

    export_output = widgets.Output()

    # Display export controls
    display(export_buttons)
    display(export_output)

    # Export handlers
    def export_csv(b):
        with export_output:
            clear_output()
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"pyusd_storage_comparison_{shorten_address(address)}_{timestamp}.csv"
            display(download_csv_direct(comparison_df, filename))

    def export_json(b):
        with export_output:
            clear_output()

            # Enhanced JSON with metadata
            export_data = {
                "metadata": {
                    "contract_address": address,
                    "block_hash1": block_hash1,
                    "block_hash2": block_hash2,
                    "analysis_time": datetime.now().isoformat(),
                    "changes_detected": int(comparison_df['changed'].sum()),
                    "slots_analyzed": len(comparison_df)
                },
                "comparison_data": comparison_df.to_dict('records')
            }

            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"pyusd_storage_comparison_{shorten_address(address)}_{timestamp}.json"
            display(download_json_direct(export_data, filename))

    def export_to_sheets(b):
        with export_output:
            clear_output()
            sheet_name = f"PYUSD Storage Comparison - {shorten_address(address)}"
            display(export_to_google_sheets_direct(comparison_df, sheet_name))

    # Connect callbacks
    export_buttons.children[0].on_click(export_csv)
    export_buttons.children[1].on_click(export_json)
    export_buttons.children[2].on_click(export_to_sheets)

    return comparison_df


def analyze_mapping_storage(contract_address, mapping_slot, keys, block_hash, tx_index=0):
    """Analyze a specific mapping in contract storage by calculating keccak hash of key+slot."""
    console.print(f"[info]Analyzing mapping at slot {mapping_slot} for {shorten_address(contract_address)}...", style="info")

    # Ensure mapping slot is in correct format
    if isinstance(mapping_slot, int):
        mapping_slot_hex = f"0x{mapping_slot:064x}"
    elif isinstance(mapping_slot, str) and mapping_slot.startswith('0x'):
        mapping_slot_hex = mapping_slot
    else:
        console.print("[error]Invalid mapping slot format.", style="error")
        return None

    results = []
    for key in keys:
        # Ensure key is in correct format (32 bytes padded address for typical ERC20 mapping)
        if isinstance(key, str) and key.startswith('0x') and len(key) == 42:
            # Convert address to padded 32-byte value
            padded_key = f"0x{key[2:].lower().zfill(64)}"
        elif isinstance(key, int):
            padded_key = f"0x{key:064x}"
        else:
            padded_key = key

        # Calculate storage slot for this mapping key
        # For typical mappings: keccak256(key + slot)
        concat_hex = padded_key[2:] + mapping_slot_hex[2:]  # Remove '0x' prefix before concatenating
        slot_to_read = Web3.keccak(hexstr=concat_hex).hex()

        # Read the storage at this calculated slot
        params = [block_hash, tx_index, contract_address, slot_to_read, 1]
        storage_result = make_rpc_request("debug_storageRangeAt", params, network='mainnet')

        if not storage_result or 'storage' not in storage_result:
            value = "Error reading slot"
            raw_value = None
        else:
            value = storage_result['storage'].get(slot_to_read, {}).get('value', 'N/A')
            raw_value = value

            # Try to decode the value
            try:
                if value != 'N/A':
                    value_int = int(value, 16)
                    if value_int < 1e12:  # Small number - probably amount
                        if mapping_slot == 0 or mapping_slot == 4:  # Common slots for ERC20 balances
                            decoded = format_value_pyusd(value_int)
                        else:
                            decoded = f"{value_int}"
                    else:
                        decoded = value
                else:
                    decoded = "0"
                    value_int = 0
            except:
                decoded = value
                value_int = None

        # Key formatting for display
        if isinstance(key, str) and key.startswith('0x') and len(key) == 42:
            # It's an address - add context
            key_display = shorten_address(key)
            key_type = "address"

            # Check if it's a known PYUSD contract
            if is_pyusd_contract(key):
                key_context = f"PYUSD: {get_contract_name(key)}"
            else:
                # Try to add extra context like "token holder"
                if mapping_slot == 0 or mapping_slot == 4:  # Balances mapping
                    key_context = "Token Holder"
                elif mapping_slot == 1 or mapping_slot == 5:  # Allowances mapping (first key is owner)
                    key_context = "Token Owner"
                else:
                    key_context = "Account"
        elif isinstance(key, int) or (isinstance(key, str) and key.isdigit()):
            # It's a number
            key_display = str(key)
            key_type = "uint256"
            key_context = "Index"
        else:
            # Other format
            key_display = str(key)
            key_type = "unknown"
            key_context = ""

        # Record the result
        results.append({
            'key': key,
            'key_display': key_display,
            'key_type': key_type,
            'key_context': key_context,
            'calculated_slot': slot_to_read,
            'raw_value': raw_value,
            'value': value,
            'decoded_value': decoded,
            'value_int': value_int if 'value_int' in locals() else None
        })

    # Create DataFrame
    mapping_df = pd.DataFrame(results)

    # Sort by value if numeric
    if 'value_int' in mapping_df.columns and mapping_df['value_int'].notna().any():
        mapping_df = mapping_df.sort_values('value_int', ascending=False)

    # Select columns for display
    display_cols = ['key_display', 'key_context', 'decoded_value', 'calculated_slot']
    display_df = mapping_df[display_cols]

    # Rename columns for display
    renamed_columns = {
        'key_display': 'Key',
        'key_context': 'Context',
        'decoded_value': 'Value',
        'calculated_slot': 'Storage Slot'
    }
    display_df = display_df.rename(columns=renamed_columns)

    console.print(f"\n[bold cyan3]📊 Mapping Analysis Results for Slot {mapping_slot}[/bold cyan3]")
    display(display_df)

    # Create balance visualization if appropriate
    if mapping_slot == 0 or mapping_slot == 4:  # ERC20 balances mapping
        console.print("\n[bold cyan3]📊 PYUSD Balance Distribution[/bold cyan3]")
        try:
            # Filter non-zero balances
            non_zero_balances = mapping_df[mapping_df['value_int'] > 0].copy()

            if not non_zero_balances.empty:
                # For better visualization, normalize values
                non_zero_balances['formatted_value'] = non_zero_balances['value_int'].apply(
                    lambda x: float(x) / 1e6 if x else 0  # Convert to PYUSD units
                )

                # Create balance distribution chart
                fig = px.pie(
                    non_zero_balances,
                    values='formatted_value',
                    names='key_display',
                    title=f'PYUSD Balance Distribution - {len(non_zero_balances)} addresses',
                    hover_data=['key_context', 'decoded_value']
                )

                fig.update_traces(textposition='inside', textinfo='percent+label')
                fig.update_layout(template="plotly_white")

                fig.show()

                # Create bar chart of top holders
                if len(non_zero_balances) > 1:
                    # Take top 10 for bar chart
                    top_holders = non_zero_balances.nlargest(10, 'formatted_value')

                    fig = px.bar(
                        top_holders,
                        y='key_display',
                        x='formatted_value',
                        color='key_context',
                        title=f'Top PYUSD Holders - Slot {mapping_slot}',
                        labels={
                            'formatted_value': 'Balance (PYUSD)',
                            'key_display': 'Address',
                            'key_context': 'Type'
                        },
                        orientation='h'  # Horizontal bars
                    )

                    fig.update_layout(template="plotly_white")
                    fig.show()
            else:
                console.print("[info]No non-zero balances found to visualize.", style="info")
        except Exception as viz_err:
            console.print(f"[warning]Could not create balance visualization: {viz_err}", style="warning")

    # Add export options
    console.print("\n[bold cyan3]📤 Export Options[/bold cyan3]")

    # Create export buttons
    export_buttons = widgets.HBox([
        widgets.Button(
            description='Export to CSV',
            button_style='primary',
            layout=widgets.Layout(width='150px')
        ),
        widgets.Button(
            description='Export as JSON',
            button_style='warning',
            layout=widgets.Layout(width='150px')
        ),
        widgets.Button(
            description='Export to Google Sheets',
            button_style='info',
            layout=widgets.Layout(width='200px')
        )
    ])

    export_output = widgets.Output()

    # Display export controls
    display(export_buttons)
    display(export_output)

    # Export handlers
    def export_csv(b):
        with export_output:
            clear_output()
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"pyusd_mapping_slot{mapping_slot}_{shorten_address(contract_address)}_{timestamp}.csv"
            display(download_csv_direct(mapping_df, filename))

    def export_json(b):
        with export_output:
            clear_output()

            # Create structured JSON with metadata
            export_data = {
                "metadata": {
                    "contract_address": contract_address,
                    "mapping_slot": mapping_slot,
                    "block_hash": block_hash,
                    "analysis_time": datetime.now().isoformat(),
                    "keys_analyzed": len(mapping_df)
                },
                "mapping_type": "balances" if mapping_slot == 0 or mapping_slot == 4 else
                              "allowances" if mapping_slot == 1 or mapping_slot == 5 else
                              "unknown",
                "mapping_data": mapping_df.to_dict('records')
            }

            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"pyusd_mapping_slot{mapping_slot}_{shorten_address(contract_address)}_{timestamp}.json"
            display(download_json_direct(export_data, filename))

    def export_to_sheets(b):
        with export_output:
            clear_output()
            sheet_name = f"PYUSD Mapping Slot {mapping_slot} - {shorten_address(contract_address)}"
            display(export_to_google_sheets_direct(mapping_df, sheet_name))

    # Connect callbacks
    export_buttons.children[0].on_click(export_csv)
    export_buttons.children[1].on_click(export_json)
    export_buttons.children[2].on_click(export_to_sheets)

    return mapping_df


def analyze_pyusd_balances(contract_address, block_hash, key_addresses, tx_index=0):
    """Deep analysis of PYUSD balances mapping with visual distribution."""
    console.print(f"[info]Analyzing PYUSD balance mapping for {len(key_addresses)} addresses...", style="info")

    # PYUSD balances are typically at slot 4 (OpenZeppelin implementation)
    balances_slot = 4
    mapping_results = []

    for address in key_addresses:
        # Calculate storage slot for this address's balance
        padded_address = f"0x{address[2:].lower().zfill(64)}"
        concat_hex = padded_address[2:] + f"{balances_slot:064x}"
        slot_to_read = Web3.keccak(hexstr=concat_hex).hex()

        # Read the storage
        params = [block_hash, tx_index, contract_address, slot_to_read, 1]
        storage_result = make_rpc_request("debug_storageRangeAt", params, network='mainnet')

        if storage_result and 'storage' in storage_result:
            value_hex = storage_result['storage'].get(slot_to_read, {}).get('value', '0x0')
            value_int = int(value_hex, 16)
            balance_pyusd = value_int / 10**6  # PYUSD uses 6 decimals

            # Check if address is a contract
            is_contract = False
            contract_name = None
            try:
                if w3_mainnet:
                    code = w3_mainnet.eth.get_code(address)
                    is_contract = len(code) > 2  # '0x' for non-contracts

                # Check if it's a known PYUSD-related contract
                if address.lower() in PYUSD_CONTRACTS:
                    contract_name = PYUSD_CONTRACTS[address.lower()]
            except:
                pass

            mapping_results.append({
                'address': address,
                'display_address': shorten_address(address),
                'balance': balance_pyusd,
                'balance_raw': value_int,
                'is_contract': is_contract,
                'contract_name': contract_name,
                'slot': slot_to_read
            })

    # Create DataFrame
    balances_df = pd.DataFrame(mapping_results)

    # Display balances table
    if not balances_df.empty:
        # Sort by balance
        balances_df = balances_df.sort_values('balance', ascending=False)

        console.print(f"\n[bold cyan3]📊 PYUSD Balance Distribution ({len(balances_df)} addresses)[/bold cyan3]")

        # Create enhanced balance table
        balance_table = Table(title="PYUSD Balances", title_style="bold cyan")
        balance_table.add_column("Address", style="dim")
        balance_table.add_column("Type", style="cyan")
        balance_table.add_column("Balance (PYUSD)", justify="right", style="green")
        balance_table.add_column("% of Sample", justify="right")

        total_sample = balances_df['balance'].sum()

        for _, row in balances_df.head(10).iterrows():
            addr_display = row['display_address']
            if row['contract_name']:
                addr_display += f" ({row['contract_name']})"

            balance = row['balance']
            percent = (balance / total_sample * 100) if total_sample > 0 else 0

            addr_type = "Contract" if row['is_contract'] else "EOA"

            balance_table.add_row(
                addr_display,
                addr_type,
                f"{balance:,.6f}",
                f"{percent:.2f}%"
            )

        if len(balances_df) > 10:
            balance_table.add_row("...", "...", f"+ {len(balances_df) - 10} more", "")

        console.print(balance_table)

        # Create balance distribution visualization
        try:
            # Prepare data for pie chart - get top holders
            if len(balances_df) > 8:
                top_holders = balances_df.nlargest(7, 'balance')
                others_sum = balances_df.nsmallest(len(balances_df) - 7, 'balance')['balance'].sum()
                others_row = pd.DataFrame([{
                    'display_address': f"Others ({len(balances_df) - 7} addresses)",
                    'balance': others_sum,
                    'is_contract': False,
                    'contract_name': None
                }])
                viz_df = pd.concat([top_holders, others_row])
            else:
                viz_df = balances_df

            # Create pie chart
            fig = px.pie(
                viz_df,
                values='balance',
                names='display_address',
                title=f'PYUSD Balance Distribution at Block {shorten_address(block_hash)}',
                hover_data=['balance', 'is_contract']
            )

            fig.update_traces(textposition='inside', textinfo='percent+label')
            fig.update_layout(template="plotly_white")

            fig.show()

            # Create bar chart showing contract vs EOA distribution
            contract_summary = balances_df.groupby('is_contract').agg(
                count=('balance', 'count'),
                total_balance=('balance', 'sum')
            ).reset_index()

            contract_summary['type'] = contract_summary['is_contract'].apply(
                lambda x: 'Smart Contracts' if x else 'User Wallets'
            )

            # Create two subplots - count and balance
            fig = make_subplots(rows=1, cols=2,
                             subplot_titles=('Address Count', 'Balance Distribution'),
                             specs=[[{"type": "pie"}, {"type": "pie"}]])

            fig.add_trace(
                go.Pie(
                    labels=contract_summary['type'],
                    values=contract_summary['count'],
                    textinfo='percent+label'
                ),
                row=1, col=1
            )

            fig.add_trace(
                go.Pie(
                    labels=contract_summary['type'],
                    values=contract_summary['total_balance'],
                    textinfo='percent+label'
                ),
                row=1, col=2
            )

            fig.update_layout(
                title_text="PYUSD Distribution: Contracts vs. User Wallets",
                template="plotly_white"
            )

            fig.show()
        except Exception as viz_err:
            console.print(f"[warning]Could not create balance visualization: {viz_err}", style="warning")

    # Add export options
    console.print("\n[bold cyan3]📤 Export Options[/bold cyan3]")

    # Create export buttons
    export_buttons = widgets.HBox([
        widgets.Button(
            description='Export to CSV',
            button_style='primary',
            layout=widgets.Layout(width='150px')
        ),
        widgets.Button(
            description='Export as JSON',
            button_style='warning',
            layout=widgets.Layout(width='150px')
        )
    ])

    export_output = widgets.Output()
    display(export_buttons)
    display(export_output)

    # Export handlers
    def export_csv(b):
        with export_output:
            clear_output()
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"pyusd_balances_{shorten_address(contract_address)}_{timestamp}.csv"
            display(download_csv_direct(balances_df, filename))

    def export_json(b):
        with export_output:
            clear_output()
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"pyusd_balances_{shorten_address(contract_address)}_{timestamp}.json"

            export_data = {
                "metadata": {
                    "contract_address": contract_address,
                    "block_hash": block_hash,
                    "analysis_time": datetime.now().isoformat(),
                    "addresses_analyzed": len(balances_df),
                    "total_balance": balances_df['balance'].sum()
                },
                "balance_data": balances_df.to_dict('records')
            }

            display(download_json_direct(export_data, filename))

    export_buttons.children[0].on_click(export_csv)
    export_buttons.children[1].on_click(export_json)

    return balances_df


def analyze_storage_history(contract_address, block_numbers, slot_to_track, network='mainnet'):
    """Track a specific storage slot across multiple blocks to see how it changes over time."""
    console.print(f"[info]Analyzing historical values of slot {slot_to_track} across {len(block_numbers)} blocks...", style="info")

    history = []

    for block_num in block_numbers:
        try:
            # Get block hash
            block_info = w3_mainnet.eth.get_block(block_num)
            block_hash = block_info['hash'].hex()
            block_timestamp = block_info['timestamp']

            # Read storage at this block
            params = [block_hash, 0, contract_address, slot_to_track, 1]
            storage_result = make_rpc_request("debug_storageRangeAt", params, network=network)

            if storage_result and 'storage' in storage_result:
                value_hex = storage_result['storage'].get(slot_to_track, {}).get('value', '0x0')
                value_int = int(value_hex, 16)

                # Special handling for total supply (assuming slot 0)
                if slot_to_track == "0x0":
                    formatted_value = f"{value_int / 10**6:,.6f} PYUSD"
                    value_type = "total_supply"
                # Special handling for paused state
                elif slot_to_track == "0x5ac1dce9f7971a63e05025b10b44b6f3c868ae576a5e4a815201051d3eae29cb":
                    formatted_value = "Paused" if value_int == 1 else "Active"
                    value_type = "paused_state"
                else:
                    formatted_value = value_hex
                    value_type = "raw"

                history.append({
                    'block': block_num,
                    'block_hash': block_hash,
                    'timestamp': block_timestamp,
                    'datetime': datetime.fromtimestamp(block_timestamp),
                    'value_hex': value_hex,
                    'value_int': value_int,
                    'formatted_value': formatted_value,
                    'value_type': value_type
                })
        except Exception as e:
            console.print(f"[warning]Error reading block {block_num}: {e}", style="warning")

    if history:
        history_df = pd.DataFrame(history)

        # Display history table
        console.print(f"\n[bold cyan3]📊 Historical Values for Slot {slot_to_track}[/bold cyan3]")

        # Create timeline visualization
        try:
            # Time series plot of values
            fig = px.line(
                history_df,
                x='datetime',
                y='value_int',
                title=f'PYUSD Storage Slot {slot_to_track} - Historical Values',
                labels={'value_int': 'Value', 'datetime': 'Block Time'},
                markers=True
            )

            fig.update_layout(template="plotly_white")
            fig.show()

            # Create table view of history data
            display_cols = ['block', 'datetime', 'formatted_value']
            display(history_df[display_cols])
        except Exception as viz_err:
            console.print(f"[warning]Could not create history visualization: {viz_err}", style="warning")

        return history_df
    else:
        console.print("[warning]No historical data retrieved.", style="warning")
        return None


def detect_storage_patterns(storage_df):
    """Automatically detect and classify storage patterns specific to PYUSD."""
    console.print(f"[info]Analyzing storage patterns across {len(storage_df)} slots...", style="info")

    patterns = {
        'erc20_standard': False,
        'proxy_pattern': False,
        'access_control': False,
        'pausable': False,
        'upgradeable': False,
        'detailed_patterns': []
    }

    # Check for ERC20 standard slots
    if 0 in storage_df['slot_int'].values:
        # Slot 0 typically has totalSupply
        patterns['erc20_standard'] = True
        patterns['detailed_patterns'].append({
            'type': 'erc20',
            'confidence': 'high',
            'description': 'Standard ERC20 storage layout detected (totalSupply at slot 0)'
        })

    # Check for proxy pattern
    impl_slot = "0x360894a13ba1a3210667c828492db98dca3e2076cc3735a920a3ca505d382bbc"
    admin_slot = "0xb53127684a568b3173ae13b9f8a6016e243e63b6e8ee1178d6a717850b5d6103"

    has_impl = impl_slot in storage_df['slot_hex'].values
    has_admin = admin_slot in storage_df['slot_hex'].values

    if has_impl or has_admin:
        patterns['proxy_pattern'] = True
        confidence = 'high' if (has_impl and has_admin) else 'medium'
        patterns['detailed_patterns'].append({
            'type': 'proxy',
            'confidence': confidence,
            'description': f'EIP-1967 proxy pattern detected ({["admin", "implementation"][has_impl]} slot found)'
        })

    # Check for pausable pattern
    paused_slot = "0x5ac1dce9f7971a63e05025b10b44b6f3c868ae576a5e4a815201051d3eae29cb"
    if paused_slot in storage_df['slot_hex'].values:
        patterns['pausable'] = True
        patterns['detailed_patterns'].append({
            'type': 'pausable',
            'confidence': 'high',
            'description': 'OpenZeppelin Pausable pattern detected (paused state slot found)'
        })

    # Check for AccessControl pattern
    role_admin_slot = "0xb09aa5aeb3702cfd50b6b62bc4532604938f21248a27a1d5ca736082b6819cc1"
    if role_admin_slot in storage_df['slot_hex'].values:
        patterns['access_control'] = True
        patterns['detailed_patterns'].append({
            'type': 'access_control',
            'confidence': 'high',
            'description': 'OpenZeppelin AccessControl pattern detected (role admin slot found)'
        })

    # Display detected patterns
    console.print(f"\n[bold cyan3]🧩 PYUSD Storage Pattern Analysis[/bold cyan3]")

    pattern_table = Table(title="Detected Storage Patterns", title_style="bold cyan")
    pattern_table.add_column("Pattern", style="cyan")
    pattern_table.add_column("Detected", justify="center")
    pattern_table.add_column("Confidence", justify="center")
    pattern_table.add_column("Description")

    for pattern in patterns['detailed_patterns']:
        pattern_table.add_row(
            pattern['type'].replace('_', ' ').title(),
            "✓",
            pattern['confidence'].upper(),
            pattern['description']
        )

    console.print(pattern_table)

    # Create pattern visualization
    try:
        # Create hierarchy diagram
        storage_graph = Digraph(comment='PYUSD Storage Patterns')
        storage_graph.attr(rankdir='TB', bgcolor='transparent')

        # Main contract node
        storage_graph.node('contract', 'PYUSD Token', shape='box', style='filled', fillcolor='lightblue')

        # Add pattern nodes
        if patterns['erc20_standard']:
            storage_graph.node('erc20', 'ERC20 Standard', shape='ellipse', style='filled', fillcolor='palegreen')
            storage_graph.edge('contract', 'erc20')

        if patterns['proxy_pattern']:
            storage_graph.node('proxy', 'Proxy Pattern', shape='ellipse', style='filled', fillcolor='lightyellow')
            storage_graph.edge('contract', 'proxy')

        if patterns['pausable']:
            storage_graph.node('pausable', 'Pausable', shape='ellipse', style='filled', fillcolor='lightpink')
            storage_graph.edge('contract', 'pausable')

        if patterns['access_control']:
            storage_graph.node('access', 'Access Control', shape='ellipse', style='filled', fillcolor='lightcyan')
            storage_graph.edge('contract', 'access')

        display(storage_graph)
        console.print("[info]This diagram shows the detected storage patterns in PYUSD contract.", style="info")
    except Exception as viz_err:
        console.print(f"[warning]Could not create pattern visualization: {viz_err}", style="warning")

    return patterns


# --- Convenience function for common ERC20 analysis ---
def analyze_erc20_storage(token_address, block_hash, accounts_to_check=None):
    """Analyze key ERC20 storage including: totalSupply, balances, allowances."""
    console.print(f"[info]Analyzing ERC20 storage for {shorten_address(token_address)}...", style="info")

    # First get basic storage
    params = [block_hash, 0, token_address, "0x0", 10]  # First 10 slots for base data
    storage_result = make_rpc_request("debug_storageRangeAt", params, network='mainnet')

    if not storage_result:
        console.print("[error]Failed to retrieve basic ERC20 storage.", style="error")
        return None

    # Analyze base contract storage
    base_analysis = analyze_storage_dump(storage_result, token_address, block_hash)

    # If accounts provided, also check balances
    balances_results = None
    if accounts_to_check and isinstance(accounts_to_check, list) and len(accounts_to_check) > 0:
        console.print(f"[info]Checking balances for {len(accounts_to_check)} accounts...", style="info")

        # Common ERC20 storage slots
        balances_slot = 4  # OpenZeppelin ERC20 uses slot 4 for balances

        # Get balances for accounts
        balances_results = analyze_mapping_storage(token_address, balances_slot, accounts_to_check, block_hash)

        # For a deeper analysis, use the specialized balances function
        console.print("\n[bold cyan3]🔍 Advanced PYUSD Balance Analysis[/bold cyan3]")
        analyze_pyusd_balances(token_address, block_hash, accounts_to_check)

    # Detect and display storage patterns
    detect_storage_patterns(base_analysis['storage_df'])

    # Return comprehensive analysis
    return {
        'base_analysis': base_analysis,
        'balances': balances_results,
        'contract_address': token_address,
        'block_hash': block_hash
    }

# --- Execute Storage analysis ---
RUN_STORAGE_DUMP = True # <<< SET TO TRUE TO RUN THIS
STORAGE_TARGET_CONTRACT = PYUSD_PROXY # Target contract (Proxy or Implementation)
STORAGE_SLOT_COUNT = 20 # Number of slots to dump from 0

if 'TARGET_BLOCK_IDENTIFIER' in locals() and TARGET_BLOCK_IDENTIFIER is not None:
    target_block_num_for_storage = TARGET_BLOCK_IDENTIFIER
    block_hash_for_storage = None

    # --- Get Block Hash ---
    # debug_storageRangeAt requires a block HASH, not number or tag
    try:
        console.print("\n\n[bold chartreuse1]💾 Inspecting Raw Contract Storage with `debug_storageRangeAt`[/bold chartreuse1]")
        console.print("─────────────────────────────────────────────────────────────", style="chartreuse1")

        console.print(f"\n\n[info]Fetching block hash for identifier '{target_block_num_for_storage}'", style="info")
        if w3_mainnet:
            # Ensure identifier is suitable for get_block
            block_identifier_param = target_block_num_for_storage
            if isinstance(target_block_num_for_storage, str) and not target_block_num_for_storage.startswith('0x'):
                if target_block_num_for_storage not in ["latest", "earliest", "pending"]:
                    try: # Convert string number to int
                        block_identifier_param = int(target_block_num_for_storage)
                    except ValueError:
                        console.print(f"[error]Invalid block identifier string '{target_block_num_for_storage}' for get_block.", style="error")
                        block_identifier_param = None

            if block_identifier_param is not None:
                block_info = w3_mainnet.eth.get_block(block_identifier_param)
                if block_info and 'hash' in block_info:
                    block_hash_for_storage = block_info['hash'].hex()
                    console.print(f"[info]Using block hash: {block_hash_for_storage}", style="info")
                else:
                    console.print(f"[error]Could not retrieve block info or hash for identifier '{target_block_num_for_storage}'.", style="error")
        else:
            console.print("[error]Mainnet client not available to fetch block hash.", style="error")

    except Exception as e:
        console.print(f"[error]Error retrieving block hash for '{target_block_num_for_storage}': {e}", style="error")

    # --- Execute RPC Call ---
    if RUN_STORAGE_DUMP and block_hash_for_storage:
        tx_index_in_block = 0 # State before first tx in the block
        start_key_slot = "0x0000000000000000000000000000000000000000000000000000000000000000"

        params = [block_hash_for_storage, tx_index_in_block, STORAGE_TARGET_CONTRACT, start_key_slot, STORAGE_SLOT_COUNT]
        console.print(f"\n\n[info]Attempting debug_storageRangeAt for {STORAGE_TARGET_CONTRACT} at block {block_hash_for_storage} (Tx Index {tx_index_in_block}, {STORAGE_SLOT_COUNT} slots)", style="info")
        storage_dump_result_data = make_rpc_request("debug_storageRangeAt", params, network='mainnet')

        if storage_dump_result_data:
            # Use Enhanced Storage analysis
            storage_analysis = analyze_storage_dump(storage_dump_result_data, STORAGE_TARGET_CONTRACT, block_hash_for_storage)

            # Optional: For in-depth ERC20 analysis with key wallets
            # Uncomment and modify the addresses list below to run this analysis
            # key_addresses = ["0x1234...", "0x5678..."]  # Add addresses you want to check
            # analyze_erc20_storage(STORAGE_TARGET_CONTRACT, block_hash_for_storage, key_addresses)

            # Optional: For historical analysis of a specific slot (e.g., total supply)
            # Uncomment and modify the block list below to run this analysis
            # historical_blocks = [17000000, 17010000, 17020000]  # Add block numbers to check
            # analyze_storage_history(STORAGE_TARGET_CONTRACT, historical_blocks, "0x0")  # Track slot 0 (total supply)
        else:
            console.print("[error]Failed to retrieve storage range. Method might be unavailable or params incorrect.", style="error")

    elif RUN_STORAGE_DUMP:
        console.print("[warning]Skipping 'debug_storageRangeAt': Block hash could not be determined.", style="warning")
    else:
        console.print("\n\n[info]Skipping 'debug_storageRangeAt' as RUN_STORAGE_DUMP is False.", style="info")

elif RUN_STORAGE_DUMP:
    console.print("[warning]Skipping 'debug_storageRangeAt': TARGET_BLOCK_IDENTIFIER invalid or not set.", style="warning")

## 1.11 🏊 `txpool_status`: Monitoring Network Congestion
---

This section utilizes the `txpool_status` RPC method to query the connected Ethereum node about the current state of its transaction pool (mempool). It returns the number of transactions currently pending (ready for inclusion in the next blocks) and queued (waiting for prerequisites like a correct nonce or sufficient sender balance).

This provides a valuable snapshot of network activity and potential congestion *from the perspective of the specific node you are connected to*.

> **🚀 Leveraging GCP's Premium RPC Capabilities**
>
> *   **Method:** `txpool_status`
> *   **Multiplier:** `50x`
> *   **GCP Advantage:** While conceptually simple, accessing mempool data can be restricted or less reliable on public nodes. GCP's dedicated infrastructure provides stable access to this information.
> *   **PYUSD Insight:** `txpool_status` helps assess:
>     *   **Current Network Load:** Understand how busy the network is when considering sending a PYUSD transaction.
>     *   **Estimated Confirmation Times:** Gauge how long a PYUSD transaction might take to get mined based on the pending queue size.
>     *   **Gas Price Strategy:** Inform decisions on appropriate gas prices needed for timely PYUSD transaction inclusion based on current congestion levels.

**Analysis Workflow:**

1.  **Fetch Status:** Calls `txpool_status` on the connected node(s) (Mainnet, optionally testnets).
2.  **Parse Counts:** Extracts the `pending` and `queued` transaction counts (converting from hex).
3.  **Analyze Congestion:** Uses helper functions (`analyze_network_congestion`, `estimate_confirmation_time`) to interpret the pending count and classify the congestion level (Low, Moderate, High, Extreme).
4.  **Recommend Gas (Mainnet):** Fetches the current `baseFeePerGas` from the latest block and uses `recommend_gas_prices` to suggest appropriate gas prices (maxFeePerGas/maxPriorityFeePerGas) for different confirmation speeds based on current congestion.
5.  **Visualize & Summarize:**
    *   **Status Tables:** Displays pending/queued counts, congestion level, and estimated confirmation time for each checked network.
    *   **Gas Recommendation Table (Mainnet):** Shows suggested gas prices (Gwei) for Slow, Standard, Fast, and Rapid confirmation targets.
    *   **Congestion Gauge:** Provides a visual indicator of the current network congestion level (based on pending count).
    *   **Network Comparison:** If multiple networks are checked, displays a comparative table and bar chart of pending/queued counts.
6.  **(Optional Advanced Analysis):** Includes code (controlled by `RUN_TXPOOL_CONTENT`, default `True`) to call the **extremely expensive (`~100x`)** `txpool_content` method to fetch *all* transactions from the pool and specifically identify pending PYUSD transactions using `analyze_txpool_for_pyusd`. This part is highly resource-intensive.
7.  **Export Options (if `txpool_content` is run):** Download details of pending PYUSD transactions.

**💡 What to Look For:**
*   **Pending Count:** A high pending count indicates network congestion.
*   **Congestion Level & Confirmation Time:** Use these to set expectations for how quickly your PYUSD transaction might be processed.
*   **Gas Recommendations:** Use the suggested Gwei values as a starting point when sending transactions, adjusting based on urgency.
*   **(If `txpool_content` run):** Observe the number and types (transfer, approve) of PYUSD transactions currently waiting in the mempool.

In [ ]:
# =============================================================================================
# 🏊 Transaction Pool Analysis using txpool_status (50x Cost)
# =============================================================================================
# This cell provides a comprehensive analysis of the Ethereum transaction pool (mempool):
# - Fetches basic pool statistics (pending/queued counts) using `txpool_status`.
# - Analyzes network congestion levels based on pending transactions and visualizes it with a gauge chart.
# - Estimates transaction confirmation times based on pool size.
# - Retrieves the current network base fee and provides recommended gas prices for different confirmation speeds (Slow, Standard, Fast, Rapid).
# - Optionally fetches detailed transaction data from the pool using the expensive `txpool_content` method.
# - Scans the detailed pool content to identify PYUSD-related transactions by checking the 'to' address and function signatures.
# - Displays detailed information about identified PYUSD transactions in an interactive table.
# - Analyzes and displays the distribution of different PYUSD functions found in the mempool.
# - Compares transaction pool status across multiple connected networks (e.g., Mainnet, Sepolia, Holesky).
# - Includes robust helper functions for direct data export to CSV, JSON, and formatted Google Sheets.
# =============================================================================================

import time
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import base64
import json
from IPython.display import HTML, display
import ipywidgets as widgets
from IPython.display import clear_output
from IPython.core.display import display, HTML

# Set larger text size for all output
display(HTML("<style>.container { font-size: 1.2em; }</style>"))

# --- Define helper functions for download capabilities ---

def download_csv_direct(df, filename=None):
    """
    Creates a direct download for CSV without intermediate display.

    Args:
        df: DataFrame to export
        filename: Optional custom filename (default: auto-generated with timestamp)

    Returns:
        HTML object that triggers browser download
    """
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"txpool_pyusd_data_{timestamp}.csv"

    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()

    # Create direct download HTML
    html = f'''
    <script>
    function download(filename, data) {{
        const a = document.createElement('a');
        a.href = data;
        a.download = filename;
        document.body.appendChild(a);
        a.click();
        document.body.removeChild(a);
    }}

    download("{filename}", "data:text/csv;base64,{payload}");
    </script>
    <div>Downloading {filename}...</div>
    '''
    return HTML(html)

def download_json_direct(data, filename=None):
    """
    Creates a direct download for JSON without intermediate display.

    Args:
        data: Dict or list to export as JSON
        filename: Optional custom filename (default: auto-generated with timestamp)

    Returns:
        HTML object that triggers browser download
    """
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"txpool_pyusd_data_{timestamp}.json"

    # Convert to JSON string (handling non-serializable objects)
    json_str = json.dumps(data, default=str, indent=2)
    b64 = base64.b64encode(json_str.encode()).decode()

    # Create direct download HTML
    html = f'''
    <script>
    function download(filename, data) {{
        const a = document.createElement('a');
        a.href = data;
        a.download = filename;
        document.body.appendChild(a);
        a.click();
        document.body.removeChild(a);
    }}

    download("{filename}", "data:application/json;base64,{b64}");
    </script>
    <div>Downloading {filename}...</div>
    '''
    return HTML(html)

def export_to_google_sheets(df, data_dict, title_prefix="PYUSD Mempool Analysis"):
    """
    Export transaction pool analysis data to Google Sheets with rich formatting.

    Args:
        df: DataFrame with transaction data
        data_dict: Dictionary with additional structured data
        title_prefix: Prefix for the sheet title

    Returns:
        HTML object that opens the created Google Sheet
    """
    # Show loading message
    console.print("[cyan3]Exporting to Google Sheets...", style="info")

    try:
        # Create a new Google Sheet with meaningful title
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        sheet_title = f"{title_prefix} {timestamp}"

        # Use the global gc_sheets client that's already authenticated
        spreadsheet = gc_sheets.create(sheet_title)

        # Get the default worksheet and rename it
        worksheet = spreadsheet.get_worksheet(0)
        worksheet.update_title("PYUSD Transactions")

        # Set up a header with transaction info
        header_values = [
            ["PYUSD Transactions in Mempool Analysis"],
            [f"Analysis Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"],
            [""],  # Empty row for spacing
        ]
        worksheet.update("A1", header_values)

        # Format the header with bold text and colored background
        worksheet.format("A1:A1", {
            "textFormat": {"bold": True, "fontSize": 14},
            "backgroundColor": {"red": 0.9, "green": 0.9, "blue": 1.0}
        })

        worksheet.format("A2:A2", {
            "textFormat": {"bold": True, "fontSize": 12}
        })

        current_row = 4  # Start after header

        # Add transaction stats summary
        if "summary" in data_dict:
            stats = data_dict["summary"]

            # Add section title
            worksheet.update(f"A{current_row}", [["Analysis Summary"]])
            worksheet.format(f"A{current_row}:A{current_row}", {
                "textFormat": {"bold": True, "fontSize": 12},
                "backgroundColor": {"red": 0.8, "green": 0.9, "blue": 1.0}
            })
            current_row += 1

            # Add stats data
            stats_rows = []
            stats_rows.append(["Metric", "Value"])  # Header row
            for key, value in stats.items():
                # Format keys and values appropriately
                formatted_key = key.replace("_", " ").title()

                # Try to format numerical values with commas
                try:
                    if isinstance(value, (int, float)):
                        formatted_value = f"{value:,}"
                    else:
                        formatted_value = str(value)
                except:
                    formatted_value = str(value)

                stats_rows.append([formatted_key, formatted_value])

            # Add stats table
            stats_start_row = current_row
            worksheet.update(f"A{stats_start_row}", stats_rows)

            # Format stats table header
            worksheet.format(f"A{stats_start_row}:B{stats_start_row}", {
                "textFormat": {"bold": True},
                "backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}
            })

            current_row += len(stats_rows) + 2  # Add extra space after table

        # Add function distribution if available
        if "function_distribution" in data_dict:
            # Add section title
            worksheet.update(f"A{current_row}", [["Function Distribution"]])
            worksheet.format(f"A{current_row}:A{current_row}", {
                "textFormat": {"bold": True, "fontSize": 12},
                "backgroundColor": {"red": 0.7, "green": 0.9, "blue": 1.0}
            })
            current_row += 1

            # Add distribution data
            dist_rows = []
            dist_rows.append(["Function", "Count", "Percentage"])  # Header row

            for func, data in data_dict["function_distribution"].items():
                dist_rows.append([func, data["count"], f"{data['percentage']:.1f}%"])

            # Add distribution table
            dist_start_row = current_row
            worksheet.update(f"A{dist_start_row}", dist_rows)

            # Format distribution table header
            worksheet.format(f"A{dist_start_row}:C{dist_start_row}", {
                "textFormat": {"bold": True},
                "backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}
            })

            current_row += len(dist_rows) + 2  # Add extra space after table

        # Add main DataFrame data
        if not df.empty:
            # Add a section title
            worksheet.update(f"A{current_row}", [["PYUSD Transactions Details"]])
            worksheet.format(f"A{current_row}:A{current_row}", {
                "textFormat": {"bold": True, "fontSize": 12},
                "backgroundColor": {"red": 0.8, "green": 0.8, "blue": 1.0}
            })
            current_row += 1

            # Convert DataFrame to list of lists for the worksheet
            df_values = [df.columns.tolist()] + df.values.tolist()

            # Format values for better readability
            for i in range(1, len(df_values)):
                for j, col in enumerate(df.columns):
                    val = df_values[i][j]

                    # Format different column types appropriately
                    if pd.isnull(val):
                        df_values[i][j] = ""
                    elif col == "gas_price_gwei" and isinstance(val, (int, float)):
                        df_values[i][j] = f"{val:.2f}"
                    else:
                        df_values[i][j] = str(val)

            worksheet.update(f"A{current_row}", df_values)

            # Format the DataFrame header
            worksheet.format(f"A{current_row}:{chr(65+len(df.columns)-1)}{current_row}", {
                "textFormat": {"bold": True},
                "backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}
            })

            # Add alternating row colors for readability
            data_rows = len(df_values)
            for i in range(2, data_rows + 1, 2):
                row_num = current_row + i - 1
                worksheet.format(f"A{row_num}:{chr(65+len(df.columns)-1)}{row_num}", {
                    "backgroundColor": {"red": 0.97, "green": 0.97, "blue": 1.0}
                })

        # Try to auto-resize columns for better readability
        try:
            worksheet.columns_auto_resize(0, 10)  # Attempt to resize first 10 columns
        except:
            pass  # Ignore if not supported

        # Clear loading message and show success message
        clear_output()
        console.print("✓ Successfully exported to Google Sheets", style="spring_green3")

        # Open the spreadsheet in a new tab
        spreadsheet_url = f"https://docs.google.com/spreadsheets/d/{spreadsheet.id}"
        html = f'''
        <script>
        window.open("{spreadsheet_url}", "_blank");
        </script>
        <div>Spreadsheet created and opened: <a href="{spreadsheet_url}" target="_blank">{sheet_title}</a></div>
        '''
        return HTML(html)

    except Exception as e:
        # Clear loading message and show error
        clear_output()
        console.print(f"❌ Error creating Google Sheet: {str(e)}", style="error")
        return HTML(f"<div style='color:red'>Error creating Google Sheet: {str(e)}</div>")


# --- Define helper functions for advanced txpool analysis ---

def estimate_confirmation_time(pending_count):
    """
    Estimate confirmation time based on pending transaction count.

    Args:
        pending_count: Number of pending transactions

    Returns:
        Human-readable estimated confirmation time string
    """
    # Ethereum processes ~15 tx/sec on average (more with optimizations)
    # Each block contains ~250 transactions and comes every ~12 seconds
    avg_tx_per_block = 250
    avg_block_time_sec = 12

    # Calculate blocks until this transaction would be processed
    blocks_to_wait = max(1, pending_count / avg_tx_per_block)

    # Calculate time in seconds
    wait_time_sec = blocks_to_wait * avg_block_time_sec

    # Return human-readable time
    if wait_time_sec < 60:
        return f"~{int(wait_time_sec)} seconds"
    elif wait_time_sec < 3600:
        return f"~{int(wait_time_sec/60)} minutes"
    else:
        return f"~{wait_time_sec/3600:.1f} hours"


def analyze_network_congestion(pending_count):
    """
    Analyze network congestion level based on pending tx count.

    Args:
        pending_count: Number of pending transactions

    Returns:
        Dictionary with congestion analysis details
    """
    if pending_count < 1000:
        return {
            "level": "Low",
            "description": "Network is not congested. Transactions should confirm quickly with standard gas prices.",
            "color": "green",
            "factor": 0.2,  # Congestion factor (0-1)
        }
    elif pending_count < 5000:
        return {
            "level": "Moderate",
            "description": "Some network congestion. Consider using slightly higher gas prices for faster confirmation.",
            "color": "yellow",
            "factor": 0.5,
        }
    elif pending_count < 15000:
        return {
            "level": "High",
            "description": "Network is congested. Higher gas prices recommended for reasonable confirmation times.",
            "color": "orange",
            "factor": 0.8,
        }
    else:
        return {
            "level": "Extreme",
            "description": "Network is extremely congested. High gas prices required for timely confirmation.",
            "color": "red",
            "factor": 1.0,
        }


def recommend_gas_prices(base_fee, congestion_factor):
    """
    Recommend gas prices based on current network conditions.

    Args:
        base_fee: Current base fee in gwei
        congestion_factor: Current network congestion factor (0-1)

    Returns:
        Dictionary with recommended gas prices for different speed tiers
    """
    # Base gas prices based on current base fee (in gwei)
    # These are multipliers on the base fee
    multipliers = {
        "slow": 1.0,      # Just the base fee for next block inclusion
        "standard": 1.1,  # Base fee + 10% for quicker inclusion
        "fast": 1.25,     # Base fee + 25% for fast inclusion
        "rapid": 1.5      # Base fee + 50% for urgent transactions
    }

    # Adjust multipliers based on congestion (increase spread during congestion)
    if congestion_factor > 0.7:  # High congestion
        multipliers["standard"] = 1.2
        multipliers["fast"] = 1.5
        multipliers["rapid"] = 2.0
    elif congestion_factor > 0.4:  # Moderate congestion
        multipliers["standard"] = 1.15
        multipliers["fast"] = 1.35
        multipliers["rapid"] = 1.7

    # Calculate recommended prices
    recommendations = {}
    for speed, multiplier in multipliers.items():
        recommendations[speed] = round(base_fee * multiplier, 2)

    return recommendations

def get_current_base_fee(network="mainnet"):
    """
    Get the current base fee from the latest block for the specified network.

    Args:
        network (str): Network to query (mainnet, holesky, sepolia)

    Returns:
        float: Current base fee in gwei
    """
    try:
        w3 = w3_clients[network]
        latest_block = w3.eth.get_block('latest')
        # Convert Wei to Gwei
        base_fee_gwei = w3.from_wei(latest_block.baseFeePerGas, 'gwei')
        return float(base_fee_gwei)
    except Exception as e:
        console.print(f"[warning]Could not get current base fee for {network}: {str(e)}", style="warning")
        # Return an estimated value if we can't get the actual value
        return 15.0  # Default fallback value in gwei

def create_congestion_gauge(congestion_factor, title="Network Congestion"):
    """
    Create a gauge chart showing network congestion level.

    Args:
        congestion_factor: Current network congestion factor (0-1)
        title: Chart title

    Returns:
        Plotly Figure object
    """
    fig = go.Figure(go.Indicator(
        mode = "gauge+number",
        value = congestion_factor * 100,
        domain = {'x': [0, 1], 'y': [0, 1]},
        title = {'text': title, 'font': {'size': 24}},  # Larger title font
        gauge = {
            'axis': {'range': [0, 100], 'tickwidth': 1, 'tickcolor': "darkgray", 'tickfont': {'size': 14}},
            'bar': {'color': "royalblue"},
            'bgcolor': "white",
            'borderwidth': 2,
            'bordercolor': "gray",
            'steps': [
                {'range': [0, 25], 'color': 'green'},
                {'range': [25, 50], 'color': 'yellow'},
                {'range': [50, 75], 'color': 'orange'},
                {'range': [75, 100], 'color': 'red'},
            ],
        }
    ))

    fig.update_layout(
        height=350,  # Slightly taller
        margin=dict(l=20, r=20, t=50, b=20),
        font=dict(size=16)  # Larger font for all text elements
    )

    return fig


def analyze_txpool_for_pyusd(txpool_content, pending_only=True):
    """
    Analyze transaction pool content to find PYUSD-related transactions.

    Args:
        txpool_content: Transaction pool content from RPC
        pending_only: Whether to analyze only pending transactions (True) or both pending and queued (False)

    Returns:
        Dictionary with PYUSD transaction analysis results
    """
    if not txpool_content:
        return {"count": 0, "transactions": []}

    pyusd_txs = []
    total_count = 0

    # Only analyze pending if specified, otherwise look at both pending and queued
    sections = ["pending"] if pending_only else ["pending", "queued"]

    for section in sections:
        if section not in txpool_content:
            continue

        for sender, nonce_dict in txpool_content[section].items():
            for nonce, tx_data in nonce_dict.items():
                total_count += 1

                # Check if this transaction is PYUSD-related
                is_pyusd_tx = False

                # Check if to address is a PYUSD contract - Handle None case properly
                to_address = tx_data.get("to", "")
                if to_address is not None:
                    to_address = to_address.lower()
                    if to_address in PYUSD_CONTRACTS:
                        is_pyusd_tx = True

                # Check input data for PYUSD function signatures
                input_data = tx_data.get("input", "")
                if len(input_data) >= 10:  # At least contains a function selector
                    method_sig = input_data[:10]
                    if method_sig in PYUSD_SIGNATURES:
                        is_pyusd_tx = True

                if is_pyusd_tx:
                    # Extract useful information
                    gas_price = int(tx_data.get("gasPrice", "0x0"), 16) / 1e9  # Convert to Gwei

                    pyusd_txs.append({
                        "hash": tx_data.get("hash", "Unknown"),
                        "from": sender,
                        "to": to_address if to_address is not None else "Contract Creation",
                        "nonce": int(nonce, 16) if isinstance(nonce, str) else nonce,
                        "function": decode_pyusd_function(input_data) if len(input_data) >= 10 else "Unknown",
                        "gas_price_gwei": gas_price,
                        "status": section
                    })

    return {
        "count": len(pyusd_txs),
        "total_analyzed": total_count,
        "transactions": pyusd_txs
    }


# =============================================================================================
# --- Execute txpool_status Analysis ---
# =============================================================================================

RUN_TXPOOL_STATUS = True # <<< SET TO TRUE TO RUN THIS (50x COST)
RUN_TXPOOL_CONTENT = True # <<< SET TO TRUE TO ANALYZE FULL POOL CONTENT (VERY EXPENSIVE, 100x Cost)

if RUN_TXPOOL_STATUS:
    console.print("\n\n[bold cyan3]🏊 Transaction Pool Analysis with `tx_pool`[/bold cyan3]")
    console.print("────────────────────────────────────────────", style="cyan3")

    console.print("\n\nAnalyzing network congestion and transaction confirmation times...")

    # Get txpool status from multiple networks if available
    networks_to_check = ["mainnet"]
    if 'holesky' in w3_clients and w3_clients['holesky'].is_connected():
        networks_to_check.append("holesky")
    if 'sepolia' in w3_clients and w3_clients['sepolia'].is_connected():
        networks_to_check.append("sepolia")

    network_results = {}

    for network in networks_to_check:
        console.print(f"\n\n[info]Fetching txpool status on {network.capitalize()}...", style="info")
        txpool_status_result = make_rpc_request("txpool_status", [], network=network)

        if txpool_status_result:
            # Result values are hex strings
            pending_count = int(txpool_status_result.get('pending', '0x0'), 16)
            queued_count = int(txpool_status_result.get('queued', '0x0'), 16)
            total_count = pending_count + queued_count

            # Analyze network congestion
            congestion_analysis = analyze_network_congestion(pending_count)
            est_confirmation_time = estimate_confirmation_time(pending_count)

            # Store for network comparison
            network_results[network] = {
                "pending": pending_count,
                "queued": queued_count,
                "total": total_count,
                "congestion": congestion_analysis,
                "confirmation_time": est_confirmation_time
            }

            # Display network results
            console.print(f"\n\n[bold cyan3]⛓️ {network.capitalize()} Transaction Pool [/bold cyan3]")
            console.print("────────────────────────────", style="cyan3")

            status_table = Table(title="", title_style="bold cyan3", border_style="cyan3")
            status_table.add_column("Metric", header_style="bold cyan3")
            status_table.add_column("Value", header_style="bold cyan3")
            status_table.add_column("Details", header_style="bold cyan3")

            status_table.add_row(
                "Pending Transactions",
                f"{pending_count:,}",
                "Waiting to be included in next blocks"
            )
            status_table.add_row(
                "Queued Transactions",
                f"{queued_count:,}",
                "Waiting for prerequisites (nonce, funds)"
            )
            status_table.add_row(
                "Total Transactions",
                f"{total_count:,}",
                "Combined pending + queued"
            )
            status_table.add_row(
                "Network Congestion",
                f"[{congestion_analysis['color']}]{congestion_analysis['level']}[/{congestion_analysis['color']}]",
                congestion_analysis['description']
            )
            status_table.add_row(
                "Est. Confirmation Time",
                est_confirmation_time,
                "Average time for new transactions"
            )

            console.print(status_table)

            # For mainnet, provide additional gas price recommendations
            console.print(f"\n\n[bold yellow3]⛽ Gas Price Recommendations for {network.capitalize()}[/bold yellow3]")
            console.print("─────────────────────────────────────────", style="yellow3")

            if network in ["mainnet", "holesky", "sepolia"]:
                # Get current base fee
                current_base_fee = get_current_base_fee(network)

                # Get gas price recommendations
                gas_recommendations = recommend_gas_prices(
                    current_base_fee,
                    congestion_analysis['factor']
                )

                # Display gas price recommendations
                gas_table = Table(title="", title_style="bold yellow3", border_style="yellow3")
                gas_table.add_column("Speed", header_style="bold yellow3")
                gas_table.add_column("Gas Price (Gwei)", header_style="bold yellow3")
                gas_table.add_column("Expected Confirmation", header_style="bold yellow3")

                gas_table.add_row(
                    "🐢 Slow",
                    f"{gas_recommendations['slow']:.2f}",
                    "Within ~5 minutes"
                )
                gas_table.add_row(
                    "🚶 Standard",
                    f"{gas_recommendations['standard']:.2f}",
                    "Within ~2 minutes"
                )
                gas_table.add_row(
                    "🏃 Fast",
                    f"{gas_recommendations['fast']:.2f}",
                    "Within ~30 seconds"
                )
                gas_table.add_row(
                    "🚀 Rapid",
                    f"{gas_recommendations['rapid']:.2f}",
                    "Next block (~12 seconds)"
                )

                console.print(gas_table)

                # Display congestion gauge visualization
                console.print(f"\n\n[bold magenta3]📶 Network Congestion Gauge for {network.capitalize()}[/bold magenta3]")
                console.print("────────────────────────────────────────", style="magenta3")

                try:
                    fig = create_congestion_gauge(congestion_analysis['factor'])
                    display(fig)
                except Exception as e:
                    console.print(f"[warning]Could not create visualization: {str(e)}", style="warning")
        else:
            console.print(f"[error]Failed to get txpool status for {network}. Method might be unavailable on this node.", style="error")

    # If we have multiple networks, show a comparison
    if len(network_results) > 1:
        console.print("\n\n[bold chartreuse1]⚖️ Network Transaction Pool Comparison[/bold chartreuse1]")
        console.print("───────────────────────────────────────", style="chartreuse1")

        comparison_table = Table(title="", title_style="bold chartreuse1", border_style="chartreuse1")
        comparison_table.add_column("Network", header_style="bold chartreuse1")
        comparison_table.add_column("Pending", header_style="bold chartreuse1")
        comparison_table.add_column("Queued", header_style="bold chartreuse1")
        comparison_table.add_column("Congestion", header_style="bold chartreuse1")

        for network, data in network_results.items():
            comparison_table.add_row(
                network.capitalize(),
                f"{data['pending']:,}",
                f"{data['queued']:,}",
                f"[{data['congestion']['color']}]{data['congestion']['level']}[/{data['congestion']['color']}]"
            )

        console.print(comparison_table)

        # Create a bar chart comparison
        try:
            console.print("\n\n[bold magenta3]🌊 Transaction Pool Comparison Across Networks[/bold magenta3]")
            console.print("────────────────────────────────────────────────", style="magenta3")

            # Prepare data for plotting
            networks = list(network_results.keys())
            pending_values = [network_results[n]['pending'] for n in networks]
            queued_values = [network_results[n]['queued'] for n in networks]

            # Create subplots with 1 row and 1 column
            fig = make_subplots(rows=1, cols=1)

            # Add bars for pending and queued transactions
            fig.add_trace(
                go.Bar(x=networks, y=pending_values, name="Pending",
                      marker_color='royalblue'),
                row=1, col=1
            )

            fig.add_trace(
                go.Bar(x=networks, y=queued_values, name="Queued",
                      marker_color='lightblue'),
                row=1, col=1
            )

            # Update layout with larger font
            fig.update_layout(
                title={
                    'text': "Transaction Pool Comparison Across Networks",
                    'font': {'size': 24}
                },
                xaxis_title={
                    'text': "Network",
                    'font': {'size': 18}
                },
                yaxis_title={
                    'text': "Number of Transactions",
                    'font': {'size': 18}
                },
                height=450,
                barmode='stack',
                font=dict(size=16)  # Increase overall font size
            )

            display(fig)
        except Exception as e:
            console.print(f"[warning]Could not create comparison visualization: {str(e)}", style="warning")

    # =============================================================================================
    # Advanced PYUSD-specific Analysis (optional, expensive)
    # =============================================================================================
    if RUN_TXPOOL_CONTENT and "mainnet" in network_results:
        console.print("\n\n[bold cyan3]🫧 PYUSD Transaction Analysis in Mempool[/bold cyan3]")
        console.print("────────────────────────────────────────", style="cyan3")

        console.print("\n\n[info]Fetching detailed transaction pool content (this is very expensive)...", style="info")

        # This is much more expensive than txpool_status
        txpool_content = make_rpc_request("txpool_content", [], network="mainnet")

        if txpool_content:
            console.print("\n\n[success]Successfully retrieved transaction pool content.", style="success")

            # Analyze the pool for PYUSD transactions
            pyusd_analysis = analyze_txpool_for_pyusd(txpool_content)

            console.print(f"[info]Found {pyusd_analysis['count']} PYUSD-related transactions out of {pyusd_analysis['total_analyzed']} transactions in the pending pool.", style="info")

            if pyusd_analysis['count'] > 0:
                # Convert to DataFrame for interactive display and export
                pyusd_tx_data = pyusd_analysis['transactions']
                pyusd_df = pd.DataFrame(pyusd_tx_data)

                # Only format numeric values, keep full addresses and hashes
                if 'gas_price_gwei' in pyusd_df.columns:
                    pyusd_df['gas_price_gwei'] = pyusd_df['gas_price_gwei'].apply(
                        lambda g: f"{g:.2f}" if isinstance(g, (int, float)) else g
                    )

                # Display interactive data table with all PYUSD transactions
                console.print("\n\n[bold cyan3]📊 PYUSD Transactions in Mempool[/bold cyan3]")
                console.print("─────────────────────────────────", style="cyan3")

                # Use IPython's display to show the interactive dataframe with full addresses
                # Set pandas display options for better readability
                with pd.option_context('display.max_rows', None,
                                      'display.max_columns', None,
                                      'display.width', None):
                    display(HTML("<style>.dataframe th, .dataframe td { font-size: 1.1em; }</style>"))
                    display(pyusd_df)

                # Analyze function distribution
                function_counts = {}
                for tx in pyusd_analysis['transactions']:
                    function = tx["function"]
                    # Check if function is a dictionary and extract a meaningful key
                    if isinstance(function, dict) and "name" in function:
                        function_key = function["name"]
                    # Fallback to string representation if it's a dict without name
                    elif isinstance(function, dict):
                        function_key = str(function)
                    # Use as is if it's already a string or other hashable type
                    else:
                        function_key = function

                    function_counts[function_key] = function_counts.get(function_key, 0) + 1

                # Prepare function distribution data for export
                function_distribution = {}
                for function, count in function_counts.items():
                    percentage = count / pyusd_analysis['count'] * 100
                    function_distribution[function] = {
                        "count": count,
                        "percentage": percentage
                    }

                # Display function distribution
                if len(function_counts) > 1:
                    console.print("\n\n[bold cyan3]🪙 PYUSD Function Distribution in Pending Transactions:[/bold cyan3]")
                    console.print("───────────────────────────────────────────────────────", style="cyan3")

                    dist_table = Table(title="", title_style="bold cyan3", border_style="cyan3")
                    dist_table.add_column("Function", header_style="cyan3")
                    dist_table.add_column("Count", justify="right", header_style="cyan3")
                    dist_table.add_column("Percentage", justify="right", header_style="cyan3")

                    for function, count in sorted(function_counts.items(), key=lambda x: x[1], reverse=True):
                        percentage = count / pyusd_analysis['count'] * 100
                        dist_table.add_row(
                            function,
                            f"{count}",
                            f"{percentage:.1f}%"
                        )

                    console.print(dist_table)

                # =============================================================================================
                # Export Options Section
                # =============================================================================================
                console.print("\n\n[bold cyan3]📤 Export Options:[/bold cyan3]")
                console.print("──────────────────", style="cyan3")

                # Create export output area
                export_output = widgets.Output()

                # Create export buttons with proper styling and larger size
                export_buttons = widgets.HBox([
                    widgets.Button(
                        description='Export to CSV',
                        button_style='primary',  # Green
                        layout=widgets.Layout(width='180px', height='40px')  # Larger buttons
                    ),
                    widgets.Button(
                        description='Export as JSON',
                        button_style='warning',  # Orange
                        layout=widgets.Layout(width='180px', height='40px')
                    ),
                    widgets.Button(
                        description='Export to Google Sheets',
                        button_style='info',     # Blue
                        layout=widgets.Layout(width='220px', height='40px')
                    )
                ])

                # Define export handlers
                def export_csv(b):
                    with export_output:
                        clear_output()
                        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                        filename = f"pyusd_mempool_transactions_{timestamp}.csv"
                        display(download_csv_direct(pyusd_df, filename))

                def export_json(b):
                    with export_output:
                        clear_output()
                        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                        filename = f"pyusd_mempool_transactions_{timestamp}.json"
                        # Prepare export data with analysis information
                        export_data = {
                            "analysis_type": "PYUSD Mempool Transactions",
                            "analysis_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                            "summary": {
                                "total_transactions_analyzed": pyusd_analysis['total_analyzed'],
                                "pyusd_transactions_found": pyusd_analysis['count'],
                                "pyusd_percentage": (pyusd_analysis['count'] / pyusd_analysis['total_analyzed'] * 100) if pyusd_analysis['total_analyzed'] > 0 else 0
                            },
                            "function_distribution": function_distribution,
                            "transactions": pyusd_tx_data
                        }
                        display(download_json_direct(export_data, filename))

                def export_to_sheets(b):
                    with export_output:
                        clear_output()
                        try:
                            # Prepare export data with analysis information
                            export_data = {
                                "summary": {
                                    "total_transactions_analyzed": pyusd_analysis['total_analyzed'],
                                    "pyusd_transactions_found": pyusd_analysis['count'],
                                    "pyusd_percentage": (pyusd_analysis['count'] / pyusd_analysis['total_analyzed'] * 100) if pyusd_analysis['total_analyzed'] > 0 else 0,
                                    "analysis_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                                },
                                "function_distribution": function_distribution
                            }
                            display(export_to_google_sheets(pyusd_df, export_data))
                        except Exception as e:
                            html = f"<div style='color:red'>Error exporting to Google Sheets: {str(e)}</div>"
                            display(HTML(html))

                            # Fallback to CSV
                            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                            filename = f"pyusd_mempool_transactions_{timestamp}.csv"
                            display(download_csv_direct(pyusd_df, filename))
                            display(HTML("<div>Falling back to CSV download due to Google Sheets error.</div>"))

                # Connect handlers to buttons
                export_buttons.children[0].on_click(export_csv)
                export_buttons.children[1].on_click(export_json)
                export_buttons.children[2].on_click(export_to_sheets)

                # Display button container and output area
                display(export_buttons)
                display(export_output)
        else:
            console.print("[error]Failed to get detailed transaction pool content. This method is often restricted.", style="error")

else:
    console.print("\n\n[info]Skipping 'txpool_status' analysis as RUN_TXPOOL_STATUS is False.", style="info")

## ✅ Conclusion: PyFlow & The GCP Advantage for Deep Blockchain Intelligence

This notebook, **PyFlow**, set out to demonstrate how deep, computationally intensive analysis of the PYUSD stablecoin could be achieved effectively and affordably. The challenge lies in the significant cost and resource requirements typically associated with advanced blockchain RPC methods needed for such insights.

**PyFlow successfully showcased a comprehensive suite of advanced analytical techniques by specifically leveraging the unique advantages of Google Cloud Platform's Blockchain Node Engine.**

**🚀 The Critical GCP Enabler: Cost-Effective High-Multiplier Methods**

The core achievement highlighted throughout this analysis is the **practical feasibility** of using high-multiplier RPC methods like:

*   `debug_traceTransaction` (`50x`) for granular execution tracing.
*   `eth_getLogs` (`50x`) for efficient event filtering.
*   `debug_storageRangeAt` (`50x`) for direct state inspection.
*   `trace_block` / `debug_traceBlock*` (`50x`) for block-level context.
*   `trace_call` (`50x`) for powerful simulations.
*   **Critically:** `trace_replayTransaction` and `trace_replayBlockTransactions` (`100x`) with `stateDiff` for precise state change analysis.

On most platforms, the **prohibitive cost multipliers** (especially the `100x` for replay methods) associated with these calls would make the analyses performed in this notebook impractical or extremely expensive for regular use or extensive research. **GCP's Blockchain Node Engine, with its generous free quotas that encompass these high-cost methods, fundamentally changes the economics of deep blockchain analysis.** It democratizes access to capabilities previously requiring specialized, costly infrastructure.

**💡 Key Capabilities Demonstrated:**

Through PyFlow, we applied these GCP-enabled methods to PYUSD, demonstrating capabilities such as:

1.  **Forensic Transaction Tracing:** Dissecting complex PYUSD flows (`debug_traceTransaction`, `trace_transaction`).
2.  **Precise State Auditing:** Observing exact balance, allowance, and supply changes (`stateDiff` via `trace_replayTransaction`).
3.  **Efficient Event Monitoring:** Isolating specific PYUSD events like Transfers and Approvals (`eth_getLogs`).
4.  **Contract Verification & Analysis:** Inspecting bytecode and storage layout (`eth_getCode`, `debug_storageRangeAt`).
5.  **Simulation & Gas Estimation:** Predicting transaction outcomes and costs (`trace_call`).
6.  **Block-Level Context:** Understanding PYUSD activity within the broader context of a block (`trace_block`, `debug_traceBlock*`).
7.  **Network Health Assessment:** Gauging congestion and estimating confirmation times (`txpool_status`).


**🎯 Impact for PYUSD & Beyond:**

By making these advanced analyses accessible, GCP empowers developers, analysts, auditors, and researchers to gain unparalleled insights into the PYUSD ecosystem and other on-chain assets. This enables more robust security practices, better gas optimization, deeper economic understanding, and enhanced regulatory compliance tooling.

**Future Directions:**

The foundation laid by PyFlow could be extended towards:

*   Real-time monitoring and alerting systems for PYUSD anomalies.
*   Integration with machine learning models for predictive analytics (e.g., fraud detection, flow prediction).
*   Building user-friendly dashboards (like the suggested Streamlit app) for specific analytical tasks.
*   Cross-chain analysis, leveraging GCP's potential support for other blockchains.
*  Cryptographic State Verification: Proving historical state using Merkle proofs (`eth_getProof`).


In conclusion, **PyFlow serves as a powerful testament to how Google Cloud's Blockchain Node Engine removes economic barriers to sophisticated blockchain analysis, unlocking a new level of transparency and understanding for critical assets like PYUSD.**